# Defining our data set
Our data set is composed of 700 poses samples
Each Pose has 30 features.
Poses are split in 7 categories. The purpose of the network will be to predict the correct category for a given sample.
We’ll use 600 samples for training and 100 samples for validation.
We’ll use a batch size of 10 for training and validation

In [1]:
import mxnet as mx
import numpy as np
import OpenSSL.SSL
import logging

logging.basicConfig(level=logging.INFO)
sample_count = 700
train_count = 600
valid_count = sample_count - train_count
feature_count = 30
category_count = 7
batch=10

/Users/abdu307/anaconda3/envs/python2/lib/python2.7/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


The categories for these 700 poses samples are represented as integers in the 0–6 range for the sequentially-generated data, 
beacause these data are generated sequentially we need to shuffle them using random.permutation.

In [2]:
# pose Data
poses = [[-0.0, -0.0, -0.024691358024691357, -0.07407407407407407, 0.01646090534979424, -0.1111111111111111, 0.012345679012345678, -0.1934156378600823, 0.00411522633744856, -0.25925925925925924, -0.07818930041152264, -0.09465020576131687, -0.09876543209876543, -0.17695473251028807, -0.09465020576131687, -0.25102880658436216, -0.0205761316872428, -0.24691358024691357, -0.037037037037037035, -0.37037037037037035, -0.05761316872427984, -0.48559670781893005, -0.06172839506172839, -0.23868312757201646, -0.05761316872427984, -0.3662551440329218, -0.06172839506172839, -0.48148148148148145, -0.03292181069958848, -0.16872427983539096], [-0.0, -0.0, -0.01646090534979424, -0.07407407407407407, 0.02880658436213992, -0.10699588477366255, 0.02880658436213992, -0.18106995884773663, 0.02880658436213992, -0.23868312757201646, -0.06172839506172839, -0.09876543209876543, -0.06995884773662552, -0.18106995884773663, -0.06584362139917696, -0.2674897119341564, 0.00823045267489712, -0.24279835390946503, 0.02880658436213992, -0.36213991769547327, 0.0411522633744856, -0.4609053497942387, -0.0411522633744856, -0.24279835390946503, -0.04526748971193416, -0.36213991769547327, -0.053497942386831275, -0.4732510288065844, -0.01646090534979424, -0.16872427983539096], [-0.0, -0.0, -0.014336917562724014, -0.06451612903225806, 0.021505376344086023, -0.0967741935483871, 0.03225806451612903, -0.15770609318996415, 0.03225806451612903, -0.21505376344086022, -0.053763440860215055, -0.07885304659498207, -0.07168458781362007, -0.14336917562724014, -0.07168458781362007, -0.20430107526881722, -0.010752688172043012, -0.1971326164874552, -0.025089605734767026, -0.3010752688172043, -0.04659498207885305, -0.3870967741935484, -0.043010752688172046, -0.1935483870967742, -0.043010752688172046, -0.2974910394265233, -0.053763440860215055, -0.3835125448028674, -0.021505376344086023, -0.13978494623655913], [-0.0, -0.0, -0.004672897196261682, -0.09345794392523364, 0.056074766355140186, -0.13551401869158877, 0.07476635514018691, -0.22429906542056074, 0.07009345794392523, -0.29439252336448596, -0.06542056074766354, -0.1308411214953271, -0.07009345794392523, -0.22897196261682243, -0.056074766355140186, -0.32710280373831774, 0.04205607476635514, -0.29906542056074764, 0.08411214953271028, -0.4392523364485981, 0.10747663551401869, -0.5514018691588785, -0.018691588785046728, -0.3037383177570093, -0.004672897196261682, -0.4439252336448598, -0.004672897196261682, -0.5560747663551402, 0.004672897196261682, -0.21495327102803738], [-0.0, -0.0, -0.020512820512820513, -0.1076923076923077, 0.046153846153846156, -0.15384615384615385, 0.06153846153846154, -0.2564102564102564, 0.06153846153846154, -0.3435897435897436, -0.08717948717948718, -0.14358974358974358, -0.1076923076923077, -0.2512820512820513, -0.09743589743589744, -0.35384615384615387, 0.015384615384615385, -0.3384615384615385, 0.02564102564102564, -0.49743589743589745, 0.020512820512820513, -0.6358974358974359, -0.05128205128205128, -0.3384615384615385, -0.07692307692307693, -0.49230769230769234, -0.09743589743589744, -0.6256410256410256, -0.015384615384615385, -0.24102564102564103], [-0.0, -0.0, -0.013452914798206279, -0.08968609865470852, 0.04484304932735426, -0.13004484304932734, 0.053811659192825115, -0.2242152466367713, 0.04484304932735426, -0.30493273542600896, -0.07174887892376682, -0.11210762331838565, -0.09865470852017937, -0.2062780269058296, -0.09865470852017937, -0.28699551569506726, -0.004484304932735426, -0.28699551569506726, -0.026905829596412557, -0.42152466367713004, -0.04035874439461883, -0.5381165919282511, -0.05829596412556054, -0.27802690582959644, -0.08520179372197309, -0.3991031390134529, -0.08968609865470852, -0.5067264573991032, -0.02242152466367713, -0.20179372197309417], [-0.0, -0.0, 0.0034602076124567475, -0.058823529411764705, 0.04498269896193772, -0.07958477508650519, 0.05536332179930796, -0.1453287197231834, 0.05536332179930796, -0.20069204152249134, -0.03460207612456748, -0.08304498269896193, -0.04844290657439446, -0.14878892733564014, -0.04152249134948097, -0.20761245674740483, 0.02422145328719723, -0.19377162629757785, 0.02422145328719723, -0.28719723183391005, 0.031141868512110725, -0.3737024221453287, -0.01384083044982699, -0.19377162629757785, -0.006920415224913495, -0.28719723183391005, -0.0, -0.36678200692041524, 0.006920415224913495, -0.1384083044982699], [-0.0, -0.0, -0.003076923076923077, -0.052307692307692305, 0.027692307692307693, -0.07384615384615385, 0.03076923076923077, -0.12615384615384614, 0.027692307692307693, -0.1753846153846154, -0.036923076923076927, -0.06769230769230769, -0.04923076923076923, -0.12, -0.05846153846153846, -0.16615384615384615, -0.0, -0.16307692307692306, -0.003076923076923077, -0.24307692307692308, -0.006153846153846154, -0.3076923076923077, -0.033846153846153845, -0.16, -0.036923076923076927, -0.24, -0.04923076923076923, -0.3076923076923077, -0.012307692307692308, -0.11384615384615385], [-0.0, -0.0, 0.003076923076923077, -0.052307692307692305, 0.033846153846153845, -0.06769230769230769, 0.046153846153846156, -0.12, 0.043076923076923075, -0.16923076923076924, -0.027692307692307693, -0.07076923076923076, -0.036923076923076927, -0.12307692307692308, -0.036923076923076927, -0.16615384615384615, 0.021538461538461538, -0.16615384615384615, 0.03076923076923077, -0.24307692307692308, 0.04923076923076923, -0.3076923076923077, -0.015384615384615385, -0.16615384615384615, -0.012307692307692308, -0.24, -0.006153846153846154, -0.3046153846153846, 0.003076923076923077, -0.12], [-0.0, -0.0, 0.006514657980456026, -0.05537459283387622, 0.03908794788273615, -0.0749185667752443, 0.05211726384364821, -0.13355048859934854, 0.035830618892508145, -0.18566775244299674, -0.02280130293159609, -0.0781758957654723, -0.026058631921824105, -0.13355048859934854, -0.026058631921824105, -0.1758957654723127, 0.02280130293159609, -0.17263843648208468, 0.016286644951140065, -0.250814332247557, 0.013029315960912053, -0.3355048859934853, -0.006514657980456026, -0.1758957654723127, -0.0, -0.254071661237785, 0.003257328990228013, -0.3322475570032573, 0.009771986970684038, -0.1270358306188925], [-0.0, -0.0, -0.007194244604316547, -0.0683453237410072, 0.03237410071942446, -0.09712230215827339, 0.04316546762589928, -0.1618705035971223, 0.04316546762589928, -0.21942446043165467, -0.046762589928057555, -0.08633093525179857, -0.06115107913669065, -0.1510791366906475, -0.05755395683453238, -0.21223021582733814, 0.007194244604316547, -0.20503597122302158, 0.02158273381294964, -0.302158273381295, 0.02877697841726619, -0.3920863309352518, -0.03237410071942446, -0.20503597122302158, -0.03597122302158273, -0.2949640287769784, -0.04316546762589928, -0.37050359712230213, -0.01079136690647482, -0.1510791366906475], [-0.0, -0.0, -0.0, -0.050473186119873815, 0.028391167192429023, -0.07255520504731862, 0.04100946372239748, -0.11987381703470032, 0.04100946372239748, -0.17034700315457413, -0.031545741324921134, -0.07255520504731862, -0.04100946372239748, -0.12618296529968454, -0.03470031545741325, -0.17981072555205047, 0.022082018927444796, -0.1640378548895899, 0.03470031545741325, -0.24605678233438485, 0.04100946372239748, -0.31230283911671924, -0.00946372239747634, -0.1640378548895899, -0.006309148264984227, -0.24290220820189273, -0.006309148264984227, -0.3186119873817035, 0.0031545741324921135, -0.11987381703470032], [-0.0, -0.0, -0.0113314447592068, -0.0453257790368272, 0.0056657223796034, -0.06232294617563739, 0.014164305949008499, -0.10198300283286119, 0.0169971671388102, -0.1529745042492918, -0.0339943342776204, -0.056657223796033995, -0.042492917847025496, -0.09631728045325778, -0.028328611898016998, -0.13314447592067988, -0.0028328611898017, -0.1359773371104816, -0.0113314447592068, -0.20396600566572237, -0.019830028328611898, -0.26628895184135976, -0.019830028328611898, -0.1359773371104816, -0.019830028328611898, -0.20396600566572237, -0.0226628895184136, -0.26628895184135976, -0.014164305949008499, -0.09915014164305949], [-0.0, -0.0, -0.011940298507462687, -0.04776119402985075, -0.0, -0.06567164179104477, 0.005970149253731343, -0.11044776119402985, 0.008955223880597015, -0.15522388059701492, -0.03283582089552239, -0.0626865671641791, -0.03880597014925373, -0.11343283582089553, -0.005970149253731343, -0.1582089552238806, -0.0, -0.15223880597014924, 0.014925373134328358, -0.2208955223880597, 0.008955223880597015, -0.2865671641791045, -0.023880597014925373, -0.15522388059701492, -0.023880597014925373, -0.22388059701492538, -0.023880597014925373, -0.28955223880597014, -0.014925373134328358, -0.11044776119402985], [-0.0, -0.0, -0.005988023952095809, -0.05089820359281437, 0.023952095808383235, -0.0688622754491018, 0.029940119760479042, -0.11676646706586827, 0.029940119760479042, -0.16467065868263472, -0.038922155688622756, -0.06287425149700598, -0.05389221556886228, -0.11377245508982035, -0.05389221556886228, -0.15868263473053892, 0.0029940119760479044, -0.1497005988023952, -0.005988023952095809, -0.2215568862275449, -0.011976047904191617, -0.2934131736526946, -0.02694610778443114, -0.1467065868263473, -0.02694610778443114, -0.2245508982035928, -0.02694610778443114, -0.2934131736526946, -0.008982035928143712, -0.10778443113772455], [-0.0, -0.0, 0.007722007722007722, -0.07722007722007722, 0.05791505791505792, -0.10038610038610038, 0.07722007722007722, -0.16988416988416988, 0.07335907335907337, -0.23166023166023167, -0.03088803088803089, -0.10810810810810811, -0.03861003861003861, -0.1776061776061776, -0.03088803088803089, -0.23938223938223938, 0.04247104247104247, -0.22393822393822393, 0.04247104247104247, -0.3359073359073359, 0.05791505791505792, -0.4362934362934363, -0.003861003861003861, -0.2277992277992278, 0.007722007722007722, -0.33204633204633205, 0.02702702702702703, -0.42084942084942084, 0.015444015444015444, -0.16602316602316602], [-0.0, -0.0, 0.03208556149732621, -0.10695187165775401, 0.09090909090909091, -0.1443850267379679, 0.0962566844919786, -0.2727272727272727, 0.058823529411764705, -0.37433155080213903, -0.0, -0.1443850267379679, -0.016042780748663103, -0.25668449197860965, -0.0481283422459893, -0.34759358288770054, 0.058823529411764705, -0.35294117647058826, 0.053475935828877004, -0.5294117647058824, 0.0481283422459893, -0.679144385026738, -0.0, -0.35294117647058826, -0.03208556149732621, -0.5080213903743316, -0.0106951871657754, -0.6524064171122995, 0.0374331550802139, -0.24598930481283424], [-0.0, -0.0, 0.012875536480686695, -0.0815450643776824, 0.06437768240343347, -0.10300429184549356, 0.06866952789699571, -0.18454935622317598, 0.012875536480686695, -0.24892703862660945, -0.02575107296137339, -0.11587982832618025, -0.04291845493562232, -0.19313304721030042, -0.04291845493562232, -0.24892703862660945, 0.06437768240343347, -0.26180257510729615, 0.0815450643776824, -0.3948497854077253, 0.10300429184549356, -0.5107296137339056, 0.004291845493562232, -0.2703862660944206, -0.0, -0.3905579399141631, 0.012875536480686695, -0.4892703862660944, 0.02575107296137339, -0.18454935622317598], [-0.0, -0.0, -0.015337423312883436, -0.049079754601226995, 0.012269938650306749, -0.0736196319018405, 0.018404907975460124, -0.1196319018404908, 0.009202453987730062, -0.15644171779141106, -0.05214723926380368, -0.06134969325153374, -0.0705521472392638, -0.11349693251533742, -0.07668711656441718, -0.16257668711656442, -0.018404907975460124, -0.15337423312883436, -0.027607361963190184, -0.2331288343558282, -0.046012269938650305, -0.3067484662576687, -0.049079754601226995, -0.147239263803681, -0.049079754601226995, -0.23006134969325154, -0.05828220858895705, -0.3128834355828221, -0.027607361963190184, -0.11042944785276074], [-0.0, -0.0, -0.008287292817679558, -0.04419889502762431, 0.016574585635359115, -0.058011049723756904, 0.027624309392265192, -0.08839779005524862, 0.011049723756906077, -0.10497237569060773, -0.03314917127071823, -0.052486187845303865, -0.06353591160220995, -0.0856353591160221, -0.049723756906077346, -0.1132596685082873, 0.0027624309392265192, -0.1298342541436464, 0.008287292817679558, -0.19337016574585636, 0.011049723756906077, -0.2513812154696133, -0.024861878453038673, -0.1298342541436464, -0.024861878453038673, -0.19060773480662985, -0.022099447513812154, -0.24861878453038674, -0.011049723756906077, -0.09392265193370165], [-0.0, -0.0, -0.010498687664041995, -0.03937007874015748, 0.007874015748031496, -0.05511811023622047, 0.013123359580052493, -0.08398950131233596, -0.0, -0.10236220472440945, -0.03412073490813648, -0.047244094488188976, -0.06036745406824147, -0.08136482939632546, -0.03937007874015748, -0.10498687664041995, -0.0026246719160104987, -0.12073490813648294, 0.0026246719160104987, -0.17060367454068243, -0.0026246719160104987, -0.22572178477690288, -0.026246719160104987, -0.12073490813648294, -0.026246719160104987, -0.17585301837270342, -0.031496062992125984, -0.23097112860892388, -0.015748031496062992, -0.08398950131233596], [-0.0, -0.0, -0.011185682326621925, -0.026845637583892617, -0.0, -0.03355704697986577, 0.006711409395973154, -0.05592841163310962, -0.0022371364653243847, -0.06711409395973154, -0.03131991051454139, -0.029082774049217, -0.053691275167785234, -0.049217002237136466, -0.0447427293064877, -0.07158836689038031, -0.013422818791946308, -0.08053691275167785, -0.020134228187919462, -0.12751677852348994, -0.026845637583892617, -0.174496644295302, -0.029082774049217, -0.07829977628635347, -0.026845637583892617, -0.1297539149888143, -0.029082774049217, -0.1767337807606264, -0.020134228187919462, -0.05592841163310962], [-0.0, -0.0, -0.007874015748031496, -0.03937007874015748, 0.007874015748031496, -0.05249343832020997, 0.013123359580052493, -0.08661417322834646, 0.010498687664041995, -0.10761154855643044, -0.028871391076115485, -0.047244094488188976, -0.05249343832020997, -0.08136482939632546, -0.028871391076115485, -0.10761154855643044, -0.0, -0.11286089238845144, -0.0026246719160104987, -0.16535433070866143, -0.013123359580052493, -0.2204724409448819, -0.02099737532808399, -0.11548556430446194, -0.015748031496062992, -0.16535433070866143, -0.023622047244094488, -0.2099737532808399, -0.010498687664041995, -0.08398950131233596], [-0.0, -0.0, -0.005376344086021506, -0.04032258064516129, 0.01881720430107527, -0.053763440860215055, 0.03763440860215054, -0.08602150537634409, 0.026881720430107527, -0.11559139784946236, -0.03225806451612903, -0.051075268817204304, -0.053763440860215055, -0.08333333333333333, -0.0456989247311828, -0.11827956989247312, 0.008064516129032258, -0.12096774193548387, 0.010752688172043012, -0.1881720430107527, 0.010752688172043012, -0.24731182795698925, -0.024193548387096774, -0.11827956989247312, -0.03494623655913978, -0.1827956989247312, -0.051075268817204304, -0.239247311827957, -0.005376344086021506, -0.08602150537634409], [-0.0, -0.0, -0.0055248618784530384, -0.04419889502762431, 0.022099447513812154, -0.055248618784530384, 0.03867403314917127, -0.09116022099447514, 0.03038674033149171, -0.12154696132596685, -0.03314917127071823, -0.049723756906077346, -0.058011049723756904, -0.08287292817679558, -0.058011049723756904, -0.11878453038674033, 0.0027624309392265192, -0.1270718232044199, -0.013812154696132596, -0.19337016574585636, -0.027624309392265192, -0.2569060773480663, -0.024861878453038673, -0.12430939226519337, -0.027624309392265192, -0.19060773480662985, -0.03038674033149171, -0.2513812154696133, -0.008287292817679558, -0.09116022099447514], [-0.0, -0.0, 0.009771986970684038, -0.05537459283387622, 0.04560260586319218, -0.06840390879478828, 0.08469055374592833, -0.11400651465798045, 0.07166123778501629, -0.1563517915309446, -0.029315960912052116, -0.0781758957654723, -0.04234527687296417, -0.1270358306188925, -0.03257328990228013, -0.16938110749185667, 0.03908794788273615, -0.16938110749185667, 0.04234527687296417, -0.2671009771986971, 0.06188925081433225, -0.3517915309446254, -0.003257328990228013, -0.16938110749185667, -0.0, -0.25732899022801303, 0.013029315960912053, -0.3322475570032573, 0.016286644951140065, -0.12052117263843648], [-0.0, -0.0, 0.004484304932735426, -0.08968609865470852, 0.06278026905829596, -0.11659192825112108, 0.11210762331838565, -0.18834080717488788, 0.08520179372197309, -0.2556053811659193, -0.053811659192825115, -0.11659192825112108, -0.06726457399103139, -0.19730941704035873, -0.08520179372197309, -0.2600896860986547, 0.026905829596412557, -0.27802690582959644, 0.017937219730941704, -0.42152466367713004, 0.008968609865470852, -0.5336322869955157, -0.03587443946188341, -0.273542600896861, -0.053811659192825115, -0.4080717488789238, -0.04035874439461883, -0.5112107623318386, 0.004484304932735426, -0.19730941704035873], [-0.0, -0.0, 0.026881720430107527, -0.11290322580645161, 0.10215053763440861, -0.14516129032258066, 0.17204301075268819, -0.23118279569892472, 0.13440860215053763, -0.3172043010752688, -0.03763440860215054, -0.16666666666666666, -0.03225806451612903, -0.26344086021505375, -0.03225806451612903, -0.3279569892473118, 0.06451612903225806, -0.34946236559139787, 0.08602150537634409, -0.5268817204301075, 0.12903225806451613, -0.6774193548387096, -0.016129032258064516, -0.3548387096774194, -0.021505376344086023, -0.5268817204301075, -0.016129032258064516, -0.6666666666666666, 0.03225806451612903, -0.25268817204301075], [-0.0, -0.0, 0.008264462809917356, -0.19008264462809918, 0.11570247933884298, -0.2396694214876033, 0.24793388429752067, -0.4049586776859504, 0.14049586776859505, -0.5206611570247934, -0.09090909090909091, -0.2644628099173554, -0.09090909090909091, -0.4214876033057851, -0.0743801652892562, -0.5371900826446281, 0.03305785123966942, -0.5950413223140496, -0.008264462809917356, -0.8842975206611571, -0.024793388429752067, -1.1487603305785123, -0.08264462809917356, -0.5867768595041323, -0.06611570247933884, -0.859504132231405, -0.008264462809917356, -0.9917355371900827, -0.0, -0.4214876033057851], [-0.0, -0.0, -0.01694915254237288, -0.11864406779661017, 0.062146892655367235, -0.15819209039548024, 0.096045197740113, -0.2542372881355932, 0.062146892655367235, -0.3220338983050847, -0.1016949152542373, -0.15819209039548024, -0.15819209039548024, -0.2598870056497175, -0.13559322033898305, -0.327683615819209, 0.02824858757062147, -0.3672316384180791, 0.062146892655367235, -0.559322033898305, 0.07909604519774012, -0.7231638418079096, -0.05649717514124294, -0.3672316384180791, -0.04519774011299435, -0.559322033898305, -0.03389830508474576, -0.7175141242937854, -0.01694915254237288, -0.2598870056497175], [-0.0, -0.0, -0.028469750889679714, -0.05693950177935943, -0.014234875444839857, -0.08540925266903915, -0.02491103202846975, -0.1387900355871886, -0.03558718861209965, -0.18505338078291814, -0.06761565836298933, -0.06761565836298933, -0.1103202846975089, -0.12099644128113879, -0.09252669039145907, -0.17437722419928825, -0.03914590747330961, -0.18861209964412812, -0.042704626334519574, -0.28825622775800713, -0.03914590747330961, -0.398576512455516, -0.0711743772241993, -0.18505338078291814, -0.05338078291814947, -0.28825622775800713, -0.046263345195729534, -0.3879003558718861, -0.0498220640569395, -0.13167259786476868], [-0.0, -0.0, -0.033444816053511704, -0.05351170568561873, -0.033444816053511704, -0.07357859531772576, -0.043478260869565216, -0.12374581939799331, -0.03678929765886288, -0.1605351170568562, -0.05351170568561873, -0.06688963210702341, -0.0802675585284281, -0.11705685618729098, -0.06688963210702341, -0.16387959866220736, -0.05351170568561873, -0.17391304347826086, -0.05351170568561873, -0.2608695652173913, -0.07023411371237458, -0.3411371237458194, -0.06688963210702341, -0.17391304347826086, -0.05351170568561873, -0.26421404682274247, -0.07692307692307693, -0.33444816053511706, -0.05351170568561873, -0.12040133779264214], [-0.0, -0.0, -0.024193548387096774, -0.04032258064516129, -0.021505376344086023, -0.053763440860215055, -0.024193548387096774, -0.08602150537634409, -0.024193548387096774, -0.11021505376344086, -0.03763440860215054, -0.0456989247311828, -0.06451612903225806, -0.07795698924731183, -0.043010752688172046, -0.11021505376344086, -0.03763440860215054, -0.11559139784946236, -0.03494623655913978, -0.1774193548387097, -0.053763440860215055, -0.23655913978494625, -0.043010752688172046, -0.11827956989247312, -0.021505376344086023, -0.1827956989247312, -0.01881720430107527, -0.24731182795698925, -0.03494623655913978, -0.08333333333333333], [-0.0, -0.0, -0.014164305949008499, -0.0453257790368272, 0.0028328611898017, -0.06232294617563739, -0.0, -0.09915014164305949, -0.0028328611898017, -0.12464589235127478, -0.036827195467422094, -0.053824362606232294, -0.0708215297450425, -0.0906515580736544, -0.04815864022662889, -0.12181303116147309, -0.0113314447592068, -0.1359773371104816, 0.0028328611898017, -0.20679886685552407, 0.0113314447592068, -0.26628895184135976, -0.039660056657223795, -0.1359773371104816, -0.042492917847025496, -0.20396600566572237, -0.05099150141643059, -0.26628895184135976, -0.019830028328611898, -0.09631728045325778], [-0.0, -0.0, -0.005988023952095809, -0.05089820359281437, 0.020958083832335328, -0.0658682634730539, 0.038922155688622756, -0.10179640718562874, 0.029940119760479042, -0.1407185628742515, -0.03592814371257485, -0.059880239520958084, -0.0688622754491018, -0.10179640718562874, -0.05688622754491018, -0.1377245508982036, 0.005988023952095809, -0.1467065868263473, -0.008982035928143712, -0.2245508982035928, -0.017964071856287425, -0.2964071856287425, -0.02694610778443114, -0.1437125748502994, -0.020958083832335328, -0.2215568862275449, -0.017964071856287425, -0.2934131736526946, -0.008982035928143712, -0.10479041916167664], [-0.0, -0.0, -0.003703703703703704, -0.06666666666666667, 0.040740740740740744, -0.08888888888888889, 0.07037037037037037, -0.14444444444444443, 0.05925925925925926, -0.18888888888888888, -0.05185185185185185, -0.08518518518518518, -0.08518518518518518, -0.14814814814814814, -0.07407407407407407, -0.2037037037037037, 0.018518518518518517, -0.2111111111111111, 0.011111111111111112, -0.32222222222222224, 0.011111111111111112, -0.4148148148148148, -0.02962962962962963, -0.2074074074074074, -0.02962962962962963, -0.31851851851851853, -0.02962962962962963, -0.40370370370370373, -0.003703703703703704, -0.14814814814814814], [-0.0, -0.0, 0.005376344086021506, -0.11290322580645161, 0.08064516129032258, -0.15591397849462366, 0.13440860215053763, -0.24731182795698925, 0.10752688172043011, -0.3225806451612903, -0.06989247311827956, -0.15053763440860216, -0.11290322580645161, -0.24193548387096775, -0.10752688172043011, -0.3225806451612903, 0.03763440860215054, -0.34946236559139787, 0.03225806451612903, -0.5268817204301075, 0.05913978494623656, -0.6666666666666666, -0.03225806451612903, -0.3548387096774194, -0.026881720430107527, -0.532258064516129, -0.021505376344086023, -0.6774193548387096, 0.005376344086021506, -0.25268817204301075], [-0.0, -0.0, -0.006024096385542169, -0.13855421686746988, 0.08433734939759036, -0.18674698795180722, 0.14457831325301204, -0.29518072289156627, 0.1144578313253012, -0.37349397590361444, -0.0963855421686747, -0.1746987951807229, -0.14457831325301204, -0.28313253012048195, -0.12650602409638553, -0.3674698795180723, 0.03614457831325301, -0.40963855421686746, 0.04819277108433735, -0.6204819277108434, 0.060240963855421686, -0.7951807228915663, -0.06626506024096386, -0.4036144578313253, -0.10843373493975904, -0.6024096385542169, -0.16265060240963855, -0.7650602409638554, -0.012048192771084338, -0.2891566265060241], [-0.0, -0.0, -0.005128205128205128, -0.1076923076923077, 0.06153846153846154, -0.14358974358974358, 0.1076923076923077, -0.23076923076923078, 0.07692307692307693, -0.29743589743589743, -0.07692307692307693, -0.13846153846153847, -0.13333333333333333, -0.22564102564102564, -0.11794871794871795, -0.2923076923076923, 0.015384615384615385, -0.3282051282051282, 0.010256410256410256, -0.49230769230769234, -0.0, -0.6256410256410256, -0.05128205128205128, -0.3230769230769231, -0.041025641025641026, -0.48717948717948717, -0.041025641025641026, -0.6307692307692307, -0.010256410256410256, -0.23076923076923078], [-0.0, -0.0, -0.0044444444444444444, -0.08, 0.05333333333333334, -0.1111111111111111, 0.08, -0.2088888888888889, 0.08, -0.29333333333333333, -0.06222222222222222, -0.1111111111111111, -0.08, -0.20444444444444446, -0.07555555555555556, -0.28444444444444444, 0.03111111111111111, -0.27555555555555555, 0.04888888888888889, -0.4177777777777778, 0.05333333333333334, -0.5377777777777778, -0.02666666666666667, -0.27555555555555555, -0.04, -0.41333333333333333, -0.05333333333333334, -0.5288888888888889, -0.0, -0.19555555555555557], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.06046511627906977, -0.12558139534883722, 0.08372093023255814, -0.22325581395348837, 0.08372093023255814, -0.30697674418604654, -0.06511627906976744, -0.12558139534883722, -0.08372093023255814, -0.2186046511627907, -0.07906976744186046, -0.29767441860465116, 0.037209302325581395, -0.2930232558139535, 0.05116279069767442, -0.4372093023255814, 0.05116279069767442, -0.5581395348837209, -0.027906976744186046, -0.29767441860465116, -0.04186046511627907, -0.4372093023255814, -0.05581395348837209, -0.5581395348837209, -0.0, -0.20930232558139536], [-0.0, -0.0, -0.004901960784313725, -0.10294117647058823, 0.06862745098039216, -0.13725490196078433, 0.08333333333333333, -0.24019607843137256, 0.0784313725490196, -0.3333333333333333, -0.07352941176470588, -0.13725490196078433, -0.09313725490196079, -0.23039215686274508, -0.09313725490196079, -0.31862745098039214, 0.029411764705882353, -0.31862745098039214, 0.04411764705882353, -0.47058823529411764, 0.04411764705882353, -0.5980392156862745, -0.0392156862745098, -0.31862745098039214, -0.05392156862745098, -0.46568627450980393, -0.06862745098039216, -0.5931372549019608, -0.004901960784313725, -0.22549019607843138], [-0.0, -0.0, -0.005154639175257732, -0.1134020618556701, 0.07216494845360824, -0.14948453608247422, 0.08762886597938144, -0.26288659793814434, 0.08247422680412371, -0.3556701030927835, -0.07731958762886598, -0.14948453608247422, -0.0979381443298969, -0.24742268041237114, -0.0979381443298969, -0.3402061855670103, 0.03608247422680412, -0.3402061855670103, 0.04639175257731959, -0.5, 0.04639175257731959, -0.634020618556701, -0.041237113402061855, -0.34536082474226804, -0.05670103092783505, -0.4948453608247423, -0.07216494845360824, -0.6288659793814433, -0.0, -0.2422680412371134], [-0.0, -0.0, -0.0, -0.0975609756097561, 0.06341463414634146, -0.13658536585365855, 0.08292682926829269, -0.23902439024390243, 0.08292682926829269, -0.33170731707317075, -0.06829268292682927, -0.13170731707317074, -0.08780487804878048, -0.22926829268292684, -0.09268292682926829, -0.3170731707317073, 0.02926829268292683, -0.3121951219512195, 0.04390243902439024, -0.4634146341463415, 0.04878048780487805, -0.5951219512195122, -0.03902439024390244, -0.3121951219512195, -0.05365853658536585, -0.4634146341463415, -0.06341463414634146, -0.5951219512195122, -0.004878048780487805, -0.22439024390243903], [-0.0, -0.0, -0.004672897196261682, -0.09345794392523364, 0.056074766355140186, -0.1308411214953271, 0.0794392523364486, -0.22897196261682243, 0.0794392523364486, -0.3130841121495327, -0.06542056074766354, -0.1261682242990654, -0.08411214953271028, -0.22897196261682243, -0.08411214953271028, -0.3177570093457944, 0.03271028037383177, -0.29906542056074764, 0.056074766355140186, -0.4392523364485981, 0.07009345794392523, -0.5607476635514018, -0.03271028037383177, -0.3037383177570093, -0.03271028037383177, -0.43457943925233644, -0.028037383177570093, -0.5560747663551402, -0.0, -0.2102803738317757], [-0.0, -0.0, -0.004651162790697674, -0.08837209302325581, 0.06046511627906977, -0.13023255813953488, 0.07441860465116279, -0.22325581395348837, 0.06976744186046512, -0.3023255813953488, -0.06976744186046512, -0.12093023255813953, -0.08837209302325581, -0.21395348837209302, -0.08372093023255814, -0.2930232558139535, 0.027906976744186046, -0.29767441860465116, 0.03255813953488372, -0.4418604651162791, 0.03255813953488372, -0.5627906976744186, -0.037209302325581395, -0.2930232558139535, -0.046511627906976744, -0.4325581395348837, -0.06511627906976744, -0.5534883720930233, -0.004651162790697674, -0.20930232558139536], [-0.0, -0.0, -0.0, -0.0975609756097561, 0.06829268292682927, -0.14634146341463414, 0.08780487804878048, -0.24878048780487805, 0.08292682926829269, -0.33170731707317075, -0.06341463414634146, -0.13658536585365855, -0.08292682926829269, -0.23414634146341465, -0.08292682926829269, -0.32195121951219513, 0.03414634146341464, -0.3170731707317073, 0.04878048780487805, -0.4634146341463415, 0.04878048780487805, -0.5951219512195122, -0.03414634146341464, -0.3170731707317073, -0.03902439024390244, -0.4585365853658537, -0.05365853658536585, -0.5902439024390244, -0.0, -0.22926829268292684], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05581395348837209, -0.13023255813953488, 0.07906976744186046, -0.22790697674418606, 0.08372093023255814, -0.30697674418604654, -0.06511627906976744, -0.12093023255813953, -0.08372093023255814, -0.22325581395348837, -0.07906976744186046, -0.3116279069767442, 0.03255813953488372, -0.2930232558139535, 0.046511627906976744, -0.4372093023255814, 0.05116279069767442, -0.5627906976744186, -0.03255813953488372, -0.2930232558139535, -0.037209302325581395, -0.4325581395348837, -0.046511627906976744, -0.5627906976744186, -0.0, -0.20930232558139536], [-0.0, -0.0, -0.004878048780487805, -0.0975609756097561, 0.05853658536585366, -0.14634146341463414, 0.09268292682926829, -0.23902439024390243, 0.09268292682926829, -0.32195121951219513, -0.07317073170731707, -0.13658536585365855, -0.09268292682926829, -0.23902439024390243, -0.0975609756097561, -0.33658536585365856, 0.03414634146341464, -0.3170731707317073, 0.04390243902439024, -0.4682926829268293, 0.04878048780487805, -0.5951219512195122, -0.03902439024390244, -0.3170731707317073, -0.04390243902439024, -0.4585365853658537, -0.05365853658536585, -0.5902439024390244, -0.004878048780487805, -0.22439024390243903], [-0.0, -0.0, -0.0, -0.0975609756097561, 0.06341463414634146, -0.14634146341463414, 0.0975609756097561, -0.23902439024390243, 0.1073170731707317, -0.32195121951219513, -0.07317073170731707, -0.13170731707317074, -0.0975609756097561, -0.23902439024390243, -0.1073170731707317, -0.32682926829268294, 0.03414634146341464, -0.3121951219512195, 0.04390243902439024, -0.4682926829268293, 0.04878048780487805, -0.6, -0.03902439024390244, -0.3121951219512195, -0.04390243902439024, -0.4634146341463415, -0.05365853658536585, -0.5902439024390244, -0.0, -0.22439024390243903], [-0.0, -0.0, -0.0, -0.0975609756097561, 0.06341463414634146, -0.14634146341463414, 0.0975609756097561, -0.24390243902439024, 0.11219512195121951, -0.32682926829268294, -0.07317073170731707, -0.13170731707317074, -0.1024390243902439, -0.23902439024390243, -0.11219512195121951, -0.32682926829268294, 0.03414634146341464, -0.3121951219512195, 0.04390243902439024, -0.4682926829268293, 0.04878048780487805, -0.6, -0.04390243902439024, -0.3121951219512195, -0.04390243902439024, -0.4634146341463415, -0.05365853658536585, -0.5902439024390244, -0.0, -0.22439024390243903], [-0.0, -0.0, -0.0, -0.0975609756097561, 0.06341463414634146, -0.14634146341463414, 0.0975609756097561, -0.24390243902439024, 0.11219512195121951, -0.32195121951219513, -0.07317073170731707, -0.13170731707317074, -0.1024390243902439, -0.23902439024390243, -0.11219512195121951, -0.32682926829268294, 0.03414634146341464, -0.3121951219512195, 0.04390243902439024, -0.4682926829268293, 0.04878048780487805, -0.5951219512195122, -0.04390243902439024, -0.3121951219512195, -0.04390243902439024, -0.4634146341463415, -0.05365853658536585, -0.5902439024390244, -0.0, -0.22439024390243903], [-0.0, -0.0, -0.004878048780487805, -0.0975609756097561, 0.05853658536585366, -0.14634146341463414, 0.0975609756097561, -0.23902439024390243, 0.11219512195121951, -0.32195121951219513, -0.07317073170731707, -0.13658536585365855, -0.1024390243902439, -0.23902439024390243, -0.11219512195121951, -0.32682926829268294, 0.03414634146341464, -0.3121951219512195, 0.04390243902439024, -0.4682926829268293, 0.04878048780487805, -0.6, -0.03902439024390244, -0.3121951219512195, -0.04390243902439024, -0.4634146341463415, -0.05365853658536585, -0.5902439024390244, -0.0, -0.22439024390243903], [-0.0, -0.0, -0.004878048780487805, -0.0975609756097561, 0.06341463414634146, -0.14634146341463414, 0.0975609756097561, -0.23902439024390243, 0.11219512195121951, -0.32195121951219513, -0.07317073170731707, -0.13658536585365855, -0.1024390243902439, -0.23902439024390243, -0.11219512195121951, -0.32682926829268294, 0.03414634146341464, -0.3121951219512195, 0.04390243902439024, -0.4682926829268293, 0.04878048780487805, -0.5951219512195122, -0.03902439024390244, -0.3121951219512195, -0.04390243902439024, -0.4634146341463415, -0.05365853658536585, -0.5902439024390244, -0.0, -0.22439024390243903], [-0.0, -0.0, -0.004878048780487805, -0.0975609756097561, 0.05853658536585366, -0.14146341463414633, 0.0975609756097561, -0.23902439024390243, 0.11219512195121951, -0.32195121951219513, -0.07317073170731707, -0.13170731707317074, -0.1024390243902439, -0.23902439024390243, -0.11219512195121951, -0.32682926829268294, 0.03414634146341464, -0.3121951219512195, 0.04390243902439024, -0.4682926829268293, 0.04878048780487805, -0.6, -0.03902439024390244, -0.3121951219512195, -0.04390243902439024, -0.4634146341463415, -0.05365853658536585, -0.5951219512195122, -0.004878048780487805, -0.22439024390243903], [-0.0, -0.0, -0.004878048780487805, -0.0975609756097561, 0.06341463414634146, -0.14146341463414633, 0.0975609756097561, -0.23902439024390243, 0.11219512195121951, -0.32195121951219513, -0.07317073170731707, -0.13658536585365855, -0.1024390243902439, -0.23902439024390243, -0.11219512195121951, -0.32682926829268294, 0.03414634146341464, -0.3121951219512195, 0.04390243902439024, -0.4682926829268293, 0.04878048780487805, -0.5951219512195122, -0.03902439024390244, -0.3121951219512195, -0.04390243902439024, -0.4634146341463415, -0.05365853658536585, -0.5902439024390244, -0.0, -0.22439024390243903], [-0.0, -0.0, -0.0, -0.10294117647058823, 0.06372549019607843, -0.14705882352941177, 0.09803921568627451, -0.24019607843137256, 0.11274509803921569, -0.3235294117647059, -0.07352941176470588, -0.13725490196078433, -0.10294117647058823, -0.24019607843137256, -0.11274509803921569, -0.3235294117647059, 0.03431372549019608, -0.3137254901960784, 0.04411764705882353, -0.47058823529411764, 0.049019607843137254, -0.6029411764705882, -0.0392156862745098, -0.3137254901960784, -0.04411764705882353, -0.46568627450980393, -0.05392156862745098, -0.5931372549019608, -0.0, -0.22549019607843138], [-0.0, -0.0, -0.004878048780487805, -0.0975609756097561, 0.05853658536585366, -0.14146341463414633, 0.0975609756097561, -0.23902439024390243, 0.11219512195121951, -0.32195121951219513, -0.07317073170731707, -0.13170731707317074, -0.1024390243902439, -0.23902439024390243, -0.11219512195121951, -0.32682926829268294, 0.03414634146341464, -0.3121951219512195, 0.04390243902439024, -0.4682926829268293, 0.04878048780487805, -0.6, -0.04390243902439024, -0.3121951219512195, -0.04390243902439024, -0.4634146341463415, -0.05365853658536585, -0.5902439024390244, -0.004878048780487805, -0.22439024390243903], [-0.0, -0.0, -0.0, -0.0975609756097561, 0.06341463414634146, -0.14634146341463414, 0.0975609756097561, -0.23902439024390243, 0.11219512195121951, -0.32195121951219513, -0.07317073170731707, -0.13658536585365855, -0.1024390243902439, -0.23902439024390243, -0.11219512195121951, -0.32682926829268294, 0.03414634146341464, -0.3121951219512195, 0.04390243902439024, -0.4682926829268293, 0.04878048780487805, -0.5951219512195122, -0.03902439024390244, -0.3121951219512195, -0.04390243902439024, -0.4634146341463415, -0.05365853658536585, -0.5902439024390244, -0.0, -0.22439024390243903], [-0.0, -0.0, -0.004878048780487805, -0.0975609756097561, 0.06341463414634146, -0.14634146341463414, 0.0975609756097561, -0.23902439024390243, 0.11219512195121951, -0.32682926829268294, -0.07317073170731707, -0.13170731707317074, -0.1024390243902439, -0.23902439024390243, -0.11219512195121951, -0.32682926829268294, 0.03414634146341464, -0.3121951219512195, 0.04390243902439024, -0.4682926829268293, 0.04878048780487805, -0.5951219512195122, -0.04390243902439024, -0.3121951219512195, -0.04390243902439024, -0.4634146341463415, -0.05365853658536585, -0.5902439024390244, -0.0, -0.22439024390243903], [-0.0, -0.0, -0.004878048780487805, -0.0975609756097561, 0.06341463414634146, -0.14634146341463414, 0.0975609756097561, -0.23902439024390243, 0.11219512195121951, -0.32195121951219513, -0.07317073170731707, -0.13658536585365855, -0.1073170731707317, -0.23902439024390243, -0.11707317073170732, -0.32682926829268294, 0.03414634146341464, -0.3121951219512195, 0.04390243902439024, -0.4634146341463415, 0.04878048780487805, -0.5951219512195122, -0.04390243902439024, -0.3121951219512195, -0.04390243902439024, -0.4634146341463415, -0.05365853658536585, -0.5902439024390244, -0.004878048780487805, -0.22439024390243903], [-0.0, -0.0, -0.004878048780487805, -0.0975609756097561, 0.06341463414634146, -0.14146341463414633, 0.0975609756097561, -0.23902439024390243, 0.1073170731707317, -0.32195121951219513, -0.07317073170731707, -0.13658536585365855, -0.1024390243902439, -0.23902439024390243, -0.11219512195121951, -0.32682926829268294, 0.03414634146341464, -0.3121951219512195, 0.04390243902439024, -0.4682926829268293, 0.04878048780487805, -0.5951219512195122, -0.04390243902439024, -0.3121951219512195, -0.04390243902439024, -0.4585365853658537, -0.05365853658536585, -0.5902439024390244, -0.004878048780487805, -0.22439024390243903], [-0.0, -0.0, -0.0, -0.1076923076923077, 0.07179487179487179, -0.14358974358974358, 0.1076923076923077, -0.24615384615384617, 0.11794871794871795, -0.3384615384615385, -0.07179487179487179, -0.14358974358974358, -0.10256410256410256, -0.2512820512820513, -0.11282051282051282, -0.3487179487179487, 0.041025641025641026, -0.3333333333333333, 0.05128205128205128, -0.49230769230769234, 0.05128205128205128, -0.6307692307692307, -0.041025641025641026, -0.3333333333333333, -0.046153846153846156, -0.49230769230769234, -0.05128205128205128, -0.6205128205128205, -0.0, -0.2358974358974359], [-0.0, -0.0, 0.01646090534979424, -0.07407407407407407, 0.06584362139917696, -0.09465020576131687, 0.09053497942386832, -0.17695473251028807, 0.09465020576131687, -0.25102880658436216, -0.012345679012345678, -0.1111111111111111, -0.024691358024691357, -0.18518518518518517, -0.037037037037037035, -0.2551440329218107, 0.0411522633744856, -0.24691358024691357, 0.04526748971193416, -0.37448559670781895, 0.07818930041152264, -0.48559670781893005, 0.00411522633744856, -0.25102880658436216, 0.00823045267489712, -0.3662551440329218, 0.01646090534979424, -0.4567901234567901, 0.024691358024691357, -0.1728395061728395], [-0.0, -0.0, 0.02459016393442623, -0.06967213114754098, 0.06557377049180328, -0.09016393442622951, 0.08196721311475409, -0.1680327868852459, 0.08196721311475409, -0.2459016393442623, 0.00819672131147541, -0.10245901639344263, 0.012295081967213115, -0.1680327868852459, -0.012295081967213115, -0.22131147540983606, 0.036885245901639344, -0.2459016393442623, 0.04918032786885246, -0.3729508196721312, 0.06147540983606557, -0.4918032786885246, -0.0, -0.2459016393442623, -0.0, -0.36065573770491804, 0.012295081967213115, -0.45491803278688525, 0.02459016393442623, -0.1680327868852459], [-0.0, -0.0, 0.02145922746781116, -0.0815450643776824, 0.07296137339055794, -0.09871244635193133, 0.0944206008583691, -0.18454935622317598, 0.09871244635193133, -0.26180257510729615, -0.008583690987124463, -0.11587982832618025, -0.017167381974248927, -0.19313304721030042, -0.030042918454935622, -0.2703862660944206, 0.04291845493562232, -0.26180257510729615, 0.055793991416309016, -0.3948497854077253, 0.06866952789699571, -0.51931330472103, -0.0, -0.26180257510729615, 0.004291845493562232, -0.38626609442060084, 0.017167381974248927, -0.48068669527896996, 0.02575107296137339, -0.18454935622317598], [-0.0, -0.0, 0.013452914798206279, -0.08968609865470852, 0.06278026905829596, -0.11210762331838565, 0.08520179372197309, -0.20179372197309417, 0.08968609865470852, -0.2825112107623318, -0.026905829596412557, -0.12556053811659193, -0.03587443946188341, -0.2062780269058296, -0.04932735426008968, -0.2825112107623318, 0.03139013452914798, -0.27802690582959644, 0.03587443946188341, -0.4170403587443946, 0.05829596412556054, -0.547085201793722, -0.017937219730941704, -0.2825112107623318, -0.008968609865470852, -0.4080717488789238, 0.004484304932735426, -0.5067264573991032, 0.013452914798206279, -0.19730941704035873], [-0.0, -0.0, 0.008928571428571428, -0.08482142857142858, 0.05803571428571429, -0.11160714285714286, 0.08035714285714286, -0.20089285714285715, 0.08035714285714286, -0.28125, -0.03125, -0.12053571428571429, -0.044642857142857144, -0.20089285714285715, -0.05357142857142857, -0.2767857142857143, 0.022321428571428572, -0.2767857142857143, 0.03571428571428571, -0.41517857142857145, 0.0625, -0.5446428571428571, -0.017857142857142856, -0.28125, -0.013392857142857142, -0.40625, 0.004464285714285714, -0.5044642857142857, 0.008928571428571428, -0.19642857142857142], [-0.0, -0.0, 0.012875536480686695, -0.0815450643776824, 0.060085836909871244, -0.10300429184549356, 0.0815450643776824, -0.19313304721030042, 0.0815450643776824, -0.26609442060085836, -0.02575107296137339, -0.11158798283261803, -0.04291845493562232, -0.19313304721030042, -0.055793991416309016, -0.26609442060085836, 0.02575107296137339, -0.26609442060085836, 0.03862660944206009, -0.3948497854077253, 0.060085836909871244, -0.51931330472103, -0.017167381974248927, -0.26609442060085836, -0.008583690987124463, -0.38626609442060084, 0.004291845493562232, -0.48068669527896996, 0.008583690987124463, -0.18454935622317598], [-0.0, -0.0, 0.008583690987124463, -0.0815450643776824, 0.060085836909871244, -0.10300429184549356, 0.0815450643776824, -0.1888412017167382, 0.08583690987124463, -0.26609442060085836, -0.030042918454935622, -0.11158798283261803, -0.05150214592274678, -0.19742489270386265, -0.060085836909871244, -0.27467811158798283, 0.030042918454935622, -0.26609442060085836, 0.03862660944206009, -0.3948497854077253, 0.06437768240343347, -0.51931330472103, -0.017167381974248927, -0.26609442060085836, -0.008583690987124463, -0.38626609442060084, 0.008583690987124463, -0.48068669527896996, 0.008583690987124463, -0.18454935622317598], [-0.0, -0.0, -0.0, -0.08620689655172414, 0.05603448275862069, -0.11206896551724138, 0.09051724137931035, -0.1896551724137931, 0.09913793103448276, -0.2672413793103448, -0.05172413793103448, -0.1206896551724138, -0.07327586206896551, -0.20689655172413793, -0.08620689655172414, -0.2801724137931034, 0.04741379310344827, -0.25862068965517243, 0.0603448275862069, -0.3922413793103448, 0.07327586206896551, -0.5086206896551724, -0.01293103448275862, -0.2672413793103448, -0.0, -0.3922413793103448, 0.008620689655172414, -0.49137931034482757, 0.008620689655172414, -0.1896551724137931], [-0.0, -0.0, -0.0205761316872428, -0.07407407407407407, 0.01646090534979424, -0.11522633744855967, 0.02880658436213992, -0.1934156378600823, 0.0411522633744856, -0.26337448559670784, -0.07407407407407407, -0.09465020576131687, -0.102880658436214, -0.1728395061728395, -0.11522633744855967, -0.24279835390946503, -0.00823045267489712, -0.24279835390946503, -0.01646090534979424, -0.3786008230452675, -0.024691358024691357, -0.4732510288065844, -0.06172839506172839, -0.23868312757201646, -0.06172839506172839, -0.37448559670781895, -0.09053497942386832, -0.48148148148148145, -0.03292181069958848, -0.1728395061728395], [-0.0, -0.0, -0.024271844660194174, -0.09223300970873786, 0.009708737864077669, -0.13592233009708737, 0.009708737864077669, -0.21844660194174756, 0.02912621359223301, -0.30097087378640774, -0.07281553398058252, -0.1262135922330097, -0.08737864077669903, -0.22330097087378642, -0.07766990291262135, -0.32038834951456313, 0.009708737864077669, -0.3106796116504854, 0.024271844660194174, -0.46116504854368934, 0.024271844660194174, -0.5776699029126213, -0.038834951456310676, -0.3155339805825243, -0.038834951456310676, -0.46601941747572817, -0.04854368932038835, -0.5970873786407767, -0.019417475728155338, -0.21844660194174756], [-0.0, -0.0, -0.014563106796116505, -0.09223300970873786, 0.019417475728155338, -0.13592233009708737, 0.02912621359223301, -0.22815533980582525, 0.043689320388349516, -0.3106796116504854, -0.06796116504854369, -0.1262135922330097, -0.07766990291262135, -0.22815533980582525, -0.07281553398058252, -0.32038834951456313, 0.014563106796116505, -0.3106796116504854, 0.02912621359223301, -0.4563106796116505, 0.024271844660194174, -0.5776699029126213, -0.02912621359223301, -0.3106796116504854, -0.02912621359223301, -0.46116504854368934, -0.04854368932038835, -0.5922330097087378, -0.014563106796116505, -0.21844660194174756], [-0.0, -0.0, -0.018518518518518517, -0.08333333333333333, 0.018518518518518517, -0.125, 0.023148148148148147, -0.20833333333333334, 0.05092592592592592, -0.28703703703703703, -0.06944444444444445, -0.11574074074074074, -0.08333333333333333, -0.21296296296296297, -0.06018518518518518, -0.3101851851851852, 0.018518518518518517, -0.2824074074074074, 0.023148148148148147, -0.42592592592592593, 0.009259259259259259, -0.5462962962962963, -0.027777777777777776, -0.28703703703703703, -0.032407407407407406, -0.4305555555555556, -0.06018518518518518, -0.5601851851851852, -0.018518518518518517, -0.19907407407407407], [-0.0, -0.0, -0.023255813953488372, -0.08837209302325581, 0.013953488372093023, -0.12558139534883722, 0.018604651162790697, -0.20930232558139536, 0.046511627906976744, -0.28837209302325584, -0.07441860465116279, -0.11627906976744186, -0.08837209302325581, -0.21395348837209302, -0.06976744186046512, -0.3116279069767442, 0.013953488372093023, -0.2837209302325581, 0.013953488372093023, -0.4325581395348837, -0.0, -0.5488372093023256, -0.03255813953488372, -0.28837209302325584, -0.04186046511627907, -0.4372093023255814, -0.06976744186046512, -0.5627906976744186, -0.018604651162790697, -0.2], [-0.0, -0.0, -0.023255813953488372, -0.08837209302325581, 0.013953488372093023, -0.12558139534883722, 0.018604651162790697, -0.20930232558139536, 0.046511627906976744, -0.28837209302325584, -0.06976744186046512, -0.11627906976744186, -0.08837209302325581, -0.21395348837209302, -0.07441860465116279, -0.30697674418604654, 0.013953488372093023, -0.28837209302325584, 0.013953488372093023, -0.4325581395348837, -0.0, -0.5488372093023256, -0.03255813953488372, -0.2930232558139535, -0.046511627906976744, -0.4325581395348837, -0.07441860465116279, -0.5627906976744186, -0.018604651162790697, -0.20465116279069767], [-0.0, -0.0, -0.023255813953488372, -0.08837209302325581, 0.013953488372093023, -0.12558139534883722, 0.018604651162790697, -0.21395348837209302, 0.046511627906976744, -0.2930232558139535, -0.07441860465116279, -0.12093023255813953, -0.08837209302325581, -0.21395348837209302, -0.07441860465116279, -0.3116279069767442, 0.013953488372093023, -0.28837209302325584, 0.013953488372093023, -0.4325581395348837, -0.004651162790697674, -0.5488372093023256, -0.03255813953488372, -0.2930232558139535, -0.046511627906976744, -0.4325581395348837, -0.07441860465116279, -0.5627906976744186, -0.018604651162790697, -0.20465116279069767], [-0.0, -0.0, -0.027906976744186046, -0.08837209302325581, 0.009302325581395349, -0.12558139534883722, 0.013953488372093023, -0.21395348837209302, 0.04186046511627907, -0.2930232558139535, -0.07441860465116279, -0.11627906976744186, -0.09302325581395349, -0.21395348837209302, -0.07906976744186046, -0.30697674418604654, 0.009302325581395349, -0.28837209302325584, 0.004651162790697674, -0.4372093023255814, -0.004651162790697674, -0.5488372093023256, -0.037209302325581395, -0.2930232558139535, -0.05116279069767442, -0.4325581395348837, -0.07906976744186046, -0.5627906976744186, -0.023255813953488372, -0.20465116279069767], [-0.0, -0.0, -0.027906976744186046, -0.08837209302325581, 0.009302325581395349, -0.13023255813953488, 0.013953488372093023, -0.20930232558139536, 0.037209302325581395, -0.28837209302325584, -0.07906976744186046, -0.12093023255813953, -0.09302325581395349, -0.21395348837209302, -0.07906976744186046, -0.30697674418604654, 0.009302325581395349, -0.28837209302325584, 0.009302325581395349, -0.4325581395348837, -0.009302325581395349, -0.5488372093023256, -0.037209302325581395, -0.2930232558139535, -0.05116279069767442, -0.4325581395348837, -0.07906976744186046, -0.5627906976744186, -0.023255813953488372, -0.20465116279069767], [-0.0, -0.0, -0.023255813953488372, -0.08837209302325581, 0.009302325581395349, -0.12558139534883722, 0.013953488372093023, -0.20930232558139536, 0.04186046511627907, -0.28837209302325584, -0.07441860465116279, -0.12093023255813953, -0.08837209302325581, -0.21395348837209302, -0.07441860465116279, -0.3116279069767442, 0.009302325581395349, -0.28837209302325584, 0.009302325581395349, -0.4325581395348837, -0.004651162790697674, -0.5488372093023256, -0.03255813953488372, -0.2930232558139535, -0.046511627906976744, -0.4325581395348837, -0.07441860465116279, -0.5581395348837209, -0.018604651162790697, -0.2], [-0.0, -0.0, -0.027906976744186046, -0.08837209302325581, 0.004651162790697674, -0.12558139534883722, 0.009302325581395349, -0.20930232558139536, 0.037209302325581395, -0.2837209302325581, -0.07441860465116279, -0.11627906976744186, -0.09302325581395349, -0.21395348837209302, -0.07906976744186046, -0.30697674418604654, 0.004651162790697674, -0.28837209302325584, 0.009302325581395349, -0.4325581395348837, -0.009302325581395349, -0.5488372093023256, -0.037209302325581395, -0.28837209302325584, -0.05116279069767442, -0.4325581395348837, -0.07906976744186046, -0.5627906976744186, -0.023255813953488372, -0.2], [-0.0, -0.0, -0.02926829268292683, -0.0975609756097561, 0.00975609756097561, -0.13658536585365855, 0.00975609756097561, -0.22926829268292684, 0.04390243902439024, -0.3170731707317073, -0.08292682926829269, -0.12682926829268293, -0.1024390243902439, -0.22926829268292684, -0.07804878048780488, -0.33170731707317075, 0.004878048780487805, -0.3024390243902439, 0.00975609756097561, -0.4585365853658537, -0.004878048780487805, -0.5804878048780487, -0.04390243902439024, -0.3073170731707317, -0.05365853658536585, -0.4585365853658537, -0.07804878048780488, -0.5951219512195122, -0.02926829268292683, -0.2146341463414634], [-0.0, -0.0, -0.02926829268292683, -0.0975609756097561, 0.00975609756097561, -0.14146341463414633, 0.00975609756097561, -0.23414634146341465, 0.04390243902439024, -0.32195121951219513, -0.08292682926829269, -0.12682926829268293, -0.1024390243902439, -0.22926829268292684, -0.08780487804878048, -0.33170731707317075, -0.0, -0.3073170731707317, 0.00975609756097561, -0.45365853658536587, -0.004878048780487805, -0.5804878048780487, -0.05365853658536585, -0.3073170731707317, -0.05365853658536585, -0.45365853658536587, -0.07804878048780488, -0.5902439024390244, -0.02926829268292683, -0.2146341463414634], [-0.0, -0.0, -0.023255813953488372, -0.08837209302325581, 0.013953488372093023, -0.13023255813953488, 0.018604651162790697, -0.21395348837209302, 0.04186046511627907, -0.28837209302325584, -0.07906976744186046, -0.11627906976744186, -0.09302325581395349, -0.21395348837209302, -0.08372093023255814, -0.3116279069767442, 0.004651162790697674, -0.2837209302325581, 0.009302325581395349, -0.4232558139534884, -0.004651162790697674, -0.5488372093023256, -0.046511627906976744, -0.2837209302325581, -0.046511627906976744, -0.42790697674418604, -0.07441860465116279, -0.5581395348837209, -0.027906976744186046, -0.2], [-0.0, -0.0, -0.02912621359223301, -0.09223300970873786, 0.014563106796116505, -0.13592233009708737, 0.024271844660194174, -0.22815533980582525, 0.04854368932038835, -0.3106796116504854, -0.0825242718446602, -0.1262135922330097, -0.0970873786407767, -0.22815533980582525, -0.0825242718446602, -0.33495145631067963, 0.009708737864077669, -0.30097087378640774, 0.014563106796116505, -0.45145631067961167, -0.0048543689320388345, -0.5776699029126213, -0.043689320388349516, -0.30097087378640774, -0.04854368932038835, -0.4563106796116505, -0.07766990291262135, -0.587378640776699, -0.02912621359223301, -0.21359223300970873], [-0.0, -0.0, -0.02912621359223301, -0.09223300970873786, 0.009708737864077669, -0.13592233009708737, 0.019417475728155338, -0.22815533980582525, 0.043689320388349516, -0.3058252427184466, -0.0825242718446602, -0.1262135922330097, -0.0970873786407767, -0.22815533980582525, -0.0825242718446602, -0.33495145631067963, 0.0048543689320388345, -0.30097087378640774, 0.014563106796116505, -0.45145631067961167, -0.0048543689320388345, -0.5776699029126213, -0.043689320388349516, -0.3058252427184466, -0.04854368932038835, -0.4563106796116505, -0.07766990291262135, -0.5922330097087378, -0.02912621359223301, -0.21359223300970873], [-0.0, -0.0, -0.027906976744186046, -0.08837209302325581, 0.009302325581395349, -0.13023255813953488, 0.013953488372093023, -0.21395348837209302, 0.037209302325581395, -0.28837209302325584, -0.08372093023255814, -0.11627906976744186, -0.09767441860465116, -0.21395348837209302, -0.08372093023255814, -0.3116279069767442, 0.004651162790697674, -0.2837209302325581, 0.004651162790697674, -0.4325581395348837, -0.009302325581395349, -0.5488372093023256, -0.046511627906976744, -0.2837209302325581, -0.05581395348837209, -0.4325581395348837, -0.07906976744186046, -0.5627906976744186, -0.03255813953488372, -0.2], [-0.0, -0.0, -0.023255813953488372, -0.08837209302325581, 0.018604651162790697, -0.13023255813953488, 0.03255813953488372, -0.21395348837209302, 0.05116279069767442, -0.2930232558139535, -0.07441860465116279, -0.12093023255813953, -0.09302325581395349, -0.2186046511627907, -0.07906976744186046, -0.3116279069767442, 0.013953488372093023, -0.28837209302325584, 0.013953488372093023, -0.4325581395348837, -0.004651162790697674, -0.5441860465116279, -0.037209302325581395, -0.28837209302325584, -0.046511627906976744, -0.4325581395348837, -0.07441860465116279, -0.5627906976744186, -0.018604651162790697, -0.2], [-0.0, -0.0, -0.022321428571428572, -0.08482142857142858, 0.022321428571428572, -0.125, 0.03571428571428571, -0.20535714285714285, 0.05803571428571429, -0.28125, -0.07142857142857142, -0.11160714285714286, -0.08482142857142858, -0.20535714285714285, -0.07589285714285714, -0.29910714285714285, 0.013392857142857142, -0.27232142857142855, 0.013392857142857142, -0.41517857142857145, -0.004464285714285714, -0.5223214285714286, -0.03571428571428571, -0.27232142857142855, -0.04017857142857143, -0.4107142857142857, -0.06696428571428571, -0.53125, -0.017857142857142856, -0.19196428571428573], [-0.0, -0.0, -0.022222222222222223, -0.08, 0.017777777777777778, -0.12, 0.03111111111111111, -0.20444444444444446, 0.044444444444444446, -0.27555555555555555, -0.07111111111111111, -0.1111111111111111, -0.08888888888888889, -0.20444444444444446, -0.08, -0.29333333333333333, 0.008888888888888889, -0.27111111111111114, 0.008888888888888889, -0.4088888888888889, -0.008888888888888889, -0.52, -0.04, -0.27111111111111114, -0.04888888888888889, -0.4088888888888889, -0.07555555555555556, -0.5333333333333333, -0.017777777777777778, -0.19111111111111112], [-0.0, -0.0, -0.022222222222222223, -0.08, 0.017777777777777778, -0.12, 0.02666666666666667, -0.2, 0.04, -0.27555555555555555, -0.07111111111111111, -0.1111111111111111, -0.08888888888888889, -0.20444444444444446, -0.08, -0.29777777777777775, 0.008888888888888889, -0.27111111111111114, 0.008888888888888889, -0.4088888888888889, -0.008888888888888889, -0.52, -0.04, -0.27111111111111114, -0.04888888888888889, -0.4088888888888889, -0.07555555555555556, -0.5288888888888889, -0.022222222222222223, -0.18666666666666668], [-0.0, -0.0, -0.027906976744186046, -0.08837209302325581, 0.009302325581395349, -0.13023255813953488, 0.018604651162790697, -0.21395348837209302, 0.04186046511627907, -0.2930232558139535, -0.08372093023255814, -0.11627906976744186, -0.09767441860465116, -0.21395348837209302, -0.08372093023255814, -0.31627906976744186, 0.004651162790697674, -0.2837209302325581, 0.004651162790697674, -0.4325581395348837, -0.009302325581395349, -0.5488372093023256, -0.046511627906976744, -0.2837209302325581, -0.05116279069767442, -0.4325581395348837, -0.07906976744186046, -0.5627906976744186, -0.027906976744186046, -0.2], [-0.0, -0.0, -0.02666666666666667, -0.08, 0.013333333333333334, -0.12, 0.022222222222222223, -0.20444444444444446, 0.04, -0.28, -0.07555555555555556, -0.1111111111111111, -0.09333333333333334, -0.20444444444444446, -0.08, -0.29777777777777775, 0.0044444444444444444, -0.26666666666666666, 0.0044444444444444444, -0.4088888888888889, -0.008888888888888889, -0.52, -0.044444444444444446, -0.26666666666666666, -0.04888888888888889, -0.40444444444444444, -0.07555555555555556, -0.5333333333333333, -0.02666666666666667, -0.18666666666666668], [-0.0, -0.0, -0.022222222222222223, -0.08, 0.017777777777777778, -0.12, 0.02666666666666667, -0.20444444444444446, 0.04, -0.27555555555555555, -0.07555555555555556, -0.1111111111111111, -0.08888888888888889, -0.20444444444444446, -0.08, -0.29777777777777775, 0.008888888888888889, -0.27111111111111114, 0.008888888888888889, -0.4088888888888889, -0.008888888888888889, -0.52, -0.04, -0.26666666666666666, -0.04888888888888889, -0.4088888888888889, -0.07555555555555556, -0.5333333333333333, -0.022222222222222223, -0.18666666666666668], [-0.0, -0.0, -0.022222222222222223, -0.08, 0.017777777777777778, -0.12, 0.02666666666666667, -0.2, 0.04, -0.27111111111111114, -0.07111111111111111, -0.10666666666666667, -0.08888888888888889, -0.2, -0.07555555555555556, -0.29333333333333333, 0.008888888888888889, -0.27111111111111114, 0.008888888888888889, -0.41333333333333333, -0.008888888888888889, -0.52, -0.035555555555555556, -0.27111111111111114, -0.04888888888888889, -0.4088888888888889, -0.07555555555555556, -0.5333333333333333, -0.017777777777777778, -0.18666666666666668], [-0.0, -0.0, -0.022222222222222223, -0.08, 0.017777777777777778, -0.12, 0.02666666666666667, -0.2, 0.04, -0.27111111111111114, -0.07111111111111111, -0.10666666666666667, -0.08888888888888889, -0.2, -0.07555555555555556, -0.29333333333333333, 0.008888888888888889, -0.27111111111111114, 0.008888888888888889, -0.4088888888888889, -0.008888888888888889, -0.52, -0.035555555555555556, -0.27111111111111114, -0.04888888888888889, -0.4088888888888889, -0.07555555555555556, -0.5333333333333333, -0.017777777777777778, -0.18666666666666668], [-0.0, -0.0, -0.017777777777777778, -0.08, 0.017777777777777778, -0.12, 0.02666666666666667, -0.2, 0.04, -0.27111111111111114, -0.07111111111111111, -0.10666666666666667, -0.08888888888888889, -0.2, -0.07555555555555556, -0.29333333333333333, 0.008888888888888889, -0.27111111111111114, 0.008888888888888889, -0.4088888888888889, -0.008888888888888889, -0.52, -0.035555555555555556, -0.27111111111111114, -0.04888888888888889, -0.4088888888888889, -0.07555555555555556, -0.5333333333333333, -0.017777777777777778, -0.18666666666666668], [-0.0, -0.0, -0.023255813953488372, -0.08837209302325581, 0.013953488372093023, -0.13023255813953488, 0.023255813953488372, -0.21395348837209302, 0.03255813953488372, -0.28837209302325584, -0.07441860465116279, -0.12093023255813953, -0.09767441860465116, -0.21395348837209302, -0.08372093023255814, -0.3116279069767442, 0.004651162790697674, -0.28837209302325584, 0.004651162790697674, -0.4325581395348837, -0.013953488372093023, -0.5488372093023256, -0.04186046511627907, -0.28837209302325584, -0.05581395348837209, -0.4325581395348837, -0.08372093023255814, -0.5627906976744186, -0.023255813953488372, -0.20465116279069767], [-0.0, -0.0, -0.022222222222222223, -0.08, 0.013333333333333334, -0.12, 0.022222222222222223, -0.2, 0.035555555555555556, -0.26666666666666666, -0.07555555555555556, -0.10666666666666667, -0.09333333333333334, -0.2, -0.08, -0.28888888888888886, 0.0044444444444444444, -0.27111111111111114, 0.0044444444444444444, -0.4088888888888889, -0.013333333333333334, -0.52, -0.04, -0.27111111111111114, -0.05333333333333334, -0.4088888888888889, -0.08, -0.5333333333333333, -0.022222222222222223, -0.19111111111111112],[-0.0, -0.0, -0.0, -0.07569721115537849, 0.04780876494023904, -0.08764940239043825, 0.10756972111553785, -0.099601593625498, 0.11952191235059761, -0.0398406374501992, -0.04780876494023904, -0.10756972111553785, -0.05976095617529881, -0.17928286852589642, -0.05976095617529881, -0.2549800796812749, 0.035856573705179286, -0.2350597609561753, 0.05179282868525897, -0.3466135458167331, 0.07569721115537849, -0.44621513944223107, -0.0199203187250996, -0.2350597609561753, -0.01195219123505976, -0.3466135458167331, -0.00796812749003984, -0.4342629482071713, 0.00796812749003984, -0.16733067729083664], [-0.0, -0.0, -0.01293103448275862, -0.08620689655172414, 0.034482758620689655, -0.10344827586206896, 0.09913793103448276, -0.11206896551724138, 0.1206896551724138, -0.03879310344827586, -0.06896551724137931, -0.11206896551724138, -0.08620689655172414, -0.1939655172413793, -0.09482758620689655, -0.27155172413793105, 0.017241379310344827, -0.25862068965517243, 0.021551724137931036, -0.3922413793103448, 0.021551724137931036, -0.5086206896551724, -0.04310344827586207, -0.2629310344827586, -0.04310344827586207, -0.3879310344827586, -0.03879310344827586, -0.4827586206896552, -0.01293103448275862, -0.1853448275862069], [-0.0, -0.0, -0.010256410256410256, -0.1076923076923077, 0.06153846153846154, -0.13846153846153847, 0.13333333333333333, -0.14358974358974358, 0.21025641025641026, -0.06666666666666667, -0.07692307692307693, -0.14358974358974358, -0.09743589743589744, -0.2512820512820513, -0.09743589743589744, -0.35384615384615387, 0.02564102564102564, -0.3282051282051282, 0.005128205128205128, -0.49743589743589745, -0.020512820512820513, -0.6256410256410256, -0.046153846153846156, -0.3282051282051282, -0.041025641025641026, -0.5025641025641026, -0.06153846153846154, -0.6564102564102564, -0.005128205128205128, -0.2358974358974359], [-0.0, -0.0, -0.03571428571428571, -0.20535714285714285, 0.08928571428571429, -0.25, 0.24107142857142858, -0.26785714285714285, 0.29464285714285715, -0.11607142857142858, -0.16071428571428573, -0.2767857142857143, -0.19642857142857142, -0.5, -0.16964285714285715, -0.6964285714285714, 0.07142857142857142, -0.625, 0.15178571428571427, -0.9375, 0.20535714285714285, -1.2142857142857142, -0.07142857142857142, -0.6339285714285714, -0.07142857142857142, -0.9642857142857143, -0.0625, -1.25, -0.008928571428571428, -0.44642857142857145], [-0.0, -0.0, -0.020134228187919462, -0.1476510067114094, 0.0738255033557047, -0.18120805369127516, 0.174496644295302, -0.2214765100671141, 0.28187919463087246, -0.10067114093959731, -0.12751677852348994, -0.19463087248322147, -0.1610738255033557, -0.348993288590604, -0.16778523489932887, -0.48322147651006714, 0.03355704697986577, -0.46308724832214765, -0.006711409395973154, -0.6845637583892618, -0.020134228187919462, -0.8791946308724832, -0.08053691275167785, -0.46308724832214765, -0.10067114093959731, -0.6711409395973155, -0.08053691275167785, -0.8590604026845637, -0.020134228187919462, -0.3288590604026846], [-0.0, -0.0, 0.013452914798206279, -0.08968609865470852, 0.06726457399103139, -0.09865470852017937, 0.14349775784753363, -0.11210762331838565, 0.17040358744394618, -0.03587443946188341, -0.04035874439461883, -0.13452914798206278, -0.04035874439461883, -0.2242152466367713, -0.03587443946188341, -0.3094170403587444, 0.08520179372197309, -0.26905829596412556, 0.1031390134529148, -0.4125560538116592, 0.11659192825112108, -0.5246636771300448, 0.017937219730941704, -0.2825112107623318, 0.013452914798206279, -0.4170403587443946, 0.008968609865470852, -0.5291479820627802, 0.03587443946188341, -0.19730941704035873], [-0.0, -0.0, -0.008658008658008658, -0.09090909090909091, 0.03896103896103896, -0.10822510822510822, 0.09523809523809523, -0.12987012987012986, 0.12554112554112554, -0.06926406926406926, -0.05627705627705628, -0.11255411255411256, -0.07792207792207792, -0.19480519480519481, -0.0735930735930736, -0.27705627705627706, 0.017316017316017316, -0.2597402597402597, -0.008658008658008658, -0.38961038961038963, -0.021645021645021644, -0.5064935064935064, -0.03463203463203463, -0.2597402597402597, -0.030303030303030304, -0.38961038961038963, -0.030303030303030304, -0.5064935064935064, -0.008658008658008658, -0.18614718614718614], [-0.0, -0.0, 0.011299435028248588, -0.11864406779661017, 0.0903954802259887, -0.14689265536723164, 0.1864406779661017, -0.1694915254237288, 0.22598870056497175, -0.07344632768361582, -0.062146892655367235, -0.1694915254237288, -0.0847457627118644, -0.2994350282485876, -0.07344632768361582, -0.4067796610169492, 0.07909604519774012, -0.3728813559322034, 0.1016949152542373, -0.5649717514124294, 0.11864406779661017, -0.7231638418079096, -0.011299435028248588, -0.384180790960452, -0.011299435028248588, -0.5649717514124294, -0.011299435028248588, -0.7175141242937854, 0.022598870056497175, -0.2655367231638418], [-0.0, -0.0, 0.0106951871657754, -0.10695187165775401, 0.08021390374331551, -0.12834224598930483, 0.1711229946524064, -0.1497326203208556, 0.1925133689839572, -0.0427807486631016, -0.06417112299465241, -0.15508021390374332, -0.06951871657754011, -0.2727272727272727, -0.06951871657754011, -0.3582887700534759, 0.06417112299465241, -0.34759358288770054, 0.058823529411764705, -0.5294117647058824, 0.0481283422459893, -0.679144385026738, -0.0213903743315508, -0.34759358288770054, -0.0481283422459893, -0.5133689839572193, -0.06951871657754011, -0.6470588235294118, 0.0213903743315508, -0.24598930481283424], [-0.0, -0.0, 0.018691588785046728, -0.09345794392523364, 0.07009345794392523, -0.11214953271028037, 0.16355140186915887, -0.10747663551401869, 0.1542056074766355, -0.014018691588785047, -0.028037383177570093, -0.14485981308411214, -0.028037383177570093, -0.24299065420560748, -0.028037383177570093, -0.3317757009345794, 0.06074766355140187, -0.2850467289719626, 0.028037383177570093, -0.4205607476635514, 0.02336448598130841, -0.5654205607476636, 0.014018691588785047, -0.2897196261682243, 0.02336448598130841, -0.4205607476635514, 0.02336448598130841, -0.5560747663551402, 0.03271028037383177, -0.205607476635514], [-0.0, -0.0, 0.00975609756097561, -0.0975609756097561, 0.07317073170731707, -0.11707317073170732, 0.16097560975609757, -0.1024390243902439, 0.17073170731707318, -0.004878048780487805, -0.04878048780487805, -0.14634146341463414, -0.06341463414634146, -0.24390243902439024, -0.07317073170731707, -0.32195121951219513, 0.05853658536585366, -0.3073170731707317, 0.04878048780487805, -0.4682926829268293, 0.04390243902439024, -0.6097560975609756, -0.01951219512195122, -0.3121951219512195, -0.04878048780487805, -0.4634146341463415, -0.07804878048780488, -0.5707317073170731, 0.024390243902439025, -0.21951219512195122], [-0.0, -0.0, 0.00398406374501992, -0.07569721115537849, 0.04780876494023904, -0.08764940239043825, 0.10358565737051793, -0.08764940239043825, 0.13147410358565736, -0.0199203187250996, -0.0398406374501992, -0.11155378486055777, -0.04780876494023904, -0.18725099601593626, -0.04780876494023904, -0.2589641434262948, 0.043824701195219126, -0.24302788844621515, 0.0398406374501992, -0.3545816733067729, 0.02390438247011952, -0.43824701195219123, -0.00796812749003984, -0.24302788844621515, -0.0, -0.3545816733067729, 0.01195219123505976, -0.43824701195219123, 0.01593625498007968, -0.17131474103585656], [-0.0, -0.0, -0.006493506493506494, -0.05194805194805195, 0.025974025974025976, -0.06493506493506493, 0.05844155844155844, -0.07142857142857142, 0.08116883116883117, -0.025974025974025976, -0.045454545454545456, -0.07467532467532467, -0.05519480519480519, -0.12987012987012986, -0.048701298701298704, -0.18831168831168832, 0.012987012987012988, -0.17207792207792208, 0.012987012987012988, -0.2564935064935065, 0.012987012987012988, -0.32792207792207795, -0.025974025974025976, -0.17207792207792208, -0.04220779220779221, -0.25, -0.06818181818181818, -0.32142857142857145, -0.003246753246753247, -0.12337662337662338], [-0.0, -0.0, -0.0, -0.052307692307692305, 0.024615384615384615, -0.06153846153846154, 0.055384615384615386, -0.06769230769230769, 0.08307692307692308, -0.024615384615384615, -0.03076923076923077, -0.06769230769230769, -0.043076923076923075, -0.11692307692307692, -0.036923076923076927, -0.16615384615384615, 0.012307692307692308, -0.15384615384615385, 0.015384615384615385, -0.2246153846153846, 0.009230769230769232, -0.28923076923076924, -0.018461538461538463, -0.15692307692307692, -0.012307692307692308, -0.2276923076923077, -0.009230769230769232, -0.28923076923076924, -0.003076923076923077, -0.11076923076923077], [-0.0, -0.0, -0.00847457627118644, -0.0423728813559322, 0.014124293785310734, -0.05367231638418079, 0.03954802259887006, -0.062146892655367235, 0.06497175141242938, -0.02824858757062147, -0.03672316384180791, -0.059322033898305086, -0.05084745762711865, -0.1016949152542373, -0.05084745762711865, -0.1384180790960452, -0.0, -0.13559322033898305, -0.005649717514124294, -0.1977401129943503, -0.00847457627118644, -0.2570621468926554, -0.031073446327683617, -0.13559322033898305, -0.03954802259887006, -0.19491525423728814, -0.05084745762711865, -0.2542372881355932, -0.011299435028248588, -0.096045197740113], [-0.0, -0.0, -0.0027624309392265192, -0.04419889502762431, 0.022099447513812154, -0.049723756906077346, 0.04419889502762431, -0.06353591160220995, 0.055248618784530384, -0.027624309392265192, -0.03038674033149171, -0.058011049723756904, -0.04143646408839779, -0.09944751381215469, -0.04143646408839779, -0.1464088397790055, 0.008287292817679558, -0.13259668508287292, 0.0027624309392265192, -0.19337016574585636, -0.0055248618784530384, -0.24861878453038674, -0.022099447513812154, -0.13259668508287292, -0.027624309392265192, -0.19337016574585636, -0.04419889502762431, -0.24585635359116023, -0.0055248618784530384, -0.09392265193370165], [-0.0, -0.0, 0.0029940119760479044, -0.05089820359281437, 0.029940119760479042, -0.05688622754491018, 0.059880239520958084, -0.0658682634730539, 0.08083832335329341, -0.023952095808383235, -0.029940119760479042, -0.0718562874251497, -0.03592814371257485, -0.11676646706586827, -0.029940119760479042, -0.16467065868263472, 0.023952095808383235, -0.15269461077844312, 0.038922155688622756, -0.2215568862275449, 0.05089820359281437, -0.281437125748503, -0.008982035928143712, -0.15568862275449102, -0.005988023952095809, -0.2245508982035928, -0.0, -0.2874251497005988, 0.0029940119760479044, -0.10778443113772455], [-0.0, -0.0, -0.0029069767441860465, -0.046511627906976744, 0.02616279069767442, -0.055232558139534885, 0.061046511627906974, -0.061046511627906974, 0.07848837209302326, -0.01744186046511628, -0.03197674418604651, -0.06395348837209303, -0.040697674418604654, -0.10755813953488372, -0.0377906976744186, -0.15988372093023256, 0.00872093023255814, -0.14534883720930233, -0.0029069767441860465, -0.21511627906976744, -0.0029069767441860465, -0.2819767441860465, -0.01744186046511628, -0.14534883720930233, -0.011627906976744186, -0.21511627906976744, -0.00872093023255814, -0.27906976744186046, -0.0029069767441860465, -0.10174418604651163], [-0.0, -0.0, 0.0031645569620253164, -0.05379746835443038, 0.0379746835443038, -0.060126582278481014, 0.08227848101265822, -0.056962025316455694, 0.07278481012658228, -0.00949367088607595, -0.03164556962025317, -0.0759493670886076, -0.0379746835443038, -0.12658227848101267, -0.03481012658227848, -0.18037974683544303, 0.03164556962025317, -0.17088607594936708, 0.04430379746835443, -0.25316455696202533, 0.060126582278481014, -0.3259493670886076, -0.00949367088607595, -0.17088607594936708, -0.006329113924050633, -0.25316455696202533, -0.0031645569620253164, -0.31962025316455694, 0.00949367088607595, -0.12025316455696203], [-0.0, -0.0, 0.0035971223021582736, -0.0683453237410072, 0.039568345323741004, -0.08273381294964029, 0.08633093525179857, -0.08992805755395683, 0.1223021582733813, -0.039568345323741004, -0.03237410071942446, -0.08992805755395683, -0.046762589928057555, -0.1474820143884892, -0.050359712230215826, -0.21223021582733814, 0.025179856115107913, -0.20503597122302158, 0.01079136690647482, -0.302158273381295, 0.0035971223021582736, -0.38489208633093525, -0.017985611510791366, -0.2014388489208633, -0.02877697841726619, -0.2949640287769784, -0.039568345323741004, -0.38489208633093525, 0.007194244604316547, -0.14388489208633093], [-0.0, -0.0, 0.008620689655172414, -0.08620689655172414, 0.0603448275862069, -0.10344827586206896, 0.125, -0.09051724137931035, 0.16379310344827586, -0.021551724137931036, -0.04310344827586207, -0.125, -0.05172413793103448, -0.20689655172413793, -0.05172413793103448, -0.28448275862068967, 0.05603448275862069, -0.2629310344827586, 0.0603448275862069, -0.3879310344827586, 0.06896551724137931, -0.5043103448275862, -0.0, -0.2672413793103448, -0.0, -0.3922413793103448, 0.008620689655172414, -0.5, 0.021551724137931036, -0.1896551724137931], [-0.0, -0.0, 0.004672897196261682, -0.09345794392523364, 0.06542056074766354, -0.11214953271028037, 0.14018691588785046, -0.09813084112149532, 0.16822429906542055, -0.014018691588785047, -0.056074766355140186, -0.13551401869158877, -0.06542056074766354, -0.22897196261682243, -0.06542056074766354, -0.308411214953271, 0.056074766355140186, -0.29906542056074764, 0.028037383177570093, -0.4485981308411215, 0.014018691588785047, -0.5654205607476636, -0.014018691588785047, -0.29906542056074764, -0.028037383177570093, -0.4392523364485981, -0.03271028037383177, -0.5654205607476636, 0.018691588785046728, -0.2102803738317757], [-0.0, -0.0, -0.0, -0.11290322580645161, 0.06989247311827956, -0.13440860215053763, 0.15591397849462366, -0.12365591397849462, 0.1881720430107527, -0.03225806451612903, -0.07526881720430108, -0.16129032258064516, -0.08602150537634409, -0.27419354838709675, -0.0913978494623656, -0.3655913978494624, 0.04838709677419355, -0.3548387096774194, 0.021505376344086023, -0.5268817204301075, 0.005376344086021506, -0.6720430107526881, -0.03225806451612903, -0.3548387096774194, -0.026881720430107527, -0.5268817204301075, -0.043010752688172046, -0.6774193548387096, 0.010752688172043012, -0.25268817204301075], [-0.0, -0.0, 0.013513513513513514, -0.0945945945945946, 0.06756756756756757, -0.11261261261261261, 0.13963963963963963, -0.0945945945945946, 0.16216216216216217, -0.009009009009009009, -0.04054054054054054, -0.13963963963963963, -0.04954954954954955, -0.22522522522522523, -0.04504504504504504, -0.3063063063063063, 0.05855855855855856, -0.28378378378378377, 0.08108108108108109, -0.42342342342342343, 0.1036036036036036, -0.5405405405405406, -0.0, -0.2882882882882883, -0.0, -0.4189189189189189, 0.009009009009009009, -0.5180180180180181, 0.02702702702702703, -0.2072072072072072], [-0.0, -0.0, 0.006920415224913495, -0.058823529411764705, 0.04844290657439446, -0.06920415224913495, 0.09688581314878893, -0.0657439446366782, 0.10726643598615918, -0.006920415224913495, -0.02768166089965398, -0.08996539792387544, -0.03806228373702422, -0.14878892733564014, -0.03460207612456748, -0.2041522491349481, 0.03460207612456748, -0.1903114186851211, 0.02768166089965398, -0.28027681660899656, 0.01384083044982699, -0.36678200692041524, -0.010380622837370242, -0.1903114186851211, -0.01384083044982699, -0.2837370242214533, -0.01384083044982699, -0.36678200692041524, 0.01384083044982699, -0.13494809688581316], [-0.0, -0.0, 0.011583011583011582, -0.07722007722007722, 0.05791505791505792, -0.08108108108108109, 0.11196911196911197, -0.06563706563706563, 0.11196911196911197, 0.003861003861003861, -0.02702702702702703, -0.11196911196911197, -0.04247104247104247, -0.18146718146718147, -0.04247104247104247, -0.24324324324324326, 0.04247104247104247, -0.2277992277992278, 0.05019305019305019, -0.33204633204633205, 0.05791505791505792, -0.42084942084942084, -0.007722007722007722, -0.2277992277992278, -0.015444015444015444, -0.33204633204633205, -0.015444015444015444, -0.42084942084942084, 0.019305019305019305, -0.16216216216216217], [-0.0, -0.0, 0.003968253968253968, -0.07142857142857142, 0.051587301587301584, -0.07936507936507936, 0.11904761904761904, -0.06349206349206349, 0.11904761904761904, 0.007936507936507936, -0.03968253968253968, -0.10714285714285714, -0.011904761904761904, -0.0, -0.05555555555555555, -0.23809523809523808, 0.03968253968253968, -0.2261904761904762, 0.015873015873015872, -0.35714285714285715, 0.011904761904761904, -0.4642857142857143, -0.015873015873015872, -0.2261904761904762, 0.003968253968253968, -0.3492063492063492, 0.015873015873015872, -0.44841269841269843, 0.011904761904761904, -0.15873015873015872], [-0.0, -0.0, 0.005128205128205128, -0.1076923076923077, 0.07179487179487179, -0.1282051282051282, 0.1641025641025641, -0.09230769230769231, 0.1794871794871795, 0.005128205128205128, -0.06153846153846154, -0.15897435897435896, -0.07692307692307693, -0.26666666666666666, -0.08717948717948718, -0.358974358974359, 0.05128205128205128, -0.3282051282051282, 0.05128205128205128, -0.5025641025641026, 0.05128205128205128, -0.6358974358974359, -0.03076923076923077, -0.3333333333333333, -0.05128205128205128, -0.49743589743589745, -0.07692307692307693, -0.6358974358974359, 0.015384615384615385, -0.2358974358974359], [-0.0, -0.0, 0.011235955056179775, -0.11235955056179775, 0.0898876404494382, -0.1348314606741573, 0.19662921348314608, -0.11235955056179775, 0.2247191011235955, -0.011235955056179775, -0.06179775280898876, -0.16853932584269662, -0.07865168539325842, -0.29213483146067415, -0.08426966292134831, -0.39325842696629215, 0.06179775280898876, -0.3539325842696629, 0.06179775280898876, -0.5561797752808989, 0.06741573033707865, -0.7359550561797753, -0.016853932584269662, -0.3651685393258427, -0.011235955056179775, -0.5561797752808989, 0.0056179775280898875, -0.7078651685393258, 0.02247191011235955, -0.25842696629213485], [-0.0, -0.0, -0.004694835680751174, -0.09859154929577464, 0.051643192488262914, -0.11267605633802817, 0.1267605633802817, -0.1267605633802817, 0.1596244131455399, -0.051643192488262914, -0.06572769953051644, -0.13145539906103287, -0.07981220657276995, -0.22065727699530516, -0.08450704225352113, -0.30985915492957744, 0.03286384976525822, -0.29577464788732394, 0.03755868544600939, -0.4413145539906103, 0.056338028169014086, -0.5774647887323944, -0.03286384976525822, -0.3004694835680751, -0.023474178403755867, -0.43661971830985913, -0.014084507042253521, -0.5492957746478874, -0.0, -0.2112676056338028], [-0.0, -0.0, 0.004, -0.08, 0.048, -0.092, 0.108, -0.1, 0.132, -0.044, -0.048, -0.108, -0.056, -0.18, -0.056, -0.248, 0.032, -0.24, 0.036, -0.348, 0.044, -0.432, -0.02, -0.244, -0.016, -0.348, -0.016, -0.444, 0.008, -0.172], [-0.0, -0.0, -0.006944444444444444, -0.0625, 0.03125, -0.0763888888888889, 0.0798611111111111, -0.09027777777777778, 0.10416666666666667, -0.041666666666666664, -0.04513888888888889, -0.08680555555555555, -0.05555555555555555, -0.14583333333333334, -0.05555555555555555, -0.21180555555555555, 0.017361111111111112, -0.1875, -0.0, -0.2777777777777778, -0.013888888888888888, -0.3576388888888889, -0.024305555555555556, -0.1909722222222222, -0.020833333333333332, -0.2777777777777778, -0.03125, -0.3576388888888889, -0.003472222222222222, -0.13541666666666666], [-0.0, -0.0, 0.007407407407407408, -0.06666666666666667, 0.04814814814814815, -0.08148148148148149, 0.1, -0.08518518518518518, 0.11851851851851852, -0.022222222222222223, -0.037037037037037035, -0.1, -0.040740740740740744, -0.16666666666666666, -0.037037037037037035, -0.23333333333333334, 0.044444444444444446, -0.21851851851851853, 0.06296296296296296, -0.3148148148148148, 0.08148148148148149, -0.4, -0.0, -0.2222222222222222, -0.0, -0.31851851851851853, 0.003703703703703704, -0.40370370370370373, 0.018518518518518517, -0.15555555555555556], [-0.0, -0.0, -0.0, -0.0683453237410072, 0.039568345323741004, -0.07913669064748201, 0.09352517985611511, -0.07913669064748201, 0.11151079136690648, -0.025179856115107913, -0.03597122302158273, -0.09712230215827339, -0.046762589928057555, -0.15467625899280577, -0.050359712230215826, -0.21223021582733814, 0.03237410071942446, -0.20503597122302158, 0.017985611510791366, -0.3057553956834532, 0.01079136690647482, -0.38489208633093525, -0.014388489208633094, -0.20863309352517986, -0.025179856115107913, -0.29856115107913667, -0.04316546762589928, -0.381294964028777, 0.007194244604316547, -0.1474820143884892], [-0.0, -0.0, 0.004149377593360996, -0.08298755186721991, 0.05394190871369295, -0.0954356846473029, 0.1078838174273859, -0.1037344398340249, 0.13278008298755187, -0.03734439834024896, -0.04149377593360996, -0.11618257261410789, -0.05394190871369295, -0.1950207468879668, -0.04564315352697095, -0.2697095435684647, 0.05394190871369295, -0.2572614107883817, 0.07053941908713693, -0.3775933609958506, 0.08298755186721991, -0.4730290456431535, -0.004149377593360996, -0.26141078838174275, -0.0, -0.37344398340248963, 0.012448132780082987, -0.46887966804979253, 0.02074688796680498, -0.1825726141078838], [-0.0, -0.0, 0.004149377593360996, -0.08298755186721991, 0.05394190871369295, -0.0954356846473029, 0.1078838174273859, -0.1078838174273859, 0.13692946058091288, -0.04149377593360996, -0.04564315352697095, -0.11618257261410789, -0.04979253112033195, -0.1950207468879668, -0.05394190871369295, -0.2697095435684647, 0.04979253112033195, -0.25311203319502074, 0.04149377593360996, -0.37344398340248963, 0.03319502074688797, -0.48132780082987553, -0.008298755186721992, -0.25311203319502074, -0.029045643153526972, -0.37344398340248963, -0.05394190871369295, -0.4730290456431535, 0.016597510373443983, -0.1825726141078838], [-0.0, -0.0, -0.0, -0.07569721115537849, 0.043824701195219126, -0.08764940239043825, 0.09163346613545817, -0.11155378486055777, 0.11952191235059761, -0.05179282868525897, -0.04780876494023904, -0.10756972111553785, -0.05179282868525897, -0.17928286852589642, -0.04780876494023904, -0.250996015936255, 0.035856573705179286, -0.2350597609561753, 0.027888446215139442, -0.35856573705179284, 0.02390438247011952, -0.4581673306772908, -0.0199203187250996, -0.2350597609561753, -0.035856573705179286, -0.3545816733067729, -0.043824701195219126, -0.450199203187251, 0.00796812749003984, -0.16733067729083664], [-0.0, -0.0, 0.0032679738562091504, -0.058823529411764705, 0.03594771241830065, -0.06862745098039216, 0.0784313725490196, -0.07516339869281045, 0.09803921568627451, -0.026143790849673203, -0.029411764705882353, -0.0784313725490196, -0.0392156862745098, -0.13071895424836602, -0.03594771241830065, -0.1895424836601307, 0.032679738562091505, -0.17320261437908496, 0.029411764705882353, -0.26143790849673204, 0.026143790849673203, -0.3300653594771242, -0.006535947712418301, -0.17647058823529413, -0.0, -0.2647058823529412, -0.0, -0.3366013071895425, 0.00980392156862745, -0.12418300653594772], [-0.0, -0.0, -0.005813953488372093, -0.046511627906976744, 0.02616279069767442, -0.055232558139534885, 0.061046511627906974, -0.061046511627906974, 0.0755813953488372, -0.023255813953488372, -0.03197674418604651, -0.06395348837209303, -0.040697674418604654, -0.10465116279069768, -0.0436046511627907, -0.1511627906976744, 0.014534883720930232, -0.14825581395348839, 0.00872093023255814, -0.2180232558139535, 0.005813953488372093, -0.2761627906976744, -0.020348837209302327, -0.14534883720930233, -0.03488372093023256, -0.21220930232558138, -0.0436046511627907, -0.26744186046511625, -0.0029069767441860465, -0.10465116279069768], [-0.0, -0.0, 0.0029069767441860465, -0.046511627906976744, 0.03197674418604651, -0.05232558139534884, 0.06976744186046512, -0.05813953488372093, 0.0755813953488372, -0.011627906976744186, -0.020348837209302327, -0.06395348837209303, -0.029069767441860465, -0.10755813953488372, -0.02616279069767442, -0.15406976744186046, 0.02616279069767442, -0.13953488372093023, 0.020348837209302327, -0.20930232558139536, 0.01744186046511628, -0.27906976744186046, -0.0, -0.14244186046511628, 0.0029069767441860465, -0.21220930232558138, 0.011627906976744186, -0.27325581395348836, 0.00872093023255814, -0.10174418604651163], [-0.0, -0.0, -0.0, -0.050473186119873815, 0.03470031545741325, -0.05993690851735016, 0.07886435331230283, -0.0694006309148265, 0.10094637223974763, -0.01892744479495268, -0.031545741324921134, -0.07570977917981073, -0.04100946372239748, -0.12933753943217666, -0.0473186119873817, -0.17981072555205047, 0.022082018927444796, -0.17350157728706625, 0.01892744479495268, -0.2555205047318612, 0.015772870662460567, -0.3217665615141956, -0.01892744479495268, -0.17034700315457413, -0.03785488958990536, -0.24921135646687698, -0.05993690851735016, -0.31545741324921134, 0.0031545741324921135, -0.11987381703470032], [-0.0, -0.0, -0.003472222222222222, -0.0625, 0.034722222222222224, -0.0763888888888889, 0.08680555555555555, -0.0798611111111111, 0.11458333333333333, -0.024305555555555556, -0.041666666666666664, -0.09027777777777778, -0.04861111111111111, -0.14930555555555555, -0.04861111111111111, -0.20833333333333334, 0.027777777777777776, -0.1875, 0.03819444444444445, -0.28125, 0.059027777777777776, -0.3680555555555556, -0.013888888888888888, -0.19444444444444445, -0.006944444444444444, -0.28125, 0.003472222222222222, -0.3576388888888889, 0.003472222222222222, -0.1388888888888889], [-0.0, -0.0, -0.00398406374501992, -0.07569721115537849, 0.0398406374501992, -0.08764940239043825, 0.09163346613545817, -0.10756972111553785, 0.13147410358565736, -0.055776892430278883, -0.055776892430278883, -0.10358565737051793, -0.06374501992031872, -0.16733067729083664, -0.06374501992031872, -0.23904382470119523, 0.0199203187250996, -0.2350597609561753, 0.00796812749003984, -0.3466135458167331, 0.00796812749003984, -0.450199203187251, -0.03187250996015936, -0.2350597609561753, -0.035856573705179286, -0.3426294820717131, -0.027888446215139442, -0.4302788844621514, -0.0, -0.16733067729083664], [-0.0, -0.0, -0.004878048780487805, -0.0975609756097561, 0.04878048780487805, -0.11707317073170732, 0.12195121951219512, -0.14146341463414633, 0.18048780487804877, -0.06341463414634146, -0.06829268292682927, -0.13170731707317074, -0.08780487804878048, -0.22926829268292684, -0.08292682926829269, -0.32682926829268294, 0.03414634146341464, -0.2975609756097561, 0.03414634146341464, -0.45365853658536587, 0.01951219512195122, -0.5804878048780487, -0.02926829268292683, -0.3024390243902439, -0.024390243902439025, -0.4585365853658537, -0.01951219512195122, -0.5951219512195122, -0.0, -0.2146341463414634], [-0.0, -0.0, -0.016129032258064516, -0.11290322580645161, 0.053763440860215055, -0.13440860215053763, 0.13978494623655913, -0.16129032258064516, 0.1881720430107527, -0.07526881720430108, -0.08602150537634409, -0.15591397849462366, -0.10215053763440861, -0.26881720430107525, -0.0967741935483871, -0.3763440860215054, 0.04838709677419355, -0.3387096774193548, 0.07526881720430108, -0.521505376344086, 0.08064516129032258, -0.6666666666666666, -0.03763440860215054, -0.34946236559139787, -0.03225806451612903, -0.5268817204301075, -0.021505376344086023, -0.6774193548387096, -0.0, -0.24193548387096775], [-0.0, -0.0, -0.013452914798206279, -0.08968609865470852, 0.04484304932735426, -0.10762331838565023, 0.11659192825112108, -0.1210762331838565, 0.14349775784753363, -0.053811659192825115, -0.07174887892376682, -0.11659192825112108, -0.09417040358744394, -0.2062780269058296, -0.09417040358744394, -0.28699551569506726, 0.02242152466367713, -0.2825112107623318, -0.0, -0.42152466367713004, -0.008968609865470852, -0.5381165919282511, -0.04484304932735426, -0.2825112107623318, -0.06726457399103139, -0.4125560538116592, -0.08520179372197309, -0.5291479820627802, -0.008968609865470852, -0.19730941704035873], [-0.0, -0.0, 0.004, -0.08, 0.048, -0.092, 0.108, -0.096, 0.132, -0.032, -0.044, -0.112, -0.048, -0.188, -0.052, -0.256, 0.036, -0.236, 0.036, -0.348, 0.044, -0.456, -0.012, -0.24, -0.008, -0.344, 0.004, -0.44, 0.008, -0.172], [-0.0, -0.0, 0.0037174721189591076, -0.07063197026022305, 0.04460966542750929, -0.07806691449814127, 0.10037174721189591, -0.08921933085501858, 0.11152416356877323, -0.02973977695167286, -0.040892193308550186, -0.09665427509293681, -0.048327137546468404, -0.15985130111524162, -0.04460966542750929, -0.22676579925650558, 0.03717472118959108, -0.21561338289962825, 0.040892193308550186, -0.31970260223048325, 0.03717472118959108, -0.4052044609665427, -0.011152416356877323, -0.21561338289962825, -0.01858736059479554, -0.31970260223048325, -0.026022304832713755, -0.4052044609665427, 0.011152416356877323, -0.1524163568773234], [-0.0, -0.0, -0.0, -0.08298755186721991, 0.04149377593360996, -0.0954356846473029, 0.11203319502074689, -0.1078838174273859, 0.13278008298755187, -0.03734439834024896, -0.04979253112033195, -0.11618257261410789, -0.058091286307053944, -0.1950207468879668, -0.05394190871369295, -0.2697095435684647, 0.04149377593360996, -0.25311203319502074, 0.03319502074688797, -0.37344398340248963, 0.016597510373443983, -0.46887966804979253, -0.008298755186721992, -0.2572614107883817, -0.008298755186721992, -0.37344398340248963, -0.0, -0.4730290456431535, 0.008298755186721992, -0.1825726141078838], [-0.0, -0.0, -0.005128205128205128, -0.1076923076923077, 0.06153846153846154, -0.13333333333333333, 0.14358974358974358, -0.14871794871794872, 0.1641025641025641, -0.05641025641025641, -0.07179487179487179, -0.14871794871794872, -0.08717948717948718, -0.26153846153846155, -0.08205128205128205, -0.358974358974359, 0.05128205128205128, -0.3333333333333333, 0.035897435897435895, -0.49230769230769234, 0.035897435897435895, -0.6410256410256411, -0.03076923076923077, -0.3333333333333333, -0.05128205128205128, -0.49230769230769234, -0.05641025641025641, -0.6256410256410256, 0.005128205128205128, -0.24102564102564103], [-0.0, -0.0, 0.00510204081632653, -0.10204081632653061, 0.0663265306122449, -0.12244897959183673, 0.15306122448979592, -0.14795918367346939, 0.17857142857142858, -0.05612244897959184, -0.0663265306122449, -0.14795918367346939, -0.09183673469387756, -0.25, -0.09183673469387756, -0.3469387755102041, 0.05612244897959184, -0.32142857142857145, 0.05102040816326531, -0.49489795918367346, 0.05102040816326531, -0.6377551020408163, -0.030612244897959183, -0.32653061224489793, -0.04591836734693878, -0.49489795918367346, -0.05612244897959184, -0.6326530612244898, 0.01020408163265306, -0.22959183673469388], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12244897959183673, 0.15306122448979592, -0.15306122448979592, 0.1836734693877551, -0.05612244897959184, -0.07142857142857142, -0.14285714285714285, -0.08673469387755102, -0.25, -0.08163265306122448, -0.3469387755102041, 0.05102040816326531, -0.32653061224489793, 0.061224489795918366, -0.49489795918367346, 0.0663265306122449, -0.6377551020408163, -0.030612244897959183, -0.33163265306122447, -0.04591836734693878, -0.49489795918367346, -0.061224489795918366, -0.6326530612244898, 0.01020408163265306, -0.22959183673469388], [-0.0, -0.0, -0.0, -0.11290322580645161, 0.06451612903225806, -0.13440860215053763, 0.15591397849462366, -0.16129032258064516, 0.1935483870967742, -0.06989247311827956, -0.07526881720430108, -0.15591397849462366, -0.0967741935483871, -0.26344086021505375, -0.0913978494623656, -0.3548387096774194, 0.053763440860215055, -0.34946236559139787, 0.06451612903225806, -0.5268817204301075, 0.06989247311827956, -0.6774193548387096, -0.03225806451612903, -0.34946236559139787, -0.04838709677419355, -0.5268817204301075, -0.06451612903225806, -0.6720430107526881, 0.010752688172043012, -0.24731182795698925], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.0663265306122449, -0.12244897959183673, 0.15306122448979592, -0.14795918367346939, 0.1836734693877551, -0.05612244897959184, -0.07142857142857142, -0.14285714285714285, -0.08673469387755102, -0.25, -0.08163265306122448, -0.336734693877551, 0.05612244897959184, -0.33163265306122447, 0.061224489795918366, -0.5, 0.0663265306122449, -0.6377551020408163, -0.025510204081632654, -0.336734693877551, -0.04081632653061224, -0.49489795918367346, -0.061224489795918366, -0.6275510204081632, 0.01020408163265306, -0.23469387755102042], [-0.0, -0.0, -0.00510204081632653, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14285714285714285, 0.17857142857142858, -0.05612244897959184, -0.07653061224489796, -0.14285714285714285, -0.09183673469387756, -0.25, -0.08673469387755102, -0.336734693877551, 0.05102040816326531, -0.33163265306122447, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.05102040816326531, -0.49489795918367346, -0.0663265306122449, -0.6326530612244898, 0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14285714285714285, 0.17857142857142858, -0.05612244897959184, -0.07653061224489796, -0.14285714285714285, -0.09183673469387756, -0.25, -0.08673469387755102, -0.336734693877551, 0.05102040816326531, -0.33163265306122447, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.04591836734693878, -0.49489795918367346, -0.0663265306122449, -0.6326530612244898, 0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14285714285714285, 0.17857142857142858, -0.04591836734693878, -0.07142857142857142, -0.14285714285714285, -0.08673469387755102, -0.24489795918367346, -0.08673469387755102, -0.33163265306122447, 0.05102040816326531, -0.32653061224489793, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.33163265306122447, -0.05102040816326531, -0.49489795918367346, -0.0663265306122449, -0.6326530612244898, 0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14285714285714285, 0.17857142857142858, -0.05102040816326531, -0.07142857142857142, -0.14285714285714285, -0.09183673469387756, -0.25, -0.08673469387755102, -0.336734693877551, 0.05102040816326531, -0.33163265306122447, 0.05612244897959184, -0.5, 0.0663265306122449, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.04591836734693878, -0.49489795918367346, -0.0663265306122449, -0.6275510204081632, 0.01020408163265306, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14285714285714285, 0.17857142857142858, -0.04591836734693878, -0.07653061224489796, -0.14285714285714285, -0.08673469387755102, -0.24489795918367346, -0.08673469387755102, -0.33163265306122447, 0.05102040816326531, -0.33163265306122447, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.04591836734693878, -0.49489795918367346, -0.0663265306122449, -0.6326530612244898, 0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14795918367346939, 0.17857142857142858, -0.05102040816326531, -0.07142857142857142, -0.14285714285714285, -0.08673469387755102, -0.24489795918367346, -0.08673469387755102, -0.33163265306122447, 0.05102040816326531, -0.33163265306122447, 0.05612244897959184, -0.5, 0.0663265306122449, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.05102040816326531, -0.49489795918367346, -0.0663265306122449, -0.6326530612244898, 0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14795918367346939, 0.17857142857142858, -0.04591836734693878, -0.07142857142857142, -0.14285714285714285, -0.08673469387755102, -0.24489795918367346, -0.08673469387755102, -0.336734693877551, 0.05102040816326531, -0.33163265306122447, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.04591836734693878, -0.49489795918367346, -0.0663265306122449, -0.6326530612244898, 0.01020408163265306, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14795918367346939, 0.17857142857142858, -0.05102040816326531, -0.07142857142857142, -0.14285714285714285, -0.08673469387755102, -0.24489795918367346, -0.08673469387755102, -0.33163265306122447, 0.05102040816326531, -0.33163265306122447, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.04591836734693878, -0.49489795918367346, -0.0663265306122449, -0.6326530612244898, 0.01020408163265306, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14285714285714285, 0.17857142857142858, -0.05102040816326531, -0.07142857142857142, -0.14285714285714285, -0.09183673469387756, -0.24489795918367346, -0.08673469387755102, -0.33163265306122447, 0.05102040816326531, -0.33163265306122447, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.04591836734693878, -0.49489795918367346, -0.0663265306122449, -0.6275510204081632, 0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14795918367346939, 0.17857142857142858, -0.04591836734693878, -0.07142857142857142, -0.14285714285714285, -0.08673469387755102, -0.24489795918367346, -0.08673469387755102, -0.33163265306122447, 0.05102040816326531, -0.32653061224489793, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.04591836734693878, -0.49489795918367346, -0.0663265306122449, -0.6326530612244898, 0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14795918367346939, 0.1836734693877551, -0.05612244897959184, -0.07142857142857142, -0.14285714285714285, -0.08673469387755102, -0.24489795918367346, -0.08673469387755102, -0.336734693877551, 0.05102040816326531, -0.32653061224489793, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.33163265306122447, -0.05102040816326531, -0.49489795918367346, -0.0663265306122449, -0.6326530612244898, 0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14795918367346939, 0.1836734693877551, -0.05612244897959184, -0.07142857142857142, -0.14285714285714285, -0.08673469387755102, -0.24489795918367346, -0.08673469387755102, -0.33163265306122447, 0.05102040816326531, -0.33163265306122447, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.04591836734693878, -0.49489795918367346, -0.0663265306122449, -0.6275510204081632, 0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14795918367346939, 0.1836734693877551, -0.05102040816326531, -0.07142857142857142, -0.14285714285714285, -0.08673469387755102, -0.25, -0.08673469387755102, -0.33163265306122447, 0.05102040816326531, -0.33163265306122447, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.04591836734693878, -0.49489795918367346, -0.0663265306122449, -0.6275510204081632, 0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12244897959183673, 0.14795918367346939, -0.14795918367346939, 0.17857142857142858, -0.05612244897959184, -0.07142857142857142, -0.14285714285714285, -0.09183673469387756, -0.24489795918367346, -0.08673469387755102, -0.33163265306122447, 0.05102040816326531, -0.33163265306122447, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.05102040816326531, -0.49489795918367346, -0.0663265306122449, -0.6275510204081632, 0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14795918367346939, 0.17857142857142858, -0.05612244897959184, -0.07142857142857142, -0.14285714285714285, -0.08673469387755102, -0.24489795918367346, -0.08673469387755102, -0.33163265306122447, 0.05102040816326531, -0.32653061224489793, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.33163265306122447, -0.05102040816326531, -0.49489795918367346, -0.0663265306122449, -0.6326530612244898, 0.00510204081632653, -0.22959183673469388], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12244897959183673, 0.14795918367346939, -0.14795918367346939, 0.17346938775510204, -0.05612244897959184, -0.07142857142857142, -0.14285714285714285, -0.09183673469387756, -0.25, -0.08673469387755102, -0.336734693877551, 0.05102040816326531, -0.33163265306122447, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.05102040816326531, -0.49489795918367346, -0.0663265306122449, -0.6326530612244898, 0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14795918367346939, -0.14795918367346939, 0.17346938775510204, -0.05612244897959184, -0.07142857142857142, -0.14285714285714285, -0.09183673469387756, -0.25, -0.08673469387755102, -0.336734693877551, 0.05102040816326531, -0.33163265306122447, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.04591836734693878, -0.49489795918367346, -0.0663265306122449, -0.6275510204081632, 0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12244897959183673, 0.14795918367346939, -0.14795918367346939, 0.17346938775510204, -0.05102040816326531, -0.07142857142857142, -0.14285714285714285, -0.08673469387755102, -0.24489795918367346, -0.08673469387755102, -0.33163265306122447, 0.05102040816326531, -0.32653061224489793, 0.05612244897959184, -0.5, 0.0663265306122449, -0.6377551020408163, -0.030612244897959183, -0.33163265306122447, -0.05102040816326531, -0.49489795918367346, -0.0663265306122449, -0.6275510204081632, 0.00510204081632653, -0.22959183673469388], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.14285714285714285, -0.14795918367346939, 0.17857142857142858, -0.05612244897959184, -0.07142857142857142, -0.14285714285714285, -0.08673469387755102, -0.24489795918367346, -0.08673469387755102, -0.336734693877551, 0.05102040816326531, -0.32653061224489793, 0.05612244897959184, -0.5, 0.061224489795918366, -0.6377551020408163, -0.030612244897959183, -0.336734693877551, -0.05102040816326531, -0.49489795918367346, -0.0663265306122449, -0.6326530612244898, 0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.1076923076923077, 0.05641025641025641, -0.1282051282051282, 0.15384615384615385, -0.11794871794871795, 0.17435897435897435, -0.010256410256410256, -0.07179487179487179, -0.14871794871794872, -0.08717948717948718, -0.24615384615384617, -0.08717948717948718, -0.3333333333333333, 0.046153846153846156, -0.3333333333333333, 0.05641025641025641, -0.5025641025641026, 0.06153846153846154, -0.6410256410256411, -0.03076923076923077, -0.3384615384615385, -0.046153846153846156, -0.49743589743589745, -0.06666666666666667, -0.6358974358974359, 0.005128205128205128, -0.2358974358974359], [-0.0, -0.0, -0.005128205128205128, -0.1076923076923077, 0.05641025641025641, -0.13333333333333333, 0.15897435897435896, -0.11794871794871795, 0.17435897435897435, -0.005128205128205128, -0.07692307692307693, -0.14871794871794872, -0.09230769230769231, -0.24615384615384617, -0.08717948717948718, -0.3384615384615385, 0.046153846153846156, -0.3333333333333333, 0.05641025641025641, -0.5025641025641026, 0.06153846153846154, -0.6410256410256411, -0.03076923076923077, -0.3384615384615385, -0.046153846153846156, -0.49743589743589745, -0.06666666666666667, -0.6307692307692307, 0.005128205128205128, -0.2358974358974359], [-0.0, -0.0, -0.005128205128205128, -0.1076923076923077, 0.05641025641025641, -0.13333333333333333, 0.15897435897435896, -0.11794871794871795, 0.17435897435897435, -0.010256410256410256, -0.07692307692307693, -0.14871794871794872, -0.09230769230769231, -0.24615384615384617, -0.08717948717948718, -0.3384615384615385, 0.046153846153846156, -0.3333333333333333, 0.05641025641025641, -0.5025641025641026, 0.06153846153846154, -0.6410256410256411, -0.03076923076923077, -0.3384615384615385, -0.046153846153846156, -0.49743589743589745, -0.06666666666666667, -0.6307692307692307, 0.005128205128205128, -0.2358974358974359], [-0.0, -0.0, -0.005128205128205128, -0.1076923076923077, 0.05641025641025641, -0.13333333333333333, 0.15897435897435896, -0.11794871794871795, 0.17435897435897435, -0.010256410256410256, -0.07179487179487179, -0.14871794871794872, -0.09230769230769231, -0.2512820512820513, -0.08717948717948718, -0.3384615384615385, 0.046153846153846156, -0.3333333333333333, 0.05641025641025641, -0.5025641025641026, 0.06666666666666667, -0.6410256410256411, -0.03076923076923077, -0.3384615384615385, -0.046153846153846156, -0.49743589743589745, -0.06666666666666667, -0.6307692307692307, 0.005128205128205128, -0.2358974358974359], [-0.0, -0.0, -0.0, -0.1076923076923077, 0.05641025641025641, -0.13333333333333333, 0.15897435897435896, -0.11794871794871795, 0.17435897435897435, -0.010256410256410256, -0.07179487179487179, -0.14358974358974358, -0.08717948717948718, -0.24615384615384617, -0.08717948717948718, -0.3333333333333333, 0.046153846153846156, -0.3333333333333333, 0.05641025641025641, -0.5025641025641026, 0.06153846153846154, -0.6410256410256411, -0.03076923076923077, -0.3384615384615385, -0.046153846153846156, -0.49743589743589745, -0.06666666666666667, -0.6358974358974359, 0.005128205128205128, -0.2358974358974359], [-0.0, -0.0, -0.0, -0.1076923076923077, 0.05641025641025641, -0.1282051282051282, 0.15897435897435896, -0.11794871794871795, 0.15897435897435896, -0.010256410256410256, -0.07179487179487179, -0.14871794871794872, -0.08717948717948718, -0.2512820512820513, -0.08717948717948718, -0.3384615384615385, 0.046153846153846156, -0.3333333333333333, 0.05641025641025641, -0.5025641025641026, 0.06153846153846154, -0.6410256410256411, -0.03076923076923077, -0.3384615384615385, -0.046153846153846156, -0.49743589743589745, -0.06666666666666667, -0.6307692307692307, 0.005128205128205128, -0.2358974358974359], [-0.0, -0.0, -0.0, -0.1076923076923077, 0.05641025641025641, -0.13333333333333333, 0.15897435897435896, -0.12307692307692308, 0.2, -0.015384615384615385, -0.07179487179487179, -0.14871794871794872, -0.08717948717948718, -0.24615384615384617, -0.08717948717948718, -0.3333333333333333, 0.046153846153846156, -0.3333333333333333, 0.05641025641025641, -0.5025641025641026, 0.06153846153846154, -0.6410256410256411, -0.03076923076923077, -0.3384615384615385, -0.046153846153846156, -0.49743589743589745, -0.06666666666666667, -0.6358974358974359, 0.005128205128205128, -0.2358974358974359], [-0.0, -0.0, -0.0, -0.1076923076923077, 0.05641025641025641, -0.1282051282051282, 0.15384615384615385, -0.12307692307692308, 0.2, -0.020512820512820513, -0.07179487179487179, -0.14871794871794872, -0.08717948717948718, -0.24615384615384617, -0.08717948717948718, -0.3384615384615385, 0.046153846153846156, -0.3333333333333333, 0.05641025641025641, -0.5025641025641026, 0.06153846153846154, -0.6410256410256411, -0.03076923076923077, -0.3384615384615385, -0.046153846153846156, -0.49743589743589745, -0.06666666666666667, -0.6358974358974359, 0.005128205128205128, -0.2358974358974359], [-0.0, -0.0, -0.0, -0.1076923076923077, 0.06153846153846154, -0.1282051282051282, 0.15384615384615385, -0.11794871794871795, 0.2, -0.02564102564102564, -0.07179487179487179, -0.14871794871794872, -0.08717948717948718, -0.2512820512820513, -0.08717948717948718, -0.3384615384615385, 0.046153846153846156, -0.3333333333333333, 0.05641025641025641, -0.5025641025641026, 0.06153846153846154, -0.6410256410256411, -0.03076923076923077, -0.3384615384615385, -0.046153846153846156, -0.49743589743589745, -0.06666666666666667, -0.6358974358974359, 0.005128205128205128, -0.2358974358974359], [-0.0, -0.0, -0.0, -0.1076923076923077, 0.06153846153846154, -0.1282051282051282, 0.15384615384615385, -0.12307692307692308, 0.2, -0.02564102564102564, -0.07179487179487179, -0.14871794871794872, -0.08717948717948718, -0.24615384615384617, -0.08717948717948718, -0.3384615384615385, 0.046153846153846156, -0.3333333333333333, 0.05641025641025641, -0.5025641025641026, 0.06153846153846154, -0.6410256410256411, -0.03076923076923077, -0.3384615384615385, -0.046153846153846156, -0.49743589743589745, -0.06666666666666667, -0.6358974358974359, 0.005128205128205128, -0.2358974358974359], [-0.0, -0.0, -0.0, -0.1076923076923077, 0.05641025641025641, -0.13333333333333333, 0.15384615384615385, -0.12307692307692308, 0.18974358974358974, -0.020512820512820513, -0.07179487179487179, -0.14871794871794872, -0.08717948717948718, -0.24615384615384617, -0.08717948717948718, -0.3333333333333333, 0.046153846153846156, -0.3333333333333333, 0.05641025641025641, -0.5025641025641026, 0.06153846153846154, -0.6410256410256411, -0.03076923076923077, -0.3384615384615385, -0.046153846153846156, -0.49743589743589745, -0.06666666666666667, -0.6358974358974359, 0.005128205128205128, -0.2358974358974359], [-0.0, -0.0, -0.005376344086021506, -0.11290322580645161, 0.053763440860215055, -0.13978494623655913, 0.13440860215053763, -0.1881720430107527, 0.1989247311827957, -0.10752688172043011, -0.08064516129032258, -0.15591397849462366, -0.10215053763440861, -0.26881720430107525, -0.0967741935483871, -0.3602150537634409, 0.04838709677419355, -0.3548387096774194, 0.053763440860215055, -0.532258064516129, 0.05913978494623656, -0.6774193548387096, -0.03763440860215054, -0.3602150537634409, -0.053763440860215055, -0.5268817204301075, -0.07526881720430108, -0.6720430107526881, -0.0, -0.25268817204301075], [-0.0, -0.0, -0.01020408163265306, -0.10204081632653061, 0.05102040816326531, -0.12755102040816327, 0.12755102040816327, -0.17857142857142858, 0.18877551020408162, -0.11224489795918367, -0.08163265306122448, -0.14285714285714285, -0.09183673469387756, -0.24489795918367346, -0.09183673469387756, -0.336734693877551, 0.04591836734693878, -0.32653061224489793, 0.05102040816326531, -0.5, 0.05612244897959184, -0.6377551020408163, -0.03571428571428571, -0.33163265306122447, -0.05102040816326531, -0.49489795918367346, -0.07142857142857142, -0.6326530612244898, -0.0, -0.23469387755102042], [-0.0, -0.0, -0.01020408163265306, -0.10204081632653061, 0.05102040816326531, -0.12755102040816327, 0.12755102040816327, -0.17857142857142858, 0.18877551020408162, -0.11734693877551021, -0.08163265306122448, -0.14285714285714285, -0.09693877551020408, -0.24489795918367346, -0.09183673469387756, -0.336734693877551, 0.04591836734693878, -0.32653061224489793, 0.05102040816326531, -0.5, 0.05612244897959184, -0.6377551020408163, -0.03571428571428571, -0.33163265306122447, -0.05102040816326531, -0.49489795918367346, -0.07142857142857142, -0.6275510204081632, -0.0, -0.23469387755102042], [-0.0, -0.0, -0.01020408163265306, -0.10204081632653061, 0.05102040816326531, -0.12755102040816327, 0.12755102040816327, -0.17857142857142858, 0.18877551020408162, -0.11224489795918367, -0.08163265306122448, -0.14285714285714285, -0.09693877551020408, -0.24489795918367346, -0.09183673469387756, -0.336734693877551, 0.04591836734693878, -0.32653061224489793, 0.05102040816326531, -0.5, 0.061224489795918366, -0.6377551020408163, -0.03571428571428571, -0.33163265306122447, -0.05102040816326531, -0.49489795918367346, -0.07142857142857142, -0.6275510204081632, -0.0, -0.23469387755102042], [-0.0, -0.0, -0.01020408163265306, -0.10204081632653061, 0.05102040816326531, -0.12755102040816327, 0.12755102040816327, -0.17857142857142858, 0.18877551020408162, -0.11734693877551021, -0.07653061224489796, -0.14285714285714285, -0.09693877551020408, -0.25, -0.09183673469387756, -0.336734693877551, 0.04591836734693878, -0.32653061224489793, 0.05102040816326531, -0.5, 0.05612244897959184, -0.6377551020408163, -0.03571428571428571, -0.33163265306122447, -0.05102040816326531, -0.49489795918367346, -0.07142857142857142, -0.6326530612244898, -0.0, -0.23469387755102042], [-0.0, -0.0, -0.00510204081632653, -0.10204081632653061, 0.05102040816326531, -0.12755102040816327, 0.12755102040816327, -0.17857142857142858, 0.18877551020408162, -0.11734693877551021, -0.07653061224489796, -0.14285714285714285, -0.09183673469387756, -0.24489795918367346, -0.09183673469387756, -0.33163265306122447, 0.04591836734693878, -0.32653061224489793, 0.05102040816326531, -0.5, 0.05612244897959184, -0.6377551020408163, -0.03571428571428571, -0.336734693877551, -0.05102040816326531, -0.49489795918367346, -0.07142857142857142, -0.6326530612244898, -0.0, -0.23469387755102042], [-0.0, -0.0, -0.00510204081632653, -0.10204081632653061, 0.05102040816326531, -0.12755102040816327, 0.12755102040816327, -0.1836734693877551, 0.18877551020408162, -0.11734693877551021, -0.07653061224489796, -0.14285714285714285, -0.09183673469387756, -0.24489795918367346, -0.09183673469387756, -0.336734693877551, 0.04591836734693878, -0.33163265306122447, 0.05102040816326531, -0.5, 0.05612244897959184, -0.6377551020408163, -0.03571428571428571, -0.336734693877551, -0.05102040816326531, -0.49489795918367346, -0.07142857142857142, -0.6326530612244898, -0.0, -0.23469387755102042], [-0.0, -0.0, -0.00510204081632653, -0.10204081632653061, 0.05102040816326531, -0.12755102040816327, 0.12755102040816327, -0.17857142857142858, 0.18877551020408162, -0.11734693877551021, -0.07653061224489796, -0.14285714285714285, -0.09183673469387756, -0.24489795918367346, -0.09183673469387756, -0.33163265306122447, 0.04591836734693878, -0.32653061224489793, 0.05102040816326531, -0.5, 0.05612244897959184, -0.6377551020408163, -0.03571428571428571, -0.336734693877551, -0.05102040816326531, -0.49489795918367346, -0.07142857142857142, -0.6326530612244898, -0.0, -0.23469387755102042], [-0.0, -0.0, -0.00510204081632653, -0.10204081632653061, 0.05612244897959184, -0.12755102040816327, 0.12755102040816327, -0.17857142857142858, 0.18877551020408162, -0.11734693877551021, -0.07653061224489796, -0.14285714285714285, -0.09183673469387756, -0.24489795918367346, -0.09183673469387756, -0.336734693877551, 0.04591836734693878, -0.32653061224489793, 0.05102040816326531, -0.5, 0.05612244897959184, -0.6377551020408163, -0.03571428571428571, -0.33163265306122447, -0.05102040816326531, -0.49489795918367346, -0.07142857142857142, -0.6326530612244898, -0.0, -0.23469387755102042], [-0.0, -0.0, -0.022727272727272728, -0.125, 0.03977272727272727, -0.1534090909090909, 0.125, -0.21022727272727273, 0.19318181818181818, -0.13636363636363635, -0.10227272727272728, -0.17045454545454544, -0.125, -0.2897727272727273, -0.11931818181818182, -0.38636363636363635, 0.03409090909090909, -0.3806818181818182, 0.03977272727272727, -0.5681818181818182, 0.045454545454545456, -0.7215909090909091, -0.056818181818181816, -0.38636363636363635, -0.07386363636363637, -0.5625, -0.09659090909090909, -0.7159090909090909, -0.017045454545454544, -0.2727272727272727], [-0.0, -0.0, -0.00510204081632653, -0.10204081632653061, 0.05612244897959184, -0.12755102040816327, 0.12755102040816327, -0.17857142857142858, 0.18877551020408162, -0.11734693877551021, -0.07653061224489796, -0.14285714285714285, -0.09183673469387756, -0.24489795918367346, -0.09183673469387756, -0.336734693877551, 0.04591836734693878, -0.33163265306122447, 0.05102040816326531, -0.5, 0.05612244897959184, -0.6377551020408163, -0.03571428571428571, -0.336734693877551, -0.05102040816326531, -0.49489795918367346, -0.07142857142857142, -0.6326530612244898, -0.0, -0.23469387755102042], [-0.0, -0.0, -0.00510204081632653, -0.10204081632653061, 0.05612244897959184, -0.12755102040816327, 0.12755102040816327, -0.1836734693877551, 0.18877551020408162, -0.11734693877551021, -0.07653061224489796, -0.14285714285714285, -0.09183673469387756, -0.24489795918367346, -0.09183673469387756, -0.336734693877551, 0.04591836734693878, -0.33163265306122447, 0.05102040816326531, -0.5, 0.061224489795918366, -0.6377551020408163, -0.03571428571428571, -0.336734693877551, -0.05102040816326531, -0.49489795918367346, -0.07142857142857142, -0.6275510204081632, -0.0, -0.23469387755102042], [-0.0, -0.0, -0.005128205128205128, -0.1076923076923077, 0.05641025641025641, -0.1282051282051282, 0.13333333333333333, -0.18461538461538463, 0.17435897435897435, -0.11794871794871795, -0.07692307692307693, -0.14358974358974358, -0.09230769230769231, -0.24615384615384617, -0.09230769230769231, -0.3384615384615385, 0.046153846153846156, -0.3333333333333333, 0.05128205128205128, -0.5025641025641026, 0.05641025641025641, -0.6410256410256411, -0.035897435897435895, -0.3384615384615385, -0.05128205128205128, -0.49743589743589745, -0.07179487179487179, -0.6307692307692307, -0.0, -0.2358974358974359], [-0.0, -0.0, -0.005128205128205128, -0.1076923076923077, 0.05641025641025641, -0.13333333333333333, 0.13333333333333333, -0.18461538461538463, 0.16923076923076924, -0.11794871794871795, -0.07692307692307693, -0.14358974358974358, -0.09230769230769231, -0.24615384615384617, -0.09230769230769231, -0.3384615384615385, 0.046153846153846156, -0.3333333333333333, 0.05128205128205128, -0.5025641025641026, 0.05641025641025641, -0.6410256410256411, -0.035897435897435895, -0.3384615384615385, -0.05128205128205128, -0.49743589743589745, -0.07179487179487179, -0.6358974358974359, -0.0, -0.2358974358974359], [-0.0, -0.0, -0.005128205128205128, -0.1076923076923077, 0.05641025641025641, -0.13333333333333333, 0.13333333333333333, -0.18461538461538463, 0.16923076923076924, -0.12307692307692308, -0.07692307692307693, -0.14358974358974358, -0.09743589743589744, -0.2512820512820513, -0.09230769230769231, -0.3435897435897436, 0.046153846153846156, -0.3333333333333333, 0.05128205128205128, -0.5025641025641026, 0.05641025641025641, -0.6410256410256411, -0.035897435897435895, -0.3384615384615385, -0.05128205128205128, -0.49743589743589745, -0.07179487179487179, -0.6307692307692307, -0.0, -0.2358974358974359], [-0.0, -0.0, -0.00975609756097561, -0.0975609756097561, 0.04878048780487805, -0.12195121951219512, 0.12195121951219512, -0.17073170731707318, 0.16097560975609757, -0.11219512195121951, -0.07804878048780488, -0.13170731707317074, -0.09268292682926829, -0.23414634146341465, -0.08780487804878048, -0.32682926829268294, 0.04390243902439024, -0.3073170731707317, 0.04878048780487805, -0.47317073170731705, 0.05365853658536585, -0.6048780487804878, -0.03414634146341464, -0.3121951219512195, -0.04878048780487805, -0.4682926829268293, -0.06829268292682927, -0.5951219512195122, -0.0, -0.21951219512195122],[-0.0, -0.0, -0.012269938650306749, -0.049079754601226995, 0.015337423312883436, -0.0705521472392638, 0.018404907975460124, -0.11349693251533742, 0.02147239263803681, -0.147239263803681, -0.04294478527607362, -0.05828220858895705, -0.08282208588957055, -0.06134969325153374, -0.08895705521472393, -0.018404907975460124, 0.003067484662576687, -0.16257668711656442, 0.006134969325153374, -0.24233128834355827, 0.012269938650306749, -0.30368098159509205, -0.03680981595092025, -0.15644171779141106, -0.046012269938650305, -0.2392638036809816, -0.05828220858895705, -0.30368098159509205, -0.015337423312883436, -0.11042944785276074], [-0.0, -0.0, -0.010752688172043012, -0.06451612903225806, 0.025089605734767026, -0.08960573476702509, 0.035842293906810034, -0.15412186379928317, 0.035842293906810034, -0.2114695340501792, -0.04659498207885305, -0.07526881720430108, -0.1003584229390681, -0.0931899641577061, -0.1039426523297491, -0.035842293906810034, -0.0, -0.2007168458781362, -0.014336917562724014, -0.2974910394265233, -0.017921146953405017, -0.3835125448028674, -0.03942652329749104, -0.1971326164874552, -0.02867383512544803, -0.2974910394265233, -0.025089605734767026, -0.3835125448028674, -0.017921146953405017, -0.13620071684587814], [-0.0, -0.0, -0.013452914798206279, -0.08968609865470852, 0.04035874439461883, -0.12556053811659193, 0.053811659192825115, -0.21076233183856502, 0.06278026905829596, -0.28699551569506726, -0.06726457399103139, -0.09865470852017937, -0.13004484304932734, -0.12556053811659193, -0.14798206278026907, -0.04932735426008968, 0.008968609865470852, -0.27802690582959644, 0.02242152466367713, -0.42152466367713004, 0.03587443946188341, -0.547085201793722, -0.053811659192825115, -0.27802690582959644, -0.04035874439461883, -0.4125560538116592, -0.03587443946188341, -0.515695067264574, -0.02242152466367713, -0.19282511210762332], [-0.0, -0.0, -0.010752688172043012, -0.11290322580645161, 0.05913978494623656, -0.16129032258064516, 0.08064516129032258, -0.27956989247311825, 0.07526881720430108, -0.3763440860215054, -0.08602150537634409, -0.12903225806451613, -0.16666666666666666, -0.15591397849462366, -0.1774193548387097, -0.053763440860215055, 0.010752688172043012, -0.3602150537634409, 0.021505376344086023, -0.5376344086021505, 0.021505376344086023, -0.6774193548387096, -0.07526881720430108, -0.3548387096774194, -0.10215053763440861, -0.5268817204301075, -0.13440860215053763, -0.6720430107526881, -0.026881720430107527, -0.24731182795698925], [-0.0, -0.0, -0.005681818181818182, -0.125, 0.06818181818181818, -0.17613636363636365, 0.09090909090909091, -0.2897727272727273, 0.09659090909090909, -0.39204545454545453, -0.08522727272727272, -0.14772727272727273, -0.17613636363636365, -0.17045454545454544, -0.19318181818181818, -0.0625, 0.028409090909090908, -0.375, 0.056818181818181816, -0.5454545454545454, 0.045454545454545456, -0.6875, -0.05113636363636364, -0.375, -0.028409090909090908, -0.5568181818181818, -0.017045454545454544, -0.7159090909090909, -0.017045454545454544, -0.26136363636363635], [-0.0, -0.0, -0.02586206896551724, -0.08620689655172414, 0.021551724137931036, -0.125, 0.02586206896551724, -0.20689655172413793, 0.021551724137931036, -0.27155172413793105, -0.07758620689655173, -0.09051724137931035, -0.14224137931034483, -0.10344827586206896, -0.15086206896551724, -0.03017241379310345, -0.021551724137931036, -0.25862068965517243, -0.03879310344827586, -0.3879310344827586, -0.04741379310344827, -0.5, -0.08620689655172414, -0.24568965517241378, -0.10344827586206896, -0.375, -0.125, -0.4870689655172414, -0.04741379310344827, -0.17672413793103448], [-0.0, -0.0, -0.011111111111111112, -0.06666666666666667, 0.03333333333333333, -0.09259259259259259, 0.040740740740740744, -0.15925925925925927, 0.040740740740740744, -0.2111111111111111, -0.05185185185185185, -0.07407407407407407, -0.1, -0.08148148148148149, -0.11481481481481481, -0.022222222222222223, 0.007407407407407408, -0.2074074074074074, 0.025925925925925925, -0.3148148148148148, 0.02962962962962963, -0.4, -0.044444444444444446, -0.2074074074074074, -0.037037037037037035, -0.3074074074074074, -0.02962962962962963, -0.4, -0.018518518518518517, -0.14074074074074075], [-0.0, -0.0, -0.012269938650306749, -0.049079754601226995, 0.012269938650306749, -0.0705521472392638, 0.018404907975460124, -0.1196319018404908, 0.015337423312883436, -0.15950920245398773, -0.04294478527607362, -0.05214723926380368, -0.07975460122699386, -0.06441717791411043, -0.0950920245398773, -0.018404907975460124, -0.012269938650306749, -0.15644171779141106, -0.015337423312883436, -0.2361963190184049, -0.018404907975460124, -0.3006134969325153, -0.049079754601226995, -0.147239263803681, -0.05521472392638037, -0.23006134969325154, -0.06748466257668712, -0.29141104294478526, -0.024539877300613498, -0.10429447852760736], [-0.0, -0.0, -0.0029850746268656717, -0.04776119402985075, 0.023880597014925373, -0.06865671641791045, 0.026865671641791045, -0.11044776119402985, 0.023880597014925373, -0.15223880597014924, -0.03283582089552239, -0.05970149253731343, -0.05970149253731343, -0.06865671641791045, -0.08059701492537313, -0.03880597014925373, -0.0, -0.14626865671641792, 0.011940298507462687, -0.21492537313432836, 0.023880597014925373, -0.27761194029850744, -0.029850746268656716, -0.14626865671641792, -0.023880597014925373, -0.21492537313432836, -0.020895522388059702, -0.2835820895522388, -0.008955223880597015, -0.1044776119402985], [-0.0, -0.0, -0.013440860215053764, -0.04032258064516129, 0.008064516129032258, -0.056451612903225805, 0.010752688172043012, -0.0913978494623656, 0.008064516129032258, -0.12634408602150538, -0.03494623655913978, -0.043010752688172046, -0.06720430107526881, -0.04838709677419355, -0.07526881720430108, -0.010752688172043012, -0.016129032258064516, -0.12096774193548387, -0.021505376344086023, -0.1827956989247312, -0.03225806451612903, -0.24193548387096775, -0.03494623655913978, -0.11827956989247312, -0.03225806451612903, -0.18548387096774194, -0.03494623655913978, -0.24193548387096775, -0.01881720430107527, -0.08333333333333333], [-0.0, -0.0, -0.0028328611898017, -0.0453257790368272, 0.019830028328611898, -0.06232294617563739, 0.0226628895184136, -0.10198300283286119, 0.019830028328611898, -0.13031161473087818, -0.031161473087818695, -0.05099150141643059, -0.0708215297450425, -0.0453257790368272, -0.0679886685552408, -0.0113314447592068, 0.0028328611898017, -0.1388101983002833, 0.0113314447592068, -0.20396600566572237, 0.0169971671388102, -0.2577903682719547, -0.028328611898016998, -0.1359773371104816, -0.031161473087818695, -0.20396600566572237, -0.039660056657223795, -0.26628895184135976, -0.0084985835694051, -0.09631728045325778], [-0.0, -0.0, -0.0056657223796034, -0.0453257790368272, 0.019830028328611898, -0.06515580736543909, 0.025495750708215296, -0.1048158640226629, 0.0226628895184136, -0.14447592067988668, -0.0339943342776204, -0.05099150141643059, -0.0679886685552408, -0.06515580736543909, -0.0708215297450425, -0.031161473087818695, -0.0028328611898017, -0.1388101983002833, -0.0113314447592068, -0.20396600566572237, -0.0226628895184136, -0.26628895184135976, -0.031161473087818695, -0.13314447592067988, -0.025495750708215296, -0.20396600566572237, -0.028328611898016998, -0.26345609065155806, -0.014164305949008499, -0.09631728045325778], [-0.0, -0.0, -0.006329113924050633, -0.05379746835443038, 0.022151898734177215, -0.07278481012658228, 0.03164556962025317, -0.12341772151898735, 0.03481012658227848, -0.16772151898734178, -0.0379746835443038, -0.060126582278481014, -0.08227848101265822, -0.06962025316455696, -0.08544303797468354, -0.0189873417721519, -0.0, -0.16455696202531644, 0.00949367088607595, -0.25, 0.012658227848101266, -0.3259493670886076, -0.03481012658227848, -0.16139240506329114, -0.03164556962025317, -0.2468354430379747, -0.0379746835443038, -0.31645569620253167, -0.012658227848101266, -0.11392405063291139], [-0.0, -0.0, -0.011152416356877323, -0.07063197026022305, 0.026022304832713755, -0.09665427509293681, 0.03345724907063197, -0.15613382899628253, 0.02973977695167286, -0.20817843866171004, -0.048327137546468404, -0.07806691449814127, -0.10037174721189591, -0.08550185873605948, -0.10780669144981413, -0.02973977695167286, -0.0037174721189591076, -0.20817843866171004, -0.007434944237918215, -0.3159851301115242, -0.01486988847583643, -0.40148698884758366, -0.040892193308550186, -0.20817843866171004, -0.040892193308550186, -0.31226765799256506, -0.055762081784386616, -0.40892193308550184, -0.01858736059479554, -0.1449814126394052], [-0.0, -0.0, -0.02252252252252252, -0.0945945945945946, 0.036036036036036036, -0.12612612612612611, 0.04954954954954955, -0.21621621621621623, 0.04954954954954955, -0.2882882882882883, -0.07657657657657657, -0.0990990990990991, -0.14414414414414414, -0.1036036036036036, -0.15765765765765766, -0.02252252252252252, 0.0045045045045045045, -0.27927927927927926, -0.0045045045045045045, -0.4189189189189189, -0.009009009009009009, -0.5405405405405406, -0.05855855855855856, -0.27927927927927926, -0.05405405405405406, -0.4099099099099099, -0.04954954954954955, -0.527027027027027, -0.02702702702702703, -0.19369369369369369], [-0.0, -0.0, -0.02702702702702703, -0.11891891891891893, 0.04864864864864865, -0.16216216216216217, 0.06486486486486487, -0.2810810810810811, 0.06486486486486487, -0.372972972972973, -0.0918918918918919, -0.13513513513513514, -0.1891891891891892, -0.12972972972972974, -0.1891891891891892, -0.032432432432432434, 0.005405405405405406, -0.3567567567567568, 0.021621621621621623, -0.5297297297297298, 0.02702702702702703, -0.6756756756756757, -0.07567567567567568, -0.35135135135135137, -0.0972972972972973, -0.518918918918919, -0.11351351351351352, -0.6702702702702703, -0.032432432432432434, -0.24864864864864866], [-0.0, -0.0, -0.014778325123152709, -0.10837438423645321, 0.04433497536945813, -0.1477832512315271, 0.06403940886699508, -0.24630541871921183, 0.054187192118226604, -0.3251231527093596, -0.07389162561576355, -0.12315270935960591, -0.15270935960591134, -0.1330049261083744, -0.16748768472906403, -0.04433497536945813, 0.0049261083743842365, -0.3103448275862069, 0.019704433497536946, -0.4482758620689655, -0.0, -0.5763546798029556, -0.054187192118226604, -0.3103448275862069, -0.03940886699507389, -0.45320197044334976, -0.03940886699507389, -0.5862068965517241, -0.024630541871921183, -0.22167487684729065], [-0.0, -0.0, -0.01593625498007968, -0.07569721115537849, 0.027888446215139442, -0.10358565737051793, 0.035856573705179286, -0.1752988047808765, 0.035856573705179286, -0.2350597609561753, -0.055776892430278883, -0.08366533864541832, -0.11952191235059761, -0.09561752988047809, -0.13147410358565736, -0.03187250996015936, -0.00398406374501992, -0.2350597609561753, -0.00398406374501992, -0.3545816733067729, -0.00796812749003984, -0.450199203187251, -0.05976095617529881, -0.23107569721115537, -0.07171314741035857, -0.3466135458167331, -0.09561752988047809, -0.43824701195219123, -0.027888446215139442, -0.16334661354581673], [-0.0, -0.0, -0.010752688172043012, -0.06451612903225806, 0.025089605734767026, -0.08602150537634409, 0.043010752688172046, -0.15053763440860216, 0.03942652329749104, -0.2007168458781362, -0.04659498207885305, -0.06810035842293907, -0.10752688172043011, -0.08602150537634409, -0.12544802867383512, -0.02867383512544803, -0.0, -0.1935483870967742, 0.010752688172043012, -0.2939068100358423, 0.014336917562724014, -0.3835125448028674, -0.03942652329749104, -0.1935483870967742, -0.03225806451612903, -0.2903225806451613, -0.02867383512544803, -0.37992831541218636, -0.017921146953405017, -0.13261648745519714], [-0.0, -0.0, -0.013071895424836602, -0.058823529411764705, 0.0196078431372549, -0.07516339869281045, 0.026143790849673203, -0.12745098039215685, 0.02287581699346405, -0.17647058823529413, -0.0457516339869281, -0.06209150326797386, -0.0915032679738562, -0.0718954248366013, -0.1111111111111111, -0.032679738562091505, -0.00980392156862745, -0.16993464052287582, -0.00980392156862745, -0.2581699346405229, -0.013071895424836602, -0.3366013071895425, -0.05228758169934641, -0.16339869281045752, -0.06535947712418301, -0.25163398692810457, -0.0784313725490196, -0.31699346405228757, -0.02287581699346405, -0.11764705882352941], [-0.0, -0.0, -0.008955223880597015, -0.04776119402985075, 0.01791044776119403, -0.06865671641791045, 0.020895522388059702, -0.11343283582089553, 0.020895522388059702, -0.15223880597014924, -0.03880597014925373, -0.056716417910447764, -0.06865671641791045, -0.06865671641791045, -0.09253731343283582, -0.026865671641791045, -0.008955223880597015, -0.14626865671641792, 0.0029850746268656717, -0.21791044776119403, 0.014925373134328358, -0.2835820895522388, -0.03582089552238806, -0.14626865671641792, -0.03582089552238806, -0.21791044776119403, -0.03880597014925373, -0.2865671641791045, -0.014925373134328358, -0.1044776119402985], [-0.0, -0.0, -0.0029154518950437317, -0.04956268221574344, 0.023323615160349854, -0.0641399416909621, 0.029154518950437316, -0.10495626822157435, 0.029154518950437316, -0.14285714285714285, -0.029154518950437316, -0.05539358600583091, -0.0641399416909621, -0.06705539358600583, -0.08163265306122448, -0.029154518950437316, -0.0, -0.14285714285714285, 0.0029154518950437317, -0.21282798833819241, 0.0029154518950437317, -0.27405247813411077, -0.029154518950437316, -0.13994169096209913, -0.026239067055393587, -0.2099125364431487, -0.026239067055393587, -0.26822157434402333, -0.008746355685131196, -0.09912536443148688], [-0.0, -0.0, -0.008064516129032258, -0.04032258064516129, 0.013440860215053764, -0.056451612903225805, 0.024193548387096774, -0.08870967741935484, 0.024193548387096774, -0.12365591397849462, -0.03225806451612903, -0.0456989247311828, -0.06720430107526881, -0.051075268817204304, -0.07795698924731183, -0.01881720430107527, 0.002688172043010753, -0.12634408602150538, 0.008064516129032258, -0.18548387096774194, 0.010752688172043012, -0.239247311827957, -0.026881720430107527, -0.12096774193548387, -0.03225806451612903, -0.1827956989247312, -0.03763440860215054, -0.23387096774193547, -0.010752688172043012, -0.08602150537634409], [-0.0, -0.0, -0.005988023952095809, -0.05089820359281437, 0.017964071856287425, -0.0718562874251497, 0.023952095808383235, -0.11377245508982035, 0.023952095808383235, -0.1467065868263473, -0.03592814371257485, -0.05688622754491018, -0.0748502994011976, -0.05688622754491018, -0.08083832335329341, -0.023952095808383235, -0.0, -0.1467065868263473, -0.0029940119760479044, -0.218562874251497, -0.014970059880239521, -0.2874251497005988, -0.029940119760479042, -0.1437125748502994, -0.023952095808383235, -0.218562874251497, -0.020958083832335328, -0.27844311377245506, -0.011976047904191617, -0.10479041916167664], [-0.0, -0.0, -0.008695652173913044, -0.043478260869565216, 0.017391304347826087, -0.06376811594202898, 0.02608695652173913, -0.1072463768115942, 0.02608695652173913, -0.14782608695652175, -0.04057971014492753, -0.04927536231884058, -0.0782608695652174, -0.057971014492753624, -0.08115942028985507, -0.014492753623188406, -0.002898550724637681, -0.14492753623188406, -0.0, -0.2144927536231884, -0.008695652173913044, -0.27246376811594203, -0.034782608695652174, -0.1391304347826087, -0.043478260869565216, -0.2144927536231884, -0.057971014492753624, -0.27246376811594203, -0.017391304347826087, -0.09855072463768116], [-0.0, -0.0, -0.013029315960912053, -0.05537459283387622, 0.016286644951140065, -0.08143322475570032, 0.026058631921824105, -0.13029315960912052, 0.035830618892508145, -0.1791530944625407, -0.04560260586319218, -0.05863192182410423, -0.09120521172638436, -0.06514657980456026, -0.09120521172638436, -0.013029315960912053, -0.003257328990228013, -0.17263843648208468, 0.013029315960912053, -0.2671009771986971, 0.016286644951140065, -0.34201954397394135, -0.04234527687296417, -0.17263843648208468, -0.035830618892508145, -0.26058631921824105, -0.03908794788273615, -0.3355048859934853, -0.019543973941368076, -0.12052117263843648], [-0.0, -0.0, -0.007168458781362007, -0.06451612903225806, 0.02867383512544803, -0.08960573476702509, 0.03942652329749104, -0.14695340501792115, 0.03942652329749104, -0.2007168458781362, -0.05017921146953405, -0.07168458781362007, -0.0967741935483871, -0.07168458781362007, -0.0931899641577061, -0.014336917562724014, -0.0, -0.2007168458781362, -0.007168458781362007, -0.30824372759856633, -0.007168458781362007, -0.3870967741935484, -0.043010752688172046, -0.2007168458781362, -0.04659498207885305, -0.3010752688172043, -0.06810035842293907, -0.3870967741935484, -0.017921146953405017, -0.13620071684587814], [-0.0, -0.0, -0.009009009009009009, -0.0945945945945946, 0.04054054054054054, -0.12612612612612611, 0.05855855855855856, -0.21621621621621623, 0.05855855855855856, -0.28378378378378377, -0.05855855855855856, -0.1036036036036036, -0.13513513513513514, -0.0990990990990991, -0.12612612612612611, -0.02252252252252252, 0.009009009009009009, -0.27927927927927926, 0.013513513513513514, -0.4189189189189189, 0.013513513513513514, -0.545045045045045, -0.04504504504504504, -0.2747747747747748, -0.04954954954954955, -0.4099099099099099, -0.05405405405405406, -0.527027027027027, -0.018018018018018018, -0.19369369369369369], [-0.0, -0.0, -0.016216216216216217, -0.11891891891891893, 0.05405405405405406, -0.16216216216216217, 0.08108108108108109, -0.2864864864864865, 0.08108108108108109, -0.372972972972973, -0.08648648648648649, -0.12972972972972974, -0.17297297297297298, -0.13513513513513514, -0.1783783783783784, -0.03783783783783784, 0.016216216216216217, -0.3567567567567568, 0.02702702702702703, -0.5243243243243243, 0.03783783783783784, -0.6756756756756757, -0.06486486486486487, -0.34594594594594597, -0.07567567567567568, -0.5243243243243243, -0.0918918918918919, -0.6756756756756757, -0.021621621621621623, -0.24864864864864866], [-0.0, -0.0, -0.02531645569620253, -0.13924050632911392, 0.06329113924050633, -0.1962025316455696, 0.08860759493670886, -0.34177215189873417, 0.08227848101265822, -0.4620253164556962, -0.11392405063291139, -0.1518987341772152, -0.21518987341772153, -0.15822784810126583, -0.24050632911392406, -0.05063291139240506, 0.012658227848101266, -0.4240506329113924, -0.012658227848101266, -0.6392405063291139, -0.03164556962025317, -0.8291139240506329, -0.0949367088607595, -0.4177215189873418, -0.13291139240506328, -0.6265822784810127, -0.14556962025316456, -0.7974683544303798, -0.03164556962025317, -0.2974683544303797], [-0.0, -0.0, -0.022727272727272728, -0.125, 0.056818181818181816, -0.16477272727272727, 0.07954545454545454, -0.2840909090909091, 0.07386363636363637, -0.3806818181818182, -0.09659090909090909, -0.13636363636363635, -0.1875, -0.1534090909090909, -0.20454545454545456, -0.05113636363636364, 0.005681818181818182, -0.3806818181818182, 0.011363636363636364, -0.5568181818181818, 0.022727272727272728, -0.7159090909090909, -0.07954545454545454, -0.3693181818181818, -0.06818181818181818, -0.5511363636363636, -0.0625, -0.7045454545454546, -0.028409090909090908, -0.26136363636363635], [-0.0, -0.0, -0.026881720430107527, -0.11290322580645161, 0.04838709677419355, -0.15053763440860216, 0.06451612903225806, -0.26344086021505375, 0.05913978494623656, -0.3655913978494624, -0.0913978494623656, -0.12365591397849462, -0.1774193548387097, -0.13440860215053763, -0.1881720430107527, -0.03763440860215054, -0.005376344086021506, -0.34946236559139787, -0.005376344086021506, -0.521505376344086, -0.005376344086021506, -0.6720430107526881, -0.08602150537634409, -0.3387096774193548, -0.11290322580645161, -0.5161290322580645, -0.13440860215053763, -0.6505376344086021, -0.03763440860215054, -0.23655913978494625], [-0.0, -0.0, -0.009389671361502348, -0.09859154929577464, 0.04225352112676056, -0.13615023474178403, 0.06103286384976526, -0.2347417840375587, 0.056338028169014086, -0.3004694835680751, -0.06572769953051644, -0.10328638497652583, -0.14084507042253522, -0.11737089201877934, -0.15492957746478872, -0.03286384976525822, 0.004694835680751174, -0.29107981220657275, 0.014084507042253521, -0.4413145539906103, 0.014084507042253521, -0.5586854460093896, -0.056338028169014086, -0.2863849765258216, -0.07042253521126761, -0.43661971830985913, -0.07981220657276995, -0.5492957746478874, -0.023474178403755867, -0.20187793427230047], [-0.0, -0.0, -0.014814814814814815, -0.06666666666666667, 0.025925925925925925, -0.09259259259259259, 0.037037037037037035, -0.15925925925925927, 0.03333333333333333, -0.21481481481481482, -0.05555555555555555, -0.07037037037037037, -0.10740740740740741, -0.07777777777777778, -0.11481481481481481, -0.018518518518518517, -0.003703703703703704, -0.2074074074074074, 0.003703703703703704, -0.31851851851851853, 0.007407407407407408, -0.4148148148148148, -0.05185185185185185, -0.2037037037037037, -0.05185185185185185, -0.3111111111111111, -0.05555555555555555, -0.4, -0.025925925925925925, -0.14074074074074075], [-0.0, -0.0, -0.00949367088607595, -0.05379746835443038, 0.0189873417721519, -0.07278481012658228, 0.028481012658227847, -0.12658227848101267, 0.022151898734177215, -0.16455696202531644, -0.04113924050632911, -0.05379746835443038, -0.07911392405063292, -0.06329113924050633, -0.0949367088607595, -0.015822784810126583, -0.00949367088607595, -0.16139240506329114, -0.0031645569620253164, -0.25, -0.006329113924050633, -0.31962025316455694, -0.04113924050632911, -0.15822784810126583, -0.04746835443037975, -0.2468354430379747, -0.05063291139240506, -0.3227848101265823, -0.0189873417721519, -0.11075949367088607], [-0.0, -0.0, -0.0084985835694051, -0.0453257790368272, 0.0169971671388102, -0.06232294617563739, 0.025495750708215296, -0.10198300283286119, 0.0226628895184136, -0.1388101983002833, -0.036827195467422094, -0.04815864022662889, -0.0708215297450425, -0.04815864022662889, -0.0764872521246459, -0.014164305949008499, -0.0028328611898017, -0.13314447592067988, 0.0056657223796034, -0.20396600566572237, 0.014164305949008499, -0.26912181303116145, -0.031161473087818695, -0.13031161473087818, -0.028328611898016998, -0.20113314447592068, -0.0226628895184136, -0.26062322946175637, -0.014164305949008499, -0.09348441926345609], [-0.0, -0.0, -0.014534883720930232, -0.046511627906976744, 0.014534883720930232, -0.06686046511627906, 0.023255813953488372, -0.11046511627906977, 0.020348837209302327, -0.14534883720930233, -0.0436046511627907, -0.04941860465116279, -0.0755813953488372, -0.05232558139534884, -0.08430232558139535, -0.01744186046511628, -0.00872093023255814, -0.14825581395348839, -0.014534883720930232, -0.2180232558139535, -0.020348837209302327, -0.27325581395348836, -0.0377906976744186, -0.14244186046511628, -0.0436046511627907, -0.21511627906976744, -0.055232558139534885, -0.26744186046511625, -0.020348837209302327, -0.10174418604651163], [-0.0, -0.0, -0.011976047904191617, -0.05089820359281437, 0.014970059880239521, -0.0688622754491018, 0.02694610778443114, -0.11377245508982035, 0.029940119760479042, -0.15269461077844312, -0.041916167664670656, -0.05389221556886228, -0.08383233532934131, -0.038922155688622756, -0.06287425149700598, 0.0029940119760479044, -0.0, -0.15269461077844312, 0.014970059880239521, -0.2245508982035928, 0.020958083832335328, -0.2904191616766467, -0.03592814371257485, -0.1497005988023952, -0.029940119760479042, -0.2245508982035928, -0.02694610778443114, -0.2874251497005988, -0.017964071856287425, -0.10778443113772455], [-0.0, -0.0, -0.003067484662576687, -0.049079754601226995, 0.024539877300613498, -0.0705521472392638, 0.03374233128834356, -0.12269938650306748, 0.03374233128834356, -0.1656441717791411, -0.03374233128834356, -0.049079754601226995, -0.0705521472392638, -0.03374233128834356, -0.06748466257668712, 0.012269938650306749, 0.003067484662576687, -0.15950920245398773, -0.006134969325153374, -0.2392638036809816, -0.012269938650306749, -0.3067484662576687, -0.03067484662576687, -0.15644171779141106, -0.03987730061349693, -0.2361963190184049, -0.05214723926380368, -0.3006134969325153, -0.009202453987730062, -0.10736196319018405], [-0.0, -0.0, -0.007722007722007722, -0.07722007722007722, 0.03088803088803089, -0.10424710424710425, 0.05019305019305019, -0.17374517374517376, 0.05405405405405406, -0.23166023166023167, -0.05019305019305019, -0.08108108108108109, -0.10810810810810811, -0.06177606177606178, -0.08494208494208494, -0.0, 0.011583011583011582, -0.2277992277992278, 0.023166023166023165, -0.3436293436293436, 0.03088803088803089, -0.4362934362934363, -0.03861003861003861, -0.22393822393822393, -0.03861003861003861, -0.3359073359073359, -0.04247104247104247, -0.416988416988417, -0.015444015444015444, -0.1583011583011583], [-0.0, -0.0, -0.0045045045045045045, -0.0945945945945946, 0.04504504504504504, -0.13063063063063063, 0.06306306306306306, -0.21621621621621623, 0.05405405405405406, -0.28378378378378377, -0.05405405405405406, -0.10810810810810811, -0.12612612612612611, -0.09009009009009009, -0.11261261261261261, -0.013513513513513514, 0.013513513513513514, -0.28378378378378377, -0.0, -0.4189189189189189, -0.0, -0.5180180180180181, -0.04504504504504504, -0.27927927927927926, -0.05855855855855856, -0.4189189189189189, -0.08108108108108109, -0.536036036036036, -0.013513513513513514, -0.1981981981981982], [-0.0, -0.0, -0.009345794392523364, -0.09345794392523364, 0.0514018691588785, -0.13551401869158877, 0.07009345794392523, -0.2383177570093458, 0.07009345794392523, -0.3130841121495327, -0.06074766355140187, -0.102803738317757, -0.14018691588785046, -0.08411214953271028, -0.13551401869158877, -0.004672897196261682, 0.014018691588785047, -0.29439252336448596, 0.018691588785046728, -0.4439252336448598, 0.028037383177570093, -0.5747663551401869, -0.0514018691588785, -0.2897196261682243, -0.04205607476635514, -0.43457943925233644, -0.037383177570093455, -0.5607476635514018, -0.014018691588785047, -0.20093457943925233], [-0.0, -0.0, -0.011363636363636364, -0.125, 0.0625, -0.17613636363636365, 0.08522727272727272, -0.3068181818181818, 0.07954545454545454, -0.4034090909090909, -0.07954545454545454, -0.14204545454545456, -0.17613636363636365, -0.11931818181818182, -0.17613636363636365, -0.022727272727272728, 0.017045454545454544, -0.375, 0.028409090909090908, -0.5738636363636364, 0.03977272727272727, -0.7215909090909091, -0.07386363636363637, -0.375, -0.09090909090909091, -0.5625, -0.10795454545454546, -0.6988636363636364, -0.022727272727272728, -0.26136363636363635], [-0.0, -0.0, -0.015463917525773196, -0.1134020618556701, 0.04639175257731959, -0.15463917525773196, 0.061855670103092786, -0.26288659793814434, 0.04639175257731959, -0.3402061855670103, -0.07731958762886598, -0.12886597938144329, -0.16494845360824742, -0.11855670103092783, -0.17010309278350516, -0.02577319587628866, -0.0, -0.33505154639175255, -0.010309278350515464, -0.4896907216494845, -0.010309278350515464, -0.6237113402061856, -0.06701030927835051, -0.32989690721649484, -0.06701030927835051, -0.4845360824742268, -0.061855670103092786, -0.6288659793814433, -0.030927835051546393, -0.23711340206185566], [-0.0, -0.0, -0.01293103448275862, -0.08620689655172414, 0.03879310344827586, -0.1206896551724138, 0.05172413793103448, -0.20689655172413793, 0.04741379310344827, -0.27586206896551724, -0.0603448275862069, -0.09482758620689655, -0.1206896551724138, -0.09482758620689655, -0.13793103448275862, -0.017241379310344827, -0.0, -0.25862068965517243, 0.01293103448275862, -0.3922413793103448, 0.03017241379310345, -0.5043103448275862, -0.06465517241379311, -0.2543103448275862, -0.0603448275862069, -0.3879310344827586, -0.07327586206896551, -0.4870689655172414, -0.02586206896551724, -0.1810344827586207], [-0.0, -0.0, -0.016597510373443983, -0.08298755186721991, 0.029045643153526972, -0.11618257261410789, 0.04564315352697095, -0.1950207468879668, 0.04149377593360996, -0.25311203319502074, -0.06224066390041494, -0.08713692946058091, -0.12033195020746888, -0.08713692946058091, -0.14107883817427386, -0.016597510373443983, -0.008298755186721992, -0.24481327800829875, 0.016597510373443983, -0.36929460580912865, 0.029045643153526972, -0.46887966804979253, -0.058091286307053944, -0.24066390041493776, -0.05394190871369295, -0.36099585062240663, -0.04979253112033195, -0.46887966804979253, -0.024896265560165973, -0.17012448132780084], [-0.0, -0.0, -0.006920415224913495, -0.058823529411764705, 0.031141868512110725, -0.07958477508650519, 0.04152249134948097, -0.14186851211072665, 0.04152249134948097, -0.1903114186851211, -0.04152249134948097, -0.06920415224913495, -0.08304498269896193, -0.0726643598615917, -0.10380622837370242, -0.020761245674740483, 0.006920415224913495, -0.18685121107266436, 0.010380622837370242, -0.2837370242214533, 0.01384083044982699, -0.36678200692041524, -0.03460207612456748, -0.18339100346020762, -0.03806228373702422, -0.28027681660899656, -0.03806228373702422, -0.35986159169550175, -0.01384083044982699, -0.12802768166089964], [-0.0, -0.0, -0.009202453987730062, -0.049079754601226995, 0.018404907975460124, -0.0705521472392638, 0.03067484662576687, -0.1196319018404908, 0.03374233128834356, -0.15950920245398773, -0.03680981595092025, -0.05521472392638037, -0.07668711656441718, -0.05828220858895705, -0.09202453987730061, -0.015337423312883436, -0.0, -0.15337423312883436, 0.012269938650306749, -0.2361963190184049, 0.027607361963190184, -0.30368098159509205, -0.03680981595092025, -0.15337423312883436, -0.03067484662576687, -0.2331288343558282, -0.024539877300613498, -0.29754601226993865, -0.015337423312883436, -0.10736196319018405], [-0.0, -0.0, -0.011940298507462687, -0.04776119402985075, 0.014925373134328358, -0.06865671641791045, 0.023880597014925373, -0.11044776119402985, 0.01791044776119403, -0.14626865671641792, -0.03880597014925373, -0.05373134328358209, -0.07761194029850746, -0.04477611940298507, -0.08955223880597014, -0.005970149253731343, -0.008955223880597015, -0.14626865671641792, -0.014925373134328358, -0.2208955223880597, -0.023880597014925373, -0.2865671641791045, -0.03880597014925373, -0.14029850746268657, -0.03880597014925373, -0.21492537313432836, -0.041791044776119404, -0.2865671641791045, -0.01791044776119403, -0.1044776119402985], [-0.0, -0.0, -0.01079136690647482, -0.0683453237410072, 0.025179856115107913, -0.09352517985611511, 0.039568345323741004, -0.1510791366906475, 0.04316546762589928, -0.2014388489208633, -0.046762589928057555, -0.07194244604316546, -0.10071942446043165, -0.0683453237410072, -0.11151079136690648, -0.01079136690647482, 0.007194244604316547, -0.2014388489208633, 0.025179856115107913, -0.3057553956834532, 0.03597122302158273, -0.38848920863309355, -0.039568345323741004, -0.2014388489208633, -0.03597122302158273, -0.29856115107913667, -0.03597122302158273, -0.3776978417266187, -0.014388489208633094, -0.14028776978417265], [-0.0, -0.0, -0.012448132780082987, -0.08298755186721991, 0.03319502074688797, -0.11618257261410789, 0.04979253112033195, -0.2033195020746888, 0.04564315352697095, -0.26141078838174275, -0.058091286307053944, -0.0912863070539419, -0.12448132780082988, -0.07883817427385892, -0.13278008298755187, -0.008298755186721992, 0.004149377593360996, -0.24896265560165975, -0.0, -0.36929460580912865, -0.0, -0.46473029045643155, -0.04979253112033195, -0.24481327800829875, -0.04979253112033195, -0.36929460580912865, -0.04979253112033195, -0.4730290456431535, -0.02074688796680498, -0.17427385892116182], [-0.0, -0.0, -0.014018691588785047, -0.09345794392523364, 0.04205607476635514, -0.1308411214953271, 0.06074766355140187, -0.22897196261682243, 0.06542056074766354, -0.308411214953271, -0.06542056074766354, -0.102803738317757, -0.13551401869158877, -0.09813084112149532, -0.14953271028037382, -0.018691588785046728, 0.009345794392523364, -0.2897196261682243, 0.009345794392523364, -0.4439252336448598, 0.004672897196261682, -0.5654205607476636, -0.056074766355140186, -0.2850467289719626, -0.0514018691588785, -0.43457943925233644, -0.056074766355140186, -0.5560747663551402, -0.02336448598130841, -0.20093457943925233], [-0.0, -0.0, -0.020512820512820513, -0.1076923076923077, 0.046153846153846156, -0.15384615384615385, 0.06666666666666667, -0.27692307692307694, 0.06153846153846154, -0.358974358974359, -0.08205128205128205, -0.11794871794871795, -0.17435897435897435, -0.10256410256410256, -0.18461538461538463, -0.015384615384615385, 0.005128205128205128, -0.3333333333333333, 0.010256410256410256, -0.49743589743589745, 0.015384615384615385, -0.6307692307692307, -0.07179487179487179, -0.3282051282051282, -0.09230769230769231, -0.48717948717948717, -0.11794871794871795, -0.6205128205128205, -0.03076923076923077, -0.23076923076923078], [-0.0, -0.0, -0.015384615384615385, -0.1076923076923077, 0.046153846153846156, -0.14871794871794872, 0.06666666666666667, -0.2564102564102564, 0.05641025641025641, -0.3333333333333333, -0.07692307692307693, -0.12307692307692308, -0.1641025641025641, -0.11794871794871795, -0.18461538461538463, -0.02564102564102564, 0.005128205128205128, -0.3282051282051282, 0.015384615384615385, -0.49230769230769234, 0.020512820512820513, -0.6307692307692307, -0.05641025641025641, -0.3282051282051282, -0.06153846153846154, -0.48717948717948717, -0.05641025641025641, -0.6205128205128205, -0.02564102564102564, -0.23076923076923078], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.04081632653061224, -0.1377551020408163, 0.0663265306122449, -0.24489795918367346, 0.0663265306122449, -0.33163265306122447, -0.08163265306122448, -0.11224489795918367, -0.1683673469387755, -0.10204081632653061, -0.17857142857142858, -0.01020408163265306, 0.01020408163265306, -0.32653061224489793, 0.02040816326530612, -0.4897959183673469, 0.030612244897959183, -0.6275510204081632, -0.061224489795918366, -0.32142857142857145, -0.061224489795918366, -0.4846938775510204, -0.0663265306122449, -0.6224489795918368, -0.025510204081632654, -0.22448979591836735], [-0.0, -0.0, -0.0106951871657754, -0.10695187165775401, 0.058823529411764705, -0.15508021390374332, 0.08021390374331551, -0.2727272727272727, 0.0855614973262032, -0.36363636363636365, -0.08021390374331551, -0.12299465240641712, -0.17647058823529413, -0.10695187165775401, -0.18181818181818182, -0.0106951871657754, 0.0213903743315508, -0.34759358288770054, 0.026737967914438502, -0.5240641711229946, 0.03208556149732621, -0.6684491978609626, -0.058823529411764705, -0.34759358288770054, -0.058823529411764705, -0.5187165775401069, -0.0748663101604278, -0.6684491978609626, -0.016042780748663103, -0.24064171122994651], [-0.0, -0.0, -0.011299435028248588, -0.11864406779661017, 0.062146892655367235, -0.1694915254237288, 0.0903954802259887, -0.2937853107344633, 0.0903954802259887, -0.3898305084745763, -0.0903954802259887, -0.13559322033898305, -0.1864406779661017, -0.11864406779661017, -0.192090395480226, -0.01694915254237288, 0.022598870056497175, -0.3728813559322034, 0.03389830508474576, -0.559322033898305, 0.03389830508474576, -0.711864406779661, -0.062146892655367235, -0.3728813559322034, -0.062146892655367235, -0.5536723163841808, -0.07909604519774012, -0.7062146892655368, -0.01694915254237288, -0.2598870056497175], [-0.0, -0.0, -0.022598870056497175, -0.11864406779661017, 0.04519774011299435, -0.1638418079096045, 0.06779661016949153, -0.2824858757062147, 0.07344632768361582, -0.384180790960452, -0.096045197740113, -0.12994350282485875, -0.1977401129943503, -0.11864406779661017, -0.2033898305084746, -0.011299435028248588, 0.005649717514124294, -0.3728813559322034, 0.022598870056497175, -0.559322033898305, 0.03389830508474576, -0.7175141242937854, -0.0847457627118644, -0.3728813559322034, -0.0847457627118644, -0.559322033898305, -0.1016949152542373, -0.711864406779661, -0.03389830508474576, -0.2598870056497175], [-0.0, -0.0, -0.0106951871657754, -0.10695187165775401, 0.058823529411764705, -0.15508021390374332, 0.08021390374331551, -0.27807486631016043, 0.08021390374331551, -0.3689839572192513, -0.08021390374331551, -0.12299465240641712, -0.17647058823529413, -0.11229946524064172, -0.18716577540106952, -0.016042780748663103, 0.016042780748663103, -0.34759358288770054, 0.0374331550802139, -0.5294117647058824, 0.0427807486631016, -0.6737967914438503, -0.06417112299465241, -0.34759358288770054, -0.06417112299465241, -0.5240641711229946, -0.0748663101604278, -0.6684491978609626, -0.0213903743315508, -0.24064171122994651], [-0.0, -0.0, -0.010752688172043012, -0.11290322580645161, 0.053763440860215055, -0.15591397849462366, 0.07526881720430108, -0.27419354838709675, 0.07526881720430108, -0.3655913978494624, -0.08602150537634409, -0.12365591397849462, -0.17204301075268819, -0.13440860215053763, -0.21505376344086022, -0.03763440860215054, 0.010752688172043012, -0.3548387096774194, 0.03225806451612903, -0.5268817204301075, 0.043010752688172046, -0.6720430107526881, -0.06989247311827956, -0.3548387096774194, -0.06989247311827956, -0.521505376344086, -0.08064516129032258, -0.6720430107526881, -0.026881720430107527, -0.24731182795698925], [-0.0, -0.0, -0.010752688172043012, -0.11290322580645161, 0.053763440860215055, -0.15591397849462366, 0.07526881720430108, -0.27419354838709675, 0.07526881720430108, -0.3655913978494624, -0.08602150537634409, -0.12365591397849462, -0.17204301075268819, -0.13440860215053763, -0.22043010752688172, -0.043010752688172046, 0.010752688172043012, -0.3548387096774194, 0.03225806451612903, -0.5268817204301075, 0.043010752688172046, -0.6720430107526881, -0.06989247311827956, -0.3548387096774194, -0.06989247311827956, -0.521505376344086, -0.08064516129032258, -0.6666666666666666, -0.026881720430107527, -0.24731182795698925], [-0.0, -0.0, -0.010752688172043012, -0.11290322580645161, 0.053763440860215055, -0.16129032258064516, 0.07526881720430108, -0.27956989247311825, 0.07526881720430108, -0.3655913978494624, -0.08602150537634409, -0.12365591397849462, -0.17204301075268819, -0.12903225806451613, -0.22043010752688172, -0.043010752688172046, 0.010752688172043012, -0.3548387096774194, 0.03225806451612903, -0.5268817204301075, 0.043010752688172046, -0.6720430107526881, -0.06989247311827956, -0.3548387096774194, -0.06989247311827956, -0.5268817204301075, -0.08064516129032258, -0.6720430107526881, -0.021505376344086023, -0.24731182795698925], [-0.0, -0.0, -0.010752688172043012, -0.11290322580645161, 0.053763440860215055, -0.15591397849462366, 0.07526881720430108, -0.27419354838709675, 0.07526881720430108, -0.3655913978494624, -0.08602150537634409, -0.12365591397849462, -0.17204301075268819, -0.12903225806451613, -0.22580645161290322, -0.03763440860215054, 0.010752688172043012, -0.3548387096774194, 0.03225806451612903, -0.5268817204301075, 0.043010752688172046, -0.6720430107526881, -0.06989247311827956, -0.3548387096774194, -0.06989247311827956, -0.521505376344086, -0.07526881720430108, -0.6720430107526881, -0.026881720430107527, -0.24731182795698925], [-0.0, -0.0, -0.010752688172043012, -0.11290322580645161, 0.053763440860215055, -0.15591397849462366, 0.07526881720430108, -0.27419354838709675, 0.07526881720430108, -0.3655913978494624, -0.08602150537634409, -0.12365591397849462, -0.17204301075268819, -0.12903225806451613, -0.23655913978494625, -0.03763440860215054, 0.010752688172043012, -0.3548387096774194, 0.03225806451612903, -0.5268817204301075, 0.043010752688172046, -0.6720430107526881, -0.06989247311827956, -0.3548387096774194, -0.06989247311827956, -0.521505376344086, -0.07526881720430108, -0.6720430107526881, -0.026881720430107527, -0.24731182795698925], [-0.0, -0.0, -0.016129032258064516, -0.11290322580645161, 0.04838709677419355, -0.16129032258064516, 0.06989247311827956, -0.27956989247311825, 0.06989247311827956, -0.3655913978494624, -0.0913978494623656, -0.12365591397849462, -0.1774193548387097, -0.12903225806451613, -0.24193548387096775, -0.043010752688172046, 0.005376344086021506, -0.3548387096774194, 0.026881720430107527, -0.5268817204301075, 0.03763440860215054, -0.6720430107526881, -0.07526881720430108, -0.3548387096774194, -0.07526881720430108, -0.521505376344086, -0.08602150537634409, -0.6720430107526881, -0.026881720430107527, -0.24731182795698925], [-0.0, -0.0, -0.010752688172043012, -0.11290322580645161, 0.053763440860215055, -0.15591397849462366, 0.07526881720430108, -0.27419354838709675, 0.07526881720430108, -0.3655913978494624, -0.08602150537634409, -0.12365591397849462, -0.1774193548387097, -0.13440860215053763, -0.23118279569892472, -0.053763440860215055, 0.010752688172043012, -0.3548387096774194, 0.03225806451612903, -0.5268817204301075, 0.043010752688172046, -0.6720430107526881, -0.06989247311827956, -0.3548387096774194, -0.06989247311827956, -0.521505376344086, -0.08064516129032258, -0.6720430107526881, -0.026881720430107527, -0.24193548387096775], [-0.0, -0.0, -0.016129032258064516, -0.11290322580645161, 0.053763440860215055, -0.15591397849462366, 0.07526881720430108, -0.26881720430107525, 0.07526881720430108, -0.3655913978494624, -0.08602150537634409, -0.12903225806451613, -0.17204301075268819, -0.15053763440860216, -0.22043010752688172, -0.06451612903225806, 0.010752688172043012, -0.3548387096774194, 0.03225806451612903, -0.5268817204301075, 0.043010752688172046, -0.6720430107526881, -0.06989247311827956, -0.3548387096774194, -0.06989247311827956, -0.521505376344086, -0.07526881720430108, -0.6720430107526881, -0.026881720430107527, -0.24193548387096775], [-0.0, -0.0, -0.016042780748663103, -0.10695187165775401, 0.053475935828877004, -0.15508021390374332, 0.06951871657754011, -0.26737967914438504, 0.0748663101604278, -0.36363636363636365, -0.0855614973262032, -0.12299465240641712, -0.1711229946524064, -0.15508021390374332, -0.2192513368983957, -0.06417112299465241, 0.0106951871657754, -0.35294117647058826, 0.03208556149732621, -0.5240641711229946, 0.0427807486631016, -0.6684491978609626, -0.06951871657754011, -0.35294117647058826, -0.06951871657754011, -0.5187165775401069, -0.0748663101604278, -0.6684491978609626, -0.026737967914438502, -0.24064171122994651], [-0.0, -0.0, -0.010752688172043012, -0.11290322580645161, 0.053763440860215055, -0.15591397849462366, 0.07526881720430108, -0.27419354838709675, 0.07526881720430108, -0.3655913978494624, -0.08602150537634409, -0.12365591397849462, -0.17204301075268819, -0.15053763440860216, -0.21505376344086022, -0.053763440860215055, 0.010752688172043012, -0.3548387096774194, 0.03225806451612903, -0.5268817204301075, 0.043010752688172046, -0.6720430107526881, -0.06989247311827956, -0.34946236559139787, -0.06989247311827956, -0.521505376344086, -0.07526881720430108, -0.6720430107526881, -0.026881720430107527, -0.24193548387096775], [-0.0, -0.0, -0.010752688172043012, -0.11290322580645161, 0.05913978494623656, -0.16129032258064516, 0.07526881720430108, -0.2849462365591398, 0.07526881720430108, -0.3709677419354839, -0.08064516129032258, -0.12365591397849462, -0.16666666666666666, -0.11827956989247312, -0.1989247311827957, -0.03225806451612903, 0.010752688172043012, -0.3548387096774194, 0.03225806451612903, -0.5268817204301075, 0.043010752688172046, -0.6720430107526881, -0.06989247311827956, -0.3548387096774194, -0.06989247311827956, -0.521505376344086, -0.07526881720430108, -0.6720430107526881, -0.021505376344086023, -0.24731182795698925], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.05102040816326531, -0.14795918367346939, 0.0663265306122449, -0.2602040816326531, 0.0663265306122449, -0.3469387755102041, -0.07653061224489796, -0.11734693877551021, -0.16326530612244897, -0.09183673469387756, -0.18877551020408162, -0.00510204081632653, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.03571428571428571, -0.6326530612244898, -0.07142857142857142, -0.32653061224489793, -0.07142857142857142, -0.4897959183673469, -0.07653061224489796, -0.6326530612244898, -0.025510204081632654, -0.22448979591836735], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.05102040816326531, -0.14795918367346939, 0.07142857142857142, -0.2602040816326531, 0.0663265306122449, -0.3469387755102041, -0.08163265306122448, -0.11734693877551021, -0.16326530612244897, -0.09693877551020408, -0.18877551020408162, -0.015306122448979591, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.03571428571428571, -0.6326530612244898, -0.07142857142857142, -0.32653061224489793, -0.07142857142857142, -0.4897959183673469, -0.08163265306122448, -0.6275510204081632, -0.025510204081632654, -0.22959183673469388], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.05102040816326531, -0.14795918367346939, 0.07142857142857142, -0.2602040816326531, 0.0663265306122449, -0.3469387755102041, -0.08163265306122448, -0.11734693877551021, -0.16326530612244897, -0.09693877551020408, -0.18877551020408162, -0.015306122448979591, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.03571428571428571, -0.6326530612244898, -0.07142857142857142, -0.32653061224489793, -0.07142857142857142, -0.4897959183673469, -0.07653061224489796, -0.6326530612244898, -0.025510204081632654, -0.22959183673469388], [-0.0, -0.0, -0.01020408163265306, -0.10204081632653061, 0.05612244897959184, -0.14795918367346939, 0.07142857142857142, -0.2602040816326531, 0.07142857142857142, -0.3469387755102041, -0.07653061224489796, -0.11734693877551021, -0.15306122448979592, -0.10204081632653061, -0.1836734693877551, -0.015306122448979591, 0.01020408163265306, -0.33163265306122447, 0.030612244897959183, -0.49489795918367346, 0.04081632653061224, -0.6326530612244898, -0.0663265306122449, -0.32653061224489793, -0.0663265306122449, -0.4897959183673469, -0.07142857142857142, -0.6326530612244898, -0.02040816326530612, -0.22959183673469388], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.05102040816326531, -0.14795918367346939, 0.07142857142857142, -0.2602040816326531, 0.0663265306122449, -0.3469387755102041, -0.08163265306122448, -0.11734693877551021, -0.16326530612244897, -0.09693877551020408, -0.18877551020408162, -0.015306122448979591, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.03571428571428571, -0.6326530612244898, -0.07142857142857142, -0.32653061224489793, -0.07142857142857142, -0.4897959183673469, -0.07653061224489796, -0.6275510204081632, -0.025510204081632654, -0.22959183673469388], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.05102040816326531, -0.14795918367346939, 0.07142857142857142, -0.2602040816326531, 0.0663265306122449, -0.3469387755102041, -0.08163265306122448, -0.11734693877551021, -0.16326530612244897, -0.10204081632653061, -0.18877551020408162, -0.02040816326530612, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.03571428571428571, -0.6326530612244898, -0.07142857142857142, -0.32653061224489793, -0.07142857142857142, -0.4897959183673469, -0.07653061224489796, -0.6326530612244898, -0.025510204081632654, -0.22959183673469388], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.05102040816326531, -0.14795918367346939, 0.07142857142857142, -0.2602040816326531, 0.0663265306122449, -0.3469387755102041, -0.08163265306122448, -0.11734693877551021, -0.16326530612244897, -0.10204081632653061, -0.18877551020408162, -0.02040816326530612, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.03571428571428571, -0.6326530612244898, -0.07142857142857142, -0.32653061224489793, -0.07142857142857142, -0.4897959183673469, -0.07653061224489796, -0.6275510204081632, -0.025510204081632654, -0.22959183673469388], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.05102040816326531, -0.14795918367346939, 0.07142857142857142, -0.2602040816326531, 0.0663265306122449, -0.3469387755102041, -0.08163265306122448, -0.11734693877551021, -0.15816326530612246, -0.10204081632653061, -0.18877551020408162, -0.02040816326530612, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.03571428571428571, -0.6326530612244898, -0.07142857142857142, -0.32653061224489793, -0.07142857142857142, -0.4897959183673469, -0.08163265306122448, -0.6326530612244898, -0.025510204081632654, -0.22959183673469388], [-0.0, -0.0, -0.022598870056497175, -0.11864406779661017, 0.04519774011299435, -0.1751412429378531, 0.062146892655367235, -0.2994350282485876, 0.062146892655367235, -0.3954802259887006, -0.096045197740113, -0.13559322033898305, -0.192090395480226, -0.12429378531073447, -0.22033898305084745, -0.02824858757062147, -0.005649717514124294, -0.3785310734463277, 0.01694915254237288, -0.559322033898305, 0.02824858757062147, -0.711864406779661, -0.0903954802259887, -0.3728813559322034, -0.0903954802259887, -0.5536723163841808, -0.1016949152542373, -0.7062146892655368, -0.03954802259887006, -0.2655367231638418], [-0.0, -0.0, -0.016042780748663103, -0.10695187165775401, 0.053475935828877004, -0.16042780748663102, 0.0748663101604278, -0.27807486631016043, 0.0748663101604278, -0.3689839572192513, -0.08021390374331551, -0.12834224598930483, -0.1711229946524064, -0.10160427807486631, -0.18181818181818182, -0.0, 0.0053475935828877, -0.35294117647058826, 0.03208556149732621, -0.5240641711229946, 0.0374331550802139, -0.6684491978609626, -0.0748663101604278, -0.34759358288770054, -0.0748663101604278, -0.5187165775401069, -0.08021390374331551, -0.6684491978609626, -0.026737967914438502, -0.24064171122994651], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.04591836734693878, -0.14285714285714285, 0.0663265306122449, -0.25510204081632654, 0.07142857142857142, -0.34183673469387754, -0.08163265306122448, -0.11224489795918367, -0.1683673469387755, -0.08673469387755102, -0.14285714285714285, 0.00510204081632653, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.030612244897959183, -0.6377551020408163, -0.07653061224489796, -0.32653061224489793, -0.07142857142857142, -0.49489795918367346, -0.08163265306122448, -0.6326530612244898, -0.030612244897959183, -0.22448979591836735], [-0.0, -0.0, -0.021505376344086023, -0.11290322580645161, 0.04838709677419355, -0.16129032258064516, 0.06989247311827956, -0.27419354838709675, 0.06989247311827956, -0.3709677419354839, -0.0913978494623656, -0.12365591397849462, -0.1774193548387097, -0.0967741935483871, -0.15053763440860216, -0.0, -0.0, -0.3548387096774194, 0.026881720430107527, -0.5268817204301075, 0.03225806451612903, -0.6720430107526881, -0.08064516129032258, -0.34946236559139787, -0.07526881720430108, -0.5268817204301075, -0.08602150537634409, -0.6720430107526881, -0.03225806451612903, -0.24193548387096775], [-0.0, -0.0, -0.016129032258064516, -0.11290322580645161, 0.053763440860215055, -0.16129032258064516, 0.07526881720430108, -0.27419354838709675, 0.07526881720430108, -0.3709677419354839, -0.08602150537634409, -0.12365591397849462, -0.17204301075268819, -0.0967741935483871, -0.14516129032258066, -0.0, 0.005376344086021506, -0.3548387096774194, 0.03225806451612903, -0.5268817204301075, 0.03763440860215054, -0.6720430107526881, -0.07526881720430108, -0.34946236559139787, -0.07526881720430108, -0.521505376344086, -0.08064516129032258, -0.6720430107526881, -0.026881720430107527, -0.24193548387096775], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.05102040816326531, -0.14285714285714285, 0.07142857142857142, -0.25510204081632654, 0.07142857142857142, -0.34183673469387754, -0.08163265306122448, -0.11224489795918367, -0.15816326530612246, -0.08163265306122448, -0.1377551020408163, -0.0, 0.01020408163265306, -0.33163265306122447, 0.030612244897959183, -0.49489795918367346, 0.03571428571428571, -0.6326530612244898, -0.07142857142857142, -0.32653061224489793, -0.07142857142857142, -0.4897959183673469, -0.07653061224489796, -0.6326530612244898, -0.025510204081632654, -0.22448979591836735], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.05102040816326531, -0.14285714285714285, 0.07142857142857142, -0.25510204081632654, 0.07653061224489796, -0.3469387755102041, -0.07653061224489796, -0.11224489795918367, -0.15306122448979592, -0.08673469387755102, -0.15306122448979592, -0.0, 0.01020408163265306, -0.32653061224489793, 0.030612244897959183, -0.49489795918367346, 0.03571428571428571, -0.6326530612244898, -0.07142857142857142, -0.32142857142857145, -0.07142857142857142, -0.4897959183673469, -0.07653061224489796, -0.6326530612244898, -0.025510204081632654, -0.22448979591836735], [-0.0, -0.0, -0.02040816326530612, -0.10204081632653061, 0.04591836734693878, -0.14285714285714285, 0.0663265306122449, -0.25510204081632654, 0.0663265306122449, -0.34183673469387754, -0.08163265306122448, -0.11224489795918367, -0.16326530612244897, -0.08673469387755102, -0.16326530612244897, 0.00510204081632653, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.030612244897959183, -0.6326530612244898, -0.07142857142857142, -0.32653061224489793, -0.07653061224489796, -0.4897959183673469, -0.08163265306122448, -0.6326530612244898, -0.030612244897959183, -0.22448979591836735], [-0.0, -0.0, -0.010752688172043012, -0.11290322580645161, 0.053763440860215055, -0.15591397849462366, 0.08064516129032258, -0.27419354838709675, 0.08064516129032258, -0.3655913978494624, -0.08064516129032258, -0.12903225806451613, -0.16666666666666666, -0.10215053763440861, -0.17204301075268819, -0.005376344086021506, 0.010752688172043012, -0.3548387096774194, 0.03225806451612903, -0.5268817204301075, 0.03763440860215054, -0.6774193548387096, -0.06989247311827956, -0.34946236559139787, -0.06989247311827956, -0.5268817204301075, -0.08064516129032258, -0.6720430107526881, -0.026881720430107527, -0.24193548387096775], [-0.0, -0.0, -0.010752688172043012, -0.11290322580645161, 0.053763440860215055, -0.15591397849462366, 0.07526881720430108, -0.27419354838709675, 0.08064516129032258, -0.3655913978494624, -0.08064516129032258, -0.12365591397849462, -0.16666666666666666, -0.10215053763440861, -0.1774193548387097, -0.010752688172043012, 0.010752688172043012, -0.3548387096774194, 0.03225806451612903, -0.5268817204301075, 0.03763440860215054, -0.6774193548387096, -0.06989247311827956, -0.34946236559139787, -0.06989247311827956, -0.5268817204301075, -0.08064516129032258, -0.6720430107526881, -0.026881720430107527, -0.24193548387096775], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.04591836734693878, -0.14285714285714285, 0.07142857142857142, -0.25510204081632654, 0.07142857142857142, -0.34183673469387754, -0.08163265306122448, -0.11734693877551021, -0.15816326530612246, -0.08673469387755102, -0.19387755102040816, -0.01020408163265306, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.030612244897959183, -0.6326530612244898, -0.07142857142857142, -0.32653061224489793, -0.07142857142857142, -0.49489795918367346, -0.08163265306122448, -0.6326530612244898, -0.025510204081632654, -0.22959183673469388], [-0.0, -0.0, -0.02040816326530612, -0.10204081632653061, 0.04591836734693878, -0.14285714285714285, 0.0663265306122449, -0.25, 0.0663265306122449, -0.34183673469387754, -0.08163265306122448, -0.11734693877551021, -0.15816326530612246, -0.08673469387755102, -0.20408163265306123, -0.02040816326530612, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.030612244897959183, -0.6326530612244898, -0.07142857142857142, -0.32653061224489793, -0.07142857142857142, -0.49489795918367346, -0.08163265306122448, -0.6326530612244898, -0.030612244897959183, -0.22959183673469388], [-0.0, -0.0, -0.02040816326530612, -0.10204081632653061, 0.04591836734693878, -0.14285714285714285, 0.0663265306122449, -0.25, 0.07142857142857142, -0.34183673469387754, -0.08163265306122448, -0.11734693877551021, -0.15816326530612246, -0.09693877551020408, -0.20408163265306123, -0.025510204081632654, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.030612244897959183, -0.6377551020408163, -0.07142857142857142, -0.32653061224489793, -0.07142857142857142, -0.49489795918367346, -0.08163265306122448, -0.6326530612244898, -0.030612244897959183, -0.22959183673469388], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.05102040816326531, -0.14285714285714285, 0.07653061224489796, -0.25510204081632654, 0.07653061224489796, -0.34183673469387754, -0.07653061224489796, -0.11734693877551021, -0.15306122448979592, -0.09183673469387756, -0.1989795918367347, -0.015306122448979591, 0.01020408163265306, -0.33163265306122447, 0.030612244897959183, -0.49489795918367346, 0.03571428571428571, -0.6377551020408163, -0.0663265306122449, -0.32653061224489793, -0.0663265306122449, -0.49489795918367346, -0.07653061224489796, -0.6326530612244898, -0.02040816326530612, -0.22959183673469388], [-0.0, -0.0, -0.02040816326530612, -0.10204081632653061, 0.04591836734693878, -0.14285714285714285, 0.07142857142857142, -0.25510204081632654, 0.07142857142857142, -0.34183673469387754, -0.08163265306122448, -0.11734693877551021, -0.15816326530612246, -0.09183673469387756, -0.20408163265306123, -0.02040816326530612, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.030612244897959183, -0.6326530612244898, -0.07142857142857142, -0.32653061224489793, -0.07142857142857142, -0.49489795918367346, -0.08163265306122448, -0.6326530612244898, -0.030612244897959183, -0.22959183673469388], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.05102040816326531, -0.14285714285714285, 0.07653061224489796, -0.25510204081632654, 0.07142857142857142, -0.34183673469387754, -0.07653061224489796, -0.11734693877551021, -0.15306122448979592, -0.09183673469387756, -0.20408163265306123, -0.025510204081632654, 0.01020408163265306, -0.33163265306122447, 0.030612244897959183, -0.49489795918367346, 0.03571428571428571, -0.6326530612244898, -0.0663265306122449, -0.32653061224489793, -0.0663265306122449, -0.49489795918367346, -0.07653061224489796, -0.6326530612244898, -0.02040816326530612, -0.22959183673469388], [-0.0, -0.0, -0.02040816326530612, -0.10204081632653061, 0.04591836734693878, -0.14285714285714285, 0.0663265306122449, -0.25, 0.0663265306122449, -0.34183673469387754, -0.08163265306122448, -0.11734693877551021, -0.15816326530612246, -0.09183673469387756, -0.20918367346938777, -0.02040816326530612, -0.0, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.030612244897959183, -0.6326530612244898, -0.07142857142857142, -0.32653061224489793, -0.07142857142857142, -0.49489795918367346, -0.08163265306122448, -0.6326530612244898, -0.030612244897959183, -0.22959183673469388], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.05102040816326531, -0.14795918367346939, 0.07142857142857142, -0.2602040816326531, 0.07142857142857142, -0.34183673469387754, -0.07653061224489796, -0.11734693877551021, -0.15306122448979592, -0.09693877551020408, -0.20408163265306123, -0.025510204081632654, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.03571428571428571, -0.6326530612244898, -0.07142857142857142, -0.33163265306122447, -0.07142857142857142, -0.4897959183673469, -0.07653061224489796, -0.6326530612244898, -0.025510204081632654, -0.22959183673469388], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.05102040816326531, -0.14795918367346939, 0.07142857142857142, -0.25510204081632654, 0.07142857142857142, -0.34183673469387754, -0.08163265306122448, -0.11734693877551021, -0.15816326530612246, -0.09693877551020408, -0.20918367346938777, -0.025510204081632654, 0.00510204081632653, -0.33163265306122447, 0.025510204081632654, -0.49489795918367346, 0.03571428571428571, -0.6326530612244898, -0.07142857142857142, -0.33163265306122447, -0.07142857142857142, -0.4897959183673469, -0.07653061224489796, -0.6275510204081632, -0.025510204081632654, -0.22959183673469388], [-0.0, -0.0, -0.015306122448979591, -0.10204081632653061, 0.04591836734693878, -0.14795918367346939, 0.0663265306122449, -0.2602040816326531, 0.0663265306122449, -0.34183673469387754, -0.08163265306122448, -0.11734693877551021, -0.15816326530612246, -0.10714285714285714, -0.21428571428571427, -0.030612244897959183, -0.0, -0.336734693877551, 0.025510204081632654, -0.49489795918367346, 0.03571428571428571, -0.6326530612244898, -0.07142857142857142, -0.33163265306122447, -0.07142857142857142, -0.49489795918367346, -0.08163265306122448, -0.6326530612244898, -0.030612244897959183, -0.22959183673469388], [-0.0, -0.0, -0.01020408163265306, -0.10204081632653061, 0.05102040816326531, -0.14795918367346939, 0.07142857142857142, -0.25510204081632654, 0.07142857142857142, -0.34183673469387754, -0.07653061224489796, -0.11734693877551021, -0.15816326530612246, -0.12244897959183673, -0.20408163265306123, -0.04591836734693878, 0.01020408163265306, -0.33163265306122447, 0.030612244897959183, -0.49489795918367346, 0.04081632653061224, -0.6326530612244898, -0.0663265306122449, -0.33163265306122447, -0.0663265306122449, -0.49489795918367346, -0.07653061224489796, -0.6326530612244898, -0.025510204081632654, -0.22959183673469388], [-0.0, -0.0, -0.015384615384615385, -0.1076923076923077, 0.046153846153846156, -0.14871794871794872, 0.06666666666666667, -0.2564102564102564, 0.06666666666666667, -0.3435897435897436, -0.08205128205128205, -0.11794871794871795, -0.15897435897435896, -0.13846153846153847, -0.20512820512820512, -0.05641025641025641, 0.005128205128205128, -0.3333333333333333, 0.02564102564102564, -0.49743589743589745, 0.035897435897435895, -0.6358974358974359, -0.07179487179487179, -0.3333333333333333, -0.07179487179487179, -0.49743589743589745, -0.07692307692307693, -0.6358974358974359, -0.03076923076923077, -0.23076923076923078],[-0.0, -0.0, -0.0029154518950437317, -0.04956268221574344, 0.02040816326530612, -0.06705539358600583, 0.01749271137026239, -0.11078717201166181, -0.026239067055393587, -0.11370262390670553, -0.026239067055393587, -0.061224489795918366, -0.03498542274052478, -0.10204081632653061, -0.037900874635568516, -0.11078717201166181, 0.0058309037900874635, -0.14868804664723032, 0.008746355685131196, -0.21574344023323616, 0.011661807580174927, -0.282798833819242, -0.023323615160349854, -0.14285714285714285, -0.037900874635568516, -0.2099125364431487, -0.04664723032069971, -0.26822157434402333, -0.0058309037900874635, -0.10495626822157435], [-0.0, -0.0, -0.005390835579514825, -0.0431266846361186, 0.018867924528301886, -0.05660377358490566, 0.02425876010781671, -0.0889487870619946, 0.01078167115902965, -0.09973045822102426, -0.03234501347708895, -0.05390835579514825, -0.03773584905660377, -0.09164420485175202, -0.01078167115902965, -0.10242587601078167, 0.008086253369272238, -0.12668463611859837, 0.01078167115902965, -0.18328840970350405, 0.016172506738544475, -0.2371967654986523, -0.0215633423180593, -0.12398921832884097, -0.0215633423180593, -0.18059299191374664, -0.018867924528301886, -0.2371967654986523, -0.008086253369272238, -0.0889487870619946], [-0.0, -0.0, -0.015306122448979591, -0.03316326530612245, -0.0, -0.05102040816326531, 0.00510204081632653, -0.08163265306122448, 0.007653061224489796, -0.09693877551020408, -0.03571428571428571, -0.04336734693877551, -0.04591836734693878, -0.07653061224489796, -0.01020408163265306, -0.09693877551020408, -0.015306122448979591, -0.10714285714285714, -0.015306122448979591, -0.16071428571428573, -0.017857142857142856, -0.2193877551020408, -0.03316326530612245, -0.10714285714285714, -0.02295918367346939, -0.16581632653061223, -0.02040816326530612, -0.22193877551020408, -0.02040816326530612, -0.07908163265306123], [-0.0, -0.0, -0.0029069767441860465, -0.046511627906976744, 0.01744186046511628, -0.061046511627906974, 0.03488372093023256, -0.10174418604651163, 0.0377906976744186, -0.11918604651162791, -0.029069767441860465, -0.06395348837209303, -0.029069767441860465, -0.10465116279069768, 0.01744186046511628, -0.12209302325581395, 0.020348837209302327, -0.14244186046511628, 0.03488372093023256, -0.2063953488372093, 0.046511627906976744, -0.27325581395348836, -0.00872093023255814, -0.14244186046511628, -0.005813953488372093, -0.21220930232558138, -0.005813953488372093, -0.2819767441860465, -0.0, -0.10174418604651163], [-0.0, -0.0, -0.008746355685131196, -0.04956268221574344, 0.01749271137026239, -0.06705539358600583, 0.026239067055393587, -0.10787172011661808, -0.0, -0.119533527696793, -0.037900874635568516, -0.05539358600583091, -0.04664723032069971, -0.10204081632653061, -0.008746355685131196, -0.11370262390670553, 0.0029154518950437317, -0.1457725947521866, -0.014577259475218658, -0.21574344023323616, -0.029154518950437316, -0.27696793002915454, -0.03206997084548105, -0.13702623906705538, -0.03498542274052478, -0.21282798833819241, -0.04956268221574344, -0.27696793002915454, -0.011661807580174927, -0.10204081632653061], [-0.0, -0.0, -0.0034602076124567475, -0.058823529411764705, 0.031141868512110725, -0.07958477508650519, 0.04844290657439446, -0.1314878892733564, 0.0657439446366782, -0.1384083044982699, -0.03460207612456748, -0.08304498269896193, -0.02422145328719723, -0.13494809688581316, 0.03460207612456748, -0.1453287197231834, 0.02422145328719723, -0.18339100346020762, 0.03460207612456748, -0.2768166089965398, 0.010380622837370242, -0.35986159169550175, -0.01384083044982699, -0.18339100346020762, -0.0034602076124567475, -0.28027681660899656, -0.010380622837370242, -0.3633217993079585, 0.0034602076124567475, -0.1314878892733564], [-0.0, -0.0, -0.004132231404958678, -0.07851239669421488, 0.04132231404958678, -0.1115702479338843, 0.07024793388429752, -0.18181818181818182, 0.04132231404958678, -0.2024793388429752, -0.05785123966942149, -0.09917355371900827, -0.06198347107438017, -0.17768595041322313, -0.0, -0.19834710743801653, 0.02066115702479339, -0.24793388429752067, 0.01652892561983471, -0.371900826446281, 0.008264462809917356, -0.4834710743801653, -0.04132231404958678, -0.2396694214876033, -0.04132231404958678, -0.36363636363636365, -0.06611570247933884, -0.47107438016528924, -0.008264462809917356, -0.17355371900826447], [-0.0, -0.0, 0.004149377593360996, -0.08298755186721991, 0.05394190871369295, -0.1078838174273859, 0.07883817427385892, -0.1908713692946058, 0.04564315352697095, -0.2074688796680498, -0.04979253112033195, -0.1037344398340249, -0.05394190871369295, -0.1825726141078838, -0.0, -0.2033195020746888, 0.02074688796680498, -0.2572614107883817, 0.008298755186721992, -0.38589211618257263, -0.008298755186721992, -0.48132780082987553, -0.03734439834024896, -0.24896265560165975, -0.06639004149377593, -0.3775933609958506, -0.1037344398340249, -0.4854771784232365, -0.004149377593360996, -0.1825726141078838], [-0.0, -0.0, 0.011299435028248588, -0.11864406779661017, 0.0847457627118644, -0.1638418079096045, 0.11299435028248588, -0.2768361581920904, 0.03954802259887006, -0.2937853107344633, -0.05649717514124294, -0.1638418079096045, -0.0847457627118644, -0.2711864406779661, -0.03389830508474576, -0.288135593220339, 0.05084745762711865, -0.3672316384180791, 0.02824858757062147, -0.559322033898305, 0.022598870056497175, -0.7344632768361582, -0.02824858757062147, -0.3615819209039548, -0.011299435028248588, -0.5536723163841808, 0.005649717514124294, -0.7062146892655368, 0.011299435028248588, -0.2598870056497175], [-0.0, -0.0, 0.011299435028248588, -0.11864406779661017, 0.096045197740113, -0.1638418079096045, 0.10734463276836158, -0.2711864406779661, 0.02824858757062147, -0.2937853107344633, -0.062146892655367235, -0.1638418079096045, -0.10734463276836158, -0.2542372881355932, -0.062146892655367235, -0.2824858757062147, 0.05649717514124294, -0.3785310734463277, 0.0903954802259887, -0.5649717514124294, 0.14124293785310735, -0.7344632768361582, -0.03954802259887006, -0.3785310734463277, -0.05084745762711865, -0.5649717514124294, -0.05649717514124294, -0.711864406779661, 0.011299435028248588, -0.2655367231638418], [-0.0, -0.0, -0.008583690987124463, -0.0815450643776824, 0.03862660944206009, -0.11587982832618025, 0.060085836909871244, -0.18025751072961374, 0.02145922746781116, -0.20600858369098712, -0.06866952789699571, -0.1072961373390558, -0.0944206008583691, -0.18025751072961374, -0.04291845493562232, -0.19742489270386265, -0.0, -0.2575107296137339, -0.012875536480686695, -0.38626609442060084, -0.017167381974248927, -0.5064377682403434, -0.060085836909871244, -0.24892703862660945, -0.04291845493562232, -0.38626609442060084, -0.03862660944206009, -0.5064377682403434, -0.02145922746781116, -0.18025751072961374], [-0.0, -0.0, -0.0035842293906810036, -0.06451612903225806, 0.03942652329749104, -0.08602150537634409, 0.05734767025089606, -0.13978494623655913, 0.025089605734767026, -0.15770609318996415, -0.04659498207885305, -0.08243727598566308, -0.06451612903225806, -0.14336917562724014, -0.014336917562724014, -0.15770609318996415, 0.014336917562724014, -0.20430107526881722, 0.03225806451612903, -0.3046594982078853, 0.05734767025089606, -0.3978494623655914, -0.03225806451612903, -0.2007168458781362, -0.03225806451612903, -0.2974910394265233, -0.02867383512544803, -0.3763440860215054, -0.007168458781362007, -0.14336917562724014], [-0.0, -0.0, -0.003257328990228013, -0.05537459283387622, 0.03257328990228013, -0.07166123778501629, 0.04560260586319218, -0.12052117263843648, 0.016286644951140065, -0.14006514657980457, -0.04234527687296417, -0.07166123778501629, -0.05537459283387622, -0.1237785016286645, -0.019543973941368076, -0.14006514657980457, 0.013029315960912053, -0.17263843648208468, 0.013029315960912053, -0.254071661237785, 0.009771986970684038, -0.3257328990228013, -0.029315960912052116, -0.16938110749185667, -0.026058631921824105, -0.254071661237785, -0.026058631921824105, -0.3257328990228013, -0.006514657980456026, -0.1237785016286645], [-0.0, -0.0, -0.0056657223796034, -0.0453257790368272, 0.0226628895184136, -0.06232294617563739, 0.036827195467422094, -0.09915014164305949, 0.014164305949008499, -0.11331444759206799, -0.0339943342776204, -0.056657223796033995, -0.04815864022662889, -0.09631728045325778, -0.031161473087818695, -0.12181303116147309, 0.0084985835694051, -0.1359773371104816, 0.0113314447592068, -0.20113314447592068, 0.0169971671388102, -0.26628895184135976, -0.025495750708215296, -0.13314447592067988, -0.0226628895184136, -0.19830028328611898, -0.025495750708215296, -0.2577903682719547, -0.0084985835694051, -0.09631728045325778], [-0.0, -0.0, -0.005249343832020997, -0.03937007874015748, 0.015748031496062992, -0.05511811023622047, 0.028871391076115485, -0.08661417322834646, 0.015748031496062992, -0.10236220472440945, -0.028871391076115485, -0.049868766404199474, -0.047244094488188976, -0.08136482939632546, -0.03412073490813648, -0.10236220472440945, -0.0, -0.11811023622047244, -0.0026246719160104987, -0.1732283464566929, -0.005249343832020997, -0.2283464566929134, -0.026246719160104987, -0.11548556430446194, -0.028871391076115485, -0.1732283464566929, -0.03674540682414698, -0.2283464566929134, -0.010498687664041995, -0.08398950131233596], [-0.0, -0.0, -0.010256410256410256, -0.038461538461538464, 0.007692307692307693, -0.05384615384615385, 0.017948717948717947, -0.08461538461538462, 0.010256410256410256, -0.09743589743589744, -0.03333333333333333, -0.046153846153846156, -0.04358974358974359, -0.08205128205128205, -0.015384615384615385, -0.1, -0.007692307692307693, -0.11025641025641025, -0.01282051282051282, -0.16153846153846155, -0.020512820512820513, -0.2153846153846154, -0.02564102564102564, -0.11025641025641025, -0.023076923076923078, -0.16153846153846155, -0.02564102564102564, -0.21025641025641026, -0.015384615384615385, -0.07948717948717948], [-0.0, -0.0, -0.007159904534606206, -0.031026252983293555, 0.00954653937947494, -0.0405727923627685, 0.026252983293556086, -0.06682577565632458, 0.016706443914081145, -0.08591885441527446, -0.028639618138424822, -0.03818615751789976, -0.0405727923627685, -0.06682577565632458, -0.016706443914081145, -0.08353221957040573, 0.002386634844868735, -0.09307875894988067, 0.002386634844868735, -0.14558472553699284, -0.00954653937947494, -0.1957040572792363, -0.021479713603818614, -0.09307875894988067, -0.028639618138424822, -0.14558472553699284, -0.045346062052505964, -0.19809069212410502, -0.00954653937947494, -0.06682577565632458], [-0.0, -0.0, -0.009569377990430622, -0.03349282296650718, 0.009569377990430622, -0.045454545454545456, 0.0215311004784689, -0.07177033492822966, 0.019138755980861243, -0.08851674641148326, -0.02631578947368421, -0.04066985645933014, -0.04066985645933014, -0.06698564593301436, -0.014354066985645933, -0.08851674641148326, -0.0023923444976076554, -0.09330143540669857, -0.004784688995215311, -0.145933014354067, -0.014354066985645933, -0.19856459330143542, -0.0215311004784689, -0.09330143540669857, -0.01674641148325359, -0.14832535885167464, -0.019138755980861243, -0.19856459330143542, -0.011961722488038277, -0.06698564593301436], [-0.0, -0.0, -0.0027624309392265192, -0.04419889502762431, 0.022099447513812154, -0.058011049723756904, 0.03867403314917127, -0.09116022099447514, 0.03038674033149171, -0.1132596685082873, -0.024861878453038673, -0.055248618784530384, -0.04419889502762431, -0.09668508287292818, -0.0055248618784530384, -0.11602209944751381, 0.013812154696132596, -0.1270718232044199, 0.022099447513812154, -0.1878453038674033, 0.024861878453038673, -0.24585635359116023, -0.016574585635359115, -0.1270718232044199, -0.022099447513812154, -0.19060773480662985, -0.024861878453038673, -0.2513812154696133, -0.0, -0.09392265193370165], [-0.0, -0.0, -0.0, -0.0453257790368272, 0.028328611898016998, -0.059490084985835696, 0.0453257790368272, -0.09915014164305949, 0.0339943342776204, -0.13314447592067988, -0.025495750708215296, -0.056657223796033995, -0.042492917847025496, -0.09631728045325778, -0.019830028328611898, -0.12464589235127478, 0.0113314447592068, -0.13314447592067988, 0.0028328611898017, -0.20396600566572237, -0.0113314447592068, -0.26628895184135976, -0.0169971671388102, -0.13031161473087818, -0.014164305949008499, -0.20396600566572237, -0.019830028328611898, -0.26912181303116145, -0.0, -0.09631728045325778], [-0.0, -0.0, -0.003257328990228013, -0.05537459283387622, 0.029315960912052116, -0.0781758957654723, 0.05211726384364821, -0.1237785016286645, 0.048859934853420196, -0.1498371335504886, -0.035830618892508145, -0.0749185667752443, -0.05863192182410423, -0.1237785016286645, -0.009771986970684038, -0.1498371335504886, 0.02280130293159609, -0.17263843648208468, 0.03257328990228013, -0.26058631921824105, 0.04234527687296417, -0.34201954397394135, -0.019543973941368076, -0.17263843648208468, -0.019543973941368076, -0.254071661237785, -0.019543973941368076, -0.34201954397394135, -0.0, -0.1237785016286645], [-0.0, -0.0, -0.007434944237918215, -0.07063197026022305, 0.02973977695167286, -0.09665427509293681, 0.05204460966542751, -0.1524163568773234, 0.02973977695167286, -0.18587360594795538, -0.048327137546468404, -0.08550185873605948, -0.07806691449814127, -0.1524163568773234, -0.026022304832713755, -0.17843866171003717, 0.007434944237918215, -0.21189591078066913, -0.0037174721189591076, -0.32342007434944237, -0.01486988847583643, -0.40892193308550184, -0.04460966542750929, -0.20074349442379183, -0.05204460966542751, -0.3159851301115242, -0.07806691449814127, -0.41263940520446096, -0.01486988847583643, -0.14869888475836432], [-0.0, -0.0, 0.004901960784313725, -0.10294117647058823, 0.06372549019607843, -0.13725490196078433, 0.09803921568627451, -0.21568627450980393, 0.07352941176470588, -0.27450980392156865, -0.058823529411764705, -0.13725490196078433, -0.07352941176470588, -0.23529411764705882, 0.004901960784313725, -0.25980392156862747, 0.05392156862745098, -0.3088235294117647, 0.07352941176470588, -0.45588235294117646, 0.06372549019607843, -0.5833333333333334, -0.024509803921568627, -0.3088235294117647, -0.00980392156862745, -0.45588235294117646, -0.004901960784313725, -0.5980392156862745, 0.00980392156862745, -0.22549019607843138], [-0.0, -0.0, -0.0, -0.1317365269461078, 0.07784431137724551, -0.17964071856287425, 0.11976047904191617, -0.2994011976047904, 0.0658682634730539, -0.3413173652694611, -0.08982035928143713, -0.17365269461077845, -0.11377245508982035, -0.2934131736526946, -0.017964071856287425, -0.32335329341317365, 0.04790419161676647, -0.40718562874251496, 0.029940119760479042, -0.6167664670658682, 0.011976047904191617, -0.7964071856287425, -0.05389221556886228, -0.40119760479041916, -0.0658682634730539, -0.5988023952095808, -0.10179640718562874, -0.7544910179640718, -0.005988023952095809, -0.2874251497005988], [-0.0, -0.0, -0.006666666666666667, -0.14, 0.09333333333333334, -0.2, 0.15333333333333332, -0.32, 0.12, -0.4066666666666667, -0.10666666666666667, -0.19333333333333333, -0.16, -0.31333333333333335, -0.08, -0.37333333333333335, 0.05333333333333334, -0.4533333333333333, 0.04666666666666667, -0.7, 0.06, -0.9066666666666666, -0.06666666666666667, -0.4533333333333333, -0.09333333333333334, -0.6933333333333334, -0.13333333333333333, -0.9066666666666666, -0.006666666666666667, -0.32666666666666666], [-0.0, -0.0, -0.0, -0.22330097087378642, 0.14563106796116504, -0.32038834951456313, 0.23300970873786409, -0.5145631067961165, 0.17475728155339806, -0.6116504854368932, -0.1553398058252427, -0.2912621359223301, -0.2621359223300971, -0.4854368932038835, -0.18446601941747573, -0.5631067961165048, 0.06796116504854369, -0.6893203883495146, 0.02912621359223301, -1.058252427184466, -0.0, -1.3786407766990292, -0.10679611650485436, -0.6893203883495146, -0.0970873786407767, -1.0485436893203883, -0.08737864077669903, -1.3592233009708738, -0.009708737864077669, -0.5048543689320388], [-0.0, -0.0, -0.0, -0.19469026548672566, 0.1415929203539823, -0.2831858407079646, 0.20353982300884957, -0.46017699115044247, 0.1592920353982301, -0.5752212389380531, -0.1415929203539823, -0.26548672566371684, -0.23008849557522124, -0.4424778761061947, -0.1415929203539823, -0.5309734513274337, 0.07079646017699115, -0.6283185840707964, 0.12389380530973451, -0.9734513274336283, 0.168141592920354, -1.247787610619469, -0.07964601769911504, -0.6283185840707964, -0.07964601769911504, -0.9646017699115044, -0.07079646017699115, -1.2300884955752212, -0.008849557522123894, -0.45132743362831856], [-0.0, -0.0, -0.006666666666666667, -0.14, 0.09333333333333334, -0.21333333333333335, 0.09333333333333334, -0.35333333333333333, 0.013333333333333334, -0.37333333333333335, -0.11333333333333333, -0.18666666666666668, -0.16666666666666666, -0.31333333333333335, -0.08666666666666667, -0.36, 0.013333333333333334, -0.46, -0.006666666666666667, -0.6933333333333334, -0.02, -0.8866666666666667, -0.1, -0.4533333333333333, -0.1, -0.6933333333333334, -0.11333333333333333, -0.9, -0.03333333333333333, -0.32666666666666666], [-0.0, -0.0, -0.005917159763313609, -0.11834319526627218, 0.0650887573964497, -0.16568047337278108, 0.09467455621301775, -0.27218934911242604, 0.04142011834319527, -0.31952662721893493, -0.09467455621301775, -0.15976331360946747, -0.14792899408284024, -0.26627218934911245, -0.07692307692307693, -0.30177514792899407, 0.011834319526627219, -0.38461538461538464, 0.047337278106508875, -0.5976331360946746, 0.0650887573964497, -0.7810650887573964, -0.08284023668639054, -0.38461538461538464, -0.07692307692307693, -0.591715976331361, -0.07100591715976332, -0.7633136094674556, -0.029585798816568046, -0.27218934911242604], [-0.0, -0.0, -0.011904761904761904, -0.125, 0.06547619047619048, -0.16666666666666666, 0.10119047619047619, -0.27380952380952384, 0.03571428571428571, -0.32142857142857145, -0.10119047619047619, -0.16666666666666666, -0.1488095238095238, -0.27976190476190477, -0.05952380952380952, -0.30952380952380953, 0.023809523809523808, -0.39880952380952384, 0.07142857142857142, -0.6011904761904762, 0.1130952380952381, -0.7797619047619048, -0.07738095238095238, -0.39880952380952384, -0.05952380952380952, -0.6011904761904762, -0.047619047619047616, -0.7678571428571429, -0.023809523809523808, -0.27976190476190477], [-0.0, -0.0, -0.022900763358778626, -0.16793893129770993, 0.07633587786259542, -0.24427480916030533, 0.12213740458015267, -0.3969465648854962, 0.04580152671755725, -0.44274809160305345, -0.1450381679389313, -0.22137404580152673, -0.183206106870229, -0.3893129770992366, -0.05343511450381679, -0.42748091603053434, 0.030534351145038167, -0.5267175572519084, -0.015267175572519083, -0.816793893129771, -0.04580152671755725, -1.0534351145038168, -0.09923664122137404, -0.5267175572519084, -0.061068702290076333, -0.816793893129771, -0.05343511450381679, -1.0534351145038168, -0.03816793893129771, -0.3816793893129771], [-0.0, -0.0, -0.00819672131147541, -0.18032786885245902, 0.10655737704918032, -0.26229508196721313, 0.1557377049180328, -0.4180327868852459, 0.09016393442622951, -0.47540983606557374, -0.13934426229508196, -0.23770491803278687, -0.18032786885245902, -0.4262295081967213, -0.03278688524590164, -0.45901639344262296, 0.05737704918032787, -0.5737704918032787, 0.07377049180327869, -0.8688524590163934, 0.07377049180327869, -1.1147540983606556, -0.09016393442622951, -0.5737704918032787, -0.09016393442622951, -0.8688524590163934, -0.12295081967213115, -1.1557377049180328, -0.01639344262295082, -0.4180327868852459], [-0.0, -0.0, -0.0, -0.1320754716981132, 0.08176100628930817, -0.18238993710691823, 0.14465408805031446, -0.3018867924528302, 0.0880503144654088, -0.29559748427672955, -0.09433962264150944, -0.1761006289308176, -0.0880503144654088, -0.3081761006289308, 0.031446540880503145, -0.3018867924528302, 0.050314465408805034, -0.4088050314465409, 0.025157232704402517, -0.6415094339622641, 0.018867924528301886, -0.8238993710691824, -0.06289308176100629, -0.4088050314465409, -0.07547169811320754, -0.6289308176100629, -0.10062893081761007, -0.8238993710691824, -0.006289308176100629, -0.29559748427672955], [-0.0, -0.0, -0.0, -0.125, 0.07142857142857142, -0.16666666666666666, 0.11904761904761904, -0.26785714285714285, 0.10714285714285714, -0.25, -0.07142857142857142, -0.16071428571428573, -0.06547619047619048, -0.27380952380952384, 0.06547619047619048, -0.26785714285714285, 0.05952380952380952, -0.38095238095238093, 0.08333333333333333, -0.5892857142857143, 0.07738095238095238, -0.7440476190476191, -0.03571428571428571, -0.38095238095238093, -0.02976190476190476, -0.5952380952380952, -0.03571428571428571, -0.7738095238095238, -0.0, -0.27380952380952384], [-0.0, -0.0, -0.013333333333333334, -0.08, 0.035555555555555556, -0.12, 0.057777777777777775, -0.19111111111111112, 0.04, -0.1688888888888889, -0.07111111111111111, -0.10666666666666667, -0.07111111111111111, -0.18222222222222223, 0.013333333333333334, -0.18222222222222223, 0.008888888888888889, -0.26666666666666666, -0.013333333333333334, -0.41333333333333333, -0.022222222222222223, -0.5333333333333333, -0.057777777777777775, -0.26222222222222225, -0.06666666666666667, -0.4088888888888889, -0.10666666666666667, -0.5244444444444445, -0.022222222222222223, -0.18666666666666668], [-0.0, -0.0, -0.0, -0.08968609865470852, 0.04932735426008968, -0.1210762331838565, 0.08968609865470852, -0.19282511210762332, 0.06726457399103139, -0.18834080717488788, -0.05829596412556054, -0.11659192825112108, -0.04035874439461883, -0.19282511210762332, 0.04484304932735426, -0.19282511210762332, 0.04035874439461883, -0.273542600896861, 0.053811659192825115, -0.3991031390134529, 0.04035874439461883, -0.5112107623318386, -0.026905829596412557, -0.273542600896861, -0.013452914798206279, -0.4080717488789238, -0.013452914798206279, -0.5336322869955157, -0.0, -0.19282511210762332], [-0.0, -0.0, -0.006920415224913495, -0.058823529411764705, 0.031141868512110725, -0.08304498269896193, 0.04498269896193772, -0.13494809688581316, 0.01384083044982699, -0.13494809688581316, -0.04152249134948097, -0.07612456747404844, -0.04844290657439446, -0.1314878892733564, 0.0034602076124567475, -0.1384083044982699, 0.01384083044982699, -0.19377162629757785, -0.0034602076124567475, -0.28719723183391005, -0.01730103806228374, -0.3633217993079585, -0.03460207612456748, -0.1903114186851211, -0.03806228373702422, -0.28719723183391005, -0.058823529411764705, -0.370242214532872, -0.010380622837370242, -0.13494809688581316], [-0.0, -0.0, -0.003257328990228013, -0.05537459283387622, 0.029315960912052116, -0.0749185667752443, 0.04234527687296417, -0.12052117263843648, 0.035830618892508145, -0.1270358306188925, -0.035830618892508145, -0.0749185667752443, -0.03908794788273615, -0.1270358306188925, 0.019543973941368076, -0.13029315960912052, 0.016286644951140065, -0.1758957654723127, 0.026058631921824105, -0.25732899022801303, 0.03257328990228013, -0.3355048859934853, -0.019543973941368076, -0.1758957654723127, -0.019543973941368076, -0.25732899022801303, -0.019543973941368076, -0.3355048859934853, -0.003257328990228013, -0.1270358306188925], [-0.0, -0.0, 0.0029850746268656717, -0.04776119402985075, 0.03283582089552239, -0.0626865671641791, 0.050746268656716415, -0.1044776119402985, 0.026865671641791045, -0.11641791044776119, -0.029850746268656716, -0.06567164179104477, -0.041791044776119404, -0.11044776119402985, -0.0, -0.11044776119402985, 0.01791044776119403, -0.15223880597014924, 0.011940298507462687, -0.22388059701492538, 0.005970149253731343, -0.2865671641791045, -0.014925373134328358, -0.15223880597014924, -0.020895522388059702, -0.22686567164179106, -0.03880597014925373, -0.29253731343283584, 0.0029850746268656717, -0.11044776119402985], [-0.0, -0.0, -0.0029154518950437317, -0.04956268221574344, 0.026239067055393587, -0.0641399416909621, 0.03498542274052478, -0.10495626822157435, 0.023323615160349854, -0.119533527696793, -0.03206997084548105, -0.0641399416909621, -0.043731778425655975, -0.10204081632653061, -0.008746355685131196, -0.10495626822157435, 0.008746355685131196, -0.14285714285714285, 0.02040816326530612, -0.21282798833819241, 0.03206997084548105, -0.27988338192419826, -0.02040816326530612, -0.14285714285714285, -0.01749271137026239, -0.2099125364431487, -0.014577259475218658, -0.27113702623906705, -0.0058309037900874635, -0.10495626822157435], [-0.0, -0.0, -0.005988023952095809, -0.05089820359281437, 0.023952095808383235, -0.0688622754491018, 0.023952095808383235, -0.11377245508982035, -0.017964071856287425, -0.11077844311377245, -0.03592814371257485, -0.059880239520958084, -0.05389221556886228, -0.10179640718562874, -0.038922155688622756, -0.10778443113772455, -0.0, -0.15269461077844312, -0.005988023952095809, -0.2275449101796407, -0.008982035928143712, -0.2904191616766467, -0.03592814371257485, -0.1467065868263473, -0.05389221556886228, -0.2215568862275449, -0.059880239520958084, -0.281437125748503, -0.011976047904191617, -0.10778443113772455], [-0.0, -0.0, -0.0027624309392265192, -0.04419889502762431, 0.024861878453038673, -0.058011049723756904, 0.03038674033149171, -0.09116022099447514, 0.008287292817679558, -0.09668508287292818, -0.03314917127071823, -0.052486187845303865, -0.04419889502762431, -0.0856353591160221, -0.022099447513812154, -0.09668508287292818, 0.0055248618784530384, -0.1270718232044199, 0.008287292817679558, -0.1878453038674033, 0.0055248618784530384, -0.2541436464088398, -0.022099447513812154, -0.12430939226519337, -0.019337016574585635, -0.19060773480662985, -0.013812154696132596, -0.24861878453038674, -0.0055248618784530384, -0.09116022099447514], [-0.0, -0.0, -0.0084985835694051, -0.0453257790368272, 0.019830028328611898, -0.06232294617563739, 0.025495750708215296, -0.09915014164305949, 0.0028328611898017, -0.10764872521246459, -0.036827195467422094, -0.056657223796033995, -0.05099150141643059, -0.0906515580736544, -0.025495750708215296, -0.09915014164305949, 0.0028328611898017, -0.1359773371104816, 0.0084985835694051, -0.20396600566572237, 0.0226628895184136, -0.26628895184135976, -0.031161473087818695, -0.13314447592067988, -0.031161473087818695, -0.20396600566572237, -0.028328611898016998, -0.26628895184135976, -0.0113314447592068, -0.09631728045325778], [-0.0, -0.0, -0.006153846153846154, -0.052307692307692305, 0.024615384615384615, -0.06769230769230769, 0.036923076923076927, -0.11384615384615385, 0.012307692307692308, -0.12, -0.04, -0.06461538461538462, -0.052307692307692305, -0.11076923076923077, -0.015384615384615385, -0.11692307692307692, 0.006153846153846154, -0.16, -0.003076923076923077, -0.23384615384615384, -0.003076923076923077, -0.3046153846153846, -0.033846153846153845, -0.15384615384615385, -0.04, -0.23076923076923078, -0.04, -0.2953846153846154, -0.009230769230769232, -0.11384615384615385], [-0.0, -0.0, -0.0035842293906810036, -0.06451612903225806, 0.035842293906810034, -0.08960573476702509, 0.043010752688172046, -0.14695340501792115, 0.021505376344086023, -0.15412186379928317, -0.05017921146953405, -0.08243727598566308, -0.05734767025089606, -0.14695340501792115, -0.014336917562724014, -0.14336917562724014, 0.014336917562724014, -0.20430107526881722, 0.02867383512544803, -0.2939068100358423, 0.03225806451612903, -0.37992831541218636, -0.03225806451612903, -0.2007168458781362, -0.021505376344086023, -0.2974910394265233, -0.017921146953405017, -0.3835125448028674, -0.007168458781362007, -0.14336917562724014], [-0.0, -0.0, -0.008264462809917356, -0.07851239669421488, 0.04132231404958678, -0.1115702479338843, 0.05785123966942149, -0.17355371900826447, 0.024793388429752067, -0.19421487603305784, -0.06198347107438017, -0.10330578512396695, -0.07851239669421488, -0.17355371900826447, -0.03305785123966942, -0.1859504132231405, 0.008264462809917356, -0.24380165289256198, -0.0, -0.36363636363636365, -0.004132231404958678, -0.4669421487603306, -0.049586776859504134, -0.2396694214876033, -0.05371900826446281, -0.36363636363636365, -0.06198347107438017, -0.47520661157024796, -0.02066115702479339, -0.17355371900826447], [-0.0, -0.0, -0.01020408163265306, -0.10204081632653061, 0.05102040816326531, -0.14285714285714285, 0.08163265306122448, -0.22959183673469388, 0.05612244897959184, -0.2602040816326531, -0.08163265306122448, -0.14285714285714285, -0.12244897959183673, -0.2193877551020408, -0.0663265306122449, -0.23469387755102042, 0.02040816326530612, -0.32142857142857145, 0.04591836734693878, -0.4897959183673469, 0.05612244897959184, -0.6326530612244898, -0.061224489795918366, -0.32142857142857145, -0.05612244897959184, -0.49489795918367346, -0.07142857142857142, -0.6326530612244898, -0.02040816326530612, -0.22959183673469388], [-0.0, -0.0, -0.011904761904761904, -0.125, 0.07142857142857142, -0.17261904761904762, 0.10714285714285714, -0.27976190476190477, 0.13095238095238096, -0.2976190476190476, -0.09523809523809523, -0.16666666666666666, -0.15476190476190477, -0.27380952380952384, -0.09523809523809523, -0.31547619047619047, 0.02976190476190476, -0.3869047619047619, 0.017857142857142856, -0.5952380952380952, -0.011904761904761904, -0.7559523809523809, -0.05952380952380952, -0.3869047619047619, -0.02976190476190476, -0.6011904761904762, -0.03571428571428571, -0.7738095238095238, -0.011904761904761904, -0.27976190476190477], [-0.0, -0.0, -0.006289308176100629, -0.1320754716981132, 0.08176100628930817, -0.18867924528301888, 0.12578616352201258, -0.31446540880503143, 0.1761006289308176, -0.32075471698113206, -0.10062893081761007, -0.1761006289308176, -0.1509433962264151, -0.2893081761006289, -0.11320754716981132, -0.32075471698113206, 0.05660377358490566, -0.4088050314465409, 0.07547169811320754, -0.6352201257861635, 0.07547169811320754, -0.8238993710691824, -0.050314465408805034, -0.41509433962264153, -0.06289308176100629, -0.6415094339622641, -0.0880503144654088, -0.8364779874213837, -0.0, -0.29559748427672955], [-0.0, -0.0, 0.006711409395973154, -0.1476510067114094, 0.087248322147651, -0.20134228187919462, 0.1342281879194631, -0.33557046979865773, 0.15436241610738255, -0.3422818791946309, -0.087248322147651, -0.20134228187919462, -0.12751677852348994, -0.3422818791946309, -0.020134228187919462, -0.3624161073825503, 0.06711409395973154, -0.4563758389261745, 0.06040268456375839, -0.6778523489932886, 0.020134228187919462, -0.87248322147651, -0.040268456375838924, -0.4563758389261745, -0.026845637583892617, -0.6778523489932886, -0.053691275167785234, -0.8926174496644296, 0.006711409395973154, -0.3288590604026846], [-0.0, -0.0, -0.005988023952095809, -0.1317365269461078, 0.0718562874251497, -0.18562874251497005, 0.10179640718562874, -0.2934131736526946, 0.11377245508982035, -0.30538922155688625, -0.08982035928143713, -0.17365269461077845, -0.11377245508982035, -0.2934131736526946, -0.023952095808383235, -0.30538922155688625, 0.03592814371257485, -0.40718562874251496, 0.029940119760479042, -0.6047904191616766, 0.011976047904191617, -0.7664670658682635, -0.05389221556886228, -0.40119760479041916, -0.0718562874251497, -0.6047904191616766, -0.11976047904191617, -0.7844311377245509, -0.005988023952095809, -0.2934131736526946], [-0.0, -0.0, 0.004901960784313725, -0.10294117647058823, 0.06372549019607843, -0.14215686274509803, 0.09803921568627451, -0.23529411764705882, 0.11274509803921569, -0.23039215686274508, -0.06372549019607843, -0.14215686274509803, -0.07352941176470588, -0.23529411764705882, -0.004901960784313725, -0.23039215686274508, 0.04411764705882353, -0.31862745098039214, 0.058823529411764705, -0.47058823529411764, 0.06862745098039216, -0.6078431372549019, -0.029411764705882353, -0.31862745098039214, -0.024509803921568627, -0.46568627450980393, -0.029411764705882353, -0.5931372549019608, 0.004901960784313725, -0.23039215686274508], [-0.0, -0.0, -0.0, -0.06896551724137931, 0.0421455938697318, -0.09578544061302682, 0.06896551724137931, -0.16475095785440613, 0.04980842911877394, -0.18007662835249041, -0.04597701149425287, -0.09578544061302682, -0.04980842911877394, -0.16475095785440613, -0.007662835249042145, -0.18007662835249041, 0.022988505747126436, -0.2260536398467433, 0.019157088122605363, -0.3333333333333333, 0.007662835249042145, -0.421455938697318, -0.02681992337164751, -0.2222222222222222, -0.03065134099616858, -0.3371647509578544, -0.04980842911877394, -0.4367816091954023, -0.0, -0.16091954022988506], [-0.0, -0.0, 0.003355704697986577, -0.05704697986577181, 0.0436241610738255, -0.0738255033557047, 0.07046979865771812, -0.1342281879194631, 0.05704697986577181, -0.1476510067114094, -0.030201342281879196, -0.08053691275167785, -0.026845637583892617, -0.13758389261744966, 0.02348993288590604, -0.14429530201342283, 0.040268456375838924, -0.18791946308724833, 0.040268456375838924, -0.2785234899328859, 0.04697986577181208, -0.35570469798657717, -0.006711409395973154, -0.18791946308724833, -0.003355704697986577, -0.2751677852348993, -0.003355704697986577, -0.348993288590604, 0.013422818791946308, -0.13087248322147652], [-0.0, -0.0, 0.0029940119760479044, -0.05089820359281437, 0.03592814371257485, -0.0658682634730539, 0.05089820359281437, -0.11077844311377245, 0.03293413173652695, -0.1317365269461078, -0.02694610778443114, -0.0658682634730539, -0.03592814371257485, -0.11676646706586827, -0.0029940119760479044, -0.12574850299401197, 0.02694610778443114, -0.15269461077844312, 0.020958083832335328, -0.2275449101796407, 0.011976047904191617, -0.2934131736526946, -0.014970059880239521, -0.15269461077844312, -0.017964071856287425, -0.23053892215568864, -0.03592814371257485, -0.3023952095808383, 0.005988023952095809, -0.10778443113772455], [-0.0, -0.0, 0.002824858757062147, -0.0423728813559322, 0.03389830508474576, -0.059322033898305086, 0.03954802259887006, -0.09887005649717515, 0.02824858757062147, -0.11864406779661017, -0.025423728813559324, -0.05649717514124294, -0.031073446327683617, -0.096045197740113, -0.014124293785310734, -0.11581920903954802, 0.01977401129943503, -0.13559322033898305, 0.01694915254237288, -0.2033898305084746, 0.014124293785310734, -0.2655367231638418, -0.011299435028248588, -0.1327683615819209, -0.014124293785310734, -0.20056497175141244, -0.025423728813559324, -0.2570621468926554, 0.005649717514124294, -0.096045197740113], [-0.0, -0.0, 0.005813953488372093, -0.046511627906976744, 0.03197674418604651, -0.061046511627906974, 0.0436046511627907, -0.10755813953488372, 0.0029069767441860465, -0.11337209302325581, -0.014534883720930232, -0.06395348837209303, -0.020348837209302327, -0.10174418604651163, -0.01744186046511628, -0.11337209302325581, 0.023255813953488372, -0.14244186046511628, 0.02616279069767442, -0.21220930232558138, 0.03488372093023256, -0.27906976744186046, 0.0029069767441860465, -0.14244186046511628, 0.0029069767441860465, -0.20930232558139536, 0.011627906976744186, -0.2616279069767442, 0.011627906976744186, -0.10174418604651163], [-0.0, -0.0, 0.0058309037900874635, -0.04956268221574344, 0.03498542274052478, -0.0641399416909621, 0.04081632653061224, -0.11078717201166181, 0.008746355685131196, -0.12244897959183673, -0.01749271137026239, -0.0641399416909621, -0.03206997084548105, -0.10204081632653061, -0.03206997084548105, -0.11661807580174927, 0.02040816326530612, -0.13994169096209913, 0.008746355685131196, -0.20116618075801748, 0.0058309037900874635, -0.27113702623906705, -0.0058309037900874635, -0.13702623906705538, -0.0058309037900874635, -0.19533527696793002, -0.0029154518950437317, -0.2594752186588921, 0.008746355685131196, -0.10204081632653061], [-0.0, -0.0, 0.0031746031746031746, -0.05714285714285714, 0.03492063492063492, -0.07301587301587302, 0.050793650793650794, -0.13015873015873017, 0.025396825396825397, -0.1492063492063492, -0.02857142857142857, -0.06984126984126984, -0.044444444444444446, -0.11746031746031746, -0.050793650793650794, -0.12698412698412698, 0.015873015873015872, -0.16825396825396827, 0.012698412698412698, -0.2507936507936508, 0.012698412698412698, -0.3238095238095238, -0.022222222222222223, -0.16507936507936508, -0.0380952380952381, -0.24761904761904763, -0.047619047619047616, -0.3238095238095238, 0.0031746031746031746, -0.12063492063492064], [-0.0, -0.0, -0.007168458781362007, -0.06451612903225806, 0.035842293906810034, -0.08602150537634409, 0.053763440860215055, -0.13620071684587814, 0.03225806451612903, -0.16845878136200718, -0.04659498207885305, -0.08243727598566308, -0.06451612903225806, -0.13261648745519714, -0.06451612903225806, -0.15412186379928317, 0.014336917562724014, -0.2007168458781362, 0.02867383512544803, -0.3046594982078853, 0.043010752688172046, -0.3978494623655914, -0.03225806451612903, -0.1971326164874552, -0.025089605734767026, -0.2974910394265233, -0.021505376344086023, -0.3763440860215054, -0.007168458781362007, -0.14336917562724014], [-0.0, -0.0, -0.0, -0.0815450643776824, 0.05150214592274678, -0.11158798283261803, 0.06866952789699571, -0.18454935622317598, 0.06866952789699571, -0.23605150214592274, -0.055793991416309016, -0.11158798283261803, -0.07725321888412018, -0.18454935622317598, -0.05150214592274678, -0.19742489270386265, 0.034334763948497854, -0.26180257510729615, 0.05150214592274678, -0.38626609442060084, 0.03862660944206009, -0.4892703862660944, -0.030042918454935622, -0.26180257510729615, -0.02575107296137339, -0.38626609442060084, -0.02145922746781116, -0.4978540772532189, 0.004291845493562232, -0.18454935622317598], [-0.0, -0.0, -0.004672897196261682, -0.09345794392523364, 0.056074766355140186, -0.1308411214953271, 0.0794392523364486, -0.21495327102803738, 0.037383177570093455, -0.22897196261682243, -0.06542056074766354, -0.1308411214953271, -0.0794392523364486, -0.22897196261682243, -0.014018691588785047, -0.2336448598130841, 0.03271028037383177, -0.29906542056074764, 0.03271028037383177, -0.4439252336448598, 0.03271028037383177, -0.5654205607476636, -0.04205607476635514, -0.29439252336448596, -0.06542056074766354, -0.43457943925233644, -0.09345794392523364, -0.5514018691588785, -0.004672897196261682, -0.21495327102803738], [-0.0, -0.0, -0.0, -0.08482142857142858, 0.0625, -0.12053571428571429, 0.07142857142857142, -0.20089285714285715, 0.05803571428571429, -0.24107142857142858, -0.05803571428571429, -0.12053571428571429, -0.07589285714285714, -0.20089285714285715, -0.013392857142857142, -0.21875, 0.03125, -0.28125, 0.04017857142857143, -0.41517857142857145, 0.05803571428571429, -0.5357142857142857, -0.04017857142857143, -0.2767857142857143, -0.05357142857142857, -0.41517857142857145, -0.0625, -0.5267857142857143, -0.0, -0.19642857142857142], [-0.0, -0.0, -0.00975609756097561, -0.0975609756097561, 0.05365853658536585, -0.14146341463414633, 0.06341463414634146, -0.22926829268292684, 0.05365853658536585, -0.2731707317073171, -0.07804878048780488, -0.13170731707317074, -0.09268292682926829, -0.22439024390243903, -0.01951219512195122, -0.23902439024390243, 0.024390243902439025, -0.3170731707317073, 0.02926829268292683, -0.4682926829268293, 0.04390243902439024, -0.6, -0.05853658536585366, -0.3121951219512195, -0.07317073170731707, -0.4634146341463415, -0.0975609756097561, -0.5853658536585366, -0.014634146341463415, -0.22439024390243903], [-0.0, -0.0, -0.01020408163265306, -0.10204081632653061, 0.05612244897959184, -0.14285714285714285, 0.0663265306122449, -0.22959183673469388, 0.05612244897959184, -0.28061224489795916, -0.08163265306122448, -0.1377551020408163, -0.10204081632653061, -0.23979591836734693, -0.030612244897959183, -0.24489795918367346, 0.025510204081632654, -0.33163265306122447, 0.05102040816326531, -0.49489795918367346, 0.07653061224489796, -0.6377551020408163, -0.05612244897959184, -0.33163265306122447, -0.05612244897959184, -0.4846938775510204, -0.061224489795918366, -0.6173469387755102, -0.015306122448979591, -0.23469387755102042], [-0.0, -0.0, -0.0048543689320388345, -0.09223300970873786, 0.05339805825242718, -0.13106796116504854, 0.07281553398058252, -0.21359223300970873, 0.04854368932038835, -0.2524271844660194, -0.06796116504854369, -0.1262135922330097, -0.09223300970873786, -0.21844660194174756, -0.02912621359223301, -0.23300970873786409, 0.02912621359223301, -0.3058252427184466, 0.043689320388349516, -0.46601941747572817, 0.04854368932038835, -0.5970873786407767, -0.04854368932038835, -0.3058252427184466, -0.04854368932038835, -0.46116504854368934, -0.05825242718446602, -0.587378640776699, -0.009708737864077669, -0.21844660194174756], [-0.0, -0.0, -0.0, -0.09345794392523364, 0.06074766355140187, -0.1308411214953271, 0.07476635514018691, -0.21495327102803738, 0.04205607476635514, -0.22429906542056074, -0.06542056074766354, -0.1261682242990654, -0.08411214953271028, -0.2102803738317757, -0.018691588785046728, -0.2102803738317757, 0.04205607476635514, -0.2897196261682243, 0.04672897196261682, -0.4439252336448598, 0.0514018691588785, -0.5700934579439252, -0.037383177570093455, -0.29439252336448596, -0.04672897196261682, -0.4392523364485981, -0.056074766355140186, -0.5607476635514018, -0.0, -0.2102803738317757], [-0.0, -0.0, -0.004651162790697674, -0.08837209302325581, 0.05116279069767442, -0.12558139534883722, 0.06511627906976744, -0.20930232558139536, 0.046511627906976744, -0.2372093023255814, -0.06976744186046512, -0.12093023255813953, -0.09302325581395349, -0.20465116279069767, -0.023255813953488372, -0.20930232558139536, 0.023255813953488372, -0.2930232558139535, 0.04186046511627907, -0.4418604651162791, 0.05116279069767442, -0.5674418604651162, -0.05116279069767442, -0.2930232558139535, -0.046511627906976744, -0.4372093023255814, -0.05116279069767442, -0.5581395348837209, -0.009302325581395349, -0.20930232558139536], [-0.0, -0.0, -0.00510204081632653, -0.10204081632653061, 0.0663265306122449, -0.14795918367346939, 0.07653061224489796, -0.23469387755102042, 0.05612244897959184, -0.27040816326530615, -0.07653061224489796, -0.14285714285714285, -0.09183673469387756, -0.23979591836734693, -0.025510204081632654, -0.24489795918367346, 0.03571428571428571, -0.33163265306122447, 0.030612244897959183, -0.49489795918367346, 0.03571428571428571, -0.6377551020408163, -0.04591836734693878, -0.33163265306122447, -0.04081632653061224, -0.4897959183673469, -0.04591836734693878, -0.6224489795918368, -0.00510204081632653, -0.23469387755102042], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05581395348837209, -0.12558139534883722, 0.06511627906976744, -0.20930232558139536, 0.03255813953488372, -0.22790697674418606, -0.06511627906976744, -0.12093023255813953, -0.08837209302325581, -0.20465116279069767, -0.03255813953488372, -0.20930232558139536, 0.023255813953488372, -0.2837209302325581, 0.013953488372093023, -0.4325581395348837, 0.013953488372093023, -0.5627906976744186, -0.04186046511627907, -0.2837209302325581, -0.03255813953488372, -0.42790697674418604, -0.03255813953488372, -0.5627906976744186, -0.009302325581395349, -0.2], [-0.0, -0.0, -0.0, -0.08, 0.04888888888888889, -0.1111111111111111, 0.07111111111111111, -0.18666666666666668, 0.044444444444444446, -0.2222222222222222, -0.06222222222222222, -0.1111111111111111, -0.08444444444444445, -0.19111111111111112, -0.022222222222222223, -0.19555555555555557, 0.02666666666666667, -0.26666666666666666, 0.017777777777777778, -0.4088888888888889, 0.013333333333333334, -0.5333333333333333, -0.04, -0.26666666666666666, -0.02666666666666667, -0.4088888888888889, -0.02666666666666667, -0.5333333333333333, -0.0044444444444444444, -0.18666666666666668], [-0.0, -0.0, -0.004651162790697674, -0.08837209302325581, 0.05116279069767442, -0.12558139534883722, 0.06976744186046512, -0.20465116279069767, 0.04186046511627907, -0.23255813953488372, -0.06511627906976744, -0.12093023255813953, -0.08837209302325581, -0.20465116279069767, -0.027906976744186046, -0.21395348837209302, 0.023255813953488372, -0.2837209302325581, 0.013953488372093023, -0.4325581395348837, 0.009302325581395349, -0.5627906976744186, -0.04186046511627907, -0.2837209302325581, -0.03255813953488372, -0.42790697674418604, -0.03255813953488372, -0.5627906976744186, -0.009302325581395349, -0.2], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05581395348837209, -0.12558139534883722, 0.06511627906976744, -0.20930232558139536, 0.037209302325581395, -0.23255813953488372, -0.06511627906976744, -0.12093023255813953, -0.08837209302325581, -0.20465116279069767, -0.023255813953488372, -0.20930232558139536, 0.023255813953488372, -0.28837209302325584, 0.013953488372093023, -0.4325581395348837, 0.009302325581395349, -0.5627906976744186, -0.04186046511627907, -0.2837209302325581, -0.03255813953488372, -0.42790697674418604, -0.03255813953488372, -0.5627906976744186, -0.004651162790697674, -0.20465116279069767], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05581395348837209, -0.12558139534883722, 0.06976744186046512, -0.20930232558139536, 0.037209302325581395, -0.22790697674418606, -0.06511627906976744, -0.12093023255813953, -0.08837209302325581, -0.20465116279069767, -0.023255813953488372, -0.20465116279069767, 0.023255813953488372, -0.2837209302325581, 0.013953488372093023, -0.4325581395348837, 0.013953488372093023, -0.5627906976744186, -0.04186046511627907, -0.2837209302325581, -0.03255813953488372, -0.42790697674418604, -0.03255813953488372, -0.5627906976744186, -0.004651162790697674, -0.20465116279069767], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05581395348837209, -0.12558139534883722, 0.06976744186046512, -0.20465116279069767, 0.04186046511627907, -0.2372093023255814, -0.06511627906976744, -0.12093023255813953, -0.08837209302325581, -0.20465116279069767, -0.027906976744186046, -0.20930232558139536, 0.023255813953488372, -0.28837209302325584, 0.013953488372093023, -0.4325581395348837, 0.013953488372093023, -0.5627906976744186, -0.04186046511627907, -0.2837209302325581, -0.03255813953488372, -0.42790697674418604, -0.03255813953488372, -0.5627906976744186, -0.004651162790697674, -0.20465116279069767], [-0.0, -0.0, -0.0, -0.0975609756097561, 0.06341463414634146, -0.13658536585365855, 0.07317073170731707, -0.22926829268292684, 0.03902439024390244, -0.24390243902439024, -0.07317073170731707, -0.13170731707317074, -0.09268292682926829, -0.22439024390243903, -0.024390243902439025, -0.22439024390243903, 0.02926829268292683, -0.3073170731707317, 0.014634146341463415, -0.4585365853658537, 0.00975609756097561, -0.5951219512195122, -0.04390243902439024, -0.3073170731707317, -0.03414634146341464, -0.4585365853658537, -0.03902439024390244, -0.5951219512195122, -0.004878048780487805, -0.21951219512195122], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05581395348837209, -0.12558139534883722, 0.06976744186046512, -0.20930232558139536, 0.037209302325581395, -0.22790697674418606, -0.06976744186046512, -0.12093023255813953, -0.09302325581395349, -0.20465116279069767, -0.027906976744186046, -0.20930232558139536, 0.027906976744186046, -0.28837209302325584, 0.013953488372093023, -0.4325581395348837, 0.009302325581395349, -0.5627906976744186, -0.04186046511627907, -0.2837209302325581, -0.037209302325581395, -0.42790697674418604, -0.037209302325581395, -0.5581395348837209, -0.004651162790697674, -0.20465116279069767], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05116279069767442, -0.12093023255813953, 0.06976744186046512, -0.19534883720930232, 0.05116279069767442, -0.2372093023255814, -0.06046511627906977, -0.12093023255813953, -0.08837209302325581, -0.2, -0.04186046511627907, -0.2186046511627907, 0.027906976744186046, -0.28837209302325584, 0.018604651162790697, -0.4325581395348837, 0.013953488372093023, -0.5627906976744186, -0.037209302325581395, -0.28837209302325584, -0.03255813953488372, -0.42790697674418604, -0.03255813953488372, -0.5627906976744186, -0.004651162790697674, -0.2], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05581395348837209, -0.12093023255813953, 0.07441860465116279, -0.19534883720930232, 0.05116279069767442, -0.2372093023255814, -0.06046511627906977, -0.12093023255813953, -0.08837209302325581, -0.2, -0.037209302325581395, -0.21395348837209302, 0.027906976744186046, -0.28837209302325584, 0.018604651162790697, -0.4325581395348837, 0.013953488372093023, -0.5627906976744186, -0.037209302325581395, -0.28837209302325584, -0.027906976744186046, -0.42790697674418604, -0.03255813953488372, -0.5581395348837209, -0.0, -0.2], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05116279069767442, -0.12093023255813953, 0.06976744186046512, -0.2, 0.05116279069767442, -0.24651162790697675, -0.06511627906976744, -0.12093023255813953, -0.09302325581395349, -0.2, -0.037209302325581395, -0.21395348837209302, 0.023255813953488372, -0.28837209302325584, 0.013953488372093023, -0.42790697674418604, 0.009302325581395349, -0.5627906976744186, -0.04186046511627907, -0.28837209302325584, -0.03255813953488372, -0.42790697674418604, -0.037209302325581395, -0.5581395348837209, -0.004651162790697674, -0.20465116279069767], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05116279069767442, -0.12093023255813953, 0.06976744186046512, -0.2, 0.046511627906976744, -0.22790697674418606, -0.06511627906976744, -0.12093023255813953, -0.09302325581395349, -0.2, -0.03255813953488372, -0.21395348837209302, 0.023255813953488372, -0.28837209302325584, 0.013953488372093023, -0.4325581395348837, 0.009302325581395349, -0.5627906976744186, -0.04186046511627907, -0.2837209302325581, -0.03255813953488372, -0.4325581395348837, -0.037209302325581395, -0.5627906976744186, -0.004651162790697674, -0.2], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05581395348837209, -0.11627906976744186, 0.07441860465116279, -0.19069767441860466, 0.05116279069767442, -0.23255813953488372, -0.06046511627906977, -0.12093023255813953, -0.08837209302325581, -0.19534883720930232, -0.046511627906976744, -0.2186046511627907, 0.027906976744186046, -0.28837209302325584, 0.018604651162790697, -0.42790697674418604, 0.013953488372093023, -0.5627906976744186, -0.037209302325581395, -0.2837209302325581, -0.027906976744186046, -0.42790697674418604, -0.03255813953488372, -0.5627906976744186, -0.0, -0.2], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05581395348837209, -0.12093023255813953, 0.07441860465116279, -0.19069767441860466, 0.05116279069767442, -0.23255813953488372, -0.06046511627906977, -0.12093023255813953, -0.08837209302325581, -0.19534883720930232, -0.04186046511627907, -0.2186046511627907, 0.027906976744186046, -0.28837209302325584, 0.018604651162790697, -0.4325581395348837, 0.013953488372093023, -0.5627906976744186, -0.037209302325581395, -0.2837209302325581, -0.027906976744186046, -0.42790697674418604, -0.03255813953488372, -0.5627906976744186, -0.0, -0.2], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05581395348837209, -0.12093023255813953, 0.07441860465116279, -0.19534883720930232, 0.05116279069767442, -0.23255813953488372, -0.06046511627906977, -0.12093023255813953, -0.08837209302325581, -0.2, -0.037209302325581395, -0.2186046511627907, 0.027906976744186046, -0.28837209302325584, 0.018604651162790697, -0.42790697674418604, 0.013953488372093023, -0.5627906976744186, -0.037209302325581395, -0.2837209302325581, -0.027906976744186046, -0.42790697674418604, -0.03255813953488372, -0.5627906976744186, -0.0, -0.2], [-0.0, -0.0, 0.004651162790697674, -0.08837209302325581, 0.05581395348837209, -0.12093023255813953, 0.07441860465116279, -0.2, 0.046511627906976744, -0.22790697674418606, -0.06046511627906977, -0.12093023255813953, -0.08837209302325581, -0.19534883720930232, -0.046511627906976744, -0.2186046511627907, 0.027906976744186046, -0.28837209302325584, 0.018604651162790697, -0.4325581395348837, 0.013953488372093023, -0.5627906976744186, -0.037209302325581395, -0.2837209302325581, -0.027906976744186046, -0.42790697674418604, -0.03255813953488372, -0.5627906976744186, -0.0, -0.2], [-0.0, -0.0, 0.004651162790697674, -0.08837209302325581, 0.05581395348837209, -0.12093023255813953, 0.07906976744186046, -0.19534883720930232, 0.05116279069767442, -0.23255813953488372, -0.06046511627906977, -0.11627906976744186, -0.09302325581395349, -0.18604651162790697, -0.05581395348837209, -0.2186046511627907, 0.027906976744186046, -0.28837209302325584, 0.018604651162790697, -0.42790697674418604, 0.013953488372093023, -0.5627906976744186, -0.037209302325581395, -0.2837209302325581, -0.027906976744186046, -0.42790697674418604, -0.03255813953488372, -0.5581395348837209, -0.0, -0.2], [-0.0, -0.0, 0.004651162790697674, -0.08837209302325581, 0.05581395348837209, -0.11627906976744186, 0.08837209302325581, -0.19069767441860466, 0.06976744186046512, -0.24186046511627907, -0.05581395348837209, -0.11627906976744186, -0.10232558139534884, -0.18604651162790697, -0.05581395348837209, -0.2186046511627907, 0.027906976744186046, -0.28837209302325584, 0.018604651162790697, -0.42790697674418604, 0.013953488372093023, -0.5627906976744186, -0.037209302325581395, -0.2837209302325581, -0.027906976744186046, -0.42790697674418604, -0.03255813953488372, -0.5627906976744186, -0.0, -0.2], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05116279069767442, -0.12093023255813953, 0.09767441860465116, -0.19069767441860466, 0.06511627906976744, -0.22790697674418606, -0.06511627906976744, -0.12093023255813953, -0.11162790697674418, -0.18604651162790697, -0.06046511627906977, -0.2186046511627907, 0.023255813953488372, -0.2837209302325581, 0.013953488372093023, -0.4325581395348837, 0.009302325581395349, -0.5627906976744186, -0.037209302325581395, -0.2837209302325581, -0.03255813953488372, -0.42790697674418604, -0.037209302325581395, -0.5627906976744186, -0.004651162790697674, -0.2], [-0.0, -0.0, 0.004651162790697674, -0.08837209302325581, 0.05581395348837209, -0.12093023255813953, 0.10232558139534884, -0.19069767441860466, 0.06976744186046512, -0.2372093023255814, -0.06046511627906977, -0.12093023255813953, -0.10697674418604651, -0.19069767441860466, -0.05116279069767442, -0.22325581395348837, 0.027906976744186046, -0.2837209302325581, 0.018604651162790697, -0.42790697674418604, 0.013953488372093023, -0.5627906976744186, -0.03255813953488372, -0.2837209302325581, -0.027906976744186046, -0.42790697674418604, -0.03255813953488372, -0.5627906976744186, -0.0, -0.2], [-0.0, -0.0, 0.004651162790697674, -0.08837209302325581, 0.05581395348837209, -0.12093023255813953, 0.10232558139534884, -0.19069767441860466, 0.06046511627906977, -0.22790697674418606, -0.06046511627906977, -0.12093023255813953, -0.10232558139534884, -0.19534883720930232, -0.037209302325581395, -0.2186046511627907, 0.027906976744186046, -0.2837209302325581, 0.018604651162790697, -0.4325581395348837, 0.013953488372093023, -0.5627906976744186, -0.037209302325581395, -0.2837209302325581, -0.027906976744186046, -0.42790697674418604, -0.03255813953488372, -0.5627906976744186, -0.0, -0.2], [-0.0, -0.0, -0.0, -0.0975609756097561, 0.05853658536585366, -0.12682926829268293, 0.1024390243902439, -0.2048780487804878, 0.05365853658536585, -0.23902439024390243, -0.06829268292682927, -0.13170731707317074, -0.1073170731707317, -0.2146341463414634, -0.03902439024390244, -0.22926829268292684, 0.02926829268292683, -0.3073170731707317, 0.014634146341463415, -0.4585365853658537, 0.00975609756097561, -0.5951219512195122, -0.04390243902439024, -0.3073170731707317, -0.03414634146341464, -0.4585365853658537, -0.03902439024390244, -0.5951219512195122, -0.004878048780487805, -0.2146341463414634], [-0.0, -0.0, 0.004651162790697674, -0.08837209302325581, 0.06046511627906977, -0.11627906976744186, 0.10697674418604651, -0.19069767441860466, 0.05116279069767442, -0.21395348837209302, -0.06046511627906977, -0.12093023255813953, -0.09767441860465116, -0.2, -0.027906976744186046, -0.20930232558139536, 0.03255813953488372, -0.2837209302325581, 0.018604651162790697, -0.4325581395348837, 0.013953488372093023, -0.5627906976744186, -0.037209302325581395, -0.2837209302325581, -0.027906976744186046, -0.4325581395348837, -0.03255813953488372, -0.5627906976744186, -0.0, -0.2], [-0.0, -0.0, 0.004651162790697674, -0.08837209302325581, 0.06046511627906977, -0.11627906976744186, 0.10697674418604651, -0.19069767441860466, 0.046511627906976744, -0.20930232558139536, -0.06046511627906977, -0.12093023255813953, -0.09767441860465116, -0.2, -0.027906976744186046, -0.20465116279069767, 0.03255813953488372, -0.2837209302325581, 0.018604651162790697, -0.4325581395348837, 0.013953488372093023, -0.5627906976744186, -0.037209302325581395, -0.2837209302325581, -0.027906976744186046, -0.4325581395348837, -0.03255813953488372, -0.5627906976744186, -0.0, -0.2], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05581395348837209, -0.11627906976744186, 0.10697674418604651, -0.19069767441860466, 0.046511627906976744, -0.21395348837209302, -0.06511627906976744, -0.12093023255813953, -0.10232558139534884, -0.20465116279069767, -0.037209302325581395, -0.20930232558139536, 0.027906976744186046, -0.28837209302325584, 0.018604651162790697, -0.4325581395348837, 0.013953488372093023, -0.5627906976744186, -0.04186046511627907, -0.2837209302325581, -0.027906976744186046, -0.4325581395348837, -0.03255813953488372, -0.5581395348837209, -0.004651162790697674, -0.2], [-0.0, -0.0, -0.0, -0.08333333333333333, 0.05555555555555555, -0.11574074074074074, 0.10185185185185185, -0.18518518518518517, 0.05555555555555555, -0.2175925925925926, -0.06481481481481481, -0.12037037037037036, -0.10185185185185185, -0.19907407407407407, -0.032407407407407406, -0.21296296296296297, 0.027777777777777776, -0.28703703703703703, 0.013888888888888888, -0.4305555555555556, 0.013888888888888888, -0.5601851851851852, -0.041666666666666664, -0.28703703703703703, -0.032407407407407406, -0.4305555555555556, -0.032407407407407406, -0.5601851851851852, -0.004629629629629629, -0.19907407407407407], [-0.0, -0.0, -0.004629629629629629, -0.08333333333333333, 0.05092592592592592, -0.11574074074074074, 0.08796296296296297, -0.18981481481481483, 0.046296296296296294, -0.2222222222222222, -0.06944444444444445, -0.12037037037037036, -0.10185185185185185, -0.2037037037037037, -0.037037037037037035, -0.20833333333333334, 0.023148148148148147, -0.28703703703703703, 0.013888888888888888, -0.4305555555555556, 0.013888888888888888, -0.5601851851851852, -0.046296296296296294, -0.28703703703703703, -0.032407407407407406, -0.4305555555555556, -0.032407407407407406, -0.5601851851851852, -0.009259259259259259, -0.19907407407407407], [-0.0, -0.0, -0.0, -0.08, 0.05333333333333334, -0.1111111111111111, 0.08, -0.18666666666666668, 0.04, -0.20444444444444446, -0.057777777777777775, -0.1111111111111111, -0.09333333333333334, -0.19111111111111112, -0.035555555555555556, -0.18666666666666668, 0.03111111111111111, -0.26666666666666666, 0.017777777777777778, -0.4088888888888889, 0.017777777777777778, -0.5333333333333333, -0.035555555555555556, -0.26666666666666666, -0.02666666666666667, -0.4088888888888889, -0.02666666666666667, -0.5333333333333333, -0.0044444444444444444, -0.18666666666666668], [-0.0, -0.0, -0.0, -0.08837209302325581, 0.05116279069767442, -0.12093023255813953, 0.07906976744186046, -0.2, 0.037209302325581395, -0.21395348837209302, -0.06511627906976744, -0.12093023255813953, -0.10697674418604651, -0.2, -0.05116279069767442, -0.2, 0.027906976744186046, -0.2837209302325581, 0.013953488372093023, -0.4325581395348837, 0.013953488372093023, -0.5627906976744186, -0.04186046511627907, -0.2837209302325581, -0.03255813953488372, -0.42790697674418604, -0.03255813953488372, -0.5627906976744186, -0.009302325581395349, -0.2], [-0.0, -0.0, -0.00975609756097561, -0.0975609756097561, 0.03902439024390244, -0.13658536585365855, 0.06829268292682927, -0.22439024390243903, 0.05365853658536585, -0.2634146341463415, -0.08292682926829269, -0.12682926829268293, -0.1073170731707317, -0.22439024390243903, -0.01951219512195122, -0.22926829268292684, 0.01951219512195122, -0.3121951219512195, 0.00975609756097561, -0.4585365853658537, 0.004878048780487805, -0.5951219512195122, -0.05365853658536585, -0.3073170731707317, -0.04390243902439024, -0.4585365853658537, -0.04390243902439024, -0.5951219512195122, -0.01951219512195122, -0.21951219512195122], [-0.0, -0.0, -0.014563106796116505, -0.09223300970873786, 0.038834951456310676, -0.13592233009708737, 0.06310679611650485, -0.22330097087378642, 0.06796116504854369, -0.27184466019417475, -0.07766990291262135, -0.1262135922330097, -0.09223300970873786, -0.21844660194174756, 0.009708737864077669, -0.22330097087378642, 0.024271844660194174, -0.3058252427184466, 0.014563106796116505, -0.45145631067961167, 0.009708737864077669, -0.5922330097087378, -0.04854368932038835, -0.30097087378640774, -0.038834951456310676, -0.45145631067961167, -0.038834951456310676, -0.587378640776699, -0.014563106796116505, -0.21359223300970873],[-0.0, -0.0, -0.0, -0.11290322580645161, 0.06989247311827956, -0.13440860215053763, 0.13440860215053763, -0.08602150537634409, 0.06451612903225806, -0.026881720430107527, -0.06989247311827956, -0.15591397849462366, -0.10215053763440861, -0.26344086021505375, -0.10215053763440861, -0.34946236559139787, 0.043010752688172046, -0.3548387096774194, 0.03225806451612903, -0.5268817204301075, 0.03225806451612903, -0.6666666666666666, -0.03763440860215054, -0.3548387096774194, -0.04838709677419355, -0.5268817204301075, -0.06451612903225806, -0.6774193548387096, 0.005376344086021506, -0.25268817204301075], [-0.0, -0.0, -0.0, -0.13924050632911392, 0.08227848101265822, -0.15822784810126583, 0.17721518987341772, -0.08860759493670886, 0.06962025316455696, -0.03164556962025317, -0.0949367088607595, -0.189873417721519, -0.12025316455696203, -0.33544303797468356, -0.10126582278481013, -0.46835443037974683, 0.056962025316455694, -0.4177215189873418, 0.06329113924050633, -0.6392405063291139, 0.08227848101265822, -0.8354430379746836, -0.0379746835443038, -0.4240506329113924, -0.0189873417721519, -0.6455696202531646, -0.006329113924050633, -0.8164556962025317, 0.006329113924050633, -0.2974683544303797], [-0.0, -0.0, -0.0, -0.1476510067114094, 0.087248322147651, -0.174496644295302, 0.18120805369127516, -0.08053691275167785, 0.0738255033557047, -0.03355704697986577, -0.09395973154362416, -0.19463087248322147, -0.1342281879194631, -0.3422818791946309, 0.013422818791946308, -0.08053691275167785, 0.03355704697986577, -0.4563758389261745, 0.026845637583892617, -0.7046979865771812, 0.013422818791946308, -0.8993288590604027, -0.0738255033557047, -0.4563758389261745, -0.12751677852348994, -0.6778523489932886, -0.11409395973154363, -0.8590604026845637, -0.013422818791946308, -0.3221476510067114], [-0.0, -0.0, -0.005952380952380952, -0.125, 0.07142857142857142, -0.1488095238095238, 0.15476190476190477, -0.07142857142857142, 0.05952380952380952, -0.017857142857142856, -0.08333333333333333, -0.17857142857142858, -0.1130952380952381, -0.30357142857142855, -0.1130952380952381, -0.4166666666666667, 0.041666666666666664, -0.39285714285714285, 0.06547619047619048, -0.5892857142857143, 0.06547619047619048, -0.7440476190476191, -0.041666666666666664, -0.39880952380952384, -0.03571428571428571, -0.5952380952380952, -0.017857142857142856, -0.7559523809523809, -0.005952380952380952, -0.27976190476190477], [-0.0, -0.0, -0.023255813953488372, -0.08837209302325581, 0.023255813953488372, -0.11627906976744186, 0.07906976744186046, -0.06976744186046512, 0.027906976744186046, -0.023255813953488372, -0.08372093023255814, -0.11627906976744186, -0.11627906976744186, -0.20465116279069767, -0.13023255813953488, -0.2930232558139535, -0.013953488372093023, -0.2930232558139535, -0.023255813953488372, -0.4325581395348837, -0.03255813953488372, -0.5581395348837209, -0.08372093023255814, -0.2837209302325581, -0.11162790697674418, -0.42790697674418604, -0.12558139534883722, -0.5348837209302325, -0.037209302325581395, -0.20465116279069767], [-0.0, -0.0, -0.0, -0.058823529411764705, 0.03806228373702422, -0.0657439446366782, 0.0726643598615917, -0.03806228373702422, 0.03460207612456748, -0.01384083044982699, -0.04152249134948097, -0.08650519031141868, -0.05190311418685121, -0.1453287197231834, -0.04498269896193772, -0.2041522491349481, 0.02422145328719723, -0.1903114186851211, 0.03460207612456748, -0.2837370242214533, 0.04844290657439446, -0.3633217993079585, -0.020761245674740483, -0.19377162629757785, -0.020761245674740483, -0.28027681660899656, -0.010380622837370242, -0.356401384083045, 0.0034602076124567475, -0.13494809688581316], [-0.0, -0.0, -0.014970059880239521, -0.05089820359281437, 0.011976047904191617, -0.059880239520958084, 0.05089820359281437, -0.05089820359281437, 0.017964071856287425, -0.029940119760479042, -0.04491017964071856, -0.06287425149700598, -0.059880239520958084, -0.11077844311377245, -0.05089820359281437, -0.15868263473053892, -0.008982035928143712, -0.1497005988023952, -0.02694610778443114, -0.218562874251497, -0.02694610778443114, -0.2904191616766467, -0.03592814371257485, -0.1497005988023952, -0.029940119760479042, -0.2215568862275449, -0.029940119760479042, -0.2844311377245509, -0.017964071856287425, -0.10778443113772455], [-0.0, -0.0, -0.012307692307692308, -0.052307692307692305, 0.012307692307692308, -0.06153846153846154, 0.046153846153846156, -0.04923076923076923, 0.024615384615384615, -0.024615384615384615, -0.04, -0.06769230769230769, -0.052307692307692305, -0.12, -0.04, -0.1723076923076923, 0.006153846153846154, -0.16, 0.024615384615384615, -0.23692307692307693, 0.04, -0.30153846153846153, -0.024615384615384615, -0.16307692307692306, -0.027692307692307693, -0.24, -0.033846153846153845, -0.3046153846153846, -0.009230769230769232, -0.11384615384615385], [-0.0, -0.0, -0.009771986970684038, -0.05537459283387622, 0.02280130293159609, -0.06840390879478828, 0.06514657980456026, -0.03908794788273615, 0.02280130293159609, -0.02280130293159609, -0.04234527687296417, -0.0749185667752443, -0.05537459283387622, -0.1237785016286645, -0.05211726384364821, -0.1758957654723127, 0.009771986970684038, -0.1758957654723127, -0.009771986970684038, -0.26058631921824105, -0.016286644951140065, -0.3355048859934853, -0.02280130293159609, -0.1758957654723127, -0.02280130293159609, -0.26384364820846906, -0.019543973941368076, -0.33876221498371334, -0.006514657980456026, -0.1237785016286645], [-0.0, -0.0, -0.004672897196261682, -0.09345794392523364, 0.04672897196261682, -0.11214953271028037, 0.11214953271028037, -0.056074766355140186, 0.028037383177570093, -0.014018691588785047, -0.07009345794392523, -0.1261682242990654, -0.08878504672897196, -0.21495327102803738, 0.037383177570093455, -0.037383177570093455, 0.028037383177570093, -0.29439252336448596, 0.028037383177570093, -0.4439252336448598, 0.018691588785046728, -0.5700934579439252, -0.037383177570093455, -0.29439252336448596, -0.056074766355140186, -0.43457943925233644, -0.07476635514018691, -0.5420560747663551, -0.004672897196261682, -0.205607476635514], [-0.0, -0.0, -0.006289308176100629, -0.1320754716981132, 0.07547169811320754, -0.16352201257861634, 0.15723270440251572, -0.08176100628930817, 0.05660377358490566, -0.031446540880503145, -0.09433962264150944, -0.18238993710691823, -0.11320754716981132, -0.31446540880503143, -0.11320754716981132, -0.4528301886792453, 0.050314465408805034, -0.42138364779874216, 0.06289308176100629, -0.6415094339622641, 0.031446540880503145, -0.8050314465408805, -0.03773584905660377, -0.4276729559748428, -0.031446540880503145, -0.6415094339622641, -0.031446540880503145, -0.8238993710691824, -0.0, -0.29559748427672955], [-0.0, -0.0, -0.0, -0.18032786885245902, 0.11475409836065574, -0.22131147540983606, 0.20491803278688525, -0.09016393442622951, 0.06557377049180328, -0.04918032786885246, -0.12295081967213115, -0.2540983606557377, -0.1557377049180328, -0.45081967213114754, 0.040983606557377046, -0.06557377049180328, 0.07377049180327869, -0.5655737704918032, 0.04918032786885246, -0.8770491803278688, 0.02459016393442623, -1.139344262295082, -0.06557377049180328, -0.5737704918032787, -0.07377049180327869, -0.860655737704918, -0.09016393442622951, -1.1065573770491803, -0.0, -0.4098360655737705], [-0.0, -0.0, 0.005952380952380952, -0.125, 0.07738095238095238, -0.15476190476190477, 0.1130952380952381, -0.05357142857142857, 0.041666666666666664, -0.03571428571428571, -0.06547619047619048, -0.17857142857142858, -0.09523809523809523, -0.30952380952380953, 0.017857142857142856, -0.05952380952380952, 0.03571428571428571, -0.39880952380952384, 0.047619047619047616, -0.6011904761904762, 0.10119047619047619, -0.7976190476190477, -0.041666666666666664, -0.40476190476190477, -0.023809523809523808, -0.6011904761904762, -0.005952380952380952, -0.7559523809523809, -0.0, -0.27976190476190477], [-0.0, -0.0, 0.007194244604316547, -0.16546762589928057, 0.08633093525179857, -0.19424460431654678, 0.1223021582733813, -0.07194244604316546, 0.07913669064748201, -0.050359712230215826, -0.06474820143884892, -0.2158273381294964, -0.1079136690647482, -0.3669064748201439, -0.0, -0.06474820143884892, 0.02877697841726619, -0.49640287769784175, -0.014388489208633094, -0.7482014388489209, -0.007194244604316547, -0.9784172661870504, -0.07194244604316546, -0.4892086330935252, -0.06474820143884892, -0.7338129496402878, 0.014388489208633094, -0.935251798561151, -0.007194244604316547, -0.35251798561151076], [-0.0, -0.0, 0.005376344086021506, -0.11290322580645161, 0.06989247311827956, -0.13978494623655913, 0.10215053763440861, -0.04838709677419355, 0.06451612903225806, -0.026881720430107527, -0.05913978494623656, -0.15591397849462366, -0.08064516129032258, -0.26881720430107525, 0.03225806451612903, -0.03763440860215054, 0.03763440860215054, -0.34946236559139787, 0.053763440860215055, -0.532258064516129, 0.10215053763440861, -0.7043010752688172, -0.03763440860215054, -0.3602150537634409, -0.021505376344086023, -0.532258064516129, -0.0, -0.6666666666666666, 0.005376344086021506, -0.24731182795698925], [-0.0, -0.0, 0.004901960784313725, -0.10294117647058823, 0.06372549019607843, -0.11764705882352941, 0.08823529411764706, -0.049019607843137254, 0.04411764705882353, -0.029411764705882353, -0.04411764705882353, -0.13725490196078433, -0.06372549019607843, -0.23529411764705882, 0.024509803921568627, -0.049019607843137254, 0.03431372549019608, -0.3088235294117647, 0.029411764705882353, -0.47058823529411764, 0.0392156862745098, -0.6029411764705882, -0.03431372549019608, -0.3137254901960784, -0.0392156862745098, -0.47058823529411764, -0.03431372549019608, -0.5980392156862745, 0.00980392156862745, -0.22058823529411764], [-0.0, -0.0, -0.0, -0.10204081632653061, 0.061224489795918366, -0.12755102040816327, 0.11224489795918367, -0.061224489795918366, 0.061224489795918366, -0.02040816326530612, -0.0663265306122449, -0.14795918367346939, -0.08673469387755102, -0.25, -0.09183673469387756, -0.3469387755102041, 0.03571428571428571, -0.33163265306122447, 0.05102040816326531, -0.5, 0.07142857142857142, -0.6326530612244898, -0.04081632653061224, -0.33163265306122447, -0.0663265306122449, -0.49489795918367346, -0.08163265306122448, -0.6275510204081632, -0.0, -0.23469387755102042], [-0.0, -0.0, -0.03333333333333333, -0.14, 0.06, -0.18, 0.16666666666666666, -0.11333333333333333, 0.06666666666666667, -0.03333333333333333, -0.14, -0.19333333333333333, -0.16666666666666666, -0.34, -0.17333333333333334, -0.47333333333333333, 0.013333333333333334, -0.46, -0.02, -0.6866666666666666, -0.04, -0.88, -0.1, -0.46, -0.12, -0.68, -0.11333333333333333, -0.86, -0.04, -0.32666666666666666], [-0.0, -0.0, -0.05303030303030303, -0.1590909090909091, 0.030303030303030304, -0.19696969696969696, 0.15151515151515152, -0.1590909090909091, 0.07575757575757576, -0.05303030303030303, -0.16666666666666666, -0.21212121212121213, -0.18181818181818182, -0.4015151515151515, -0.1590909090909091, -0.5606060606060606, -0.0, -0.5227272727272727, 0.015151515151515152, -0.7878787878787878, 0.022727272727272728, -1.0151515151515151, -0.09848484848484848, -0.5303030303030303, -0.12121212121212122, -0.7878787878787878, -0.13636363636363635, -1.0227272727272727, -0.05303030303030303, -0.3712121212121212], [-0.0, -0.0, -0.03125, -0.08482142857142858, 0.017857142857142856, -0.11607142857142858, 0.08482142857142858, -0.08928571428571429, 0.03571428571428571, -0.03571428571428571, -0.08928571428571429, -0.11160714285714286, -0.10267857142857142, -0.20089285714285715, -0.10267857142857142, -0.29017857142857145, -0.008928571428571428, -0.2767857142857143, -0.026785714285714284, -0.41517857142857145, -0.03125, -0.5223214285714286, -0.0625, -0.2767857142857143, -0.07142857142857142, -0.41517857142857145, -0.09375, -0.5401785714285714, -0.03571428571428571, -0.19196428571428573], [-0.0, -0.0, -0.014814814814814815, -0.06666666666666667, 0.02962962962962963, -0.07777777777777778, 0.07777777777777778, -0.05185185185185185, 0.025925925925925925, -0.018518518518518517, -0.06296296296296296, -0.09259259259259259, -0.06666666666666667, -0.16666666666666666, -0.014814814814814815, -0.08148148148148149, 0.007407407407407408, -0.2111111111111111, 0.018518518518518517, -0.3111111111111111, 0.025925925925925925, -0.40370370370370373, -0.03333333333333333, -0.21481481481481482, -0.03333333333333333, -0.3111111111111111, -0.037037037037037035, -0.4, -0.014814814814814815, -0.14814814814814814], [-0.0, -0.0, -0.017964071856287425, -0.05089820359281437, 0.005988023952095809, -0.05688622754491018, 0.05089820359281437, -0.04790419161676647, 0.017964071856287425, -0.017964071856287425, -0.05389221556886228, -0.059880239520958084, -0.0688622754491018, -0.10479041916167664, -0.0658682634730539, -0.15568862275449102, -0.017964071856287425, -0.1467065868263473, -0.029940119760479042, -0.218562874251497, -0.041916167664670656, -0.2874251497005988, -0.04491017964071856, -0.1467065868263473, -0.041916167664670656, -0.2215568862275449, -0.04790419161676647, -0.2874251497005988, -0.02694610778443114, -0.10179640718562874], [-0.0, -0.0, -0.013468013468013467, -0.06060606060606061, 0.016835016835016835, -0.06734006734006734, 0.05723905723905724, -0.04713804713804714, 0.016835016835016835, -0.020202020202020204, -0.04713804713804714, -0.08417508417508418, -0.04713804713804714, -0.1447811447811448, -0.03367003367003367, -0.2053872053872054, 0.010101010101010102, -0.18181818181818182, 0.02356902356902357, -0.26262626262626265, 0.02356902356902357, -0.3333333333333333, -0.020202020202020204, -0.18855218855218855, -0.016835016835016835, -0.265993265993266, -0.026936026936026935, -0.3400673400673401, -0.010101010101010102, -0.13131313131313133], [-0.0, -0.0, -0.014534883720930232, -0.046511627906976744, 0.00872093023255814, -0.055232558139534885, 0.040697674418604654, -0.05232558139534884, 0.014534883720930232, -0.01744186046511628, -0.046511627906976744, -0.061046511627906974, -0.055232558139534885, -0.10755813953488372, -0.05232558139534884, -0.1511627906976744, -0.011627906976744186, -0.14244186046511628, -0.01744186046511628, -0.20930232558139536, -0.02616279069767442, -0.2761627906976744, -0.03488372093023256, -0.14244186046511628, -0.029069767441860465, -0.21220930232558138, -0.029069767441860465, -0.27325581395348836, -0.020348837209302327, -0.10174418604651163], [-0.0, -0.0, -0.0031645569620253164, -0.05379746835443038, 0.02531645569620253, -0.06329113924050633, 0.06645569620253164, -0.04746835443037975, 0.02531645569620253, -0.02531645569620253, -0.0379746835443038, -0.07278481012658228, -0.04746835443037975, -0.12025316455696203, -0.04430379746835443, -0.17721518987341772, 0.0189873417721519, -0.16139240506329114, 0.03164556962025317, -0.25, 0.04430379746835443, -0.31962025316455694, -0.0189873417721519, -0.16455696202531644, -0.022151898734177215, -0.25, -0.02531645569620253, -0.3259493670886076, -0.0, -0.11392405063291139], [-0.0, -0.0, -0.010067114093959731, -0.05704697986577181, 0.026845637583892617, -0.06711409395973154, 0.05704697986577181, -0.053691275167785234, 0.020134228187919462, -0.026845637583892617, -0.0436241610738255, -0.08053691275167785, -0.06040268456375839, -0.1342281879194631, -0.06040268456375839, -0.1912751677852349, 0.010067114093959731, -0.18120805369127516, -0.006711409395973154, -0.2684563758389262, -0.016778523489932886, -0.348993288590604, -0.026845637583892617, -0.18120805369127516, -0.026845637583892617, -0.27181208053691275, -0.03355704697986577, -0.348993288590604, -0.006711409395973154, -0.12751677852348994], [-0.0, -0.0, -0.008264462809917356, -0.07851239669421488, 0.04132231404958678, -0.09504132231404959, 0.08264462809917356, -0.05371900826446281, 0.028925619834710745, -0.01652892561983471, -0.05371900826446281, -0.1115702479338843, -0.07024793388429752, -0.1859504132231405, -0.06611570247933884, -0.256198347107438, 0.02066115702479339, -0.25206611570247933, 0.03305785123966942, -0.3760330578512397, 0.05371900826446281, -0.4834710743801653, -0.028925619834710745, -0.256198347107438, -0.02066115702479339, -0.371900826446281, -0.02066115702479339, -0.45867768595041325, -0.0, -0.18181818181818182], [-0.0, -0.0, -0.0048543689320388345, -0.09223300970873786, 0.04854368932038835, -0.11165048543689321, 0.10194174757281553, -0.05825242718446602, 0.03398058252427184, -0.019417475728155338, -0.06796116504854369, -0.13106796116504854, -0.08737864077669903, -0.22815533980582525, -0.09223300970873786, -0.3155339805825243, 0.02912621359223301, -0.30097087378640774, 0.019417475728155338, -0.46601941747572817, 0.019417475728155338, -0.6019417475728155, -0.038834951456310676, -0.30097087378640774, -0.038834951456310676, -0.46601941747572817, -0.03398058252427184, -0.5970873786407767, -0.0048543689320388345, -0.21359223300970873], [-0.0, -0.0, 0.006369426751592357, -0.1464968152866242, 0.07643312101910828, -0.16560509554140126, 0.12738853503184713, -0.07006369426751592, 0.050955414012738856, -0.044585987261146494, -0.07006369426751592, -0.20382165605095542, -0.10828025477707007, -0.3248407643312102, 0.025477707006369428, -0.050955414012738856, 0.044585987261146494, -0.42038216560509556, 0.044585987261146494, -0.643312101910828, 0.044585987261146494, -0.8407643312101911, -0.044585987261146494, -0.42038216560509556, -0.06369426751592357, -0.643312101910828, -0.07006369426751592, -0.821656050955414, 0.006369426751592357, -0.29936305732484075], [-0.0, -0.0, 0.014778325123152709, -0.10837438423645321, 0.054187192118226604, -0.12315270935960591, 0.059113300492610835, -0.14285714285714285, 0.029556650246305417, -0.06896551724137931, -0.029556650246305417, -0.1330049261083744, -0.059113300492610835, -0.2315270935960591, -0.06896551724137931, -0.31527093596059114, 0.019704433497536946, -0.3103448275862069, 0.029556650246305417, -0.46798029556650245, 0.06896551724137931, -0.6206896551724138, -0.029556650246305417, -0.31527093596059114, -0.019704433497536946, -0.45320197044334976, 0.0049261083743842365, -0.5714285714285714, 0.009852216748768473, -0.22167487684729065], [-0.0, -0.0, 0.022857142857142857, -0.13142857142857142, 0.06857142857142857, -0.15428571428571428, 0.08571428571428572, -0.14857142857142858, 0.045714285714285714, -0.06857142857142857, -0.011428571428571429, -0.15428571428571428, 0.045714285714285714, -0.12571428571428572, 0.03428571428571429, -0.09714285714285714, 0.03428571428571429, -0.37714285714285717, 0.011428571428571429, -0.56, 0.02857142857142857, -0.6457142857142857, -0.02857142857142857, -0.37142857142857144, -0.05714285714285714, -0.5428571428571428, 0.011428571428571429, -0.6457142857142857, 0.022857142857142857, -0.26285714285714284], [-0.0, -0.0, -0.0, -0.0975609756097561, 0.05365853658536585, -0.11707317073170732, 0.09268292682926829, -0.04878048780487805, 0.01951219512195122, -0.02926829268292683, -0.06341463414634146, -0.13658536585365855, -0.08292682926829269, -0.23414634146341465, -0.08292682926829269, -0.32682926829268294, 0.02926829268292683, -0.3024390243902439, 0.04878048780487805, -0.4682926829268293, 0.07804878048780488, -0.6, -0.03902439024390244, -0.3121951219512195, -0.02926829268292683, -0.4634146341463415, -0.02926829268292683, -0.5853658536585366, -0.0, -0.21951219512195122], [-0.0, -0.0, -0.0213903743315508, -0.10695187165775401, 0.0481283422459893, -0.13368983957219252, 0.10695187165775401, -0.06951871657754011, 0.0374331550802139, -0.026737967914438502, -0.0962566844919786, -0.15508021390374332, -0.12299465240641712, -0.2620320855614973, -0.12834224598930483, -0.36363636363636365, 0.0106951871657754, -0.34759358288770054, 0.016042780748663103, -0.5294117647058824, 0.0053475935828877, -0.6737967914438503, -0.0748663101604278, -0.34759358288770054, -0.11229946524064172, -0.5187165775401069, -0.13903743315508021, -0.6470588235294118, -0.026737967914438502, -0.24598930481283424], [-0.0, -0.0, -0.015873015873015872, -0.07142857142857142, 0.027777777777777776, -0.09126984126984126, 0.07936507936507936, -0.05555555555555555, 0.023809523809523808, -0.015873015873015872, -0.06349206349206349, -0.09523809523809523, -0.07936507936507936, -0.1746031746031746, -0.01984126984126984, -0.05952380952380952, 0.011904761904761904, -0.23412698412698413, 0.015873015873015872, -0.34523809523809523, 0.015873015873015872, -0.43253968253968256, -0.031746031746031744, -0.23809523809523808, -0.031746031746031744, -0.3492063492063492, -0.031746031746031744, -0.4444444444444444, -0.011904761904761904, -0.1626984126984127], [-0.0, -0.0, -0.013422818791946308, -0.05704697986577181, 0.020134228187919462, -0.06711409395973154, 0.06040268456375839, -0.050335570469798654, 0.026845637583892617, -0.020134228187919462, -0.053691275167785234, -0.07718120805369127, -0.06040268456375839, -0.13758389261744966, -0.050335570469798654, -0.19798657718120805, 0.003355704697986577, -0.18120805369127516, 0.010067114093959731, -0.2684563758389262, 0.010067114093959731, -0.3523489932885906, -0.03355704697986577, -0.18120805369127516, -0.03691275167785235, -0.2684563758389262, -0.04697986577181208, -0.3422818791946309, -0.013422818791946308, -0.12416107382550336], [-0.0, -0.0, -0.00872093023255814, -0.046511627906976744, 0.020348837209302327, -0.05232558139534884, 0.04941860465116279, -0.05813953488372093, 0.01744186046511628, -0.03488372093023256, -0.040697674418604654, -0.061046511627906974, -0.046511627906976744, -0.10755813953488372, -0.01744186046511628, -0.06395348837209303, 0.0029069767441860465, -0.14244186046511628, -0.005813953488372093, -0.20930232558139536, -0.023255813953488372, -0.2703488372093023, -0.023255813953488372, -0.14534883720930233, -0.029069767441860465, -0.21220930232558138, -0.0436046511627907, -0.27325581395348836, -0.011627906976744186, -0.10174418604651163], [-0.0, -0.0, -0.018072289156626505, -0.0572289156626506, 0.0030120481927710845, -0.060240963855421686, 0.03313253012048193, -0.045180722891566265, 0.006024096385542169, -0.024096385542168676, -0.045180722891566265, -0.06927710843373494, -0.045180722891566265, -0.12349397590361445, -0.030120481927710843, -0.16566265060240964, -0.0030120481927710845, -0.14759036144578314, 0.006024096385542169, -0.21084337349397592, -0.015060240963855422, -0.28012048192771083, -0.02710843373493976, -0.1566265060240964, -0.024096385542168676, -0.21686746987951808, -0.03313253012048193, -0.28012048192771083, -0.018072289156626505, -0.11144578313253012], [-0.0, -0.0, -0.012012012012012012, -0.05405405405405406, 0.015015015015015015, -0.06006006006006006, 0.05105105105105105, -0.05405405405405406, 0.015015015015015015, -0.02702702702702703, -0.03903903903903904, -0.06906906906906907, -0.05405405405405406, -0.11711711711711711, -0.04804804804804805, -0.16216216216216217, -0.003003003003003003, -0.15015015015015015, -0.015015015015015015, -0.21621621621621623, -0.015015015015015015, -0.2852852852852853, -0.03003003003003003, -0.15315315315315314, -0.018018018018018018, -0.21921921921921922, -0.018018018018018018, -0.27927927927927926, -0.012012012012012012, -0.10810810810810811], [-0.0, -0.0, -0.006153846153846154, -0.052307692307692305, 0.021538461538461538, -0.05846153846153846, 0.055384615384615386, -0.04, 0.018461538461538463, -0.012307692307692308, -0.036923076923076927, -0.07076923076923076, -0.04923076923076923, -0.12307692307692308, -0.04, -0.17846153846153845, 0.021538461538461538, -0.15692307692307692, 0.04, -0.23692307692307693, 0.05846153846153846, -0.3046153846153846, -0.015384615384615385, -0.16307692307692306, -0.009230769230769232, -0.24, -0.009230769230769232, -0.3046153846153846, -0.0, -0.11384615384615385], [-0.0, -0.0, -0.006711409395973154, -0.05704697986577181, 0.030201342281879196, -0.07046979865771812, 0.06040268456375839, -0.06375838926174497, 0.03355704697986577, -0.03691275167785235, -0.04697986577181208, -0.08053691275167785, -0.05704697986577181, -0.13758389261744966, -0.053691275167785234, -0.19463087248322147, 0.010067114093959731, -0.18120805369127516, 0.003355704697986577, -0.2651006711409396, -0.006711409395973154, -0.348993288590604, -0.026845637583892617, -0.17785234899328858, -0.026845637583892617, -0.2684563758389262, -0.030201342281879196, -0.3523489932885906, -0.006711409395973154, -0.12751677852348994], [-0.0, -0.0, -0.017241379310344827, -0.08620689655172414, 0.03879310344827586, -0.10344827586206896, 0.08620689655172414, -0.05603448275862069, 0.03017241379310345, -0.01293103448275862, -0.06465517241379311, -0.1206896551724138, -0.08189655172413793, -0.20689655172413793, -0.07327586206896551, -0.28448275862068967, 0.021551724137931036, -0.2672413793103448, 0.034482758620689655, -0.40086206896551724, 0.05603448275862069, -0.5172413793103449, -0.034482758620689655, -0.2672413793103448, -0.02586206896551724, -0.39655172413793105, -0.021551724137931036, -0.4956896551724138, -0.004310344827586207, -0.1896551724137931], [-0.0, -0.0, 0.02577319587628866, -0.1134020618556701, 0.07216494845360824, -0.12886597938144329, 0.08762886597938144, -0.14948453608247422, 0.041237113402061855, -0.061855670103092786, -0.015463917525773196, -0.14432989690721648, -0.03608247422680412, -0.23711340206185566, 0.015463917525773196, -0.0979381443298969, 0.03608247422680412, -0.3402061855670103, 0.03608247422680412, -0.5051546391752577, 0.02577319587628866, -0.654639175257732, -0.02577319587628866, -0.33505154639175255, -0.05670103092783505, -0.4948453608247423, -0.06701030927835051, -0.6288659793814433, 0.020618556701030927, -0.23195876288659795], [-0.0, -0.0, 0.005681818181818182, -0.125, 0.0625, -0.1534090909090909, 0.09659090909090909, -0.07386363636363637, 0.045454545454545456, -0.03977272727272727, -0.056818181818181816, -0.17613636363636365, -0.07954545454545454, -0.29545454545454547, -0.08522727272727272, -0.4034090909090909, 0.03409090909090909, -0.36363636363636365, 0.017045454545454544, -0.5511363636363636, 0.07386363636363637, -0.7159090909090909, -0.03409090909090909, -0.36363636363636365, -0.03409090909090909, -0.5511363636363636, -0.0, -0.6875, 0.005681818181818182, -0.26704545454545453], [-0.0, -0.0, -0.0, -0.18604651162790697, 0.07751937984496124, -0.21705426356589147, 0.13178294573643412, -0.10077519379844961, 0.05426356589147287, -0.05426356589147287, -0.10077519379844961, -0.24031007751937986, -0.13953488372093023, -0.40310077519379844, -0.17054263565891473, -0.5736434108527132, 0.023255813953488372, -0.5271317829457365, 0.007751937984496124, -0.8217054263565892, -0.0, -1.062015503875969, -0.09302325581395349, -0.5271317829457365, -0.13953488372093023, -0.8062015503875969, -0.10852713178294573, -1.0077519379844961, -0.015503875968992248, -0.3798449612403101], [-0.0, -0.0, -0.007194244604316547, -0.16546762589928057, 0.08633093525179857, -0.18705035971223022, 0.19424460431654678, -0.1079136690647482, 0.08633093525179857, -0.02877697841726619, -0.1079136690647482, -0.22302158273381295, -0.12949640287769784, -0.38848920863309355, -0.11510791366906475, -0.5467625899280576, 0.07194244604316546, -0.48201438848920863, 0.1079136690647482, -0.7266187050359713, 0.10071942446043165, -0.920863309352518, -0.02877697841726619, -0.5035971223021583, -0.0, -0.7410071942446043, 0.007194244604316547, -0.9496402877697842, 0.007194244604316547, -0.35251798561151076], [-0.0, -0.0, -0.02912621359223301, -0.09223300970873786, 0.024271844660194174, -0.12135922330097088, 0.0970873786407767, -0.0825242718446602, 0.03398058252427184, -0.02912621359223301, -0.0970873786407767, -0.12135922330097088, -0.12135922330097088, -0.21844660194174756, -0.12135922330097088, -0.3106796116504854, -0.009708737864077669, -0.30097087378640774, -0.02912621359223301, -0.4563106796116505, -0.04854368932038835, -0.5825242718446602, -0.07281553398058252, -0.2961165048543689, -0.06796116504854369, -0.4563106796116505, -0.07766990291262135, -0.5970873786407767, -0.03398058252427184, -0.2087378640776699], [-0.0, -0.0, -0.03773584905660377, -0.1320754716981132, 0.0440251572327044, -0.15723270440251572, 0.13836477987421383, -0.11320754716981132, 0.05660377358490566, -0.03773584905660377, -0.1320754716981132, -0.1761006289308176, -0.14465408805031446, -0.3270440251572327, -0.11949685534591195, -0.4528301886792453, 0.012578616352201259, -0.42138364779874216, -0.0, -0.6352201257861635, -0.012578616352201259, -0.7924528301886793, -0.06918238993710692, -0.42138364779874216, -0.06918238993710692, -0.6477987421383647, -0.06918238993710692, -0.8490566037735849, -0.03773584905660377, -0.29559748427672955], [-0.0, -0.0, -0.038834951456310676, -0.22330097087378642, 0.07766990291262135, -0.27184466019417475, 0.24271844660194175, -0.20388349514563106, 0.11650485436893204, -0.0970873786407767, -0.20388349514563106, -0.30097087378640774, -0.24271844660194175, -0.5533980582524272, 0.038834951456310676, -0.07766990291262135, 0.05825242718446602, -0.6990291262135923, 0.05825242718446602, -1.0776699029126213, 0.038834951456310676, -1.3689320388349515, -0.0970873786407767, -0.7184466019417476, -0.1650485436893204, -1.0679611650485437, -0.2524271844660194, -1.3786407766990292, -0.02912621359223301, -0.5048543689320388], [-0.0, -0.0, -0.03529411764705882, -0.27058823529411763, 0.1411764705882353, -0.3411764705882353, 0.32941176470588235, -0.2235294117647059, 0.1411764705882353, -0.11764705882352941, -0.23529411764705882, -0.3764705882352941, -0.29411764705882354, -0.6941176470588235, 0.03529411764705882, -0.10588235294117647, 0.058823529411764705, -0.8705882352941177, 0.011764705882352941, -1.3411764705882352, -0.058823529411764705, -1.6705882352941177, -0.1411764705882353, -0.8823529411764706, -0.12941176470588237, -1.3647058823529412, -0.16470588235294117, -1.7294117647058824, -0.047058823529411764, -0.6235294117647059], [-0.0, -0.0, -0.08771929824561403, -0.42105263157894735, 0.17543859649122806, -0.543859649122807, 0.45614035087719296, -0.3157894736842105, 0.19298245614035087, -0.15789473684210525, -0.38596491228070173, -0.631578947368421, -0.45614035087719296, -1.105263157894737, 0.03508771929824561, -0.22807017543859648, 0.10526315789473684, -1.3333333333333333, 0.17543859649122806, -2.017543859649123, 0.19298245614035087, -2.543859649122807, -0.21052631578947367, -1.3859649122807018, -0.19298245614035087, -2.0526315789473686, -0.19298245614035087, -2.6315789473684212, -0.07017543859649122, -0.9824561403508771], [-0.0, -0.0, -0.02531645569620253, -0.13924050632911392, 0.056962025316455694, -0.17088607594936708, 0.14556962025316456, -0.13291139240506328, 0.06329113924050633, -0.05063291139240506, -0.12658227848101267, -0.189873417721519, -0.1518987341772152, -0.33544303797468356, -0.16455696202531644, -0.4620253164556962, 0.0189873417721519, -0.43037974683544306, 0.03164556962025317, -0.6582278481012658, 0.03164556962025317, -0.8481012658227848, -0.08227848101265822, -0.43037974683544306, -0.12658227848101267, -0.6392405063291139, -0.1518987341772152, -0.7974683544303798, -0.03164556962025317, -0.310126582278481], [-0.0, -0.0, -0.01195219123505976, -0.07569721115537849, 0.03187250996015936, -0.08764940239043825, 0.07171314741035857, -0.07569721115537849, 0.02390438247011952, -0.0398406374501992, -0.05976095617529881, -0.09561752988047809, -0.0796812749003984, -0.16334661354581673, -0.08366533864541832, -0.2350597609561753, 0.00398406374501992, -0.23904382470119523, -0.00796812749003984, -0.3545816733067729, -0.00398406374501992, -0.44223107569721115, -0.04780876494023904, -0.23904382470119523, -0.05976095617529881, -0.350597609561753, -0.06374501992031872, -0.4541832669322709, -0.01593625498007968, -0.16334661354581673], [-0.0, -0.0, -0.007407407407407408, -0.06666666666666667, 0.03333333333333333, -0.08148148148148149, 0.07037037037037037, -0.07777777777777778, 0.02962962962962963, -0.14444444444444443, -0.04814814814814815, -0.09259259259259259, -0.06296296296296296, -0.16666666666666666, -0.007407407407407408, -0.15185185185185185, 0.018518518518518517, -0.21851851851851853, 0.03333333333333333, -0.32592592592592595, 0.05555555555555555, -0.4111111111111111, -0.02962962962962963, -0.21851851851851853, -0.037037037037037035, -0.31851851851851853, -0.044444444444444446, -0.3962962962962963, -0.003703703703703704, -0.15555555555555556], [-0.0, -0.0, -0.0031545741324921135, -0.050473186119873815, 0.031545741324921134, -0.05993690851735016, 0.06309148264984227, -0.05993690851735016, 0.028391167192429023, -0.10725552050473186, -0.03785488958990536, -0.07255520504731862, -0.0473186119873817, -0.12302839116719243, -0.0473186119873817, -0.17665615141955837, 0.012618296529968454, -0.1640378548895899, 0.00946372239747634, -0.24290220820189273, 0.006309148264984227, -0.31230283911671924, -0.01892744479495268, -0.1640378548895899, -0.01892744479495268, -0.24605678233438485, -0.01892744479495268, -0.3186119873817035, -0.0, -0.1167192429022082], [-0.0, -0.0, -0.005970149253731343, -0.04776119402985075, 0.01791044776119403, -0.056716417910447764, 0.04776119402985075, -0.06865671641791045, 0.020895522388059702, -0.06567164179104477, -0.03880597014925373, -0.05970149253731343, -0.05373134328358209, -0.1044776119402985, -0.020895522388059702, -0.05970149253731343, -0.0029850746268656717, -0.14328358208955225, -0.014925373134328358, -0.2208955223880597, -0.01791044776119403, -0.29253731343283584, -0.029850746268656716, -0.14328358208955225, -0.023880597014925373, -0.2208955223880597, -0.01791044776119403, -0.28955223880597014, -0.014925373134328358, -0.10149253731343283], [-0.0, -0.0, -0.006920415224913495, -0.058823529411764705, 0.02768166089965398, -0.06920415224913495, 0.0726643598615917, -0.0726643598615917, 0.031141868512110725, -0.06228373702422145, -0.04152249134948097, -0.08304498269896193, -0.05536332179930796, -0.14186851211072665, -0.05190311418685121, -0.2041522491349481, 0.020761245674740483, -0.1903114186851211, 0.03806228373702422, -0.29411764705882354, 0.058823529411764705, -0.370242214532872, -0.020761245674740483, -0.19377162629757785, -0.02422145328719723, -0.2906574394463668, -0.02768166089965398, -0.370242214532872, -0.0, -0.1314878892733564], [-0.0, -0.0, -0.007142857142857143, -0.060714285714285714, 0.03214285714285714, -0.075, 0.060714285714285714, -0.08571428571428572, 0.017857142857142856, -0.05357142857142857, -0.05, -0.08214285714285714, -0.07142857142857142, -0.1392857142857143, -0.07142857142857142, -0.2, 0.007142857142857143, -0.19642857142857142, -0.010714285714285714, -0.29642857142857143, -0.025, -0.3892857142857143, -0.039285714285714285, -0.19285714285714287, -0.03571428571428571, -0.29642857142857143, -0.04285714285714286, -0.38571428571428573, -0.010714285714285714, -0.13214285714285715], [-0.0, -0.0, -0.004672897196261682, -0.09345794392523364, 0.056074766355140186, -0.10747663551401869, 0.08411214953271028, -0.1308411214953271, 0.03271028037383177, -0.102803738317757, -0.06542056074766354, -0.1261682242990654, -0.08411214953271028, -0.21495327102803738, -0.0794392523364486, -0.3037383177570093, 0.03271028037383177, -0.2897196261682243, 0.04205607476635514, -0.43457943925233644, 0.03271028037383177, -0.5420560747663551, -0.03271028037383177, -0.29439252336448596, -0.018691588785046728, -0.4392523364485981, -0.014018691588785047, -0.5560747663551402, -0.0, -0.20093457943925233], [-0.0, -0.0, -0.015228426395939087, -0.09644670050761421, 0.05583756345177665, -0.12690355329949238, 0.04060913705583756, -0.20812182741116753, 0.02030456852791878, -0.2893401015228426, -0.08121827411167512, -0.12690355329949238, -0.116751269035533, -0.22842639593908629, -0.13705583756345177, -0.3248730964467005, -0.0, -0.3248730964467005, -0.01015228426395939, -0.49746192893401014, -0.02030456852791878, -0.6395939086294417, -0.07614213197969544, -0.3197969543147208, -0.1319796954314721, -0.48223350253807107, -0.16243654822335024, -0.6091370558375635, -0.025380710659898477, -0.2233502538071066], [-0.0, -0.0, -0.004878048780487805, -0.0975609756097561, 0.05365853658536585, -0.11707317073170732, 0.07804878048780488, -0.14146341463414633, 0.02926829268292683, -0.0975609756097561, -0.07317073170731707, -0.13170731707317074, -0.09268292682926829, -0.23414634146341465, -0.09268292682926829, -0.33170731707317075, 0.03414634146341464, -0.3121951219512195, 0.05853658536585366, -0.4682926829268293, 0.07317073170731707, -0.5951219512195122, -0.03902439024390244, -0.3170731707317073, -0.03414634146341464, -0.4682926829268293, -0.03414634146341464, -0.5853658536585366, -0.004878048780487805, -0.2146341463414634], [-0.0, -0.0, -0.010256410256410256, -0.1076923076923077, 0.05641025641025641, -0.1282051282051282, 0.08205128205128205, -0.15384615384615385, 0.02564102564102564, -0.1076923076923077, -0.07692307692307693, -0.14871794871794872, -0.09743589743589744, -0.24615384615384617, -0.09230769230769231, -0.3435897435897436, 0.035897435897435895, -0.3282051282051282, 0.041025641025641026, -0.49743589743589745, 0.041025641025641026, -0.6307692307692307, -0.041025641025641026, -0.3333333333333333, -0.05641025641025641, -0.49743589743589745, -0.06666666666666667, -0.6307692307692307, -0.005128205128205128, -0.23076923076923078], [-0.0, -0.0, -0.005076142131979695, -0.09644670050761421, 0.06091370558375635, -0.1218274111675127, 0.07106598984771574, -0.16751269035532995, 0.04060913705583756, -0.1218274111675127, -0.07614213197969544, -0.14213197969543148, -0.09137055837563451, -0.24873096446700507, -0.08629441624365482, -0.350253807106599, 0.03553299492385787, -0.3248730964467005, 0.050761421319796954, -0.49238578680203043, 0.07614213197969544, -0.6345177664974619, -0.04060913705583756, -0.3299492385786802, -0.030456852791878174, -0.49238578680203043, -0.030456852791878174, -0.6142131979695431, -0.0, -0.22842639593908629], [-0.0, -0.0, -0.011235955056179775, -0.11235955056179775, 0.07303370786516854, -0.1404494382022472, 0.08426966292134831, -0.19101123595505617, 0.028089887640449437, -0.11797752808988764, -0.0898876404494382, -0.15730337078651685, -0.11797752808988764, -0.2752808988764045, -0.11235955056179775, -0.38202247191011235, 0.028089887640449437, -0.3707865168539326, 0.02247191011235955, -0.5617977528089888, 0.016853932584269662, -0.7134831460674157, -0.05056179775280899, -0.3707865168539326, -0.06741573033707865, -0.5561797752808989, -0.07865168539325842, -0.7134831460674157, -0.011235955056179775, -0.25842696629213485], [-0.0, -0.0, -0.012578616352201259, -0.1320754716981132, 0.08176100628930817, -0.15723270440251572, 0.10062893081761007, -0.20125786163522014, 0.031446540880503145, -0.12578616352201258, -0.1069182389937107, -0.18238993710691823, -0.12578616352201258, -0.32075471698113206, -0.12578616352201258, -0.44654088050314467, 0.0440251572327044, -0.42138364779874216, 0.07547169811320754, -0.6415094339622641, 0.0880503144654088, -0.8301886792452831, -0.05660377358490566, -0.4339622641509434, -0.050314465408805034, -0.6477987421383647, -0.03773584905660377, -0.8176100628930818, -0.006289308176100629, -0.29559748427672955], [-0.0, -0.0, -0.017857142857142856, -0.125, 0.05357142857142857, -0.1488095238095238, 0.11904761904761904, -0.125, 0.05357142857142857, -0.041666666666666664, -0.10714285714285714, -0.16666666666666666, -0.13095238095238096, -0.30357142857142855, -0.125, -0.4226190476190476, 0.03571428571428571, -0.3869047619047619, 0.023809523809523808, -0.6011904761904762, 0.017857142857142856, -0.7678571428571429, -0.05952380952380952, -0.39285714285714285, -0.08333333333333333, -0.6011904761904762, -0.09523809523809523, -0.7619047619047619, -0.011904761904761904, -0.27380952380952384], [-0.0, -0.0, -0.018867924528301886, -0.1320754716981132, 0.05660377358490566, -0.15723270440251572, 0.12578616352201258, -0.13836477987421383, 0.06289308176100629, -0.05660377358490566, -0.11320754716981132, -0.18238993710691823, -0.13836477987421383, -0.32075471698113206, -0.13836477987421383, -0.44654088050314467, 0.031446540880503145, -0.42138364779874216, 0.031446540880503145, -0.6415094339622641, 0.018867924528301886, -0.8176100628930818, -0.06918238993710692, -0.4276729559748428, -0.0880503144654088, -0.6352201257861635, -0.10062893081761007, -0.8113207547169812, -0.018867924528301886, -0.29559748427672955], [-0.0, -0.0, -0.006329113924050633, -0.13924050632911392, 0.06962025316455696, -0.14556962025316456, 0.13291139240506328, -0.13291139240506328, 0.06329113924050633, -0.056962025316455694, -0.0949367088607595, -0.18354430379746836, -0.12658227848101267, -0.3291139240506329, -0.10759493670886076, -0.45569620253164556, 0.056962025316455694, -0.4240506329113924, 0.05063291139240506, -0.6518987341772152, 0.05063291139240506, -0.8291139240506329, -0.04430379746835443, -0.43037974683544306, -0.06329113924050633, -0.6455696202531646, -0.0949367088607595, -0.8227848101265823, -0.0, -0.2974683544303797], [-0.0, -0.0, -0.023809523809523808, -0.125, 0.041666666666666664, -0.1488095238095238, 0.1130952380952381, -0.1130952380952381, 0.041666666666666664, -0.047619047619047616, -0.1130952380952381, -0.16666666666666666, -0.13690476190476192, -0.30357142857142855, -0.13095238095238096, -0.4166666666666667, 0.03571428571428571, -0.39880952380952384, 0.02976190476190476, -0.6071428571428571, 0.041666666666666664, -0.7738095238095238, -0.06547619047619048, -0.40476190476190477, -0.08333333333333333, -0.6011904761904762, -0.09523809523809523, -0.7619047619047619, -0.017857142857142856, -0.27976190476190477], [-0.0, -0.0, -0.017857142857142856, -0.125, 0.047619047619047616, -0.1488095238095238, 0.11904761904761904, -0.1130952380952381, 0.047619047619047616, -0.047619047619047616, -0.10714285714285714, -0.17261904761904762, -0.13095238095238096, -0.30357142857142855, -0.125, -0.4166666666666667, 0.041666666666666664, -0.39880952380952384, 0.03571428571428571, -0.6071428571428571, 0.047619047619047616, -0.7738095238095238, -0.05357142857142857, -0.40476190476190477, -0.07142857142857142, -0.6071428571428571, -0.08928571428571429, -0.7619047619047619, -0.011904761904761904, -0.27976190476190477], [-0.0, -0.0, -0.012658227848101266, -0.13924050632911392, 0.06329113924050633, -0.15822784810126583, 0.13291139240506328, -0.13924050632911392, 0.06962025316455696, -0.056962025316455694, -0.10759493670886076, -0.189873417721519, -0.12658227848101267, -0.3227848101265823, -0.12658227848101267, -0.44936708860759494, 0.0379746835443038, -0.4240506329113924, 0.0379746835443038, -0.6455696202531646, 0.03164556962025317, -0.8227848101265823, -0.056962025316455694, -0.43670886075949367, -0.0759493670886076, -0.6392405063291139, -0.0949367088607595, -0.810126582278481, -0.012658227848101266, -0.3037974683544304], [-0.0, -0.0, -0.03125, -0.125, 0.05, -0.1625, 0.11875, -0.1125, 0.05, -0.04375, -0.11875, -0.175, -0.1375, -0.31875, -0.1375, -0.4375, 0.01875, -0.41875, 0.025, -0.63125, 0.03125, -0.8125, -0.06875, -0.425, -0.06875, -0.63125, -0.05, -0.80625, -0.03125, -0.29375], [-0.0, -0.0, -0.018867924528301886, -0.1320754716981132, 0.06289308176100629, -0.16352201257861634, 0.13836477987421383, -0.11320754716981132, 0.05660377358490566, -0.0440251572327044, -0.1069182389937107, -0.18238993710691823, -0.13836477987421383, -0.3270440251572327, -0.12578616352201258, -0.44654088050314467, 0.03773584905660377, -0.42138364779874216, 0.031446540880503145, -0.6289308176100629, 0.031446540880503145, -0.8176100628930818, -0.050314465408805034, -0.4276729559748428, -0.050314465408805034, -0.6352201257861635, -0.050314465408805034, -0.8113207547169812, -0.012578616352201259, -0.3018867924528302], [-0.0, -0.0, -0.023809523809523808, -0.125, 0.047619047619047616, -0.14285714285714285, 0.10119047619047619, -0.13690476190476192, 0.03571428571428571, -0.07142857142857142, -0.10714285714285714, -0.16666666666666666, -0.13690476190476192, -0.2976190476190476, -0.125, -0.4166666666666667, 0.02976190476190476, -0.39285714285714285, 0.023809523809523808, -0.5952380952380952, 0.017857142857142856, -0.7619047619047619, -0.05952380952380952, -0.39880952380952384, -0.05952380952380952, -0.5952380952380952, -0.05357142857142857, -0.7619047619047619, -0.017857142857142856, -0.27380952380952384], [-0.0, -0.0, -0.01775147928994083, -0.11834319526627218, 0.05917159763313609, -0.1301775147928994, 0.07692307692307693, -0.16568047337278108, 0.023668639053254437, -0.09467455621301775, -0.10059171597633136, -0.15976331360946747, -0.13609467455621302, -0.28994082840236685, -0.1301775147928994, -0.40828402366863903, 0.023668639053254437, -0.3905325443786982, 0.01775147928994083, -0.591715976331361, 0.01775147928994083, -0.7633136094674556, -0.05917159763313609, -0.39644970414201186, -0.05917159763313609, -0.591715976331361, -0.05325443786982249, -0.757396449704142, -0.011834319526627219, -0.27218934911242604], [-0.0, -0.0, -0.0125, -0.125, 0.06875, -0.1625, 0.0625, -0.24375, 0.03125, -0.15, -0.1, -0.175, -0.1375, -0.3125, -0.13125, -0.4375, 0.025, -0.41875, 0.025, -0.63125, 0.025, -0.80625, -0.0625, -0.425, -0.0625, -0.63125, -0.05625, -0.80625, -0.0125, -0.29375], [-0.0, -0.0, -0.011764705882352941, -0.11176470588235295, 0.06470588235294118, -0.14705882352941177, 0.058823529411764705, -0.2235294117647059, 0.029411764705882353, -0.12941176470588237, -0.09411764705882353, -0.1588235294117647, -0.12941176470588237, -0.28823529411764703, -0.12352941176470589, -0.40588235294117647, 0.023529411764705882, -0.38823529411764707, 0.023529411764705882, -0.5882352941176471, 0.01764705882352941, -0.7529411764705882, -0.058823529411764705, -0.38823529411764707, -0.058823529411764705, -0.5882352941176471, -0.052941176470588235, -0.7529411764705882, -0.011764705882352941, -0.27058823529411763], [-0.0, -0.0, -0.00625, -0.125, 0.075, -0.1625, 0.075, -0.23125, 0.03125, -0.14375, -0.1, -0.18125, -0.13125, -0.3125, -0.125, -0.4375, 0.03125, -0.41875, 0.03125, -0.63125, 0.025, -0.80625, -0.05625, -0.425, -0.05625, -0.63125, -0.05, -0.80625, -0.00625, -0.29375], [-0.0, -0.0, -0.011764705882352941, -0.11176470588235295, 0.07058823529411765, -0.13529411764705881, 0.07647058823529412, -0.20588235294117646, 0.029411764705882353, -0.11764705882352941, -0.09411764705882353, -0.1588235294117647, -0.12941176470588237, -0.28823529411764703, -0.11764705882352941, -0.40588235294117647, 0.029411764705882353, -0.38823529411764707, 0.023529411764705882, -0.5882352941176471, 0.023529411764705882, -0.7529411764705882, -0.052941176470588235, -0.3941176470588235, -0.052941176470588235, -0.5882352941176471, -0.047058823529411764, -0.7529411764705882, -0.0058823529411764705, -0.27058823529411763], [-0.0, -0.0, -0.01694915254237288, -0.11864406779661017, 0.05649717514124294, -0.12429378531073447, 0.06779661016949153, -0.1638418079096045, 0.005649717514124294, -0.096045197740113, -0.0903954802259887, -0.15254237288135594, -0.12429378531073447, -0.2711864406779661, -0.11864406779661017, -0.384180790960452, 0.02824858757062147, -0.3728813559322034, 0.01694915254237288, -0.559322033898305, 0.01694915254237288, -0.7175141242937854, -0.05649717514124294, -0.3728813559322034, -0.05649717514124294, -0.559322033898305, -0.05084745762711865, -0.7175141242937854, -0.011299435028248588, -0.2542372881355932], [-0.0, -0.0, -0.017964071856287425, -0.1317365269461078, 0.059880239520958084, -0.1377245508982036, 0.0658682634730539, -0.17365269461077845, -0.0, -0.10179640718562874, -0.09580838323353294, -0.16766467065868262, -0.1317365269461078, -0.2934131736526946, -0.1317365269461078, -0.41317365269461076, 0.029940119760479042, -0.39520958083832336, 0.023952095808383235, -0.5988023952095808, 0.017964071856287425, -0.7664670658682635, -0.059880239520958084, -0.40119760479041916, -0.059880239520958084, -0.5988023952095808, -0.05389221556886228, -0.7664670658682635, -0.011976047904191617, -0.2754491017964072], [-0.0, -0.0, -0.023952095808383235, -0.1317365269461078, 0.04790419161676647, -0.1377245508982036, 0.0658682634730539, -0.16766467065868262, -0.005988023952095809, -0.10179640718562874, -0.10179640718562874, -0.16766467065868262, -0.1377245508982036, -0.2934131736526946, -0.1377245508982036, -0.41317365269461076, 0.023952095808383235, -0.40119760479041916, 0.011976047904191617, -0.5988023952095808, 0.011976047904191617, -0.7664670658682635, -0.0658682634730539, -0.40119760479041916, -0.0658682634730539, -0.5988023952095808, -0.059880239520958084, -0.7664670658682635, -0.017964071856287425, -0.2754491017964072], [-0.0, -0.0, -0.01694915254237288, -0.11864406779661017, 0.05649717514124294, -0.12429378531073447, 0.06779661016949153, -0.15819209039548024, 0.005649717514124294, -0.0903954802259887, -0.0903954802259887, -0.15254237288135594, -0.12429378531073447, -0.2768361581920904, -0.11864406779661017, -0.3898305084745763, 0.02824858757062147, -0.3672316384180791, 0.01694915254237288, -0.559322033898305, 0.01694915254237288, -0.7175141242937854, -0.05649717514124294, -0.3728813559322034, -0.05649717514124294, -0.559322033898305, -0.05084745762711865, -0.7175141242937854, -0.011299435028248588, -0.2542372881355932], [-0.0, -0.0, -0.023809523809523808, -0.125, 0.047619047619047616, -0.14285714285714285, 0.08928571428571429, -0.08928571428571429, 0.03571428571428571, -0.041666666666666664, -0.1130952380952381, -0.16666666666666666, -0.14285714285714285, -0.2916666666666667, -0.03571428571428571, -0.09523809523809523, 0.017857142857142856, -0.39880952380952384, 0.005952380952380952, -0.5952380952380952, 0.005952380952380952, -0.7619047619047619, -0.07142857142857142, -0.39880952380952384, -0.07142857142857142, -0.5952380952380952, -0.06547619047619048, -0.7619047619047619, -0.02976190476190476, -0.27380952380952384], [-0.0, -0.0, -0.023809523809523808, -0.125, 0.047619047619047616, -0.14285714285714285, 0.09523809523809523, -0.07738095238095238, 0.02976190476190476, -0.02976190476190476, -0.1130952380952381, -0.16666666666666666, -0.14285714285714285, -0.2976190476190476, -0.02976190476190476, -0.08928571428571429, 0.017857142857142856, -0.39285714285714285, 0.011904761904761904, -0.5952380952380952, 0.005952380952380952, -0.7619047619047619, -0.07142857142857142, -0.39880952380952384, -0.07142857142857142, -0.5952380952380952, -0.06547619047619048, -0.7619047619047619, -0.02976190476190476, -0.27380952380952384], [-0.0, -0.0, -0.011834319526627219, -0.11834319526627218, 0.0650887573964497, -0.14792899408284024, 0.15384615384615385, -0.08875739644970414, 0.07692307692307693, -0.03550295857988166, -0.10059171597633136, -0.17159763313609466, -0.1301775147928994, -0.30177514792899407, -0.01775147928994083, -0.08875739644970414, 0.029585798816568046, -0.39644970414201186, 0.023668639053254437, -0.591715976331361, 0.01775147928994083, -0.757396449704142, -0.05917159763313609, -0.39644970414201186, -0.05917159763313609, -0.591715976331361, -0.05325443786982249, -0.757396449704142, -0.011834319526627219, -0.2781065088757396], [-0.0, -0.0, -0.012578616352201259, -0.1320754716981132, 0.06918238993710692, -0.16352201257861634, 0.15723270440251572, -0.1069182389937107, 0.08176100628930817, -0.050314465408805034, -0.11949685534591195, -0.18867924528301888, -0.14465408805031446, -0.32075471698113206, -0.018867924528301886, -0.10062893081761007, 0.025157232704402517, -0.4276729559748428, 0.012578616352201259, -0.6352201257861635, 0.012578616352201259, -0.8113207547169812, -0.06918238993710692, -0.4276729559748428, -0.06918238993710692, -0.6352201257861635, -0.06289308176100629, -0.8113207547169812, -0.018867924528301886, -0.3018867924528302], [-0.0, -0.0, -0.006289308176100629, -0.1320754716981132, 0.07547169811320754, -0.15723270440251572, 0.1509433962264151, -0.12578616352201258, 0.031446540880503145, -0.10062893081761007, -0.10062893081761007, -0.18238993710691823, -0.1320754716981132, -0.31446540880503143, -0.1320754716981132, -0.4339622641509434, 0.03773584905660377, -0.41509433962264153, 0.025157232704402517, -0.6289308176100629, 0.025157232704402517, -0.8113207547169812, -0.05660377358490566, -0.42138364779874216, -0.06289308176100629, -0.6289308176100629, -0.050314465408805034, -0.8050314465408805, -0.006289308176100629, -0.29559748427672955], [-0.0, -0.0, -0.011834319526627219, -0.11834319526627218, 0.05917159763313609, -0.15976331360946747, 0.1301775147928994, -0.1301775147928994, 0.05325443786982249, -0.14792899408284024, -0.10059171597633136, -0.16568047337278108, -0.1301775147928994, -0.28994082840236685, -0.1242603550295858, -0.40828402366863903, 0.023668639053254437, -0.3905325443786982, 0.023668639053254437, -0.591715976331361, 0.01775147928994083, -0.757396449704142, -0.05325443786982249, -0.3905325443786982, -0.05917159763313609, -0.591715976331361, -0.05325443786982249, -0.757396449704142, -0.011834319526627219, -0.27218934911242604], [-0.0, -0.0, -0.011834319526627219, -0.11834319526627218, 0.0650887573964497, -0.15384615384615385, 0.1301775147928994, -0.1301775147928994, 0.03550295857988166, -0.11834319526627218, -0.09467455621301775, -0.16568047337278108, -0.1301775147928994, -0.28994082840236685, -0.1242603550295858, -0.40828402366863903, 0.029585798816568046, -0.3905325443786982, 0.023668639053254437, -0.591715976331361, 0.01775147928994083, -0.757396449704142, -0.05325443786982249, -0.3905325443786982, -0.05917159763313609, -0.591715976331361, -0.05325443786982249, -0.757396449704142, -0.011834319526627219, -0.27218934911242604], [-0.0, -0.0, -0.011904761904761904, -0.125, 0.05952380952380952, -0.15476190476190477, 0.05357142857142857, -0.25595238095238093, 0.05357142857142857, -0.34523809523809523, -0.09523809523809523, -0.16666666666666666, -0.13095238095238096, -0.2916666666666667, -0.125, -0.4107142857142857, 0.023809523809523808, -0.39880952380952384, 0.017857142857142856, -0.5952380952380952, 0.017857142857142856, -0.7619047619047619, -0.05952380952380952, -0.39880952380952384, -0.05952380952380952, -0.5952380952380952, -0.05357142857142857, -0.7619047619047619, -0.017857142857142856, -0.27976190476190477], [-0.0, -0.0, -0.011904761904761904, -0.125, 0.05952380952380952, -0.15476190476190477, 0.05357142857142857, -0.25595238095238093, 0.023809523809523808, -0.19642857142857142, -0.10119047619047619, -0.16666666666666666, -0.13095238095238096, -0.2916666666666667, -0.125, -0.4107142857142857, 0.023809523809523808, -0.39285714285714285, 0.017857142857142856, -0.5952380952380952, 0.017857142857142856, -0.7619047619047619, -0.05952380952380952, -0.39880952380952384, -0.05952380952380952, -0.5952380952380952, -0.05357142857142857, -0.7619047619047619, -0.017857142857142856, -0.27380952380952384], [-0.0, -0.0, -0.011904761904761904, -0.125, 0.05952380952380952, -0.15476190476190477, 0.05357142857142857, -0.25595238095238093, 0.023809523809523808, -0.18452380952380953, -0.09523809523809523, -0.16666666666666666, -0.13095238095238096, -0.2916666666666667, -0.125, -0.4107142857142857, 0.023809523809523808, -0.39285714285714285, 0.023809523809523808, -0.5952380952380952, 0.017857142857142856, -0.7619047619047619, -0.05952380952380952, -0.39880952380952384, -0.05952380952380952, -0.5952380952380952, -0.05357142857142857, -0.7619047619047619, -0.011904761904761904, -0.27976190476190477], [-0.0, -0.0, -0.011235955056179775, -0.11235955056179775, 0.056179775280898875, -0.1404494382022472, 0.05056179775280899, -0.23595505617977527, 0.028089887640449437, -0.17415730337078653, -0.0898876404494382, -0.15168539325842698, -0.12359550561797752, -0.2696629213483146, -0.11797752808988764, -0.38202247191011235, 0.02247191011235955, -0.3651685393258427, 0.02247191011235955, -0.5561797752808989, 0.016853932584269662, -0.7134831460674157, -0.056179775280898875, -0.3707865168539326, -0.056179775280898875, -0.5561797752808989, -0.05056179775280899, -0.7134831460674157, -0.016853932584269662, -0.25280898876404495], [-0.0, -0.0, -0.011904761904761904, -0.125, 0.05952380952380952, -0.16071428571428573, 0.05357142857142857, -0.2619047619047619, 0.023809523809523808, -0.18452380952380953, -0.09523809523809523, -0.16666666666666666, -0.13095238095238096, -0.2916666666666667, -0.125, -0.4107142857142857, 0.023809523809523808, -0.39285714285714285, 0.023809523809523808, -0.5952380952380952, 0.017857142857142856, -0.7619047619047619, -0.05952380952380952, -0.39880952380952384, -0.05952380952380952, -0.5952380952380952, -0.05357142857142857, -0.7619047619047619, -0.017857142857142856, -0.27380952380952384], [-0.0, -0.0, -0.011235955056179775, -0.11235955056179775, 0.056179775280898875, -0.14606741573033707, 0.05056179775280899, -0.23595505617977527, 0.02247191011235955, -0.16853932584269662, -0.0898876404494382, -0.15168539325842698, -0.12359550561797752, -0.2696629213483146, -0.11797752808988764, -0.38764044943820225, 0.02247191011235955, -0.3651685393258427, 0.02247191011235955, -0.5561797752808989, 0.016853932584269662, -0.7134831460674157, -0.056179775280898875, -0.3651685393258427, -0.056179775280898875, -0.5561797752808989, -0.05056179775280899, -0.7134831460674157, -0.011235955056179775, -0.25842696629213485], [-0.0, -0.0, -0.011235955056179775, -0.11235955056179775, 0.056179775280898875, -0.14606741573033707, 0.05056179775280899, -0.23595505617977527, 0.02247191011235955, -0.16853932584269662, -0.0898876404494382, -0.15168539325842698, -0.12359550561797752, -0.2696629213483146, -0.11235955056179775, -0.38764044943820225, 0.02247191011235955, -0.3651685393258427, 0.02247191011235955, -0.5561797752808989, 0.016853932584269662, -0.7134831460674157, -0.056179775280898875, -0.3651685393258427, -0.056179775280898875, -0.5561797752808989, -0.05056179775280899, -0.7134831460674157, -0.011235955056179775, -0.25842696629213485], [-0.0, -0.0, -0.011904761904761904, -0.125, 0.05952380952380952, -0.16071428571428573, 0.05357142857142857, -0.2619047619047619, 0.023809523809523808, -0.18452380952380953, -0.09523809523809523, -0.16666666666666666, -0.13095238095238096, -0.2916666666666667, -0.11904761904761904, -0.4166666666666667, 0.023809523809523808, -0.39285714285714285, 0.023809523809523808, -0.5952380952380952, 0.017857142857142856, -0.7619047619047619, -0.05952380952380952, -0.39285714285714285, -0.05952380952380952, -0.5952380952380952, -0.05357142857142857, -0.7619047619047619, -0.011904761904761904, -0.27380952380952384], [-0.0, -0.0, -0.011235955056179775, -0.11235955056179775, 0.056179775280898875, -0.14606741573033707, 0.05056179775280899, -0.24157303370786518, 0.05056179775280899, -0.33146067415730335, -0.0898876404494382, -0.15168539325842698, -0.12359550561797752, -0.2696629213483146, -0.11235955056179775, -0.38764044943820225, 0.02247191011235955, -0.3651685393258427, 0.02247191011235955, -0.5561797752808989, 0.016853932584269662, -0.7134831460674157, -0.056179775280898875, -0.3707865168539326, -0.056179775280898875, -0.5561797752808989, -0.05056179775280899, -0.7134831460674157, -0.011235955056179775, -0.25280898876404495], [-0.0, -0.0, -0.011904761904761904, -0.125, 0.05952380952380952, -0.16071428571428573, 0.05357142857142857, -0.26785714285714285, 0.05357142857142857, -0.35714285714285715, -0.09523809523809523, -0.16666666666666666, -0.13095238095238096, -0.2916666666666667, -0.125, -0.4166666666666667, 0.023809523809523808, -0.39285714285714285, 0.023809523809523808, -0.5952380952380952, 0.017857142857142856, -0.7619047619047619, -0.05952380952380952, -0.39285714285714285, -0.05952380952380952, -0.5952380952380952, -0.05357142857142857, -0.7619047619047619, -0.011904761904761904, -0.27380952380952384], [-0.0, -0.0, -0.011834319526627219, -0.11834319526627218, 0.05917159763313609, -0.15976331360946747, 0.05325443786982249, -0.26627218934911245, 0.05917159763313609, -0.35502958579881655, -0.09467455621301775, -0.16568047337278108, -0.1301775147928994, -0.28994082840236685, -0.11834319526627218, -0.41420118343195267, 0.023668639053254437, -0.3905325443786982, 0.023668639053254437, -0.591715976331361, 0.01775147928994083, -0.757396449704142, -0.05917159763313609, -0.3905325443786982, -0.05917159763313609, -0.591715976331361, -0.05325443786982249, -0.757396449704142, -0.011834319526627219, -0.27218934911242604],[-0.0, -0.0, -0.008264462809917356, -0.07851239669421488, 0.045454545454545456, -0.1115702479338843, 0.05371900826446281, -0.19008264462809918, 0.02066115702479339, -0.06611570247933884, -0.049586776859504134, -0.08264462809917356, -0.10743801652892562, -0.0371900826446281, -0.028925619834710745, -0.012396694214876033, 0.012396694214876033, -0.24380165289256198, 0.024793388429752067, -0.371900826446281, 0.03305785123966942, -0.48760330578512395, -0.049586776859504134, -0.2396694214876033, -0.06611570247933884, -0.36363636363636365, -0.08677685950413223, -0.4628099173553719, -0.01652892561983471, -0.1652892561983471], [-0.0, -0.0, -0.0, -0.05704697986577181, 0.030201342281879196, -0.08053691275167785, 0.03691275167785235, -0.13758389261744966, 0.040268456375838924, -0.18791946308724833, -0.03355704697986577, -0.06040268456375839, -0.07718120805369127, -0.02348993288590604, -0.020134228187919462, -0.006711409395973154, 0.010067114093959731, -0.18120805369127516, 0.010067114093959731, -0.2651006711409396, 0.006711409395973154, -0.3422818791946309, -0.030201342281879196, -0.17785234899328858, -0.040268456375838924, -0.2684563758389262, -0.06375838926174497, -0.3523489932885906, -0.006711409395973154, -0.12080536912751678], [-0.0, -0.0, 0.006734006734006734, -0.06060606060606061, 0.03367003367003367, -0.08080808080808081, 0.04040404040404041, -0.12794612794612795, 0.037037037037037035, -0.1717171717171717, -0.02356902356902357, -0.06734006734006734, -0.06060606060606061, -0.030303030303030304, -0.016835016835016835, -0.013468013468013467, 0.020202020202020204, -0.18181818181818182, 0.02356902356902357, -0.26936026936026936, -0.0, -0.3434343434343434, -0.010101010101010102, -0.17845117845117844, -0.003367003367003367, -0.26936026936026936, -0.010101010101010102, -0.3501683501683502, 0.006734006734006734, -0.12457912457912458], [-0.0, -0.0, -0.0, -0.06451612903225806, 0.035842293906810034, -0.08602150537634409, 0.04659498207885305, -0.14336917562724014, 0.043010752688172046, -0.18996415770609318, -0.03225806451612903, -0.07526881720430108, -0.07885304659498207, -0.025089605734767026, -0.017921146953405017, -0.017921146953405017, 0.014336917562724014, -0.2007168458781362, 0.014336917562724014, -0.2939068100358423, 0.010752688172043012, -0.3835125448028674, -0.02867383512544803, -0.1971326164874552, -0.04659498207885305, -0.2903225806451613, -0.06810035842293907, -0.3763440860215054, -0.0035842293906810036, -0.13620071684587814], [-0.0, -0.0, 0.00980392156862745, -0.10294117647058823, 0.06862745098039216, -0.14215686274509803, 0.08823529411764706, -0.23529411764705882, 0.09803921568627451, -0.29411764705882354, -0.0392156862745098, -0.12745098039215685, -0.10784313725490197, -0.058823529411764705, -0.0196078431372549, -0.029411764705882353, 0.05392156862745098, -0.3137254901960784, 0.08333333333333333, -0.46078431372549017, 0.08333333333333333, -0.5784313725490197, -0.014705882352941176, -0.3137254901960784, -0.0, -0.46078431372549017, -0.0, -0.5931372549019608, 0.014705882352941176, -0.22058823529411764], [-0.0, -0.0, 0.00510204081632653, -0.10204081632653061, 0.0663265306122449, -0.14285714285714285, 0.08673469387755102, -0.25, 0.08673469387755102, -0.33163265306122447, -0.05102040816326531, -0.11224489795918367, -0.12755102040816327, -0.04081632653061224, -0.04591836734693878, -0.00510204081632653, 0.025510204081632654, -0.32653061224489793, 0.015306122448979591, -0.5, -0.0, -0.6326530612244898, -0.04081632653061224, -0.32142857142857145, -0.0663265306122449, -0.4846938775510204, -0.08673469387755102, -0.6122448979591837, -0.0, -0.22448979591836735], [-0.0, -0.0, 0.020134228187919462, -0.1476510067114094, 0.11409395973154363, -0.2080536912751678, 0.15436241610738255, -0.37583892617449666, 0.1342281879194631, -0.48322147651006714, -0.06040268456375839, -0.18120805369127516, -0.18120805369127516, -0.09395973154362416, -0.053691275167785234, -0.026845637583892617, 0.06711409395973154, -0.4563758389261745, 0.087248322147651, -0.6912751677852349, 0.11409395973154363, -0.8590604026845637, -0.040268456375838924, -0.44966442953020136, -0.040268456375838924, -0.6845637583892618, -0.03355704697986577, -0.87248322147651, 0.013422818791946308, -0.3221476510067114], [-0.0, -0.0, 0.039473684210526314, -0.3026315789473684, 0.25, -0.4473684210526316, 0.3157894736842105, -0.8421052631578947, 0.2631578947368421, -1.0657894736842106, -0.14473684210526316, -0.3815789473684211, -0.3684210526315789, -0.19736842105263158, -0.10526315789473684, -0.07894736842105263, 0.11842105263157894, -0.9868421052631579, 0.07894736842105263, -1.513157894736842, 0.039473684210526314, -1.9210526315789473, -0.11842105263157894, -0.9736842105263158, -0.18421052631578946, -1.4736842105263157, -0.14473684210526316, -1.8289473684210527, 0.039473684210526314, -0.6973684210526315], [-0.0, -0.0, 0.05263157894736842, -0.42105263157894735, 0.3157894736842105, -0.6140350877192983, 0.43859649122807015, -1.087719298245614, 0.42105263157894735, -1.4385964912280702, -0.17543859649122806, -0.5263157894736842, -0.543859649122807, -0.2631578947368421, -0.19298245614035087, -0.12280701754385964, 0.19298245614035087, -1.3508771929824561, 0.2807017543859649, -2.017543859649123, 0.2631578947368421, -2.5789473684210527, -0.12280701754385964, -1.3333333333333333, -0.07017543859649122, -2.017543859649123, -0.017543859649122806, -2.5789473684210527, 0.05263157894736842, -0.9473684210526315], [-0.0, -0.0, -0.019417475728155338, -0.22330097087378642, 0.1262135922330097, -0.3300970873786408, 0.1650485436893204, -0.5728155339805825, 0.1262135922330097, -0.7572815533980582, -0.13592233009708737, -0.2621359223300971, -0.3106796116504854, -0.11650485436893204, -0.10679611650485436, -0.038834951456310676, 0.019417475728155338, -0.7087378640776699, -0.0, -1.0679611650485437, -0.02912621359223301, -1.3689320388349515, -0.14563106796116504, -0.6893203883495146, -0.18446601941747573, -1.029126213592233, -0.2621359223300971, -1.3203883495145632, -0.038834951456310676, -0.5048543689320388], [-0.0, -0.0, -0.0056179775280898875, -0.11235955056179775, 0.06179775280898876, -0.16292134831460675, 0.0898876404494382, -0.29213483146067415, 0.0898876404494382, -0.38764044943820225, -0.07303370786516854, -0.1348314606741573, -0.15730337078651685, -0.0449438202247191, -0.0449438202247191, -0.0056179775280898875, 0.028089887640449437, -0.3651685393258427, 0.06741573033707865, -0.5449438202247191, 0.0449438202247191, -0.6910112359550562, -0.05056179775280899, -0.3651685393258427, -0.03932584269662921, -0.5449438202247191, -0.033707865168539325, -0.7078651685393258, -0.0056179775280898875, -0.25842696629213485], [-0.0, -0.0, -0.01646090534979424, -0.07407407407407407, 0.03292181069958848, -0.1111111111111111, 0.0411522633744856, -0.1934156378600823, 0.037037037037037035, -0.2551440329218107, -0.05761316872427984, -0.07818930041152264, -0.1111111111111111, -0.024691358024691357, -0.02880658436213992, -0.012345679012345678, -0.0, -0.23868312757201646, -0.00823045267489712, -0.3662551440329218, -0.012345679012345678, -0.4732510288065844, -0.053497942386831275, -0.23045267489711935, -0.06584362139917696, -0.35802469135802467, -0.09465020576131687, -0.4609053497942387, -0.024691358024691357, -0.16049382716049382], [-0.0, -0.0, -0.007380073800738007, -0.06273062730627306, 0.02214022140221402, -0.08856088560885608, 0.02214022140221402, -0.15129151291512916, 0.02952029520295203, -0.1992619926199262, -0.04059040590405904, -0.06273062730627306, -0.0, -0.025830258302583026, -0.025830258302583026, -0.007380073800738007, -0.0, -0.2066420664206642, 0.014760147601476014, -0.3025830258302583, 0.007380073800738007, -0.3874538745387454, -0.04059040590405904, -0.2029520295202952, -0.03690036900369004, -0.3062730627306273, -0.03690036900369004, -0.3985239852398524, -0.01845018450184502, -0.13653136531365315], [-0.0, -0.0, -0.006153846153846154, -0.052307692307692305, 0.015384615384615385, -0.07384615384615385, 0.012307692307692308, 0.006153846153846154, 0.009230769230769232, -0.027692307692307693, -0.03076923076923077, -0.05846153846153846, -0.06153846153846154, -0.027692307692307693, -0.033846153846153845, -0.009230769230769232, -0.006153846153846154, -0.15692307692307692, -0.006153846153846154, -0.2276923076923077, -0.012307692307692308, -0.2953846153846154, -0.033846153846153845, -0.15384615384615385, -0.036923076923076927, -0.2276923076923077, -0.04923076923076923, -0.2953846153846154, -0.012307692307692308, -0.11076923076923077], [-0.0, -0.0, -0.0, -0.04956268221574344, 0.014577259475218658, -0.0641399416909621, 0.02040816326530612, -0.10204081632653061, -0.014577259475218658, -0.029154518950437316, -0.014577259475218658, -0.05539358600583091, -0.03206997084548105, -0.08163265306122448, -0.023323615160349854, -0.026239067055393587, 0.008746355685131196, -0.14285714285714285, 0.014577259475218658, -0.20699708454810495, 0.01749271137026239, -0.27113702623906705, -0.01749271137026239, -0.13994169096209913, -0.02040816326530612, -0.2099125364431487, -0.026239067055393587, -0.27696793002915454, -0.0, -0.09912536443148688], [-0.0, -0.0, 0.0058309037900874635, -0.04956268221574344, 0.026239067055393587, -0.061224489795918366, 0.03498542274052478, -0.09912536443148688, 0.014577259475218658, -0.03206997084548105, -0.01749271137026239, -0.052478134110787174, -0.043731778425655975, -0.023323615160349854, -0.011661807580174927, -0.01749271137026239, 0.02040816326530612, -0.13702623906705538, 0.03498542274052478, -0.20699708454810495, 0.011661807580174927, -0.26822157434402333, -0.0058309037900874635, -0.13994169096209913, 0.0029154518950437317, -0.21282798833819241, 0.0029154518950437317, -0.27113702623906705, 0.008746355685131196, -0.09620991253644315], [-0.0, -0.0, -0.0, -0.05555555555555555, 0.024691358024691357, -0.07407407407407407, 0.033950617283950615, -0.11728395061728394, 0.030864197530864196, -0.15123456790123457, -0.027777777777777776, -0.05864197530864197, -0.06172839506172839, -0.030864197530864196, -0.024691358024691357, -0.018518518518518517, 0.006172839506172839, -0.16049382716049382, 0.009259259259259259, -0.22839506172839505, 0.0030864197530864196, -0.2962962962962963, -0.024691358024691357, -0.1574074074074074, -0.030864197530864196, -0.23148148148148148, -0.05246913580246913, -0.30246913580246915, -0.0030864197530864196, -0.1111111111111111], [-0.0, -0.0, 0.009202453987730062, -0.049079754601226995, 0.03680981595092025, -0.06748466257668712, 0.04294478527607362, -0.11349693251533742, 0.03987730061349693, -0.15030674846625766, -0.015337423312883436, -0.06134969325153374, -0.049079754601226995, -0.03067484662576687, -0.03987730061349693, -0.02147239263803681, 0.024539877300613498, -0.15030674846625766, 0.015337423312883436, -0.22699386503067484, 0.003067484662576687, -0.29754601226993865, -0.006134969325153374, -0.15030674846625766, -0.003067484662576687, -0.22392638036809817, -0.006134969325153374, -0.2883435582822086, 0.012269938650306749, -0.10429447852760736], [-0.0, -0.0, 0.0035842293906810036, -0.06451612903225806, 0.03942652329749104, -0.08602150537634409, 0.05734767025089606, -0.15412186379928317, 0.06093189964157706, -0.2078853046594982, -0.02867383512544803, -0.07168458781362007, -0.06810035842293907, -0.035842293906810034, -0.021505376344086023, -0.017921146953405017, 0.025089605734767026, -0.2007168458781362, 0.02867383512544803, -0.2974910394265233, 0.025089605734767026, -0.3763440860215054, -0.021505376344086023, -0.1971326164874552, -0.025089605734767026, -0.2939068100358423, -0.03942652329749104, -0.3727598566308244, -0.0, -0.13620071684587814], [-0.0, -0.0, 0.009852216748768473, -0.10837438423645321, 0.07389162561576355, -0.1477832512315271, 0.09852216748768473, -0.2512315270935961, 0.09852216748768473, -0.33004926108374383, -0.03940886699507389, -0.12807881773399016, -0.11330049261083744, -0.06896551724137931, -0.019704433497536946, -0.024630541871921183, 0.04433497536945813, -0.31527093596059114, 0.06403940886699508, -0.458128078817734, 0.054187192118226604, -0.5615763546798029, -0.019704433497536946, -0.31527093596059114, -0.009852216748768473, -0.4630541871921182, -0.009852216748768473, -0.5911330049261084, 0.014778325123152709, -0.22167487684729065], [-0.0, -0.0, 0.018867924528301886, -0.1320754716981132, 0.1069182389937107, -0.1949685534591195, 0.1320754716981132, -0.3584905660377358, 0.11320754716981132, -0.4591194968553459, -0.05660377358490566, -0.16352201257861634, -0.1509433962264151, -0.08176100628930817, -0.06918238993710692, -0.025157232704402517, 0.050314465408805034, -0.4339622641509434, 0.0440251572327044, -0.6540880503144654, 0.025157232704402517, -0.8176100628930818, -0.050314465408805034, -0.4276729559748428, -0.07547169811320754, -0.6289308176100629, -0.07547169811320754, -0.779874213836478, 0.012578616352201259, -0.29559748427672955], [-0.0, -0.0, 0.017857142857142856, -0.20535714285714285, 0.15178571428571427, -0.30357142857142855, 0.19642857142857142, -0.5357142857142857, 0.15178571428571427, -0.6964285714285714, -0.08928571428571429, -0.25892857142857145, -0.25, -0.14285714285714285, -0.07142857142857142, -0.0625, 0.08035714285714286, -0.625, 0.10714285714285714, -0.9553571428571429, 0.16964285714285715, -1.2232142857142858, -0.08035714285714286, -0.625, -0.0625, -0.9464285714285714, -0.05357142857142857, -1.2232142857142858, 0.017857142857142856, -0.45535714285714285], [-0.0, -0.0, -0.0, -0.15714285714285714, 0.1, -0.2357142857142857, 0.12142857142857143, -0.42142857142857143, 0.08571428571428572, -0.55, -0.08571428571428572, -0.17857142857142858, -0.2, -0.08571428571428572, -0.05714285714285714, -0.02857142857142857, 0.02857142857142857, -0.4857142857142857, 0.014285714285714285, -0.75, 0.014285714285714285, -0.95, -0.09285714285714286, -0.4714285714285714, -0.07857142857142857, -0.7357142857142858, -0.08571428571428572, -0.9642857142857143, -0.02142857142857143, -0.34285714285714286], [-0.0, -0.0, -0.0106951871657754, -0.10695187165775401, 0.058823529411764705, -0.15508021390374332, 0.08021390374331551, -0.27807486631016043, 0.06951871657754011, -0.3689839572192513, -0.06951871657754011, -0.12834224598930483, -0.15508021390374332, -0.0481283422459893, -0.0427807486631016, -0.0053475935828877, 0.016042780748663103, -0.3422459893048128, 0.0374331550802139, -0.5347593582887701, 0.0481283422459893, -0.6898395721925134, -0.06951871657754011, -0.33689839572192515, -0.0855614973262032, -0.5187165775401069, -0.10160427807486631, -0.6631016042780749, -0.016042780748663103, -0.24064171122994651], [-0.0, -0.0, -0.004132231404958678, -0.07851239669421488, 0.04132231404958678, -0.1115702479338843, 0.049586776859504134, -0.19421487603305784, 0.04132231404958678, -0.256198347107438, -0.049586776859504134, -0.08677685950413223, -0.10743801652892562, -0.0371900826446281, -0.0371900826446281, -0.012396694214876033, 0.008264462809917356, -0.24380165289256198, -0.004132231404958678, -0.371900826446281, -0.02066115702479339, -0.47107438016528924, -0.04132231404958678, -0.23553719008264462, -0.028925619834710745, -0.36363636363636365, -0.028925619834710745, -0.4793388429752066, -0.012396694214876033, -0.1652892561983471], [-0.0, -0.0, -0.0037174721189591076, -0.07063197026022305, 0.02973977695167286, -0.09665427509293681, 0.03345724907063197, -0.15985130111524162, 0.03345724907063197, -0.20446096654275092, -0.03717472118959108, -0.07434944237918216, -0.07434944237918216, -0.03345724907063197, -0.01486988847583643, -0.01486988847583643, 0.0037174721189591076, -0.20817843866171004, 0.026022304832713755, -0.3159851301115242, 0.05204460966542751, -0.4052044609665427, -0.03717472118959108, -0.20446096654275092, -0.03717472118959108, -0.31226765799256506, -0.040892193308550186, -0.40148698884758366, -0.011152416356877323, -0.1412639405204461], [-0.0, -0.0, -0.005988023952095809, -0.05089820359281437, 0.0029940119760479044, -0.06287425149700598, 0.008982035928143712, -0.10179640718562874, -0.014970059880239521, -0.03293413173652695, -0.014970059880239521, -0.05688622754491018, -0.038922155688622756, -0.023952095808383235, -0.020958083832335328, -0.029940119760479042, -0.008982035928143712, -0.1437125748502994, -0.011976047904191617, -0.218562874251497, -0.020958083832335328, -0.2874251497005988, -0.020958083832335328, -0.1437125748502994, -0.017964071856287425, -0.218562874251497, -0.023952095808383235, -0.2874251497005988, -0.008982035928143712, -0.10179640718562874], [-0.0, -0.0, -0.005797101449275362, -0.043478260869565216, 0.002898550724637681, -0.057971014492753624, 0.008695652173913044, -0.0927536231884058, 0.008695652173913044, -0.11884057971014493, -0.020289855072463767, -0.05507246376811594, -0.005797101449275362, -0.014492753623188406, -0.005797101449275362, -0.0463768115942029, -0.002898550724637681, -0.1391304347826087, -0.005797101449275362, -0.20579710144927535, -0.02318840579710145, -0.26956521739130435, -0.011594202898550725, -0.1391304347826087, -0.005797101449275362, -0.20579710144927535, -0.020289855072463767, -0.26666666666666666, -0.008695652173913044, -0.09855072463768116], [-0.0, -0.0, -0.0, -0.048295454545454544, 0.014204545454545454, -0.05965909090909091, 0.022727272727272728, -0.09375, -0.011363636363636364, -0.02556818181818182, -0.017045454545454544, -0.056818181818181816, -0.011363636363636364, -0.011363636363636364, -0.014204545454545454, -0.04261363636363636, 0.008522727272727272, -0.13636363636363635, 0.014204545454545454, -0.19886363636363635, 0.002840909090909091, -0.26420454545454547, -0.011363636363636364, -0.13352272727272727, -0.019886363636363636, -0.20454545454545456, -0.04261363636363636, -0.26704545454545453, -0.0, -0.09659090909090909], [-0.0, -0.0, 0.013071895424836602, -0.058823529411764705, 0.0457516339869281, -0.06862745098039216, 0.05228758169934641, -0.12091503267973856, 0.049019607843137254, -0.1568627450980392, -0.016339869281045753, -0.06535947712418301, -0.042483660130718956, -0.029411764705882353, -0.013071895424836602, -0.02287581699346405, 0.0392156862745098, -0.17320261437908496, 0.05228758169934641, -0.25163398692810457, 0.03594771241830065, -0.32679738562091504, -0.0, -0.17320261437908496, 0.006535947712418301, -0.2581699346405229, 0.006535947712418301, -0.33986928104575165, 0.0196078431372549, -0.11764705882352941], [-0.0, -0.0, -0.0, -0.06666666666666667, 0.040740740740740744, -0.0962962962962963, 0.05555555555555555, -0.15925925925925927, 0.05555555555555555, -0.21481481481481482, -0.03333333333333333, -0.07777777777777778, -0.08518518518518518, -0.03333333333333333, -0.018518518518518517, -0.011111111111111112, 0.014814814814814815, -0.2111111111111111, 0.007407407407407408, -0.3111111111111111, -0.007407407407407408, -0.3962962962962963, -0.02962962962962963, -0.2037037037037037, -0.03333333333333333, -0.3074074074074074, -0.04814814814814815, -0.40370370370370373, -0.003703703703703704, -0.14444444444444443], [-0.0, -0.0, 0.016216216216216217, -0.11891891891891893, 0.08648648648648649, -0.16216216216216217, 0.11891891891891893, -0.2702702702702703, 0.11891891891891893, -0.3621621621621622, -0.03783783783783784, -0.14594594594594595, -0.11891891891891893, -0.08648648648648649, -0.02702702702702703, -0.02702702702702703, 0.06486486486486487, -0.35135135135135137, 0.08108108108108109, -0.5297297297297298, 0.0918918918918919, -0.6756756756756757, -0.010810810810810811, -0.35135135135135137, 0.005405405405405406, -0.5243243243243243, 0.010810810810810811, -0.6648648648648648, 0.021621621621621623, -0.24864864864864866], [-0.0, -0.0, 0.025157232704402517, -0.1320754716981132, 0.11949685534591195, -0.1949685534591195, 0.13836477987421383, -0.36477987421383645, 0.11320754716981132, -0.4591194968553459, -0.03773584905660377, -0.16981132075471697, -0.1509433962264151, -0.11320754716981132, -0.05660377358490566, -0.0440251572327044, 0.06918238993710692, -0.4339622641509434, 0.07547169811320754, -0.6666666666666666, 0.0880503144654088, -0.8364779874213837, -0.031446540880503145, -0.4276729559748428, -0.08176100628930817, -0.6415094339622641, -0.11949685534591195, -0.8113207547169812, 0.018867924528301886, -0.3018867924528302], [-0.0, -0.0, 0.01951219512195122, -0.0975609756097561, 0.08292682926829269, -0.13658536585365855, 0.11707317073170732, -0.23414634146341465, 0.11219512195121951, -0.3170731707317073, -0.02926829268292683, -0.11219512195121951, -0.08292682926829269, -0.04878048780487805, -0.00975609756097561, -0.014634146341463415, 0.06341463414634146, -0.3024390243902439, 0.07317073170731707, -0.4585365853658537, 0.03902439024390244, -0.5756097560975609, -0.004878048780487805, -0.3024390243902439, 0.014634146341463415, -0.4634146341463415, 0.02926829268292683, -0.6, 0.024390243902439025, -0.2097560975609756], [-0.0, -0.0, 0.008298755186721992, -0.08298755186721991, 0.05394190871369295, -0.11618257261410789, 0.07053941908713693, -0.1908713692946058, 0.06224066390041494, -0.24896265560165975, -0.03734439834024896, -0.08713692946058091, -0.06639004149377593, -0.04149377593360996, -0.02074688796680498, -0.012448132780082987, 0.02074688796680498, -0.24481327800829875, 0.03319502074688797, -0.36099585062240663, 0.029045643153526972, -0.46887966804979253, -0.029045643153526972, -0.24481327800829875, -0.04149377593360996, -0.36099585062240663, -0.05394190871369295, -0.45643153526970953, -0.0, -0.17012448132780084], [-0.0, -0.0, -0.0, -0.06451612903225806, 0.03225806451612903, -0.08602150537634409, 0.04659498207885305, -0.14695340501792115, 0.043010752688172046, -0.2007168458781362, -0.03225806451612903, -0.07168458781362007, -0.06451612903225806, -0.043010752688172046, -0.05017921146953405, -0.017921146953405017, 0.014336917562724014, -0.1971326164874552, 0.025089605734767026, -0.2831541218637993, 0.021505376344086023, -0.35842293906810035, -0.025089605734767026, -0.1971326164874552, -0.014336917562724014, -0.2903225806451613, -0.010752688172043012, -0.36917562724014336, -0.007168458781362007, -0.13261648745519714], [-0.0, -0.0, -0.0029154518950437317, -0.04956268221574344, 0.02040816326530612, -0.06705539358600583, 0.023323615160349854, -0.11078717201166181, 0.01749271137026239, -0.1457725947521866, -0.029154518950437316, -0.04956268221574344, -0.052478134110787174, -0.029154518950437316, -0.023323615160349854, -0.02040816326530612, -0.0029154518950437317, -0.14285714285714285, -0.014577259475218658, -0.20699708454810495, -0.023323615160349854, -0.27113702623906705, -0.03206997084548105, -0.13702623906705538, -0.037900874635568516, -0.20408163265306123, -0.04664723032069971, -0.26822157434402333, -0.011661807580174927, -0.09912536443148688], [-0.0, -0.0, -0.0029154518950437317, -0.04956268221574344, 0.02040816326530612, -0.06705539358600583, 0.026239067055393587, -0.10787172011661808, 0.02040816326530612, -0.029154518950437316, -0.029154518950437316, -0.05539358600583091, -0.05830903790087463, -0.02040816326530612, -0.037900874635568516, -0.014577259475218658, 0.0029154518950437317, -0.14285714285714285, 0.014577259475218658, -0.20408163265306123, 0.023323615160349854, -0.2653061224489796, -0.026239067055393587, -0.13994169096209913, -0.023323615160349854, -0.20699708454810495, -0.029154518950437316, -0.26822157434402333, -0.0058309037900874635, -0.10204081632653061], [-0.0, -0.0, -0.008522727272727272, -0.048295454545454544, 0.011363636363636364, -0.06534090909090909, 0.017045454545454544, -0.10227272727272728, 0.011363636363636364, -0.13636363636363635, -0.03125, -0.048295454545454544, 0.008522727272727272, -0.011363636363636364, -0.017045454545454544, -0.008522727272727272, -0.014204545454545454, -0.13636363636363635, -0.019886363636363636, -0.20170454545454544, -0.03409090909090909, -0.26420454545454547, -0.03409090909090909, -0.13352272727272727, -0.03409090909090909, -0.20170454545454544, -0.03977272727272727, -0.26136363636363635, -0.014204545454545454, -0.09659090909090909], [-0.0, -0.0, -0.0, -0.05537459283387622, 0.029315960912052116, -0.0781758957654723, 0.035830618892508145, -0.13029315960912052, 0.03908794788273615, -0.17263843648208468, -0.03257328990228013, -0.06188925081433225, -0.0749185667752443, -0.02280130293159609, -0.026058631921824105, -0.006514657980456026, 0.009771986970684038, -0.1758957654723127, 0.026058631921824105, -0.26384364820846906, 0.03908794788273615, -0.33876221498371334, -0.03257328990228013, -0.17263843648208468, -0.03257328990228013, -0.25732899022801303, -0.035830618892508145, -0.3289902280130293, -0.006514657980456026, -0.12052117263843648], [-0.0, -0.0, -0.0038461538461538464, -0.07307692307692308, 0.03076923076923077, -0.10384615384615385, 0.046153846153846156, -0.16538461538461538, 0.038461538461538464, -0.21923076923076923, -0.038461538461538464, -0.08076923076923077, -0.08461538461538462, -0.04230769230769231, -0.05, -0.019230769230769232, -0.0, -0.21923076923076923, -0.0038461538461538464, -0.3230769230769231, -0.019230769230769232, -0.4153846153846154, -0.038461538461538464, -0.2153846153846154, -0.04230769230769231, -0.3192307692307692, -0.06538461538461539, -0.41923076923076924, -0.011538461538461539, -0.15384615384615385], [-0.0, -0.0, -0.0049261083743842365, -0.10837438423645321, 0.054187192118226604, -0.1477832512315271, 0.07881773399014778, -0.2512315270935961, 0.07881773399014778, -0.3251231527093596, -0.054187192118226604, -0.12315270935960591, -0.1330049261083744, -0.07389162561576355, -0.03940886699507389, -0.029556650246305417, 0.029556650246305417, -0.31527093596059114, 0.04926108374384237, -0.4482758620689655, 0.029556650246305417, -0.5615763546798029, -0.03940886699507389, -0.31527093596059114, -0.024630541871921183, -0.458128078817734, -0.024630541871921183, -0.5862068965517241, -0.0049261083743842365, -0.22167487684729065], [-0.0, -0.0, -0.0, -0.0975609756097561, 0.06341463414634146, -0.13658536585365855, 0.08780487804878048, -0.24390243902439024, 0.07804878048780488, -0.32682926829268294, -0.05365853658536585, -0.11219512195121951, -0.13170731707317074, -0.06341463414634146, -0.06341463414634146, -0.00975609756097561, 0.03414634146341464, -0.3073170731707317, 0.024390243902439025, -0.4682926829268293, 0.014634146341463415, -0.6, -0.03902439024390244, -0.3024390243902439, -0.05853658536585366, -0.4585365853658537, -0.09268292682926829, -0.5804878048780487, -0.004878048780487805, -0.2097560975609756], [-0.0, -0.0, 0.018867924528301886, -0.1320754716981132, 0.1069182389937107, -0.1949685534591195, 0.13836477987421383, -0.3522012578616352, 0.11949685534591195, -0.4528301886792453, -0.050314465408805034, -0.16352201257861634, -0.16981132075471697, -0.1069182389937107, -0.07547169811320754, -0.031446540880503145, 0.05660377358490566, -0.42138364779874216, 0.050314465408805034, -0.6289308176100629, 0.0440251572327044, -0.7987421383647799, -0.0440251572327044, -0.4088050314465409, -0.05660377358490566, -0.6289308176100629, -0.08176100628930817, -0.8238993710691824, 0.012578616352201259, -0.29559748427672955], [-0.0, -0.0, 0.015267175572519083, -0.16793893129770993, 0.12213740458015267, -0.24427480916030533, 0.16030534351145037, -0.4580152671755725, 0.11450381679389313, -0.5725190839694656, -0.08396946564885496, -0.1984732824427481, -0.21374045801526717, -0.1297709923664122, -0.0916030534351145, -0.05343511450381679, 0.05343511450381679, -0.5343511450381679, 0.030534351145038167, -0.8091603053435115, 0.015267175572519083, -1.0534351145038168, -0.06870229007633588, -0.5114503816793893, -0.061068702290076333, -0.8015267175572519, -0.04580152671755725, -1.0305343511450382, 0.007633587786259542, -0.366412213740458], [-0.0, -0.0, -0.0, -0.11864406779661017, 0.0847457627118644, -0.1638418079096045, 0.1016949152542373, -0.2994350282485876, 0.0847457627118644, -0.4011299435028249, -0.062146892655367235, -0.14124293785310735, -0.1638418079096045, -0.0847457627118644, -0.07344632768361582, -0.022598870056497175, 0.02824858757062147, -0.3785310734463277, 0.05084745762711865, -0.5649717514124294, 0.07909604519774012, -0.7401129943502824, -0.05649717514124294, -0.3728813559322034, -0.062146892655367235, -0.5480225988700564, -0.062146892655367235, -0.6949152542372882, -0.005649717514124294, -0.2598870056497175], [-0.0, -0.0, -0.0, -0.0975609756097561, 0.06341463414634146, -0.13658536585365855, 0.09268292682926829, -0.24878048780487805, 0.07317073170731707, -0.33170731707317075, -0.05365853658536585, -0.11219512195121951, -0.13170731707317074, -0.06341463414634146, -0.05853658536585366, -0.014634146341463415, 0.02926829268292683, -0.3073170731707317, 0.02926829268292683, -0.4682926829268293, 0.024390243902439025, -0.5951219512195122, -0.04390243902439024, -0.3024390243902439, -0.05853658536585366, -0.4585365853658537, -0.07317073170731707, -0.5902439024390244, -0.004878048780487805, -0.2097560975609756], [-0.0, -0.0, 0.004830917874396135, -0.08695652173913043, 0.06763285024154589, -0.13526570048309178, 0.09178743961352658, -0.2463768115942029, 0.0821256038647343, -0.3285024154589372, -0.05314009661835749, -0.11594202898550725, -0.13043478260869565, -0.05314009661835749, -0.04830917874396135, -0.014492753623188406, 0.033816425120772944, -0.30917874396135264, 0.04830917874396135, -0.463768115942029, 0.04830917874396135, -0.5893719806763285, -0.03864734299516908, -0.30434782608695654, -0.03864734299516908, -0.45893719806763283, -0.04830917874396135, -0.5893719806763285, -0.0, -0.20772946859903382], [-0.0, -0.0, 0.00510204081632653, -0.10204081632653061, 0.07142857142857142, -0.14795918367346939, 0.09693877551020408, -0.2602040816326531, 0.08673469387755102, -0.3469387755102041, -0.05612244897959184, -0.12244897959183673, -0.1377551020408163, -0.07653061224489796, -0.07653061224489796, -0.030612244897959183, 0.030612244897959183, -0.33163265306122447, 0.04591836734693878, -0.49489795918367346, 0.04591836734693878, -0.6326530612244898, -0.04081632653061224, -0.32653061224489793, -0.04591836734693878, -0.49489795918367346, -0.05102040816326531, -0.6275510204081632, -0.0, -0.22959183673469388], [-0.0, -0.0, 0.005376344086021506, -0.11290322580645161, 0.08064516129032258, -0.15591397849462366, 0.11290322580645161, -0.27956989247311825, 0.0967741935483871, -0.3709677419354839, -0.04838709677419355, -0.13440860215053763, -0.14516129032258066, -0.08602150537634409, -0.04838709677419355, -0.03225806451612903, 0.04838709677419355, -0.3548387096774194, 0.043010752688172046, -0.5268817204301075, 0.03763440860215054, -0.6720430107526881, -0.03225806451612903, -0.34946236559139787, -0.04838709677419355, -0.521505376344086, -0.06451612903225806, -0.6612903225806451, 0.010752688172043012, -0.24193548387096775], [-0.0, -0.0, 0.01020408163265306, -0.10204081632653061, 0.08163265306122448, -0.14285714285714285, 0.10204081632653061, -0.2602040816326531, 0.09183673469387756, -0.3469387755102041, -0.04591836734693878, -0.12244897959183673, -0.11224489795918367, -0.10204081632653061, -0.02040816326530612, -0.05612244897959184, 0.04081632653061224, -0.33163265306122447, 0.04591836734693878, -0.49489795918367346, 0.04591836734693878, -0.6326530612244898, -0.03571428571428571, -0.32653061224489793, -0.03571428571428571, -0.4897959183673469, -0.04081632653061224, -0.6275510204081632, 0.00510204081632653, -0.22448979591836735], [-0.0, -0.0, 0.005128205128205128, -0.1076923076923077, 0.08205128205128205, -0.14358974358974358, 0.10256410256410256, -0.26666666666666666, 0.09230769230769231, -0.3487179487179487, -0.05128205128205128, -0.12307692307692308, -0.13333333333333333, -0.09230769230769231, -0.03076923076923077, -0.05128205128205128, 0.035897435897435895, -0.3333333333333333, 0.041025641025641026, -0.49743589743589745, 0.041025641025641026, -0.6358974358974359, -0.041025641025641026, -0.3282051282051282, -0.041025641025641026, -0.49230769230769234, -0.046153846153846156, -0.6307692307692307, -0.0, -0.23076923076923078], [-0.0, -0.0, 0.018691588785046728, -0.09345794392523364, 0.08411214953271028, -0.1261682242990654, 0.102803738317757, -0.2336448598130841, 0.08878504672897196, -0.308411214953271, -0.03271028037383177, -0.10747663551401869, -0.09345794392523364, -0.08411214953271028, -0.02336448598130841, -0.028037383177570093, 0.04205607476635514, -0.29439252336448596, 0.04672897196261682, -0.4439252336448598, 0.04672897196261682, -0.5700934579439252, -0.028037383177570093, -0.2897196261682243, -0.028037383177570093, -0.4392523364485981, -0.03271028037383177, -0.5607476635514018, 0.014018691588785047, -0.19626168224299065], [-0.0, -0.0, 0.0196078431372549, -0.10294117647058823, 0.08823529411764706, -0.13725490196078433, 0.11274509803921569, -0.25, 0.09803921568627451, -0.3284313725490196, -0.0392156862745098, -0.11274509803921569, -0.06862745098039216, -0.10784313725490197, -0.0196078431372549, -0.04411764705882353, 0.049019607843137254, -0.3137254901960784, 0.05392156862745098, -0.47058823529411764, 0.05392156862745098, -0.6029411764705882, -0.024509803921568627, -0.3088235294117647, -0.024509803921568627, -0.46568627450980393, -0.029411764705882353, -0.5980392156862745, 0.0196078431372549, -0.21568627450980393], [-0.0, -0.0, 0.019704433497536946, -0.10837438423645321, 0.08866995073891626, -0.13793103448275862, 0.10344827586206896, -0.2512315270935961, 0.09359605911330049, -0.33004926108374383, -0.03940886699507389, -0.11330049261083744, -0.06896551724137931, -0.10837438423645321, -0.024630541871921183, -0.04433497536945813, 0.04433497536945813, -0.31527093596059114, 0.04926108374384237, -0.4729064039408867, 0.04926108374384237, -0.6059113300492611, -0.029556650246305417, -0.3103448275862069, -0.029556650246305417, -0.46798029556650245, -0.034482758620689655, -0.6009852216748769, 0.014778325123152709, -0.21674876847290642], [-0.0, -0.0, 0.0196078431372549, -0.10294117647058823, 0.08823529411764706, -0.13725490196078433, 0.10784313725490197, -0.25, 0.09313725490196079, -0.3284313725490196, -0.0392156862745098, -0.11274509803921569, -0.07352941176470588, -0.10784313725490197, -0.024509803921568627, -0.04411764705882353, 0.04411764705882353, -0.3137254901960784, 0.049019607843137254, -0.47058823529411764, 0.049019607843137254, -0.6029411764705882, -0.029411764705882353, -0.3088235294117647, -0.029411764705882353, -0.46568627450980393, -0.03431372549019608, -0.5980392156862745, 0.014705882352941176, -0.2107843137254902], [-0.0, -0.0, 0.0196078431372549, -0.10294117647058823, 0.08823529411764706, -0.13725490196078433, 0.10784313725490197, -0.25, 0.09803921568627451, -0.3284313725490196, -0.0392156862745098, -0.11274509803921569, -0.07352941176470588, -0.10784313725490197, -0.024509803921568627, -0.04411764705882353, 0.04411764705882353, -0.3137254901960784, 0.049019607843137254, -0.47058823529411764, 0.049019607843137254, -0.6029411764705882, -0.029411764705882353, -0.3088235294117647, -0.029411764705882353, -0.46568627450980393, -0.03431372549019608, -0.5980392156862745, 0.014705882352941176, -0.21568627450980393], [-0.0, -0.0, 0.0196078431372549, -0.10294117647058823, 0.08823529411764706, -0.13725490196078433, 0.10294117647058823, -0.25, 0.09313725490196079, -0.3284313725490196, -0.0392156862745098, -0.11274509803921569, -0.0784313725490196, -0.10784313725490197, -0.0196078431372549, -0.049019607843137254, 0.04411764705882353, -0.3137254901960784, 0.049019607843137254, -0.47058823529411764, 0.049019607843137254, -0.6029411764705882, -0.029411764705882353, -0.3088235294117647, -0.029411764705882353, -0.46568627450980393, -0.03431372549019608, -0.5980392156862745, 0.014705882352941176, -0.21568627450980393], [-0.0, -0.0, 0.018779342723004695, -0.09859154929577464, 0.08450704225352113, -0.1267605633802817, 0.10328638497652583, -0.2347417840375587, 0.09389671361502347, -0.30985915492957744, -0.03286384976525822, -0.10328638497652583, -0.056338028169014086, -0.107981220657277, -0.014084507042253521, -0.04225352112676056, 0.046948356807511735, -0.29577464788732394, 0.051643192488262914, -0.4460093896713615, 0.051643192488262914, -0.5727699530516432, -0.023474178403755867, -0.29107981220657275, -0.023474178403755867, -0.4413145539906103, -0.028169014084507043, -0.568075117370892, 0.018779342723004695, -0.20187793427230047], [-0.0, -0.0, 0.023474178403755867, -0.09859154929577464, 0.0892018779342723, -0.1267605633802817, 0.10328638497652583, -0.2347417840375587, 0.09389671361502347, -0.30985915492957744, -0.03286384976525822, -0.10328638497652583, -0.056338028169014086, -0.10328638497652583, -0.014084507042253521, -0.03755868544600939, 0.046948356807511735, -0.29577464788732394, 0.051643192488262914, -0.4460093896713615, 0.051643192488262914, -0.5727699530516432, -0.023474178403755867, -0.29107981220657275, -0.023474178403755867, -0.4413145539906103, -0.028169014084507043, -0.5633802816901409, 0.018779342723004695, -0.20187793427230047], [-0.0, -0.0, 0.0196078431372549, -0.10294117647058823, 0.09313725490196079, -0.14215686274509803, 0.10784313725490197, -0.25, 0.09803921568627451, -0.3284313725490196, -0.03431372549019608, -0.11274509803921569, -0.049019607843137254, -0.10294117647058823, -0.00980392156862745, -0.03431372549019608, 0.049019607843137254, -0.3137254901960784, 0.05392156862745098, -0.47058823529411764, 0.05392156862745098, -0.6029411764705882, -0.024509803921568627, -0.3088235294117647, -0.024509803921568627, -0.46568627450980393, -0.029411764705882353, -0.5980392156862745, 0.0196078431372549, -0.21568627450980393], [-0.0, -0.0, 0.020618556701030927, -0.1134020618556701, 0.0979381443298969, -0.14948453608247422, 0.1134020618556701, -0.26804123711340205, 0.10309278350515463, -0.35051546391752575, -0.03608247422680412, -0.12371134020618557, -0.05670103092783505, -0.10309278350515463, -0.010309278350515464, -0.041237113402061855, 0.05154639175257732, -0.33505154639175255, 0.05670103092783505, -0.5, 0.05670103092783505, -0.6391752577319587, -0.02577319587628866, -0.32989690721649484, -0.02577319587628866, -0.4948453608247423, -0.030927835051546393, -0.6288659793814433, 0.020618556701030927, -0.23195876288659795], [-0.0, -0.0, 0.028169014084507043, -0.09859154929577464, 0.09389671361502347, -0.13145539906103287, 0.11267605633802817, -0.2347417840375587, 0.06103286384976526, -0.0892018779342723, -0.028169014084507043, -0.10328638497652583, -0.056338028169014086, -0.07981220657276995, 0.004694835680751174, -0.03286384976525822, 0.056338028169014086, -0.29577464788732394, 0.06103286384976526, -0.4460093896713615, 0.06103286384976526, -0.5727699530516432, -0.014084507042253521, -0.29107981220657275, -0.014084507042253521, -0.43661971830985913, -0.018779342723004695, -0.5633802816901409, 0.023474178403755867, -0.20187793427230047], [-0.0, -0.0, -0.0, -0.1134020618556701, 0.07216494845360824, -0.15463917525773196, 0.08762886597938144, -0.26804123711340205, 0.08247422680412371, -0.35051546391752575, -0.05670103092783505, -0.1134020618556701, -0.0979381443298969, -0.07731958762886598, -0.020618556701030927, -0.02577319587628866, 0.02577319587628866, -0.3402061855670103, 0.030927835051546393, -0.5, 0.030927835051546393, -0.6391752577319587, -0.05154639175257732, -0.32989690721649484, -0.05154639175257732, -0.4948453608247423, -0.05670103092783505, -0.6288659793814433, -0.005154639175257732, -0.23195876288659795], [-0.0, -0.0, -0.0, -0.125, 0.07065217391304347, -0.16847826086956522, 0.08695652173913043, -0.29347826086956524, 0.08152173913043478, -0.375, -0.059782608695652176, -0.125, -0.10326086956521739, -0.08152173913043478, -0.021739130434782608, -0.02717391304347826, 0.021739130434782608, -0.3641304347826087, 0.02717391304347826, -0.532608695652174, 0.02717391304347826, -0.6793478260869565, -0.059782608695652176, -0.3532608695652174, -0.059782608695652176, -0.5271739130434783, -0.06521739130434782, -0.6684782608695652, -0.010869565217391304, -0.25], [-0.0, -0.0, 0.010810810810810811, -0.11891891891891893, 0.08108108108108109, -0.16756756756756758, 0.10270270270270271, -0.2864864864864865, 0.0918918918918919, -0.372972972972973, -0.04864864864864865, -0.12972972972972974, -0.0918918918918919, -0.0918918918918919, -0.016216216216216217, -0.043243243243243246, 0.03783783783783784, -0.3567567567567568, 0.043243243243243246, -0.5351351351351351, 0.03783783783783784, -0.6756756756756757, -0.043243243243243246, -0.35135135135135137, -0.043243243243243246, -0.5243243243243243, -0.05405405405405406, -0.6648648648648648, 0.005405405405405406, -0.24864864864864866], [-0.0, -0.0, -0.004878048780487805, -0.0975609756097561, 0.05853658536585366, -0.14146341463414633, 0.07804878048780488, -0.24878048780487805, 0.06829268292682927, -0.32682926829268294, -0.05853658536585366, -0.11219512195121951, -0.12195121951219512, -0.07317073170731707, -0.03902439024390244, -0.03414634146341464, 0.01951219512195122, -0.3073170731707317, 0.024390243902439025, -0.4682926829268293, 0.01951219512195122, -0.6, -0.05365853658536585, -0.3024390243902439, -0.05365853658536585, -0.4634146341463415, -0.06341463414634146, -0.5902439024390244, -0.00975609756097561, -0.2097560975609756], [-0.0, -0.0, -0.004878048780487805, -0.0975609756097561, 0.05853658536585366, -0.13658536585365855, 0.07804878048780488, -0.24878048780487805, 0.06341463414634146, -0.33170731707317075, -0.05853658536585366, -0.11219512195121951, -0.12682926829268293, -0.07804878048780488, -0.06341463414634146, -0.03414634146341464, 0.01951219512195122, -0.3024390243902439, 0.01951219512195122, -0.4682926829268293, 0.014634146341463415, -0.6, -0.05365853658536585, -0.2975609756097561, -0.05853658536585366, -0.4634146341463415, -0.06829268292682927, -0.5902439024390244, -0.014634146341463415, -0.2097560975609756], [-0.0, -0.0, -0.0, -0.1076923076923077, 0.06666666666666667, -0.14871794871794872, 0.09230769230769231, -0.26666666666666666, 0.07692307692307693, -0.35384615384615387, -0.05641025641025641, -0.12307692307692308, -0.13846153846153847, -0.08717948717948718, -0.06666666666666667, -0.05128205128205128, 0.03076923076923077, -0.3333333333333333, 0.03076923076923077, -0.5025641025641026, 0.020512820512820513, -0.6358974358974359, -0.046153846153846156, -0.3282051282051282, -0.05128205128205128, -0.49230769230769234, -0.06666666666666667, -0.6256410256410256, -0.005128205128205128, -0.23076923076923078], [-0.0, -0.0, 0.004878048780487805, -0.0975609756097561, 0.07317073170731707, -0.13658536585365855, 0.0975609756097561, -0.24878048780487805, 0.07804878048780488, -0.33170731707317075, -0.04878048780487805, -0.11707317073170732, -0.13170731707317074, -0.07804878048780488, -0.04878048780487805, -0.03414634146341464, 0.03414634146341464, -0.3121951219512195, 0.03414634146341464, -0.4682926829268293, 0.02926829268292683, -0.6, -0.03902439024390244, -0.3073170731707317, -0.04390243902439024, -0.4634146341463415, -0.05365853658536585, -0.5902439024390244, 0.004878048780487805, -0.2146341463414634], [-0.0, -0.0, 0.014634146341463415, -0.0975609756097561, 0.08292682926829269, -0.13658536585365855, 0.1024390243902439, -0.24878048780487805, 0.08780487804878048, -0.33170731707317075, -0.04390243902439024, -0.11219512195121951, -0.06829268292682927, -0.14146341463414633, -0.02926829268292683, -0.06341463414634146, 0.04390243902439024, -0.3073170731707317, 0.04390243902439024, -0.4682926829268293, 0.03902439024390244, -0.6, -0.02926829268292683, -0.3024390243902439, -0.03414634146341464, -0.4634146341463415, -0.04390243902439024, -0.5902439024390244, 0.014634146341463415, -0.2097560975609756], [-0.0, -0.0, 0.01951219512195122, -0.0975609756097561, 0.08292682926829269, -0.13658536585365855, 0.1073170731707317, -0.24878048780487805, 0.09268292682926829, -0.33170731707317075, -0.03902439024390244, -0.11219512195121951, -0.06829268292682927, -0.12682926829268293, -0.02926829268292683, -0.04878048780487805, 0.04390243902439024, -0.3073170731707317, 0.04878048780487805, -0.47317073170731705, 0.04390243902439024, -0.6, -0.02926829268292683, -0.3073170731707317, -0.02926829268292683, -0.4634146341463415, -0.03902439024390244, -0.5902439024390244, 0.014634146341463415, -0.2097560975609756], [-0.0, -0.0, 0.0196078431372549, -0.10294117647058823, 0.08823529411764706, -0.13725490196078433, 0.10784313725490197, -0.25, 0.09313725490196079, -0.3333333333333333, -0.03431372549019608, -0.11274509803921569, -0.06862745098039216, -0.12745098039215685, -0.024509803921568627, -0.05392156862745098, 0.049019607843137254, -0.3088235294117647, 0.049019607843137254, -0.47549019607843135, 0.04411764705882353, -0.6029411764705882, -0.024509803921568627, -0.30392156862745096, -0.029411764705882353, -0.46568627450980393, -0.0392156862745098, -0.5931372549019608, 0.014705882352941176, -0.2107843137254902], [-0.0, -0.0, 0.014705882352941176, -0.10294117647058823, 0.08333333333333333, -0.13725490196078433, 0.10294117647058823, -0.25, 0.08823529411764706, -0.3333333333333333, -0.0392156862745098, -0.11274509803921569, -0.07352941176470588, -0.12745098039215685, -0.029411764705882353, -0.049019607843137254, 0.04411764705882353, -0.3088235294117647, 0.04411764705882353, -0.47058823529411764, 0.0392156862745098, -0.6029411764705882, -0.029411764705882353, -0.30392156862745096, -0.03431372549019608, -0.46568627450980393, -0.04411764705882353, -0.5931372549019608, 0.00980392156862745, -0.2107843137254902], [-0.0, -0.0, 0.00980392156862745, -0.10294117647058823, 0.07352941176470588, -0.14215686274509803, 0.09313725490196079, -0.25, 0.08333333333333333, -0.3284313725490196, -0.04411764705882353, -0.11764705882352941, -0.09803921568627451, -0.11274509803921569, -0.0392156862745098, -0.04411764705882353, 0.03431372549019608, -0.3088235294117647, 0.024509803921568627, -0.46568627450980393, 0.00980392156862745, -0.5980392156862745, -0.03431372549019608, -0.30392156862745096, -0.05392156862745098, -0.45588235294117646, -0.06862745098039216, -0.5833333333333334, -0.0, -0.21568627450980393], [-0.0, -0.0, 0.004464285714285714, -0.08482142857142858, 0.0625, -0.125, 0.08035714285714286, -0.21875, 0.06696428571428571, -0.29464285714285715, -0.044642857142857144, -0.09375, -0.10267857142857142, -0.08035714285714286, -0.03571428571428571, -0.026785714285714284, 0.022321428571428572, -0.26785714285714285, 0.022321428571428572, -0.4107142857142857, 0.022321428571428572, -0.5401785714285714, -0.044642857142857144, -0.25892857142857145, -0.04017857142857143, -0.4107142857142857, -0.04017857142857143, -0.5357142857142857, -0.004464285714285714, -0.18303571428571427], [-0.0, -0.0, -0.0, -0.09345794392523364, 0.056074766355140186, -0.13551401869158877, 0.07476635514018691, -0.2383177570093458, 0.07476635514018691, -0.3130841121495327, -0.0514018691588785, -0.102803738317757, -0.09813084112149532, -0.09345794392523364, -0.03271028037383177, -0.03271028037383177, 0.014018691588785047, -0.2803738317757009, 0.037383177570093455, -0.43457943925233644, 0.04672897196261682, -0.5654205607476636, -0.04672897196261682, -0.2803738317757009, -0.03271028037383177, -0.4252336448598131, -0.02336448598130841, -0.5607476635514018, -0.009345794392523364, -0.19626168224299065], [-0.0, -0.0, -0.0, -0.0815450643776824, 0.05150214592274678, -0.11587982832618025, 0.07725321888412018, -0.2017167381974249, 0.0815450643776824, -0.2703862660944206, -0.05150214592274678, -0.08583690987124463, -0.07296137339055794, -0.09012875536480687, -0.02575107296137339, -0.02575107296137339, 0.02575107296137339, -0.2532188841201717, 0.05150214592274678, -0.38626609442060084, 0.060085836909871244, -0.5107296137339056, -0.034334763948497854, -0.2532188841201717, -0.017167381974248927, -0.38197424892703863, -0.004291845493562232, -0.5021459227467812, -0.004291845493562232, -0.1759656652360515], [-0.0, -0.0, 0.004672897196261682, -0.09345794392523364, 0.06542056074766354, -0.1261682242990654, 0.08878504672897196, -0.22897196261682243, 0.0794392523364486, -0.3130841121495327, -0.04205607476635514, -0.102803738317757, -0.102803738317757, -0.09813084112149532, -0.04205607476635514, -0.03271028037383177, 0.03271028037383177, -0.2850467289719626, 0.037383177570093455, -0.4439252336448598, 0.037383177570093455, -0.5747663551401869, -0.037383177570093455, -0.2803738317757009, -0.037383177570093455, -0.43457943925233644, -0.037383177570093455, -0.5654205607476636, -0.0, -0.19626168224299065], [-0.0, -0.0, 0.005128205128205128, -0.1076923076923077, 0.07692307692307693, -0.14871794871794872, 0.09743589743589744, -0.26153846153846155, 0.08717948717948718, -0.3487179487179487, -0.05128205128205128, -0.12307692307692308, -0.11794871794871795, -0.10256410256410256, -0.03076923076923077, -0.046153846153846156, 0.03076923076923077, -0.3282051282051282, 0.02564102564102564, -0.48717948717948717, 0.02564102564102564, -0.6358974358974359, -0.041025641025641026, -0.3230769230769231, -0.035897435897435895, -0.48205128205128206, -0.041025641025641026, -0.6307692307692307, -0.0, -0.22564102564102564], [-0.0, -0.0, 0.010256410256410256, -0.1076923076923077, 0.08205128205128205, -0.14871794871794872, 0.10256410256410256, -0.26153846153846155, 0.09230769230769231, -0.3487179487179487, -0.041025641025641026, -0.12307692307692308, -0.11282051282051282, -0.08717948717948718, -0.02564102564102564, -0.035897435897435895, 0.035897435897435895, -0.3230769230769231, 0.035897435897435895, -0.48717948717948717, 0.02564102564102564, -0.6358974358974359, -0.035897435897435895, -0.31794871794871793, -0.02564102564102564, -0.48205128205128206, -0.02564102564102564, -0.6358974358974359, 0.005128205128205128, -0.22564102564102564], [-0.0, -0.0, 0.010256410256410256, -0.1076923076923077, 0.07692307692307693, -0.14871794871794872, 0.09743589743589744, -0.26666666666666666, 0.09230769230769231, -0.3487179487179487, -0.046153846153846156, -0.12307692307692308, -0.11282051282051282, -0.08717948717948718, -0.02564102564102564, -0.035897435897435895, 0.03076923076923077, -0.3230769230769231, 0.03076923076923077, -0.48717948717948717, 0.02564102564102564, -0.6358974358974359, -0.041025641025641026, -0.31794871794871793, -0.02564102564102564, -0.48205128205128206, -0.02564102564102564, -0.6358974358974359, 0.005128205128205128, -0.22564102564102564], [-0.0, -0.0, 0.006024096385542169, -0.13855421686746988, 0.09036144578313253, -0.1927710843373494, 0.1144578313253012, -0.3313253012048193, 0.10240963855421686, -0.43373493975903615, -0.060240963855421686, -0.16265060240963855, -0.14457831325301204, -0.10843373493975904, -0.04216867469879518, -0.04216867469879518, 0.03614457831325301, -0.4036144578313253, 0.03614457831325301, -0.5903614457831325, 0.030120481927710843, -0.7650602409638554, -0.04819277108433735, -0.39759036144578314, -0.03614457831325301, -0.5843373493975904, -0.030120481927710843, -0.7650602409638554, -0.0, -0.28313253012048195], [-0.0, -0.0, 0.005376344086021506, -0.11290322580645161, 0.07526881720430108, -0.16129032258064516, 0.0967741935483871, -0.2849462365591398, 0.0913978494623656, -0.3709677419354839, -0.053763440860215055, -0.13440860215053763, -0.13440860215053763, -0.08064516129032258, -0.043010752688172046, -0.021505376344086023, 0.026881720430107527, -0.34946236559139787, 0.03225806451612903, -0.5161290322580645, 0.021505376344086023, -0.6720430107526881, -0.04838709677419355, -0.34408602150537637, -0.03763440860215054, -0.510752688172043, -0.03225806451612903, -0.6720430107526881, -0.0, -0.24193548387096775], [-0.0, -0.0, 0.010752688172043012, -0.11290322580645161, 0.08602150537634409, -0.16129032258064516, 0.10752688172043011, -0.27956989247311825, 0.0967741935483871, -0.3709677419354839, -0.04838709677419355, -0.13440860215053763, -0.12365591397849462, -0.08602150537634409, -0.03763440860215054, -0.026881720430107527, 0.03763440860215054, -0.34946236559139787, 0.03763440860215054, -0.5161290322580645, 0.03225806451612903, -0.6720430107526881, -0.03763440860215054, -0.34408602150537637, -0.026881720430107527, -0.5161290322580645, -0.021505376344086023, -0.6720430107526881, 0.005376344086021506, -0.24193548387096775], [-0.0, -0.0, 0.010810810810810811, -0.11891891891891893, 0.08648648648648649, -0.16216216216216217, 0.10810810810810811, -0.2864864864864865, 0.0972972972972973, -0.372972972972973, -0.04864864864864865, -0.13513513513513514, -0.12432432432432433, -0.08648648648648649, -0.03783783783783784, -0.02702702702702703, 0.03783783783783784, -0.35135135135135137, 0.03783783783783784, -0.518918918918919, 0.032432432432432434, -0.6756756756756757, -0.03783783783783784, -0.34594594594594597, -0.02702702702702703, -0.5135135135135135, -0.021621621621621623, -0.6756756756756757, 0.005405405405405406, -0.24324324324324326], [-0.0, -0.0, 0.005376344086021506, -0.11290322580645161, 0.08064516129032258, -0.16129032258064516, 0.10215053763440861, -0.2849462365591398, 0.0913978494623656, -0.3709677419354839, -0.053763440860215055, -0.13440860215053763, -0.12903225806451613, -0.08602150537634409, -0.03763440860215054, -0.026881720430107527, 0.03225806451612903, -0.34946236559139787, 0.03225806451612903, -0.5161290322580645, 0.026881720430107527, -0.6720430107526881, -0.043010752688172046, -0.34408602150537637, -0.03225806451612903, -0.5161290322580645, -0.026881720430107527, -0.6720430107526881, -0.0, -0.24193548387096775], [-0.0, -0.0, 0.005376344086021506, -0.11290322580645161, 0.08064516129032258, -0.16129032258064516, 0.10752688172043011, -0.2849462365591398, 0.0967741935483871, -0.3709677419354839, -0.053763440860215055, -0.13440860215053763, -0.12903225806451613, -0.08602150537634409, -0.03763440860215054, -0.03225806451612903, 0.03225806451612903, -0.34946236559139787, 0.03225806451612903, -0.5161290322580645, 0.026881720430107527, -0.6720430107526881, -0.043010752688172046, -0.34408602150537637, -0.026881720430107527, -0.5161290322580645, -0.026881720430107527, -0.6720430107526881, 0.005376344086021506, -0.24193548387096775], [-0.0, -0.0, 0.005376344086021506, -0.11290322580645161, 0.08064516129032258, -0.16129032258064516, 0.10215053763440861, -0.2849462365591398, 0.0913978494623656, -0.3709677419354839, -0.053763440860215055, -0.13440860215053763, -0.12903225806451613, -0.08602150537634409, -0.03763440860215054, -0.026881720430107527, 0.03225806451612903, -0.34946236559139787, 0.03225806451612903, -0.5161290322580645, 0.026881720430107527, -0.6720430107526881, -0.043010752688172046, -0.34408602150537637, -0.026881720430107527, -0.5161290322580645, -0.026881720430107527, -0.6720430107526881, -0.0, -0.24193548387096775], [-0.0, -0.0, 0.005376344086021506, -0.11290322580645161, 0.08064516129032258, -0.16129032258064516, 0.10215053763440861, -0.2849462365591398, 0.0913978494623656, -0.3709677419354839, -0.053763440860215055, -0.13440860215053763, -0.12903225806451613, -0.08602150537634409, -0.03763440860215054, -0.021505376344086023, 0.03225806451612903, -0.34946236559139787, 0.03225806451612903, -0.5161290322580645, 0.026881720430107527, -0.6720430107526881, -0.043010752688172046, -0.34408602150537637, -0.03225806451612903, -0.5161290322580645, -0.026881720430107527, -0.6720430107526881, -0.0, -0.24193548387096775], [-0.0, -0.0, 0.010752688172043012, -0.11290322580645161, 0.08064516129032258, -0.16129032258064516, 0.10752688172043011, -0.27956989247311825, 0.0967741935483871, -0.3709677419354839, -0.04838709677419355, -0.13440860215053763, -0.12365591397849462, -0.08602150537634409, -0.03763440860215054, -0.026881720430107527, 0.03763440860215054, -0.34946236559139787, 0.03763440860215054, -0.5161290322580645, 0.03225806451612903, -0.6720430107526881, -0.03763440860215054, -0.34946236559139787, -0.026881720430107527, -0.5161290322580645, -0.021505376344086023, -0.6720430107526881, 0.005376344086021506, -0.24193548387096775], [-0.0, -0.0, 0.015306122448979591, -0.10204081632653061, 0.08163265306122448, -0.14795918367346939, 0.10204081632653061, -0.2602040816326531, 0.09693877551020408, -0.3469387755102041, -0.04591836734693878, -0.12755102040816327, -0.11224489795918367, -0.07653061224489796, -0.030612244897959183, -0.025510204081632654, 0.04081632653061224, -0.32653061224489793, 0.04081632653061224, -0.4846938775510204, 0.03571428571428571, -0.6326530612244898, -0.030612244897959183, -0.32142857142857145, -0.02040816326530612, -0.47959183673469385, -0.015306122448979591, -0.6326530612244898, 0.01020408163265306, -0.22448979591836735], [-0.0, -0.0, 0.00510204081632653, -0.10204081632653061, 0.07142857142857142, -0.14795918367346939, 0.09693877551020408, -0.2602040816326531, 0.08673469387755102, -0.3469387755102041, -0.05102040816326531, -0.12244897959183673, -0.12244897959183673, -0.07653061224489796, -0.03571428571428571, -0.02040816326530612, 0.030612244897959183, -0.32142857142857145, 0.030612244897959183, -0.4846938775510204, 0.025510204081632654, -0.6326530612244898, -0.04081632653061224, -0.32142857142857145, -0.030612244897959183, -0.47959183673469385, -0.025510204081632654, -0.6326530612244898, -0.0, -0.22448979591836735], [-0.0, -0.0, 0.005376344086021506, -0.11290322580645161, 0.08064516129032258, -0.16129032258064516, 0.10215053763440861, -0.27956989247311825, 0.0913978494623656, -0.3709677419354839, -0.053763440860215055, -0.13440860215053763, -0.12903225806451613, -0.08602150537634409, -0.043010752688172046, -0.03763440860215054, 0.03225806451612903, -0.34408602150537637, 0.03225806451612903, -0.5161290322580645, 0.026881720430107527, -0.6720430107526881, -0.043010752688172046, -0.34408602150537637, -0.03225806451612903, -0.510752688172043, -0.026881720430107527, -0.6720430107526881, -0.0, -0.24193548387096775], [-0.0, -0.0, 0.005405405405405406, -0.11891891891891893, 0.07567567567567568, -0.16216216216216217, 0.10270270270270271, -0.2810810810810811, 0.0918918918918919, -0.372972972972973, -0.05405405405405406, -0.13513513513513514, -0.13513513513513514, -0.08648648648648649, -0.03783783783783784, -0.03783783783783784, 0.02702702702702703, -0.35135135135135137, 0.02702702702702703, -0.518918918918919, 0.021621621621621623, -0.6756756756756757, -0.043243243243243246, -0.34594594594594597, -0.032432432432432434, -0.5135135135135135, -0.032432432432432434, -0.6756756756756757, -0.0, -0.24324324324324326], [-0.0, -0.0, 0.010256410256410256, -0.1076923076923077, 0.07692307692307693, -0.14871794871794872, 0.09743589743589744, -0.26153846153846155, 0.08717948717948718, -0.3487179487179487, -0.046153846153846156, -0.12307692307692308, -0.12307692307692308, -0.08205128205128205, -0.035897435897435895, -0.03076923076923077, 0.03076923076923077, -0.3230769230769231, 0.03076923076923077, -0.48717948717948717, 0.02564102564102564, -0.6358974358974359, -0.041025641025641026, -0.3230769230769231, -0.02564102564102564, -0.48205128205128206, -0.02564102564102564, -0.6358974358974359, -0.0, -0.22564102564102564], [-0.0, -0.0, 0.005376344086021506, -0.11290322580645161, 0.08064516129032258, -0.16129032258064516, 0.10215053763440861, -0.27956989247311825, 0.0913978494623656, -0.3709677419354839, -0.04838709677419355, -0.13440860215053763, -0.12903225806451613, -0.0913978494623656, -0.03763440860215054, -0.043010752688172046, 0.03225806451612903, -0.34946236559139787, 0.03225806451612903, -0.5161290322580645, 0.026881720430107527, -0.6720430107526881, -0.043010752688172046, -0.34408602150537637, -0.03225806451612903, -0.510752688172043, -0.026881720430107527, -0.6720430107526881, -0.0, -0.24193548387096775], [-0.0, -0.0, 0.005128205128205128, -0.1076923076923077, 0.07692307692307693, -0.14871794871794872, 0.09743589743589744, -0.26153846153846155, 0.08717948717948718, -0.3487179487179487, -0.05128205128205128, -0.12307692307692308, -0.12307692307692308, -0.08205128205128205, -0.041025641025641026, -0.03076923076923077, 0.03076923076923077, -0.3230769230769231, 0.03076923076923077, -0.48717948717948717, 0.02564102564102564, -0.6358974358974359, -0.041025641025641026, -0.31794871794871793, -0.03076923076923077, -0.48205128205128206, -0.02564102564102564, -0.6358974358974359, -0.0, -0.22564102564102564], [-0.0, -0.0, 0.01020408163265306, -0.10204081632653061, 0.07653061224489796, -0.14795918367346939, 0.09693877551020408, -0.2602040816326531, 0.09183673469387756, -0.3469387755102041, -0.04591836734693878, -0.12244897959183673, -0.11734693877551021, -0.07653061224489796, -0.03571428571428571, -0.030612244897959183, 0.03571428571428571, -0.32142857142857145, 0.03571428571428571, -0.4846938775510204, 0.030612244897959183, -0.6326530612244898, -0.03571428571428571, -0.32142857142857145, -0.025510204081632654, -0.47959183673469385, -0.02040816326530612, -0.6326530612244898, 0.00510204081632653, -0.22448979591836735], [-0.0, -0.0, 0.0048543689320388345, -0.09223300970873786, 0.06796116504854369, -0.13592233009708737, 0.08737864077669903, -0.24271844660194175, 0.0825242718446602, -0.32524271844660196, -0.04854368932038835, -0.11165048543689321, -0.11650485436893204, -0.07281553398058252, -0.043689320388349516, -0.024271844660194174, 0.02912621359223301, -0.30097087378640774, 0.02912621359223301, -0.4563106796116505, 0.024271844660194174, -0.5970873786407767, -0.038834951456310676, -0.2961165048543689, -0.02912621359223301, -0.45145631067961167, -0.024271844660194174, -0.5970873786407767, -0.0, -0.2087378640776699],[-0.0, -0.0, 0.0106951871657754, -0.10695187165775401, 0.08021390374331551, -0.12299465240641712, 0.1657754010695187, -0.08021390374331551, 0.1497326203208556, 0.016042780748663103, -0.053475935828877004, -0.13903743315508021, -0.1443850267379679, -0.11229946524064172, -0.15508021390374332, -0.016042780748663103, 0.053475935828877004, -0.3315508021390374, 0.06417112299465241, -0.5026737967914439, 0.058823529411764705, -0.6417112299465241, -0.0213903743315508, -0.33689839572192515, -0.016042780748663103, -0.5080213903743316, -0.0106951871657754, -0.6631016042780749, 0.016042780748663103, -0.23529411764705882], [-0.0, -0.0, -0.0, -0.05704697986577181, 0.03691275167785235, -0.06711409395973154, 0.08053691275167785, -0.053691275167785234, 0.07046979865771812, -0.0, -0.040268456375838924, -0.06040268456375839, -0.08389261744966443, -0.0436241610738255, -0.07718120805369127, 0.003355704697986577, 0.006711409395973154, -0.18456375838926176, 0.010067114093959731, -0.2751677852348993, 0.013422818791946308, -0.35906040268456374, -0.03691275167785235, -0.18120805369127516, -0.040268456375838924, -0.2651006711409396, -0.0436241610738255, -0.3221476510067114, -0.010067114093959731, -0.12416107382550336], [-0.0, -0.0, -0.0029069767441860465, -0.046511627906976744, 0.023255813953488372, -0.05813953488372093, 0.05813953488372093, -0.05813953488372093, 0.06395348837209303, -0.01744186046511628, -0.029069767441860465, -0.05232558139534884, -0.06976744186046512, -0.0377906976744186, -0.061046511627906974, -0.005813953488372093, -0.0029069767441860465, -0.14244186046511628, -0.005813953488372093, -0.20930232558139536, -0.00872093023255814, -0.2703488372093023, -0.02616279069767442, -0.13953488372093023, -0.029069767441860465, -0.20930232558139536, -0.040697674418604654, -0.2703488372093023, -0.00872093023255814, -0.09883720930232558], [-0.0, -0.0, 0.0029154518950437317, -0.04956268221574344, 0.026239067055393587, -0.05539358600583091, 0.05539358600583091, -0.052478134110787174, 0.05830903790087463, -0.01749271137026239, -0.023323615160349854, -0.05539358600583091, -0.0641399416909621, -0.04081632653061224, -0.052478134110787174, -0.0, 0.008746355685131196, -0.13702623906705538, 0.0058309037900874635, -0.20408163265306123, -0.008746355685131196, -0.2653061224489796, -0.014577259475218658, -0.13994169096209913, -0.0058309037900874635, -0.20699708454810495, -0.011661807580174927, -0.2594752186588921, -0.0029154518950437317, -0.09912536443148688], [-0.0, -0.0, -0.005813953488372093, -0.046511627906976744, 0.020348837209302327, -0.05813953488372093, 0.04941860465116279, -0.061046511627906974, 0.04941860465116279, -0.00872093023255814, -0.03197674418604651, -0.05232558139534884, -0.07267441860465117, -0.040697674418604654, -0.061046511627906974, -0.0, 0.005813953488372093, -0.14825581395348839, 0.005813953488372093, -0.21511627906976744, -0.0029069767441860465, -0.27325581395348836, -0.023255813953488372, -0.14534883720930233, -0.02616279069767442, -0.21511627906976744, -0.05813953488372093, -0.2703488372093023, -0.00872093023255814, -0.10174418604651163], [-0.0, -0.0, -0.0, -0.052307692307692305, 0.027692307692307693, -0.06153846153846154, 0.06153846153846154, -0.052307692307692305, 0.05846153846153846, -0.009230769230769232, -0.03076923076923077, -0.05846153846153846, -0.07384615384615385, -0.036923076923076927, -0.05846153846153846, 0.006153846153846154, 0.018461538461538463, -0.15692307692307692, 0.03076923076923077, -0.2276923076923077, 0.009230769230769232, -0.2953846153846154, -0.018461538461538463, -0.15692307692307692, -0.009230769230769232, -0.24, -0.006153846153846154, -0.3046153846153846, -0.003076923076923077, -0.1076923076923077], [-0.0, -0.0, 0.0038461538461538464, -0.07307692307692308, 0.046153846153846156, -0.08846153846153847, 0.1, -0.08461538461538462, 0.08076923076923077, -0.023076923076923078, -0.038461538461538464, -0.08461538461538462, -0.1, -0.06153846153846154, -0.08461538461538462, -0.0038461538461538464, 0.019230769230769232, -0.23076923076923078, 0.0038461538461538464, -0.3346153846153846, -0.0038461538461538464, -0.4230769230769231, -0.038461538461538464, -0.22692307692307692, -0.046153846153846156, -0.3269230769230769, -0.057692307692307696, -0.4076923076923077, -0.007692307692307693, -0.1576923076923077], [-0.0, -0.0, 0.010752688172043012, -0.11290322580645161, 0.07526881720430108, -0.14516129032258066, 0.16666666666666666, -0.12365591397849462, 0.12903225806451613, -0.026881720430107527, -0.053763440860215055, -0.14516129032258066, -0.15591397849462366, -0.10752688172043011, -0.12365591397849462, -0.016129032258064516, 0.053763440860215055, -0.34408602150537637, 0.05913978494623656, -0.5161290322580645, 0.05913978494623656, -0.6720430107526881, -0.026881720430107527, -0.34946236559139787, -0.016129032258064516, -0.521505376344086, -0.005376344086021506, -0.6666666666666666, 0.010752688172043012, -0.24731182795698925], [-0.0, -0.0, -0.005988023952095809, -0.1317365269461078, 0.0718562874251497, -0.15568862275449102, 0.16167664670658682, -0.1317365269461078, 0.11976047904191617, -0.03592814371257485, -0.0718562874251497, -0.1497005988023952, -0.16167664670658682, -0.12574850299401197, -0.17964071856287425, -0.03592814371257485, 0.017964071856287425, -0.38922155688622756, -0.005988023952095809, -0.6107784431137725, -0.017964071856287425, -0.7904191616766467, -0.07784431137724551, -0.38323353293413176, -0.12574850299401197, -0.5868263473053892, -0.11377245508982035, -0.7485029940119761, -0.017964071856287425, -0.2754491017964072], [-0.0, -0.0, 0.017964071856287425, -0.1317365269461078, 0.08383233532934131, -0.1497005988023952, 0.19760479041916168, -0.1317365269461078, 0.16167664670658682, -0.023952095808383235, -0.05389221556886228, -0.16167664670658682, -0.1497005988023952, -0.1437125748502994, -0.17964071856287425, -0.05389221556886228, 0.059880239520958084, -0.38922155688622756, 0.05389221556886228, -0.5988023952095808, 0.08982035928143713, -0.7784431137724551, -0.041916167664670656, -0.39520958083832336, -0.011976047904191617, -0.6047904191616766, -0.0, -0.7544910179640718, 0.011976047904191617, -0.2754491017964072], [-0.0, -0.0, -0.007142857142857143, -0.15714285714285714, 0.08571428571428572, -0.19285714285714287, 0.22857142857142856, -0.17142857142857143, 0.2, -0.02857142857142857, -0.1, -0.17857142857142858, -0.24285714285714285, -0.14285714285714285, -0.25, -0.014285714285714285, 0.02857142857142857, -0.4857142857142857, 0.02857142857142857, -0.7571428571428571, 0.02857142857142857, -0.9857142857142858, -0.1, -0.4785714285714286, -0.15714285714285714, -0.7428571428571429, -0.20714285714285716, -0.95, -0.02857142857142857, -0.3357142857142857], [-0.0, -0.0, -0.010256410256410256, -0.1076923076923077, 0.05641025641025641, -0.1282051282051282, 0.13333333333333333, -0.11282051282051282, 0.14871794871794872, -0.03076923076923077, -0.07692307692307693, -0.11282051282051282, -0.17435897435897435, -0.08717948717948718, -0.14871794871794872, -0.0, 0.020512820512820513, -0.3128205128205128, 0.020512820512820513, -0.47692307692307695, 0.015384615384615385, -0.6102564102564103, -0.05641025641025641, -0.3076923076923077, -0.046153846153846156, -0.48717948717948717, -0.05641025641025641, -0.6358974358974359, -0.015384615384615385, -0.2205128205128205], [-0.0, -0.0, -0.011904761904761904, -0.07142857142857142, 0.031746031746031744, -0.0873015873015873, 0.07936507936507936, -0.07936507936507936, 0.09126984126984126, -0.01984126984126984, -0.05952380952380952, -0.07539682539682539, -0.11904761904761904, -0.051587301587301584, -0.0992063492063492, 0.007936507936507936, 0.003968253968253968, -0.23015873015873015, 0.023809523809523808, -0.34523809523809523, 0.027777777777777776, -0.4444444444444444, -0.051587301587301584, -0.23015873015873015, -0.03968253968253968, -0.3492063492063492, -0.03968253968253968, -0.4523809523809524, -0.01984126984126984, -0.15079365079365079], [-0.0, -0.0, -0.009230769230769232, -0.052307692307692305, 0.021538461538461538, -0.06153846153846154, 0.05846153846153846, -0.055384615384615386, 0.06461538461538462, -0.015384615384615385, -0.046153846153846156, -0.04923076923076923, -0.08, -0.03076923076923077, -0.07076923076923076, 0.012307692307692308, -0.0, -0.15692307692307692, -0.006153846153846154, -0.23384615384615384, -0.015384615384615385, -0.30153846153846153, -0.04, -0.15384615384615385, -0.04, -0.23384615384615384, -0.03076923076923077, -0.2923076923076923, -0.015384615384615385, -0.10461538461538461], [-0.0, -0.0, 0.010135135135135136, -0.06418918918918919, 0.04054054054054054, -0.07432432432432433, 0.07094594594594594, -0.057432432432432436, 0.04054054054054054, -0.006756756756756757, -0.013513513513513514, -0.07432432432432433, -0.0472972972972973, -0.0472972972972973, -0.04054054054054054, -0.013513513513513514, 0.02364864864864865, -0.18243243243243243, 0.02027027027027027, -0.2635135135135135, 0.02027027027027027, -0.33783783783783783, -0.006756756756756757, -0.18243243243243243, -0.013513513513513514, -0.2635135135135135, 0.0033783783783783786, -0.32432432432432434, 0.010135135135135136, -0.12837837837837837], [-0.0, -0.0, 0.0030864197530864196, -0.05555555555555555, 0.024691358024691357, -0.05864197530864197, 0.04938271604938271, -0.05246913580246913, 0.021604938271604937, -0.012345679012345678, -0.021604938271604937, -0.06172839506172839, -0.046296296296296294, -0.06481481481481481, -0.06172839506172839, -0.033950617283950615, 0.006172839506172839, -0.16049382716049382, 0.012345679012345678, -0.2345679012345679, 0.009259259259259259, -0.2932098765432099, -0.018518518518518517, -0.16049382716049382, -0.027777777777777776, -0.22839506172839505, -0.021604938271604937, -0.28703703703703703, 0.0030864197530864196, -0.1111111111111111], [-0.0, -0.0, 0.002840909090909091, -0.048295454545454544, 0.02556818181818182, -0.056818181818181816, 0.05397727272727273, -0.0625, 0.05113636363636364, -0.017045454545454544, -0.022727272727272728, -0.056818181818181816, -0.056818181818181816, -0.048295454545454544, -0.0625, -0.02556818181818182, 0.014204545454545454, -0.13636363636363635, 0.014204545454545454, -0.19602272727272727, 0.014204545454545454, -0.2528409090909091, -0.014204545454545454, -0.13636363636363635, -0.005681818181818182, -0.19602272727272727, 0.002840909090909091, -0.2528409090909091, -0.0, -0.09659090909090909], [-0.0, -0.0, -0.002770083102493075, -0.04709141274238227, 0.019390581717451522, -0.055401662049861494, 0.04986149584487535, -0.055401662049861494, 0.04155124653739612, -0.0110803324099723, -0.030470914127423823, -0.05263157894736842, -0.06371191135734072, -0.038781163434903045, -0.060941828254847646, -0.00554016620498615, -0.0, -0.1329639889196676, -0.00554016620498615, -0.19113573407202217, -0.008310249307479225, -0.24653739612188366, -0.030470914127423823, -0.13019390581717452, -0.030470914127423823, -0.19113573407202217, -0.04155124653739612, -0.24653739612188366, -0.013850415512465374, -0.09141274238227147], [-0.0, -0.0, -0.002688172043010753, -0.04032258064516129, 0.01881720430107527, -0.0456989247311828, 0.0456989247311828, -0.04838709677419355, 0.043010752688172046, -0.002688172043010753, -0.026881720430107527, -0.043010752688172046, -0.06451612903225806, -0.03225806451612903, -0.051075268817204304, 0.005376344086021506, 0.010752688172043012, -0.12634408602150538, 0.01881720430107527, -0.18548387096774194, 0.024193548387096774, -0.24193548387096775, -0.01881720430107527, -0.12634408602150538, -0.016129032258064516, -0.18548387096774194, -0.016129032258064516, -0.23387096774193547, -0.002688172043010753, -0.08602150537634409], [-0.0, -0.0, -0.0030864197530864196, -0.05555555555555555, 0.027777777777777776, -0.06481481481481481, 0.06481481481481481, -0.06481481481481481, 0.06172839506172839, -0.021604938271604937, -0.027777777777777776, -0.06481481481481481, -0.07407407407407407, -0.05246913580246913, -0.05864197530864197, -0.006172839506172839, 0.012345679012345678, -0.1574074074074074, -0.0, -0.2345679012345679, -0.012345679012345678, -0.2993827160493827, -0.018518518518518517, -0.1574074074074074, -0.018518518518518517, -0.2345679012345679, -0.027777777777777776, -0.2993827160493827, -0.0030864197530864196, -0.11419753086419752], [-0.0, -0.0, 0.0031645569620253164, -0.05379746835443038, 0.03481012658227848, -0.06329113924050633, 0.07278481012658228, -0.056962025316455694, 0.06645569620253164, -0.006329113924050633, -0.02531645569620253, -0.06329113924050633, -0.0759493670886076, -0.04430379746835443, -0.06329113924050633, 0.0031645569620253164, 0.0189873417721519, -0.16455696202531644, 0.02531645569620253, -0.2468354430379747, 0.03164556962025317, -0.31962025316455694, -0.0189873417721519, -0.16455696202531644, -0.012658227848101266, -0.2468354430379747, -0.015822784810126583, -0.3227848101265823, 0.0031645569620253164, -0.11392405063291139], [-0.0, -0.0, 0.006557377049180328, -0.06229508196721312, 0.036065573770491806, -0.06885245901639345, 0.07213114754098361, -0.05245901639344262, 0.03934426229508197, -0.003278688524590164, -0.019672131147540985, -0.07213114754098361, -0.04918032786885246, -0.07213114754098361, -0.06229508196721312, -0.03278688524590164, 0.013114754098360656, -0.1737704918032787, 0.003278688524590164, -0.25245901639344265, 0.006557377049180328, -0.32131147540983607, -0.019672131147540985, -0.1737704918032787, -0.019672131147540985, -0.24918032786885247, -0.013114754098360656, -0.3016393442622951, 0.003278688524590164, -0.12459016393442623], [-0.0, -0.0, -0.0, -0.05555555555555555, 0.021604938271604937, -0.05864197530864197, 0.046296296296296294, -0.05246913580246913, 0.015432098765432098, -0.018518518518518517, -0.024691358024691357, -0.06172839506172839, -0.05246913580246913, -0.05864197530864197, -0.06481481481481481, -0.033950617283950615, 0.006172839506172839, -0.16049382716049382, -0.006172839506172839, -0.23148148148148148, -0.0, -0.28703703703703703, -0.018518518518518517, -0.1574074074074074, -0.030864197530864196, -0.23148148148148148, -0.009259259259259259, -0.2808641975308642, -0.0030864197530864196, -0.1111111111111111], [-0.0, -0.0, 0.005390835579514825, -0.0431266846361186, 0.026954177897574125, -0.04582210242587601, 0.05929919137466307, -0.04582210242587601, 0.0431266846361186, -0.008086253369272238, -0.016172506738544475, -0.05121293800539083, -0.04043126684636118, -0.04851752021563342, -0.05390835579514825, -0.02425876010781671, 0.013477088948787063, -0.12398921832884097, 0.018867924528301886, -0.18059299191374664, 0.0215633423180593, -0.23450134770889489, -0.008086253369272238, -0.12398921832884097, -0.0026954177897574125, -0.18059299191374664, 0.0026954177897574125, -0.22911051212938005, 0.005390835579514825, -0.0862533692722372], [-0.0, -0.0, 0.005494505494505495, -0.038461538461538464, 0.03021978021978022, -0.04395604395604396, 0.057692307692307696, -0.04945054945054945, 0.04395604395604396, -0.0027472527472527475, -0.016483516483516484, -0.04395604395604396, -0.046703296703296704, -0.03571428571428571, -0.04120879120879121, -0.038461538461538464, 0.01098901098901099, -0.13186813186813187, 0.016483516483516484, -0.19230769230769232, 0.024725274725274724, -0.25824175824175827, -0.013736263736263736, -0.12912087912087913, -0.019230769230769232, -0.18956043956043955, -0.024725274725274724, -0.23901098901098902, 0.008241758241758242, -0.08791208791208792], [-0.0, -0.0, -0.0, -0.04956268221574344, 0.026239067055393587, -0.061224489795918366, 0.061224489795918366, -0.061224489795918366, 0.0641399416909621, -0.02040816326530612, -0.023323615160349854, -0.05830903790087463, -0.0058309037900874635, 0.014577259475218658, -0.0641399416909621, -0.029154518950437316, 0.0058309037900874635, -0.14285714285714285, -0.0, -0.20699708454810495, -0.0, -0.26822157434402333, -0.02040816326530612, -0.14285714285714285, -0.014577259475218658, -0.20699708454810495, -0.0029154518950437317, -0.2594752186588921, -0.0029154518950437317, -0.10204081632653061], [-0.0, -0.0, -0.012307692307692308, -0.052307692307692305, 0.018461538461538463, -0.06461538461538462, 0.04, -0.07692307692307693, 0.04923076923076923, -0.043076923076923075, -0.04, -0.055384615384615386, -0.07384615384615385, -0.04, -0.06153846153846154, 0.003076923076923077, -0.0, -0.15384615384615385, 0.012307692307692308, -0.22153846153846155, 0.012307692307692308, -0.28615384615384615, -0.036923076923076927, -0.15384615384615385, -0.03076923076923077, -0.2276923076923077, -0.03076923076923077, -0.28923076923076924, -0.018461538461538463, -0.1076923076923077], [-0.0, -0.0, -0.0056657223796034, -0.0453257790368272, 0.0113314447592068, -0.053824362606232294, 0.039660056657223795, -0.059490084985835696, 0.04815864022662889, -0.028328611898016998, -0.028328611898016998, -0.05099150141643059, -0.06515580736543909, -0.031161473087818695, -0.05099150141643059, 0.0028328611898017, -0.0113314447592068, -0.13314447592067988, -0.014164305949008499, -0.19830028328611898, -0.014164305949008499, -0.26628895184135976, -0.019830028328611898, -0.13314447592067988, -0.014164305949008499, -0.19830028328611898, -0.014164305949008499, -0.2577903682719547, -0.014164305949008499, -0.09348441926345609], [-0.0, -0.0, -0.006134969325153374, -0.049079754601226995, 0.02147239263803681, -0.06134969325153374, 0.046012269938650305, -0.06134969325153374, 0.05214723926380368, -0.03374233128834356, -0.03374233128834356, -0.05521472392638037, -0.0736196319018405, -0.027607361963190184, -0.049079754601226995, 0.015337423312883436, 0.006134969325153374, -0.15644171779141106, 0.02147239263803681, -0.2331288343558282, 0.02147239263803681, -0.30368098159509205, -0.027607361963190184, -0.15950920245398773, -0.03067484662576687, -0.2361963190184049, -0.03680981595092025, -0.3067484662576687, -0.009202453987730062, -0.10736196319018405], [-0.0, -0.0, -0.006514657980456026, -0.05537459283387622, 0.026058631921824105, -0.07166123778501629, 0.06188925081433225, -0.07166123778501629, 0.06840390879478828, -0.02280130293159609, -0.035830618892508145, -0.06188925081433225, -0.08794788273615635, -0.035830618892508145, -0.06188925081433225, 0.006514657980456026, 0.003257328990228013, -0.17263843648208468, -0.009771986970684038, -0.25732899022801303, -0.019543973941368076, -0.33876221498371334, -0.029315960912052116, -0.16938110749185667, -0.02280130293159609, -0.25732899022801303, -0.02280130293159609, -0.3355048859934853, -0.009771986970684038, -0.11726384364820847], [-0.0, -0.0, -0.0, -0.08968609865470852, 0.053811659192825115, -0.11210762331838565, 0.11659192825112108, -0.10762331838565023, 0.10762331838565023, -0.03587443946188341, -0.04932735426008968, -0.10762331838565023, -0.13004484304932734, -0.07174887892376682, -0.1031390134529148, -0.0, 0.03587443946188341, -0.273542600896861, 0.03139013452914798, -0.4170403587443946, 0.026905829596412557, -0.5291479820627802, -0.03139013452914798, -0.273542600896861, -0.04484304932735426, -0.4125560538116592, -0.07623318385650224, -0.5201793721973094, -0.0, -0.18834080717488788], [-0.0, -0.0, 0.011299435028248588, -0.11864406779661017, 0.0847457627118644, -0.14124293785310735, 0.1864406779661017, -0.12429378531073447, 0.14689265536723164, -0.02824858757062147, -0.05649717514124294, -0.14124293785310735, -0.15819209039548024, -0.12994350282485875, -0.1751412429378531, -0.03389830508474576, 0.04519774011299435, -0.3672316384180791, 0.05084745762711865, -0.5536723163841808, 0.05084745762711865, -0.7005649717514124, -0.04519774011299435, -0.3672316384180791, -0.02824858757062147, -0.5649717514124294, -0.011299435028248588, -0.7175141242937854, 0.005649717514124294, -0.2542372881355932], [-0.0, -0.0, 0.05357142857142857, -0.20535714285714285, 0.125, -0.24107142857142858, 0.21428571428571427, -0.14285714285714285, 0.09821428571428571, -0.026785714285714284, -0.017857142857142856, -0.26785714285714285, -0.125, -0.2767857142857143, -0.19642857142857142, -0.16071428571428573, 0.07142857142857142, -0.6428571428571429, 0.07142857142857142, -0.9642857142857143, 0.0625, -1.25, -0.05357142857142857, -0.6339285714285714, -0.09821428571428571, -0.9375, -0.08035714285714286, -1.1785714285714286, 0.03571428571428571, -0.44642857142857145], [-0.0, -0.0, 0.0650887573964497, -0.11834319526627218, 0.09467455621301775, -0.1301775147928994, 0.04142011834319527, -0.1893491124260355, 0.023668639053254437, -0.08875739644970414, 0.05325443786982249, -0.15384615384615385, 0.005917159763313609, -0.1952662721893491, -0.0650887573964497, -0.1301775147928994, 0.05917159763313609, -0.3905325443786982, 0.01775147928994083, -0.5798816568047337, 0.047337278106508875, -0.7692307692307693, 0.05325443786982249, -0.38461538461538464, 0.047337278106508875, -0.5739644970414202, 0.07100591715976332, -0.7514792899408284, 0.0650887573964497, -0.27218934911242604], [-0.0, -0.0, 0.06486486486486487, -0.11891891891891893, 0.0918918918918919, -0.13513513513513514, 0.03783783783783784, -0.15135135135135136, 0.016216216216216217, -0.07027027027027027, 0.05405405405405406, -0.14054054054054055, 0.016216216216216217, -0.12972972972972974, -0.032432432432432434, -0.08108108108108109, 0.06486486486486487, -0.3567567567567568, 0.07027027027027027, -0.518918918918919, 0.0918918918918919, -0.6918918918918919, 0.03783783783783784, -0.34594594594594597, 0.016216216216216217, -0.5027027027027027, 0.016216216216216217, -0.6324324324324324, 0.06486486486486487, -0.24324324324324326], [-0.0, -0.0, 0.008, -0.08, 0.052, -0.092, 0.112, -0.084, 0.084, -0.016, -0.036, -0.092, -0.088, -0.092, -0.112, -0.036, 0.024, -0.232, 0.024, -0.344, 0.04, -0.448, -0.024, -0.232, -0.016, -0.344, -0.0, -0.444, 0.004, -0.16], [-0.0, -0.0, -0.006514657980456026, -0.05537459283387622, 0.029315960912052116, -0.06840390879478828, 0.0781758957654723, -0.05863192182410423, 0.07166123778501629, -0.013029315960912053, -0.04234527687296417, -0.05863192182410423, -0.09120521172638436, -0.05211726384364821, -0.0749185667752443, -0.0, 0.009771986970684038, -0.17263843648208468, 0.02280130293159609, -0.26058631921824105, 0.04560260586319218, -0.34201954397394135, -0.035830618892508145, -0.16938110749185667, -0.03257328990228013, -0.26058631921824105, -0.03257328990228013, -0.3289902280130293, -0.009771986970684038, -0.11726384364820847], [-0.0, -0.0, -0.009230769230769232, -0.052307692307692305, 0.018461538461538463, -0.06153846153846154, 0.046153846153846156, -0.06461538461538462, 0.05846153846153846, -0.024615384615384615, -0.036923076923076927, -0.05846153846153846, -0.08, -0.036923076923076927, -0.05846153846153846, 0.006153846153846154, 0.003076923076923077, -0.15384615384615385, 0.006153846153846154, -0.23076923076923078, -0.0, -0.30153846153846153, -0.03076923076923077, -0.15384615384615385, -0.027692307692307693, -0.23384615384615384, -0.04, -0.3076923076923077, -0.012307692307692308, -0.1076923076923077], [-0.0, -0.0, -0.003076923076923077, -0.052307692307692305, 0.021538461538461538, -0.06153846153846154, 0.04923076923076923, -0.07076923076923076, 0.06461538461538462, -0.033846153846153845, -0.033846153846153845, -0.05846153846153846, -0.07384615384615385, -0.043076923076923075, -0.05846153846153846, 0.003076923076923077, 0.009230769230769232, -0.15076923076923077, -0.003076923076923077, -0.23076923076923078, -0.015384615384615385, -0.3076923076923077, -0.024615384615384615, -0.15076923076923077, -0.015384615384615385, -0.23384615384615384, -0.018461538461538463, -0.31076923076923074, -0.006153846153846154, -0.10461538461538461], [-0.0, -0.0, -0.0, -0.06666666666666667, 0.040740740740740744, -0.07777777777777778, 0.08518518518518518, -0.07777777777777778, 0.07777777777777778, -0.014814814814814815, -0.040740740740740744, -0.07777777777777778, -0.09259259259259259, -0.06296296296296296, -0.07037037037037037, -0.007407407407407408, 0.022222222222222223, -0.2111111111111111, 0.037037037037037035, -0.31851851851851853, 0.04814814814814815, -0.4111111111111111, -0.02962962962962963, -0.2111111111111111, -0.025925925925925925, -0.31851851851851853, -0.03333333333333333, -0.4111111111111111, -0.003703703703703704, -0.14074074074074075], [-0.0, -0.0, -0.0047169811320754715, -0.10377358490566038, 0.05188679245283019, -0.12264150943396226, 0.12735849056603774, -0.1179245283018868, 0.10377358490566038, -0.02830188679245283, -0.05188679245283019, -0.12264150943396226, -0.1320754716981132, -0.11320754716981132, -0.11320754716981132, -0.02830188679245283, 0.02830188679245283, -0.29245283018867924, 0.0047169811320754715, -0.42924528301886794, -0.014150943396226415, -0.5471698113207547, -0.04245283018867924, -0.28773584905660377, -0.03773584905660377, -0.42924528301886794, -0.05188679245283019, -0.5566037735849056, -0.0047169811320754715, -0.20754716981132076], [-0.0, -0.0, 0.011363636363636364, -0.125, 0.07954545454545454, -0.14772727272727273, 0.18181818181818182, -0.13068181818181818, 0.14772727272727273, -0.03409090909090909, -0.0625, -0.14772727272727273, -0.16477272727272727, -0.13636363636363635, -0.16477272727272727, -0.045454545454545456, 0.03977272727272727, -0.36363636363636365, 0.022727272727272728, -0.5681818181818182, 0.03409090909090909, -0.7386363636363636, -0.05113636363636364, -0.36363636363636365, -0.03409090909090909, -0.5568181818181818, -0.022727272727272728, -0.7102272727272727, 0.005681818181818182, -0.2556818181818182], [-0.0, -0.0, -0.005988023952095809, -0.1317365269461078, 0.0658682634730539, -0.16167664670658682, 0.17964071856287425, -0.1497005988023952, 0.1497005988023952, -0.04790419161676647, -0.08383233532934131, -0.1497005988023952, -0.20958083832335328, -0.1317365269461078, -0.17964071856287425, -0.023952095808383235, 0.023952095808383235, -0.39520958083832336, 0.0658682634730539, -0.5988023952095808, 0.10778443113772455, -0.7724550898203593, -0.0718562874251497, -0.39520958083832336, -0.05389221556886228, -0.5988023952095808, -0.04790419161676647, -0.7604790419161677, -0.017964071856287425, -0.2754491017964072], [-0.0, -0.0, -0.011363636363636364, -0.125, 0.0625, -0.14772727272727273, 0.16477272727272727, -0.13068181818181818, 0.13068181818181818, -0.03409090909090909, -0.07954545454545454, -0.13636363636363635, -0.19886363636363635, -0.125, -0.19886363636363635, -0.028409090909090908, 0.028409090909090908, -0.3693181818181818, 0.005681818181818182, -0.5625, -0.005681818181818182, -0.7272727272727273, -0.06818181818181818, -0.3693181818181818, -0.08522727272727272, -0.5568181818181818, -0.11363636363636363, -0.7159090909090909, -0.017045454545454544, -0.26136363636363635], [-0.0, -0.0, -0.004310344827586207, -0.08620689655172414, 0.04310344827586207, -0.09913793103448276, 0.11206896551724138, -0.09913793103448276, 0.09913793103448276, -0.017241379310344827, -0.0603448275862069, -0.09051724137931035, -0.13793103448275862, -0.07327586206896551, -0.1206896551724138, -0.004310344827586207, 0.01293103448275862, -0.25862068965517243, 0.01293103448275862, -0.3922413793103448, 0.02586206896551724, -0.5172413793103449, -0.05172413793103448, -0.25862068965517243, -0.05603448275862069, -0.3922413793103448, -0.05172413793103448, -0.4870689655172414, -0.017241379310344827, -0.17672413793103448], [-0.0, -0.0, -0.004329004329004329, -0.09090909090909091, 0.05194805194805195, -0.1038961038961039, 0.11688311688311688, -0.09956709956709957, 0.09090909090909091, -0.021645021645021644, -0.05627705627705628, -0.09956709956709957, -0.12987012987012986, -0.08225108225108226, -0.12554112554112554, -0.021645021645021644, 0.017316017316017316, -0.2683982683982684, 0.004329004329004329, -0.39826839826839827, -0.004329004329004329, -0.5064935064935064, -0.047619047619047616, -0.26406926406926406, -0.06060606060606061, -0.3939393939393939, -0.0735930735930736, -0.5064935064935064, -0.012987012987012988, -0.18181818181818182], [-0.0, -0.0, 0.014778325123152709, -0.10837438423645321, 0.06896551724137931, -0.12315270935960591, 0.15763546798029557, -0.12807881773399016, 0.14285714285714285, -0.029556650246305417, -0.04433497536945813, -0.12315270935960591, -0.1330049261083744, -0.10344827586206896, -0.12315270935960591, -0.019704433497536946, 0.04926108374384237, -0.3054187192118227, 0.06896551724137931, -0.45320197044334976, 0.07881773399014778, -0.5911330049261084, -0.034482758620689655, -0.3054187192118227, -0.029556650246305417, -0.4630541871921182, -0.019704433497536946, -0.5960591133004927, 0.009852216748768473, -0.21182266009852216], [-0.0, -0.0, 0.005649717514124294, -0.11864406779661017, 0.0847457627118644, -0.15254237288135594, 0.1864406779661017, -0.14124293785310735, 0.15819209039548024, -0.04519774011299435, -0.05649717514124294, -0.14689265536723164, -0.1694915254237288, -0.12994350282485875, -0.1638418079096045, -0.03954802259887006, 0.05649717514124294, -0.3728813559322034, 0.04519774011299435, -0.559322033898305, 0.03954802259887006, -0.7175141242937854, -0.03954802259887006, -0.3672316384180791, -0.05084745762711865, -0.559322033898305, -0.05649717514124294, -0.7062146892655368, 0.011299435028248588, -0.2598870056497175], [-0.0, -0.0, -0.0, -0.1317365269461078, 0.0658682634730539, -0.15568862275449102, 0.18562874251497005, -0.15568862275449102, 0.16167664670658682, -0.05389221556886228, -0.07784431137724551, -0.15568862275449102, -0.19760479041916168, -0.12574850299401197, -0.17964071856287425, -0.023952095808383235, 0.041916167664670656, -0.40119760479041916, 0.041916167664670656, -0.5988023952095808, 0.05389221556886228, -0.7664670658682635, -0.059880239520958084, -0.39520958083832336, -0.04790419161676647, -0.5988023952095808, -0.05389221556886228, -0.7664670658682635, -0.005988023952095809, -0.2754491017964072], [-0.0, -0.0, -0.0, -0.11864406779661017, 0.06779661016949153, -0.14689265536723164, 0.1807909604519774, -0.13559322033898305, 0.15254237288135594, -0.03954802259887006, -0.07344632768361582, -0.14689265536723164, -0.1864406779661017, -0.12429378531073447, -0.1638418079096045, -0.01694915254237288, 0.04519774011299435, -0.3672316384180791, 0.03954802259887006, -0.559322033898305, 0.05084745762711865, -0.7175141242937854, -0.05084745762711865, -0.3672316384180791, -0.04519774011299435, -0.559322033898305, -0.05084745762711865, -0.7175141242937854, -0.0, -0.2598870056497175], [-0.0, -0.0, 0.005917159763313609, -0.11834319526627218, 0.07692307692307693, -0.14201183431952663, 0.1952662721893491, -0.14201183431952663, 0.1834319526627219, -0.029585798816568046, -0.07100591715976332, -0.14201183431952663, -0.1893491124260355, -0.1242603550295858, -0.1893491124260355, -0.011834319526627219, 0.05325443786982249, -0.38461538461538464, 0.047337278106508875, -0.591715976331361, 0.05325443786982249, -0.7633136094674556, -0.05325443786982249, -0.38461538461538464, -0.05917159763313609, -0.5976331360946746, -0.08284023668639054, -0.757396449704142, 0.005917159763313609, -0.26627218934911245], [-0.0, -0.0, -0.0, -0.11834319526627218, 0.07100591715976332, -0.14201183431952663, 0.1893491124260355, -0.14201183431952663, 0.21893491124260356, -0.03550295857988166, -0.07100591715976332, -0.14201183431952663, -0.1834319526627219, -0.1301775147928994, -0.23076923076923078, -0.023668639053254437, 0.047337278106508875, -0.38461538461538464, 0.05917159763313609, -0.591715976331361, 0.07100591715976332, -0.7633136094674556, -0.047337278106508875, -0.38461538461538464, -0.047337278106508875, -0.591715976331361, -0.047337278106508875, -0.757396449704142, -0.0, -0.27218934911242604], [-0.0, -0.0, -0.012658227848101266, -0.13924050632911392, 0.06962025316455696, -0.15822784810126583, 0.189873417721519, -0.1518987341772152, 0.22151898734177214, -0.03164556962025317, -0.08860759493670886, -0.1518987341772152, -0.21518987341772153, -0.13924050632911392, -0.2468354430379747, -0.02531645569620253, 0.0379746835443038, -0.4240506329113924, 0.0379746835443038, -0.6392405063291139, 0.04430379746835443, -0.8227848101265823, -0.06962025316455696, -0.4240506329113924, -0.06962025316455696, -0.6455696202531646, -0.08227848101265822, -0.8164556962025317, -0.012658227848101266, -0.2911392405063291], [-0.0, -0.0, -0.012658227848101266, -0.13924050632911392, 0.06962025316455696, -0.16455696202531644, 0.189873417721519, -0.1518987341772152, 0.22151898734177214, -0.03164556962025317, -0.08860759493670886, -0.1518987341772152, -0.2088607594936709, -0.14556962025316456, -0.2468354430379747, -0.02531645569620253, 0.0379746835443038, -0.4240506329113924, 0.0379746835443038, -0.6392405063291139, 0.0379746835443038, -0.8227848101265823, -0.0759493670886076, -0.4240506329113924, -0.06962025316455696, -0.6455696202531646, -0.08227848101265822, -0.8164556962025317, -0.012658227848101266, -0.2911392405063291], [-0.0, -0.0, -0.005952380952380952, -0.125, 0.06547619047619048, -0.1488095238095238, 0.17857142857142858, -0.13690476190476192, 0.20833333333333334, -0.023809523809523808, -0.08333333333333333, -0.13690476190476192, -0.19642857142857142, -0.13095238095238096, -0.23214285714285715, -0.017857142857142856, 0.03571428571428571, -0.39285714285714285, 0.03571428571428571, -0.5952380952380952, 0.041666666666666664, -0.7678571428571429, -0.06547619047619048, -0.3869047619047619, -0.06547619047619048, -0.5952380952380952, -0.07738095238095238, -0.7619047619047619, -0.011904761904761904, -0.26785714285714285], [-0.0, -0.0, -0.012658227848101266, -0.13924050632911392, 0.06962025316455696, -0.15822784810126583, 0.18354430379746836, -0.15822784810126583, 0.21518987341772153, -0.0379746835443038, -0.08860759493670886, -0.15822784810126583, -0.21518987341772153, -0.14556962025316456, -0.2468354430379747, -0.02531645569620253, 0.0379746835443038, -0.4240506329113924, 0.03164556962025317, -0.6392405063291139, 0.0379746835443038, -0.8227848101265823, -0.0759493670886076, -0.4177215189873418, -0.0759493670886076, -0.6392405063291139, -0.08860759493670886, -0.8164556962025317, -0.012658227848101266, -0.2911392405063291], [-0.0, -0.0, -0.006329113924050633, -0.13924050632911392, 0.06962025316455696, -0.16455696202531644, 0.18354430379746836, -0.15822784810126583, 0.22151898734177214, -0.03164556962025317, -0.08860759493670886, -0.15822784810126583, -0.2088607594936709, -0.1518987341772152, -0.25316455696202533, -0.02531645569620253, 0.0379746835443038, -0.4177215189873418, 0.03164556962025317, -0.6392405063291139, 0.0379746835443038, -0.8227848101265823, -0.0759493670886076, -0.41139240506329117, -0.0759493670886076, -0.6392405063291139, -0.08860759493670886, -0.8164556962025317, -0.012658227848101266, -0.2911392405063291], [-0.0, -0.0, -0.006329113924050633, -0.13924050632911392, 0.06962025316455696, -0.16455696202531644, 0.189873417721519, -0.16455696202531644, 0.22784810126582278, -0.0379746835443038, -0.08860759493670886, -0.15822784810126583, -0.21518987341772153, -0.1518987341772152, -0.25316455696202533, -0.03164556962025317, 0.0379746835443038, -0.4177215189873418, 0.03164556962025317, -0.6392405063291139, 0.0379746835443038, -0.8227848101265823, -0.0759493670886076, -0.4177215189873418, -0.0759493670886076, -0.6392405063291139, -0.08860759493670886, -0.8164556962025317, -0.012658227848101266, -0.2911392405063291], [-0.0, -0.0, -0.0, -0.1320754716981132, 0.07547169811320754, -0.16352201257861634, 0.1949685534591195, -0.1761006289308176, 0.22012578616352202, -0.03773584905660377, -0.07547169811320754, -0.15723270440251572, -0.18238993710691823, -0.1509433962264151, -0.22012578616352202, -0.050314465408805034, 0.050314465408805034, -0.42138364779874216, 0.03773584905660377, -0.6352201257861635, 0.031446540880503145, -0.8176100628930818, -0.050314465408805034, -0.41509433962264153, -0.06918238993710692, -0.6352201257861635, -0.09433962264150944, -0.8113207547169812, -0.0, -0.29559748427672955], [-0.0, -0.0, 0.004830917874396135, -0.08695652173913043, 0.06280193236714976, -0.10628019323671498, 0.12560386473429952, -0.057971014492753624, 0.14009661835748793, 0.01932367149758454, -0.043478260869565216, -0.1111111111111111, -0.12077294685990338, -0.04830917874396135, -0.1111111111111111, 0.033816425120772944, 0.03864734299516908, -0.2946859903381642, 0.04830917874396135, -0.45893719806763283, 0.057971014492753624, -0.5942028985507246, -0.033816425120772944, -0.2995169082125604, -0.033816425120772944, -0.45893719806763283, -0.033816425120772944, -0.5893719806763285, 0.004830917874396135, -0.2028985507246377], [-0.0, -0.0, 0.0044444444444444444, -0.08, 0.057777777777777775, -0.09777777777777778, 0.10666666666666667, -0.057777777777777775, 0.1111111111111111, 0.013333333333333334, -0.04, -0.09777777777777778, -0.10222222222222223, -0.035555555555555556, -0.08, 0.035555555555555556, 0.035555555555555556, -0.27555555555555555, 0.044444444444444446, -0.4088888888888889, 0.05333333333333334, -0.5244444444444445, -0.03111111111111111, -0.27111111111111114, -0.03111111111111111, -0.4088888888888889, -0.03111111111111111, -0.5155555555555555, -0.0, -0.18222222222222223], [-0.0, -0.0, 0.012295081967213115, -0.06967213114754098, 0.05737704918032787, -0.08196721311475409, 0.10655737704918032, -0.040983606557377046, 0.09836065573770492, 0.01639344262295082, -0.036885245901639344, -0.0860655737704918, -0.09016393442622951, -0.03278688524590164, -0.0778688524590164, 0.036885245901639344, 0.03278688524590164, -0.2459016393442623, 0.028688524590163935, -0.36475409836065575, 0.028688524590163935, -0.47540983606557374, -0.02459016393442623, -0.23770491803278687, -0.03278688524590164, -0.36475409836065575, -0.04918032786885246, -0.4713114754098361, 0.00819672131147541, -0.1598360655737705], [-0.0, -0.0, 0.015810276679841896, -0.06719367588932806, 0.05533596837944664, -0.07509881422924901, 0.10276679841897234, -0.03557312252964427, 0.08695652173913043, 0.023715415019762844, -0.03557312252964427, -0.08300395256916997, -0.08300395256916997, -0.02766798418972332, -0.07114624505928854, 0.039525691699604744, 0.03557312252964427, -0.233201581027668, 0.03162055335968379, -0.35177865612648224, 0.03162055335968379, -0.45454545454545453, -0.023715415019762844, -0.22924901185770752, -0.02766798418972332, -0.35177865612648224, -0.03557312252964427, -0.4505928853754941, 0.011857707509881422, -0.1541501976284585], [-0.0, -0.0, 0.01646090534979424, -0.07407407407407407, 0.05761316872427984, -0.0823045267489712, 0.102880658436214, -0.0411522633744856, 0.09053497942386832, 0.01646090534979424, -0.037037037037037035, -0.08641975308641975, -0.08641975308641975, -0.03292181069958848, -0.06995884773662552, 0.03292181069958848, 0.037037037037037035, -0.24691358024691357, 0.03292181069958848, -0.3662551440329218, 0.037037037037037035, -0.4732510288065844, -0.024691358024691357, -0.23868312757201646, -0.02880658436213992, -0.37037037037037035, -0.037037037037037035, -0.4732510288065844, 0.00823045267489712, -0.16049382716049382], [-0.0, -0.0, 0.011857707509881422, -0.06719367588932806, 0.05533596837944664, -0.07509881422924901, 0.09881422924901186, -0.03557312252964427, 0.08695652173913043, 0.019762845849802372, -0.03557312252964427, -0.08300395256916997, -0.08695652173913043, -0.023715415019762844, -0.06719367588932806, 0.039525691699604744, 0.03162055335968379, -0.22924901185770752, 0.03162055335968379, -0.35177865612648224, 0.03557312252964427, -0.45454545454545453, -0.023715415019762844, -0.22529644268774704, -0.02766798418972332, -0.35177865612648224, -0.03557312252964427, -0.4505928853754941, 0.007905138339920948, -0.15019762845849802], [-0.0, -0.0, 0.011904761904761904, -0.07142857142857142, 0.05555555555555555, -0.07539682539682539, 0.0992063492063492, -0.03968253968253968, 0.0873015873015873, 0.015873015873015872, -0.03571428571428571, -0.08333333333333333, -0.0873015873015873, -0.027777777777777776, -0.06746031746031746, 0.03968253968253968, 0.03571428571428571, -0.23015873015873015, 0.031746031746031744, -0.3492063492063492, 0.03571428571428571, -0.4523809523809524, -0.023809523809523808, -0.2261904761904762, -0.027777777777777776, -0.3531746031746032, -0.031746031746031744, -0.4523809523809524, 0.007936507936507936, -0.15079365079365079], [-0.0, -0.0, 0.00796812749003984, -0.07569721115537849, 0.04780876494023904, -0.07569721115537849, 0.08764940239043825, -0.027888446215139442, 0.07569721115537849, 0.027888446215139442, -0.0398406374501992, -0.0796812749003984, -0.08764940239043825, -0.01593625498007968, -0.055776892430278883, 0.043824701195219126, 0.027888446215139442, -0.23107569721115537, 0.027888446215139442, -0.350597609561753, 0.027888446215139442, -0.4541832669322709, -0.027888446215139442, -0.22709163346613545, -0.03187250996015936, -0.3545816733067729, -0.035856573705179286, -0.4541832669322709, 0.00398406374501992, -0.15139442231075698], [-0.0, -0.0, 0.017241379310344827, -0.08620689655172414, 0.05603448275862069, -0.08620689655172414, 0.09913793103448276, -0.03879310344827586, 0.07327586206896551, 0.021551724137931036, -0.034482758620689655, -0.08620689655172414, -0.06896551724137931, -0.03017241379310345, -0.04310344827586207, 0.034482758620689655, 0.03879310344827586, -0.2543103448275862, 0.03879310344827586, -0.3879310344827586, 0.03879310344827586, -0.5, -0.021551724137931036, -0.25, -0.02586206896551724, -0.3879310344827586, -0.03017241379310345, -0.5, 0.01293103448275862, -0.1724137931034483], [-0.0, -0.0, 0.012396694214876033, -0.07851239669421488, 0.05371900826446281, -0.07851239669421488, 0.09090909090909091, -0.0371900826446281, 0.07024793388429752, 0.02066115702479339, -0.0371900826446281, -0.08264462809917356, -0.07024793388429752, -0.024793388429752067, -0.045454545454545456, 0.0371900826446281, 0.03305785123966942, -0.2396694214876033, 0.03305785123966942, -0.3677685950413223, 0.0371900826446281, -0.47520661157024796, -0.024793388429752067, -0.2396694214876033, -0.028925619834710745, -0.371900826446281, -0.03305785123966942, -0.47520661157024796, 0.008264462809917356, -0.16115702479338842], [-0.0, -0.0, 0.024793388429752067, -0.07851239669421488, 0.06611570247933884, -0.07851239669421488, 0.10330578512396695, -0.0371900826446281, 0.08264462809917356, 0.02066115702479339, -0.024793388429752067, -0.07851239669421488, -0.05785123966942149, -0.028925619834710745, -0.03305785123966942, 0.0371900826446281, 0.045454545454545456, -0.24380165289256198, 0.045454545454545456, -0.3677685950413223, 0.045454545454545456, -0.47520661157024796, -0.012396694214876033, -0.2396694214876033, -0.01652892561983471, -0.371900826446281, -0.024793388429752067, -0.47520661157024796, 0.02066115702479339, -0.16115702479338842], [-0.0, -0.0, 0.02066115702479339, -0.07851239669421488, 0.06198347107438017, -0.07851239669421488, 0.09917355371900827, -0.0371900826446281, 0.07851239669421488, 0.024793388429752067, -0.028925619834710745, -0.07851239669421488, -0.06198347107438017, -0.024793388429752067, -0.0371900826446281, 0.0371900826446281, 0.04132231404958678, -0.2396694214876033, 0.04132231404958678, -0.36363636363636365, 0.04132231404958678, -0.47520661157024796, -0.01652892561983471, -0.23553719008264462, -0.02066115702479339, -0.3677685950413223, -0.024793388429752067, -0.47520661157024796, 0.01652892561983471, -0.16115702479338842], [-0.0, -0.0, 0.021551724137931036, -0.08620689655172414, 0.0603448275862069, -0.08620689655172414, 0.10344827586206896, -0.03879310344827586, 0.08189655172413793, 0.017241379310344827, -0.034482758620689655, -0.08620689655172414, -0.06896551724137931, -0.034482758620689655, -0.03879310344827586, 0.034482758620689655, 0.04310344827586207, -0.2543103448275862, 0.04310344827586207, -0.3879310344827586, 0.04310344827586207, -0.5, -0.017241379310344827, -0.25, -0.021551724137931036, -0.3879310344827586, -0.03017241379310345, -0.5, 0.01293103448275862, -0.1724137931034483], [-0.0, -0.0, 0.008, -0.08, 0.044, -0.08, 0.084, -0.048, 0.084, 0.008, -0.04, -0.084, -0.088, -0.02, -0.068, 0.032, 0.024, -0.228, 0.02, -0.352, 0.02, -0.452, -0.032, -0.228, -0.032, -0.348, -0.04, -0.452, -0.0, -0.152], [-0.0, -0.0, 0.003816793893129771, -0.0648854961832061, 0.04580152671755725, -0.08015267175572519, 0.0916030534351145, -0.03435114503816794, 0.09541984732824428, 0.019083969465648856, -0.03816793893129771, -0.07633587786259542, -0.08015267175572519, -0.03435114503816794, -0.0916030534351145, 0.026717557251908396, 0.019083969465648856, -0.21374045801526717, 0.019083969465648856, -0.32061068702290074, 0.011450381679389313, -0.4198473282442748, -0.022900763358778626, -0.2099236641221374, -0.015267175572519083, -0.32061068702290074, -0.007633587786259542, -0.41603053435114506, -0.0, -0.1450381679389313], [-0.0, -0.0, 0.00796812749003984, -0.07569721115537849, -0.043824701195219126, -0.08764940239043825, -0.09163346613545817, -0.043824701195219126, 0.0796812749003984, 0.02390438247011952, 0.05179282868525897, -0.08764940239043825, 0.09561752988047809, -0.0398406374501992, -0.099601593625498, 0.043824701195219126, 0.01195219123505976, -0.2350597609561753, 0.01593625498007968, -0.3426294820717131, 0.00796812749003984, -0.44621513944223107, -0.0, -0.23107569721115537, -0.01195219123505976, -0.3426294820717131, -0.00796812749003984, -0.43824701195219123, 0.00398406374501992, -0.1593625498007968], [-0.0, -0.0, 0.007936507936507936, -0.07142857142857142, 0.051587301587301584, -0.08333333333333333, 0.09523809523809523, -0.04365079365079365, 0.1111111111111111, 0.011904761904761904, -0.03968253968253968, -0.08333333333333333, -0.07936507936507936, -0.03968253968253968, -0.0992063492063492, 0.01984126984126984, 0.027777777777777776, -0.2261904761904762, 0.023809523809523808, -0.3412698412698413, 0.011904761904761904, -0.4444444444444444, -0.01984126984126984, -0.2261904761904762, -0.011904761904761904, -0.3412698412698413, -0.007936507936507936, -0.4365079365079365, 0.003968253968253968, -0.15079365079365079], [-0.0, -0.0, 0.01195219123505976, -0.07569721115537849, 0.055776892430278883, -0.08366533864541832, 0.099601593625498, -0.043824701195219126, 0.11553784860557768, 0.01195219123505976, -0.035856573705179286, -0.08764940239043825, -0.07569721115537849, -0.04780876494023904, -0.099601593625498, 0.01593625498007968, 0.027888446215139442, -0.23107569721115537, 0.0199203187250996, -0.3466135458167331, 0.01195219123505976, -0.44621513944223107, -0.01593625498007968, -0.22709163346613545, -0.01195219123505976, -0.3426294820717131, -0.00796812749003984, -0.43824701195219123, 0.00796812749003984, -0.1553784860557769], [-0.0, -0.0, 0.011904761904761904, -0.07142857142857142, 0.05555555555555555, -0.07936507936507936, 0.10317460317460317, -0.051587301587301584, 0.11507936507936507, 0.007936507936507936, -0.03571428571428571, -0.0873015873015873, -0.07936507936507936, -0.047619047619047616, -0.10317460317460317, 0.011904761904761904, 0.031746031746031744, -0.23015873015873015, 0.023809523809523808, -0.3412698412698413, 0.011904761904761904, -0.4444444444444444, -0.015873015873015872, -0.2261904761904762, -0.011904761904761904, -0.3412698412698413, -0.007936507936507936, -0.4365079365079365, 0.011904761904761904, -0.15476190476190477], [-0.0, -0.0, 0.011904761904761904, -0.07142857142857142, 0.05555555555555555, -0.08333333333333333, 0.10317460317460317, -0.05952380952380952, 0.11904761904761904, -0.0, -0.031746031746031744, -0.0873015873015873, -0.07936507936507936, -0.05952380952380952, -0.10714285714285714, 0.003968253968253968, 0.027777777777777776, -0.23015873015873015, 0.023809523809523808, -0.3412698412698413, 0.011904761904761904, -0.4444444444444444, -0.01984126984126984, -0.2261904761904762, -0.011904761904761904, -0.3373015873015873, -0.007936507936507936, -0.4365079365079365, 0.007936507936507936, -0.15476190476190477], [-0.0, -0.0, 0.007936507936507936, -0.07142857142857142, 0.05555555555555555, -0.08333333333333333, 0.10714285714285714, -0.06349206349206349, 0.11904761904761904, -0.003968253968253968, -0.031746031746031744, -0.0873015873015873, -0.08333333333333333, -0.05555555555555555, -0.10714285714285714, 0.003968253968253968, 0.027777777777777776, -0.23015873015873015, 0.01984126984126984, -0.3412698412698413, 0.011904761904761904, -0.4444444444444444, -0.01984126984126984, -0.2261904761904762, -0.011904761904761904, -0.3412698412698413, -0.007936507936507936, -0.44047619047619047, 0.007936507936507936, -0.15476190476190477], [-0.0, -0.0, 0.01195219123505976, -0.07569721115537849, 0.055776892430278883, -0.08366533864541832, 0.10756972111553785, -0.07171314741035857, 0.12350597609561753, -0.01195219123505976, -0.03187250996015936, -0.09163346613545817, -0.08764940239043825, -0.06374501992031872, -0.10756972111553785, -0.00796812749003984, 0.03187250996015936, -0.23107569721115537, 0.0199203187250996, -0.3426294820717131, 0.01195219123505976, -0.44621513944223107, -0.0199203187250996, -0.22709163346613545, -0.01195219123505976, -0.3426294820717131, -0.00796812749003984, -0.44223107569721115, 0.00796812749003984, -0.1553784860557769], [-0.0, -0.0, 0.01195219123505976, -0.07569721115537849, 0.055776892430278883, -0.08366533864541832, 0.11155378486055777, -0.07569721115537849, 0.12350597609561753, -0.01195219123505976, -0.03187250996015936, -0.08764940239043825, -0.08764940239043825, -0.07171314741035857, -0.11155378486055777, -0.00796812749003984, 0.03187250996015936, -0.23107569721115537, 0.0199203187250996, -0.3466135458167331, 0.01195219123505976, -0.44621513944223107, -0.0199203187250996, -0.22709163346613545, -0.01195219123505976, -0.3426294820717131, -0.00796812749003984, -0.44223107569721115, 0.00796812749003984, -0.1553784860557769], [-0.0, -0.0, 0.01195219123505976, -0.07569721115537849, 0.055776892430278883, -0.08366533864541832, 0.11155378486055777, -0.07569721115537849, 0.12350597609561753, -0.01195219123505976, -0.03187250996015936, -0.08764940239043825, -0.08764940239043825, -0.06772908366533864, -0.10756972111553785, -0.00796812749003984, 0.03187250996015936, -0.23107569721115537, 0.0199203187250996, -0.3426294820717131, 0.01195219123505976, -0.44621513944223107, -0.0199203187250996, -0.22709163346613545, -0.01195219123505976, -0.3426294820717131, -0.00796812749003984, -0.43824701195219123, 0.00796812749003984, -0.1553784860557769], [-0.0, -0.0, 0.007936507936507936, -0.07142857142857142, 0.051587301587301584, -0.08333333333333333, 0.10714285714285714, -0.07142857142857142, 0.11904761904761904, -0.007936507936507936, -0.031746031746031744, -0.0873015873015873, -0.09523809523809523, -0.05952380952380952, -0.10317460317460317, -0.0, 0.031746031746031744, -0.23015873015873015, 0.01984126984126984, -0.3412698412698413, 0.011904761904761904, -0.4444444444444444, -0.01984126984126984, -0.2261904761904762, -0.011904761904761904, -0.3412698412698413, -0.007936507936507936, -0.4365079365079365, 0.007936507936507936, -0.15476190476190477], [-0.0, -0.0, 0.007936507936507936, -0.07142857142857142, 0.051587301587301584, -0.08333333333333333, 0.10317460317460317, -0.07142857142857142, 0.11507936507936507, -0.011904761904761904, -0.03571428571428571, -0.0873015873015873, -0.0992063492063492, -0.05952380952380952, -0.10317460317460317, -0.003968253968253968, 0.027777777777777776, -0.23015873015873015, 0.015873015873015872, -0.34523809523809523, 0.007936507936507936, -0.4444444444444444, -0.023809523809523808, -0.2261904761904762, -0.015873015873015872, -0.3412698412698413, -0.011904761904761904, -0.44047619047619047, 0.003968253968253968, -0.15476190476190477], [-0.0, -0.0, 0.007936507936507936, -0.07142857142857142, 0.051587301587301584, -0.08333333333333333, 0.10317460317460317, -0.07539682539682539, 0.11507936507936507, -0.011904761904761904, -0.03571428571428571, -0.0873015873015873, -0.0992063492063492, -0.06349206349206349, -0.10714285714285714, -0.003968253968253968, 0.027777777777777776, -0.23015873015873015, 0.015873015873015872, -0.3412698412698413, 0.007936507936507936, -0.4444444444444444, -0.023809523809523808, -0.2261904761904762, -0.015873015873015872, -0.3412698412698413, -0.011904761904761904, -0.44047619047619047, 0.003968253968253968, -0.15476190476190477], [-0.0, -0.0, 0.00398406374501992, -0.07569721115537849, 0.04780876494023904, -0.08764940239043825, 0.10358565737051793, -0.07569721115537849, 0.11553784860557768, -0.01195219123505976, -0.035856573705179286, -0.08764940239043825, -0.10358565737051793, -0.05976095617529881, -0.10756972111553785, -0.0, 0.02390438247011952, -0.23107569721115537, 0.0199203187250996, -0.3386454183266932, 0.00796812749003984, -0.44223107569721115, -0.02390438247011952, -0.22709163346613545, -0.0199203187250996, -0.3386454183266932, -0.01195219123505976, -0.43824701195219123, 0.00398406374501992, -0.1553784860557769], [-0.0, -0.0, 0.007936507936507936, -0.07142857142857142, 0.047619047619047616, -0.08333333333333333, 0.10714285714285714, -0.07539682539682539, 0.11507936507936507, -0.011904761904761904, -0.03571428571428571, -0.0873015873015873, -0.0992063492063492, -0.06349206349206349, -0.1111111111111111, -0.003968253968253968, 0.027777777777777776, -0.23015873015873015, 0.01984126984126984, -0.3412698412698413, 0.011904761904761904, -0.4444444444444444, -0.01984126984126984, -0.2261904761904762, -0.015873015873015872, -0.3412698412698413, -0.007936507936507936, -0.4365079365079365, 0.003968253968253968, -0.15476190476190477], [-0.0, -0.0, 0.003968253968253968, -0.07142857142857142, 0.047619047619047616, -0.0873015873015873, 0.10317460317460317, -0.07539682539682539, 0.11507936507936507, -0.011904761904761904, -0.03571428571428571, -0.0873015873015873, -0.10317460317460317, -0.06349206349206349, -0.11507936507936507, -0.003968253968253968, 0.023809523809523808, -0.23015873015873015, 0.01984126984126984, -0.3412698412698413, 0.007936507936507936, -0.4444444444444444, -0.023809523809523808, -0.2261904761904762, -0.01984126984126984, -0.3373015873015873, -0.011904761904761904, -0.4365079365079365, 0.003968253968253968, -0.15476190476190477], [-0.0, -0.0, 0.003968253968253968, -0.07142857142857142, 0.047619047619047616, -0.0873015873015873, 0.10317460317460317, -0.07142857142857142, 0.1111111111111111, -0.011904761904761904, -0.03571428571428571, -0.0873015873015873, -0.10317460317460317, -0.05952380952380952, -0.10714285714285714, 0.003968253968253968, 0.023809523809523808, -0.23015873015873015, 0.01984126984126984, -0.3412698412698413, 0.007936507936507936, -0.4444444444444444, -0.023809523809523808, -0.2261904761904762, -0.01984126984126984, -0.3373015873015873, -0.011904761904761904, -0.4365079365079365, 0.003968253968253968, -0.15476190476190477], [-0.0, -0.0, 0.003968253968253968, -0.07142857142857142, 0.047619047619047616, -0.08333333333333333, 0.10317460317460317, -0.07539682539682539, 0.11507936507936507, -0.011904761904761904, -0.03968253968253968, -0.0873015873015873, -0.0992063492063492, -0.05952380952380952, -0.10317460317460317, -0.003968253968253968, 0.027777777777777776, -0.23015873015873015, 0.015873015873015872, -0.3412698412698413, 0.007936507936507936, -0.4444444444444444, -0.023809523809523808, -0.2261904761904762, -0.015873015873015872, -0.3412698412698413, -0.011904761904761904, -0.44047619047619047, 0.003968253968253968, -0.15476190476190477], [-0.0, -0.0, 0.003968253968253968, -0.07142857142857142, 0.047619047619047616, -0.08333333333333333, 0.10317460317460317, -0.07142857142857142, 0.11507936507936507, -0.007936507936507936, -0.03968253968253968, -0.0873015873015873, -0.0992063492063492, -0.05555555555555555, -0.10317460317460317, -0.0, 0.023809523809523808, -0.2261904761904762, 0.01984126984126984, -0.3412698412698413, 0.007936507936507936, -0.4444444444444444, -0.023809523809523808, -0.2222222222222222, -0.015873015873015872, -0.3373015873015873, -0.011904761904761904, -0.4365079365079365, 0.003968253968253968, -0.15476190476190477], [-0.0, -0.0, 0.003968253968253968, -0.07142857142857142, 0.047619047619047616, -0.08333333333333333, 0.0992063492063492, -0.07936507936507936, 0.11904761904761904, -0.015873015873015872, -0.03571428571428571, -0.0873015873015873, -0.0992063492063492, -0.06349206349206349, -0.11507936507936507, -0.003968253968253968, 0.027777777777777776, -0.23015873015873015, 0.01984126984126984, -0.3412698412698413, 0.007936507936507936, -0.4444444444444444, -0.023809523809523808, -0.2261904761904762, -0.015873015873015872, -0.3373015873015873, -0.011904761904761904, -0.4365079365079365, 0.003968253968253968, -0.15476190476190477], [-0.0, -0.0, 0.003968253968253968, -0.07142857142857142, 0.047619047619047616, -0.08333333333333333, 0.10714285714285714, -0.07936507936507936, 0.12301587301587301, -0.015873015873015872, -0.03571428571428571, -0.0873015873015873, -0.09523809523809523, -0.07142857142857142, -0.12301587301587301, -0.011904761904761904, 0.027777777777777776, -0.23015873015873015, 0.015873015873015872, -0.3412698412698413, 0.007936507936507936, -0.4444444444444444, -0.01984126984126984, -0.2261904761904762, -0.01984126984126984, -0.3373015873015873, -0.011904761904761904, -0.4365079365079365, 0.003968253968253968, -0.15476190476190477], [-0.0, -0.0, 0.007936507936507936, -0.07142857142857142, 0.051587301587301584, -0.08333333333333333, 0.10714285714285714, -0.07936507936507936, 0.12301587301587301, -0.01984126984126984, -0.03571428571428571, -0.0873015873015873, -0.09523809523809523, -0.07142857142857142, -0.11904761904761904, -0.007936507936507936, 0.027777777777777776, -0.23015873015873015, 0.015873015873015872, -0.34523809523809523, 0.007936507936507936, -0.4444444444444444, -0.01984126984126984, -0.2261904761904762, -0.015873015873015872, -0.3412698412698413, -0.011904761904761904, -0.44047619047619047, 0.003968253968253968, -0.15476190476190477], [-0.0, -0.0, 0.007936507936507936, -0.07142857142857142, 0.051587301587301584, -0.08333333333333333, 0.1111111111111111, -0.07936507936507936, 0.12698412698412698, -0.007936507936507936, -0.031746031746031744, -0.0873015873015873, -0.09126984126984126, -0.07142857142857142, -0.11904761904761904, -0.011904761904761904, 0.027777777777777776, -0.23412698412698413, 0.015873015873015872, -0.34523809523809523, 0.007936507936507936, -0.4444444444444444, -0.01984126984126984, -0.2261904761904762, -0.015873015873015872, -0.3412698412698413, -0.011904761904761904, -0.4365079365079365, 0.007936507936507936, -0.15476190476190477], [-0.0, -0.0, 0.007936507936507936, -0.07142857142857142, 0.051587301587301584, -0.08333333333333333, 0.1111111111111111, -0.07936507936507936, 0.12301587301587301, -0.007936507936507936, -0.031746031746031744, -0.0873015873015873, -0.09126984126984126, -0.07142857142857142, -0.11904761904761904, -0.011904761904761904, 0.027777777777777776, -0.23412698412698413, 0.015873015873015872, -0.34523809523809523, 0.007936507936507936, -0.4444444444444444, -0.01984126984126984, -0.2261904761904762, -0.015873015873015872, -0.3412698412698413, -0.011904761904761904, -0.4365079365079365, 0.007936507936507936, -0.15476190476190477], [-0.0, -0.0, 0.007936507936507936, -0.07142857142857142, 0.051587301587301584, -0.08333333333333333, 0.1111111111111111, -0.07936507936507936, 0.12301587301587301, -0.011904761904761904, -0.031746031746031744, -0.0873015873015873, -0.09126984126984126, -0.07142857142857142, -0.11507936507936507, -0.007936507936507936, 0.027777777777777776, -0.23015873015873015, 0.015873015873015872, -0.34523809523809523, 0.007936507936507936, -0.4444444444444444, -0.01984126984126984, -0.2261904761904762, -0.015873015873015872, -0.3412698412698413, -0.011904761904761904, -0.44047619047619047, 0.007936507936507936, -0.15476190476190477], [-0.0, -0.0, 0.007936507936507936, -0.07142857142857142, 0.051587301587301584, -0.08333333333333333, 0.1111111111111111, -0.07936507936507936, 0.12301587301587301, -0.015873015873015872, -0.03571428571428571, -0.0873015873015873, -0.0992063492063492, -0.06746031746031746, -0.11507936507936507, -0.007936507936507936, 0.027777777777777776, -0.23412698412698413, 0.015873015873015872, -0.3492063492063492, 0.007936507936507936, -0.44841269841269843, -0.01984126984126984, -0.23015873015873015, -0.015873015873015872, -0.34523809523809523, -0.011904761904761904, -0.44047619047619047, 0.007936507936507936, -0.15476190476190477], [-0.0, -0.0, 0.003968253968253968, -0.07142857142857142, 0.047619047619047616, -0.08333333333333333, 0.10317460317460317, -0.08333333333333333, 0.11904761904761904, -0.015873015873015872, -0.03571428571428571, -0.0873015873015873, -0.0992063492063492, -0.06746031746031746, -0.11904761904761904, -0.007936507936507936, 0.027777777777777776, -0.23015873015873015, 0.015873015873015872, -0.3412698412698413, 0.007936507936507936, -0.4444444444444444, -0.023809523809523808, -0.2261904761904762, -0.015873015873015872, -0.3412698412698413, -0.011904761904761904, -0.4365079365079365, 0.003968253968253968, -0.15476190476190477]]    

# shuffle data
idx = np.random.permutation(sample_count)

poses_shuffled = []
for i in range(sample_count):
    poses_shuffled.append(poses[idx[i]])

Y = np.zeros([sample_count])
for i in range(0,sample_count):
    Y[i] = i/100
    
y_shuffled = []
for i in range(sample_count):
    y_shuffled.append(Y[idx[i]])
    
poses_shuffled = mx.nd.array(np.asarray(poses_shuffled))
y_shuffled =  mx.nd.array(np.asarray(y_shuffled))
print poses_shuffled.shape
print y_shuffled.shape

(700L, 30L)
(700L,)


# Splitting the data set
Next, we’re splitting the data set 6 to 1 for training and validation. We use the NDArray.crop function to do this. Here, the data set is completely random, so we can use the top 600 sample for training and the bottom 100 sample for validation.

In [3]:
X_train = mx.nd.crop(poses_shuffled, begin=(0,0), end=(train_count,feature_count))
X_valid = mx.nd.crop(poses_shuffled, begin=(train_count,0), end=(sample_count,feature_count))
Y_train = y_shuffled[0:train_count]
Y_valid = y_shuffled[train_count:sample_count]

# Building the network

In [4]:
data = mx.sym.Variable('data')
fc1 = mx.sym.FullyConnected(data, name='fc1', num_hidden=64)
relu1 = mx.sym.Activation(fc1, name='relu1', act_type="relu")
fc2 = mx.sym.FullyConnected(relu1, name='fc2', num_hidden=category_count)
out = mx.sym.SoftmaxOutput(fc2, name='softmax')
mod = mx.mod.Module(out)

# Building the data iterator

In [ ]:
train_iter = mx.io.NDArrayIter(data=X_train,label=Y_train,batch_size=batch)

# Training the model

In [ ]:
# bind the input symbol to the actual data set (samples and labels). 
mod.bind(data_shapes=train_iter.provide_data, label_shapes=train_iter.provide_label)
# initializer
mod.init_params(initializer=mx.init.Xavier(magnitude=2.))
mod.init_optimizer(optimizer='sgd', optimizer_params=(('learning_rate', 0.1), ))
mod.fit(train_iter, num_epoch=1000)

/Users/abdu307/anaconda3/envs/python2/lib/python2.7/site-packages/mxnet/module/base_module.py:464: UserWarning: Parameters already initialized and force_init=False. init_params call ignored.
  allow_missing=allow_missing, force_init=force_init)
INFO:root:Epoch[0] Train-accuracy=0.186667
INFO:root:Epoch[0] Time cost=0.051
INFO:root:Epoch[1] Train-accuracy=0.303333
INFO:root:Epoch[1] Time cost=0.052
INFO:root:Epoch[2] Train-accuracy=0.400000
INFO:root:Epoch[2] Time cost=0.055
INFO:root:Epoch[3] Train-accuracy=0.460000
INFO:root:Epoch[3] Time cost=0.051
INFO:root:Epoch[4] Train-accuracy=0.540000
INFO:root:Epoch[4] Time cost=0.060
INFO:root:Epoch[5] Train-accuracy=0.605000
INFO:root:Epoch[5] Time cost=0.051
INFO:root:Epoch[6] Train-accuracy=0.653333
INFO:root:Epoch[6] Time cost=0.050
INFO:root:Epoch[7] Train-accuracy=0.688333
INFO:root:Epoch[7] Time cost=0.057
INFO:root:Epoch[8] Train-accuracy=0.718333
INFO:root:Epoch[8] Time cost=0.051
INFO:root:Epoch[9] Train-accuracy=0.733333
INFO:root:

# Validating the model

In [ ]:
pred_iter = mx.io.NDArrayIter(data=X_valid,label=Y_valid, batch_size=batch)
pred_count = valid_count
correct_preds = total_correct_preds = 0
for preds, i_batch, batch in mod.iter_predict(pred_iter):
    label = batch.label[0].asnumpy().astype(int)
    pred_label = preds[0].asnumpy().argmax(axis=1)
    correct_preds = np.sum(pred_label==label)
    total_correct_preds = total_correct_preds + correct_preds
print('Validation accuracy: %2.2f' % (1.0*total_correct_preds/pred_count))

# save a model to pose-symbol.json and pose-1000.params

In [ ]:
import os
filename = os.getcwd()+"/pose"
mod.save_checkpoint(filename,1000)

# leading and  Testing the model 

In [ ]:
import os
import mxnet as mx
import numpy as np
import logging
logging.basicConfig(level=logging.INFO)
sample_count = 7*20
train_count = 4*20
valid_count = sample_count - train_count
feature_count = 15*2
category_count = 7
batch=10

X_pred = mx.nd.zeros((10,feature_count))
Y_pred = mx.nd.empty((10,))

pred_iter = mx.io.NDArrayIter(data=X_pred,label=Y_pred, batch_size=batch)

filename = os.getcwd()+"/pose"
sym, arg_params, aux_params = mx.model.load_checkpoint(filename, 1000)

new_model = mx.mod.Module(symbol=sym)
new_model.bind(pred_iter.provide_data, pred_iter.provide_label)
new_model.set_params(arg_params, aux_params)


In [ ]:
for i in range(10):
    pose = mx.nd.array(poses[i])

    X_pred = mx.nd.empty((10,feature_count))
    Y_pred = mx.nd.empty((10,))

    X_pred[0] = pose

    #print(X_pred)
    pred_iter = mx.io.NDArrayIter(data=X_pred,label=Y_pred, batch_size=batch)

    a = new_model.predict(pred_iter)[0]
    a= list(a.asnumpy())
    print max(a)
    p = a.index(max(a))
    print ("real pose:",p,"   predicted pose:",Y[i])
